Wikipedia analysis using Latent Dirichlet Allocation (LDA)
==========================================================

Authors: Axel Berg, Johan Grönqvist, Jens Gulin

Link to presentation:
https://lunduniversityo365-my.sharepoint.com/:v:/g/personal/ax8107be*lu*se/EU2uVibnZShNsXfbPWXEmJQBDcc6dR9kFVCGCgo1TSckcQ?e=Gow6lI

Introduction
------------

As part of the course assignment to set up a scalable pipeline, we
decided to run topic analysis on Wikipedia articles. Latent Dirichlet
Allocation (LDA) extracts topics from the corpus and we uses those to
assign each article to the most covered topic. As a proof-of-concept
system, we make a simple recommender system, recommending the highest
scoring from each topic.

Although the pipeline is meant to be generalizable, this workbook is not
streamlined, but also explores the data. This is to help presentation,
and also to support any effort to re-use the pipeline on another data
source. If this was an automatic job on a stable data source, the
throughput would benefit from removing the intermediate snapshot storage
and sample output.

We currently run the pipeline on Swedish Wikipedia. Changing the
download and data cleaning slightly, it should work on other sources.
Most of the cells are language agnostic, but stopwords and tokenization
needs to be adapted to new languages. Swedish Wikipedia also have a
peculiarity, having a high number of articles automatically generated
from structured data by bots. We decided to discard those article from
analysis since their relatively high number and uniform wording skewed
the results. The detection of autogenerated articles is thus specific
and could be skipped or changed for other languages.

Scope
-----

This notebook examplifies topic analysis on Swedish Wikipedia. For
overview and documenation, the code is divided into sections. Use the
outline to navigate to relevant parts. The following stages illustrates
the flow of the pipeline.

1.  Download a data dump of all pages from Wikipedia (compressed XML of
    current state)
2.  Clean the data
    1.  Discard unwanted pages. Keep only article-space pages not
        automatically generated by algorithms.
    2.  Filter the xml to extract meta data and raw article text.
    3.  Remove links and other markup-formatting
3.  Generate the LDA model
4.  Remove stopwords
5.  Train the model
6.  Analyse the model
7.  Pick an article and explore its topic.
8.  Recommend similar articles.

Results
-------

The pipeline works well and the topics produced seems ok. We have done
no further qualitative evaluation and no systematic search for optimal
hyper-parameters. In regards to runtime performance, the scalable
cluster handles the work load in a fair manner and we have not focused
on optimization at this point.

There are several potential improvement that are outside of the scope of
this project, but hat we would expect to improve execution time,
selected topics, and use-cases utilizing the resulting model.

References
----------

-   https://en.wikipedia.org/wiki/Latent*Dirichlet*allocation
-   https://spark.apache.org/docs/latest/api/scala/org/apache/spark/mllib/clustering/LDA.html
-   https://dumps.wikimedia.org/svwiki/20201120/

Folders and Files
=================

To allow pipelines running in parallel simplify configurability, work is
parameterized in file and folder structure.

To simplify exploration, the data is saved to file as snapshots, to
avoid the need to rerun previous sections in every session. We here set
some variables that will help us locate those files correctly.

In [ ]:
val lang = "sv"
val freeze_date = "20201120"
val xml_file = lang + "wiki-" + freeze_date + "-pages-articles.xml"

val group = "05_lda"  // directory name to separate file structure, allows to run different instances in parallel
val localpath=s"file:/databricks/driver/$group/"
val dir = s"dbfs:/datasets/$group/"

val filtered_articles_file = s"${dir}filtered_articles_${lang}_${freeze_date}"
val lda_countVector_file = s"${dir}lda_countvector_${lang}_${freeze_date}"
val lda_file = s"${dir}lda_${lang}_${freeze_date}.model"
val stopword_path = s"${dir}stoppord.csv"


  

>     lang: String = sv
>     freeze_date: String = 20201120
>     xml_file: String = svwiki-20201120-pages-articles.xml
>     group: String = 05_lda
>     localpath: String = file:/databricks/driver/05_lda/
>     dir: String = dbfs:/datasets/05_lda/
>     filtered_articles_file: String = dbfs:/datasets/05_lda/filtered_articles_sv_20201120
>     lda_countVector_file: String = dbfs:/datasets/05_lda/lda_countvector_sv_20201120
>     lda_file: String = dbfs:/datasets/05_lda/lda_sv_20201120.model
>     stopword_path: String = dbfs:/datasets/05_lda/stoppord.csv

In [ ]:
export group=05_lda
mkdir $group
cd $group
pwd
ls

  

>     /databricks/driver/05_lda

  

Downloading
===========

Wikipedia data and stopword file for selected language.

### Swedish Stopwords

We download a list of suitable stopwords from a separate location.

In [ ]:
wget https://raw.githubusercontent.com/peterdalle/svensktext/master/stoppord/stoppord.csv

  

>     --2020-12-10 14:18:52--  https://raw.githubusercontent.com/peterdalle/svensktext/master/stoppord/stoppord.csv
>     Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.52.133
>     Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.52.133|:443... connected.
>     HTTP request sent, awaiting response... 200 OK
>     Length: 1936 (1.9K) [text/plain]
>     Saving to: ‘stoppord.csv’
>
>          0K .                                                     100% 23.1M=0s
>
>     2020-12-10 14:18:53 (23.1 MB/s) - ‘stoppord.csv’ saved [1936/1936]

  

Move stopwords to DBFS

In [ ]:
val file ="file:/databricks/driver/stoppord.csv"
val dir = "dbfs:/datasets/05_lda/"

dbutils.fs.mkdirs(dir)
dbutils.fs.cp(file, dir)

  

>     file: String = file:/databricks/driver/stoppord.csv
>     dir: String = dbfs:/datasets/05_lda/
>     res1: Boolean = true

  

We check that the file exists, and has some contents.

In [ ]:
val stopword_path = "dbfs:/datasets/05_lda/stoppord.csv"
display(dbutils.fs.ls(stopword_path))

  

[TABLE]

  

Wikipedia Download
------------------

Wikipedia downloads are available as compressed files, and language and
data is chosen via the name of the file. For the current date, this is
the file share. https://dumps.wikimedia.org/svwiki/20201120/

There are several files, we used "pages-articles". This monolith file
requires unpacking on driver node. To allow parallel download and
unpacking the multistream may be a better choice.

In [ ]:
wget https://dumps.wikimedia.org/svwiki/20201120/svwiki-20201120-pages-articles.xml.bz2

  

>     --2020-11-30 09:16:33--  https://dumps.wikimedia.org/svwiki/20201120/svwiki-20201120-pages-articles.xml.bz2
>     Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.155.106, 2620:0:861:4:208:80:155:106
>     Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.155.106|:443... connected.
>     HTTP request sent, awaiting response... 200 OK
>     Length: 1525903265 (1.4G) [application/octet-stream]
>     Saving to: ‘svwiki-20201120-pages-articles.xml.bz2’
>
>          0K .......... .......... .......... .......... ..........  0%  636K 39m2s
>         50K .......... .......... .......... .......... ..........  0%  636K 39m1s
>        100K .......... .......... .......... .......... ..........  0% 1.45M 31m36s
>        150K .......... .......... .......... .......... ..........  0%  116M 23m45s
>        200K .......... .......... .......... .......... ..........  0% 1.09M 23m27s
>        250K .......... .......... .......... .......... ..........  0% 92.2M 19m35s
>        300K .......... .......... .......... .......... ..........  0% 1.45M 19m11s
>        350K .......... .......... .......... .......... ..........  0%  131M 16m48s
>        400K .......... .......... .......... .......... ..........  0%  160M 14m57s
>        450K .......... .......... .......... .......... ..........  0% 1.12M 15m37s
>        500K .......... .......... .......... .......... ..........  0%  139M 14m13s
>        550K .......... .......... .......... .......... ..........  0%  129M 13m3s
>        600K .......... .......... .......... .......... ..........  0%  231M 12m3s
>        650K .......... .......... .......... .......... ..........  0% 1.49M 12m21s
>        700K .......... .......... .......... .......... ..........  0%  153M 11m32s
>        750K .......... .......... .......... .......... ..........  0%  128M 10m50s
>        800K .......... .......... .......... .......... ..........  0%  148M 10m12s
>        850K .......... .......... .......... .......... ..........  0%  155M 9m39s
>        900K .......... .......... .......... .......... ..........  0%  155M 9m9s
>        950K .......... .......... .......... .......... ..........  0% 1.13M 9m46s
>       1000K .......... .......... .......... .......... ..........  0%  125M 9m18s
>       1050K .......... .......... .......... .......... ..........  0%  168M 8m53s
>       1100K .......... .......... .......... .......... ..........  0%  202M 8m31s
>       1150K .......... .......... .......... .......... ..........  0%  178M 8m10s
>       1200K .......... .......... .......... .......... ..........  0%  186M 7m50s
>       1250K .......... .......... .......... .......... ..........  0%  172M 7m33s
>       1300K .......... .......... .......... .......... ..........  0% 18.3M 7m19s
>       1350K .......... .......... .......... .......... ..........  0% 1.66M 7m34s
>       1400K .......... .......... .......... .......... ..........  0%  131M 7m19s
>       1450K .......... .......... .......... .......... ..........  0%  166M 7m5s
>       1500K .......... .......... .......... .......... ..........  0%  147M 6m51s
>       1550K .......... .......... .......... .......... ..........  0%  125M 6m39s
>       1600K .......... .......... .......... .......... ..........  0%  154M 6m27s
>       1650K .......... .......... .......... .......... ..........  0% 3.33M 6m28s
>       1700K .......... .......... .......... .......... ..........  0% 3.60M 6m29s
>       1750K .......... .......... .......... .......... ..........  0% 2.83M 6m32s
>       1800K .......... .......... .......... .......... ..........  0%  150M 6m22s
>       1850K .......... .......... .......... .......... ..........  0% 3.19M 6m24s
>       1900K .......... .......... .......... .......... ..........  0% 3.47M 6m25s
>       1950K .......... .......... .......... .......... ..........  0% 5.15M 6m22s
>       2000K .......... .......... .......... .......... ..........  0% 3.91M 6m22s
>       2050K .......... .......... .......... .......... ..........  0%  111M 6m13s
>       2100K .......... .......... .......... .......... ..........  0% 3.09M 6m15s
>       2150K .......... .......... .......... .......... ..........  0% 4.13M 6m15s
>       2200K .......... .......... .......... .......... ..........  0% 4.42M 6m14s
>       2250K .......... .......... .......... .......... ..........  0%  158M 6m6s
>       2300K .......... .......... .......... .......... ..........  0% 3.96M 6m6s
>       2350K .......... .......... .......... .......... ..........  0% 2.94M 6m9s
>       2400K .......... .......... .......... .......... ..........  0% 3.54M 6m9s
>       2450K .......... .......... .......... .......... ..........  0% 5.16M 6m8s
>       2500K .......... .......... .......... .......... ..........  0%  129M 6m1s
>       2550K .......... .......... .......... .......... ..........  0% 4.13M 6m0s
>       2600K .......... .......... .......... .......... ..........  0% 3.04M 6m3s
>       2650K .......... .......... .......... .......... ..........  0% 2.89M 6m5s
>       2700K .......... .......... .......... .......... ..........  0%  148M 5m59s
>       2750K .......... .......... .......... .......... ..........  0% 2.94M 6m1s
>       2800K .......... .......... .......... .......... ..........  0% 2.81M 6m4s
>       2850K .......... .......... .......... .......... ..........  0% 3.18M 6m5s
>       2900K .......... .......... .......... .......... ..........  0% 2.94M 6m8s
>       2950K .......... .......... .......... .......... ..........  0%  169M 6m2s
>       3000K .......... .......... .......... .......... ..........  0% 3.03M 6m4s
>       3050K .......... .......... .......... .......... ..........  0% 3.30M 6m5s
>       3100K .......... .......... .......... .......... ..........  0% 4.11M 6m5s
>       3150K .......... .......... .......... .......... ..........  0% 3.04M 6m6s
>       3200K .......... .......... .......... .......... ..........  0%  149M 6m1s
>       3250K .......... .......... .......... .......... ..........  0% 3.15M 6m2s
>       3300K .......... .......... .......... .......... ..........  0% 3.06M 6m4s
>       3350K .......... .......... .......... .......... ..........  0% 3.13M 6m5s
>       3400K .......... .......... .......... .......... ..........  0%  125M 6m0s
>       3450K .......... .......... .......... .......... ..........  0% 3.11M 6m2s
>       3500K .......... .......... .......... .......... ..........  0% 3.92M 6m2s
>       3550K .......... .......... .......... .......... ..........  0% 3.18M 6m3s
>       3600K .......... .......... .......... .......... ..........  0% 3.19M 6m5s
>       3650K .......... .......... .......... .......... ..........  0%  181M 6m0s
>       3700K .......... .......... .......... .......... ..........  0% 3.13M 6m1s
>       3750K .......... .......... .......... .......... ..........  0% 3.38M 6m2s
>       3800K .......... .......... .......... .......... ..........  0% 3.58M 6m2s
>       3850K .......... .......... .......... .......... ..........  0%  155M 5m58s
>       3900K .......... .......... .......... .......... ..........  0% 3.21M 5m59s
>       3950K .......... .......... .......... .......... ..........  0% 3.62M 6m0s
>       4000K .......... .......... .......... .......... ..........  0% 4.40M 5m59s
>       4050K .......... .......... .......... .......... ..........  0% 4.02M 5m59s
>       4100K .......... .......... .......... .......... ..........  0%  146M 5m55s
>       4150K .......... .......... .......... .......... ..........  0% 4.20M 5m55s
>       4200K .......... .......... .......... .......... ..........  0% 3.84M 5m55s
>       4250K .......... .......... .......... .......... ..........  0% 4.71M 5m55s
>       4300K .......... .......... .......... .......... ..........  0%  144M 5m51s
>       4350K .......... .......... .......... .......... ..........  0% 3.87M 5m51s
>       4400K .......... .......... .......... .......... ..........  0% 3.22M 5m52s
>       4450K .......... .......... .......... .......... ..........  0% 3.02M 5m53s
>       4500K .......... .......... .......... .......... ..........  0% 3.98M 5m54s
>       4550K .......... .......... .......... .......... ..........  0%  122M 5m50s
>       4600K .......... .......... .......... .......... ..........  0% 2.95M 5m51s
>       4650K .......... .......... .......... .......... ..........  0% 3.25M 5m52s
>       4700K .......... .......... .......... .......... ..........  0% 3.10M 5m54s
>       4750K .......... .......... .......... .......... ..........  0% 1.94M 5m58s
>       4800K .......... .......... .......... .......... ..........  0%  139M 5m54s
>       4850K .......... .......... .......... .......... ..........  0% 4.61M 5m54s
>       4900K .......... .......... .......... .......... ..........  0% 4.13M 5m54s
>       4950K .......... .......... .......... .......... ..........  0% 3.11M 5m55s
>       5000K .......... .......... .......... .......... ..........  0%  180M 5m51s
>       5050K .......... .......... .......... .......... ..........  0% 4.27M 5m51s
>       5100K .......... .......... .......... .......... ..........  0% 4.26M 5m51s
>       5150K .......... .......... .......... .......... ..........  0% 3.04M 5m52s
>       5200K .......... .......... .......... .......... ..........  0% 6.14M 5m51s
>       5250K .......... .......... .......... .......... ..........  0%  126M 5m48s
>       5300K .......... .......... .......... .......... ..........  0% 3.15M 5m49s
>       5350K .......... .......... .......... .......... ..........  0% 4.92M 5m48s
>       5400K .......... .......... .......... .......... ..........  0% 4.31M 5m48s
>       5450K .......... .......... .......... .......... ..........  0%  124M 5m45s
>       5500K .......... .......... .......... .......... ..........  0% 4.16M 5m45s
>       5550K .......... .......... .......... .......... ..........  0% 3.92M 5m45s
>       5600K .......... .......... .......... .......... ..........  0% 3.63M 5m46s
>       5650K .......... .......... .......... .......... ..........  0% 3.73M 5m46s
>       5700K .......... .......... .......... .......... ..........  0%  159M 5m43s
>       5750K .......... .......... .......... .......... ..........  0% 3.83M 5m44s
>       5800K .......... .......... .......... .......... ..........  0% 3.16M 5m45s
>       5850K .......... .......... .......... .......... ..........  0% 3.65M 5m45s
>       5900K .......... .......... .......... .......... ..........  0%  140M 5m42s
>       5950K .......... .......... .......... .......... ..........  0% 3.06M 5m43s
>       6000K .......... .......... .......... .......... ..........  0% 4.60M 5m43s
>       6050K .......... .......... .......... .......... ..........  0% 4.08M 5m43s
>       6100K .......... .......... .......... .......... ..........  0% 4.14M 5m43s
>       6150K .......... .......... .......... .......... ..........  0%  123M 5m41s
>       6200K .......... .......... .......... .......... ..........  0% 4.11M 5m41s
>       6250K .......... .......... .......... .......... ..........  0% 3.34M 5m41s
>       6300K .......... .......... .......... .......... ..........  0% 5.49M 5m41s
>       6350K .......... .......... .......... .......... ..........  0% 3.28M 5m41s
>       6400K .......... .......... .......... .......... ..........  0%  154M 5m39s
>       6450K .......... .......... .......... .......... ..........  0% 3.41M 5m40s
>       6500K .......... .......... .......... .......... ..........  0% 4.11M 5m40s
>       6550K .......... .......... .......... .......... ..........  0% 3.34M 5m40s
>       6600K .......... .......... .......... .......... ..........  0%  161M 5m38s
>       6650K .......... .......... .......... .......... ..........  0% 1.65M 5m42s
>       6700K .......... .......... .......... .......... ..........  0% 4.39M 5m42s
>       6750K .......... .......... .......... .......... ..........  0% 3.26M 5m43s
>       6800K .......... .......... .......... .......... ..........  0% 3.33M 5m43s
>       6850K .......... .......... .......... .......... ..........  0%  144M 5m41s
>       6900K .......... .......... .......... .......... ..........  0% 3.34M 5m41s
>       6950K .......... .......... .......... .......... ..........  0% 3.35M 5m42s
>       7000K .......... .......... .......... .......... ..........  0% 3.30M 5m43s
>       7050K .......... .......... .......... .......... ..........  0%  182M 5m40s
>       7100K .......... .......... .......... .......... ..........  0% 3.35M 5m41s
>       7150K .......... .......... .......... .......... ..........  0% 4.28M 5m41s
>       7200K .......... .......... .......... .......... ..........  0% 5.78M 5m40s
>       7250K .......... .......... .......... .......... ..........  0% 3.36M 5m41s
>       7300K .......... .......... .......... .......... ..........  0%  154M 5m39s
>       7350K .......... .......... .......... .......... ..........  0% 3.39M 5m39s
>       7400K .......... .......... .......... .......... ..........  0% 3.37M 5m40s
>       7450K .......... .......... .......... .......... ..........  0% 4.40M 5m40s
>       7500K .......... .......... .......... .......... ..........  0%  156M 5m38s
>       7550K .......... .......... .......... .......... ..........  0% 3.44M 5m38s
>       7600K .......... .......... .......... .......... ..........  0% 3.45M 5m39s
>       7650K .......... .......... .......... .......... ..........  0% 3.49M 5m39s
>       7700K .......... .......... .......... .......... ..........  0% 3.49M 5m40s
>       7750K .......... .......... .......... .......... ..........  0%  114M 5m37s
>       7800K .......... .......... .......... .......... ..........  0% 3.42M 5m38s
>       7850K .......... .......... .......... .......... ..........  0%  712K 5m49s
>       7900K .......... .......... .......... .......... ..........  0% 2.18M 5m51s
>       7950K .......... .......... .......... .......... ..........  0% 16.9M 5m49s
>       8000K .......... .......... .......... .......... ..........  0%  167M 5m47s
>       8050K .......... .......... .......... .......... ..........  0% 3.22M 5m48s
>       8100K .......... .......... .......... .......... ..........  0% 3.35M 5m48s
>       8150K .......... .......... .......... .......... ..........  0% 3.64M 5m49s
>       8200K .......... .......... .......... .......... ..........  0%  123M 5m47s
>       8250K .......... .......... .......... .......... ..........  0% 4.12M 5m47s
>       8300K .......... .......... .......... .......... ..........  0% 4.85M 5m46s
>       8350K .......... .......... .......... .......... ..........  0% 3.37M 5m47s
>       8400K .......... .......... .......... .......... ..........  0% 4.76M 5m47s
>       8450K .......... .......... .......... .......... ..........  0%  133M 5m45s
>       8500K .......... .......... .......... .......... ..........  0% 3.65M 5m45s
>       8550K .......... .......... .......... .......... ..........  0% 4.26M 5m45s
>       8600K .......... .......... .......... .......... ..........  0% 3.45M 5m45s
>       8650K .......... .......... .......... .......... ..........  0%  167M 5m43s
>       8700K .......... .......... .......... .......... ..........  0% 3.47M 5m44s
>       8750K .......... .......... .......... .......... ..........  0% 3.48M 5m44s
>       8800K .......... .......... .......... .......... ..........  0% 3.45M 5m44s
>       8850K .......... .......... .......... .......... ..........  0% 3.44M 5m45s
>       8900K .......... .......... .......... .......... ..........  0%  122M 5m43s
>       8950K .......... .......... .......... .......... ..........  0% 4.21M 5m43s
>       9000K .......... .......... .......... .......... ..........  0% 4.50M 5m43s
>       9050K .......... .......... .......... .......... ..........  0% 3.32M 5m43s
>       9100K .......... .......... .......... .......... ..........  0%  141M 5m42s
>       9150K .......... .......... .......... .......... ..........  0% 5.64M 5m41s
>       9200K .......... .......... .......... .......... ..........  0% 3.26M 5m42s
>       9250K .......... .......... .......... .......... ..........  0% 4.49M 5m41s
>       9300K .......... .......... .......... .......... ..........  0% 3.30M 5m42s
>       9350K .......... .......... .......... .......... ..........  0% 87.0M 5m40s
>       9400K .......... .......... .......... .......... ..........  0% 3.37M 5m41s
>       9450K .......... .......... .......... .......... ..........  0% 4.02M 5m41s
>       9500K .......... .......... .......... .......... ..........  0% 3.26M 5m41s
>       9550K .......... .......... .......... .......... ..........  0% 4.00M 5m41s
>       9600K .......... .......... .......... .......... ..........  0%  154M 5m40s
>       9650K .......... .......... .......... .......... ..........  0% 3.80M 5m40s
>       9700K .......... .......... .......... .......... ..........  0% 3.99M 5m40s
>       9750K .......... .......... .......... .......... ..........  0% 3.18M 5m41s
>       9800K .......... .......... .......... .......... ..........  0%  234M 5m39s
>       9850K .......... .......... .......... .......... ..........  0% 3.06M 5m40s
>       9900K .......... .......... .......... .......... ..........  0% 3.33M 5m40s
>       9950K .......... .......... .......... .......... ..........  0% 3.95M 5m40s
>      10000K .......... .......... .......... .......... ..........  0% 3.14M 5m41s
>      10050K .......... .......... .......... .......... ..........  0%  175M 5m39s
>      10100K .......... .......... .......... .......... ..........  0% 3.24M 5m40s
>      10150K .......... .......... .......... .......... ..........  0% 3.06M 5m40s
>      10200K .......... .......... .......... .......... ..........  0% 3.21M 5m41s
>      10250K .......... .......... .......... .......... ..........  0% 99.8M 5m39s
>      10300K .......... .......... .......... .......... ..........  0% 3.23M 5m40s
>      10350K .......... .......... .......... .......... ..........  0% 3.26M 5m40s
>      10400K .......... .......... .......... .......... ..........  0% 3.14M 5m41s
>      10450K .......... .......... .......... .......... ..........  0% 3.08M 5m41s
>      10500K .......... .......... .......... .......... ..........  0%  146M 5m40s
>      10550K .......... .......... .......... .......... ..........  0% 3.11M 5m40s
>      10600K .......... .......... .......... .......... ..........  0% 3.21M 5m41s
>      10650K .......... .......... .......... .......... ..........  0% 4.03M 5m41s
>      10700K .......... .......... .......... .......... ..........  0%  125M 5m39s
>      10750K .......... .......... .......... .......... ..........  0% 4.31M 5m39s
>      10800K .......... .......... .......... .......... ..........  0% 3.16M 5m40s
>      10850K .......... .......... .......... .......... ..........  0% 3.21M 5m40s
>      10900K .......... .......... .......... .......... ..........  0% 3.19M 5m41s
>      10950K .......... .......... .......... .......... ..........  0%  136M 5m39s
>      11000K .......... .......... .......... .......... ..........  0% 3.18M 5m40s
>      11050K .......... .......... .......... .......... ..........  0% 3.19M 5m40s
>      11100K .......... .......... .......... .......... ..........  0% 5.20M 5m40s
>      11150K .......... .......... .......... .......... ..........  0% 4.31M 5m40s
>      11200K .......... .......... .......... .......... ..........  0%  131M 5m38s
>      11250K .......... .......... .......... .......... ..........  0% 3.20M 5m39s
>      11300K .......... .......... .......... .......... ..........  0% 3.07M 5m40s
>      11350K .......... .......... .......... .......... ..........  0% 3.16M 5m40s
>      11400K .......... .......... .......... .......... ..........  0%  136M 5m39s
>      11450K .......... .......... .......... .......... ..........  0% 4.18M 5m39s
>      11500K .......... .......... .......... .......... ..........  0% 3.91M 5m39s
>      11550K .......... .......... .......... .......... ..........  0% 4.12M 5m39s
>      11600K .......... .......... .......... .......... ..........  0% 4.41M 5m39s
>      11650K .......... .......... .......... .......... ..........  0%  135M 5m37s
>      11700K .......... .......... .......... .......... ..........  0% 3.17M 5m38s
>      11750K .......... .......... .......... .......... ..........  0% 3.20M 5m38s
>      11800K .......... .......... .......... .......... ..........  0% 3.24M 5m39s
>      11850K .......... .......... .......... .......... ..........  0%  197M 5m37s
>      11900K .......... .......... .......... .......... ..........  0% 3.26M 5m38s
>      11950K .......... .......... .......... .......... ..........  0% 4.31M 5m38s
>      12000K .......... .......... .......... .......... ..........  0% 4.12M 5m38s
>      12050K .......... .......... .......... .......... ..........  0% 4.00M 5m38s
>      12100K .......... .......... .......... .......... ..........  0%  149M 5m36s
>      12150K .......... .......... .......... .......... ..........  0% 3.35M 5m37s
>      12200K .......... .......... .......... .......... ..........  0% 4.87M 5m37s
>      12250K .......... .......... .......... .......... ..........  0% 3.28M 5m37s
>      12300K .......... .......... .......... .......... ..........  0%  196M 5m36s
>      12350K .......... .......... .......... .......... ..........  0% 3.22M 5m36s
>      12400K .......... .......... .......... .......... ..........  0% 3.59M 5m36s
>      12450K .......... .......... .......... .......... ..........  0% 3.13M 5m37s
>      12500K .......... .......... .......... .......... ..........  0% 4.09M 5m37s
>      12550K .......... .......... .......... .......... ..........  0%  114M 5m36s
>      12600K .......... .......... .......... .......... ..........  0% 4.58M 5m36s
>      12650K .......... .......... .......... .......... ..........  0% 4.27M 5m36s
>      12700K .......... .......... .......... .......... ..........  0% 3.15M 5m36s
>      12750K .......... .......... .......... .......... ..........  0% 3.18M 5m36s
>      12800K .......... .......... .......... .......... ..........  0%  180M 5m35s
>      12850K .......... .......... .......... .......... ..........  0% 3.14M 5m36s
>      12900K .......... .......... .......... .......... ..........  0% 3.18M 5m36s
>      12950K .......... .......... .......... .......... ..........  0% 3.34M 5m36s
>      13000K .......... .......... .......... .......... ..........  0%  161M 5m35s
>      13050K .......... .......... .......... .......... ..........  0% 3.95M 5m35s
>      13100K .......... .......... .......... .......... ..........  0% 4.23M 5m35s
>      13150K .......... .......... .......... .......... ..........  0% 3.19M 5m36s
>      13200K .......... .......... .......... .......... ..........  0% 3.15M 5m36s
>      13250K .......... .......... .......... .......... ..........  0%  138M 5m35s
>      13300K .......... .......... .......... .......... ..........  0% 3.22M 5m35s
>      13350K .......... .......... .......... .......... ..........  0% 3.27M 5m36s
>      13400K .......... .......... .......... .......... ..........  0% 3.25M 5m36s
>      13450K .......... .......... .......... .......... ..........  0%  141M 5m35s
>      13500K .......... .......... .......... .......... ..........  0% 4.49M 5m35s
>      13550K .......... .......... .......... .......... ..........  0% 3.24M 5m35s
>      13600K .......... .......... .......... .......... ..........  0% 3.25M 5m36s
>      13650K .......... .......... .......... .......... ..........  0% 3.28M 5m36s
>      13700K .......... .......... .......... .......... ..........  0%  144M 5m35s
>      13750K .......... .......... .......... .......... ..........  0% 3.31M 5m35s
>      13800K .......... .......... .......... .......... ..........  0% 3.68M 5m35s
>      13850K .......... .......... .......... .......... ..........  0% 5.33M 5m35s
>      13900K .......... .......... .......... .......... ..........  0%  165M 5m34s
>      13950K .......... .......... .......... .......... ..........  0% 3.15M 5m34s
>      14000K .......... .......... .......... .......... ..........  0% 2.97M 5m35s
>      14050K .......... .......... .......... .......... ..........  0% 3.11M 5m35s
>      14100K .......... .......... .......... .......... ..........  0% 3.16M 5m36s
>      14150K .......... .......... .......... .......... ..........  0%  145M 5m35s
>      14200K .......... .......... .......... .......... ..........  0% 3.09M 5m35s
>      14250K .......... .......... .......... .......... ..........  0% 3.16M 5m35s
>      14300K .......... .......... .......... .......... ..........  0% 3.18M 5m36s
>      14350K .......... .......... .......... .......... ..........  0% 3.15M 5m36s
>      14400K .......... .......... .......... .......... ..........  0%  188M 5m35s
>      14450K .......... .......... .......... .......... ..........  0% 3.18M 5m36s
>      14500K .......... .......... .......... .......... ..........  0% 5.12M 5m35s
>      14550K .......... .......... .......... .......... ..........  0% 3.17M 5m36s
>      14600K .......... .......... .......... .......... ..........  0%  138M 5m35s
>      14650K .......... .......... .......... .......... ..........  0% 3.22M 5m35s
>      14700K .......... .......... .......... .......... ..........  0% 3.24M 5m35s
>      14750K .......... .......... .......... .......... ..........  0% 3.28M 5m36s
>      14800K .......... .......... .......... .......... ..........  0% 3.26M 5m36s
>      14850K .......... .......... .......... .......... ..........  0%  128M 5m35s
>      14900K .......... .......... .......... .......... ..........  1% 4.01M 5m35s
>      14950K .......... .......... .......... .......... ..........  1% 3.15M 5m35s
>      15000K .......... .......... .......... .......... ..........  1% 4.21M 5m35s
>      15050K .......... .......... .......... .......... ..........  1%  189M 5m34s
>      15100K .......... .......... .......... .......... ..........  1% 4.26M 5m34s
>      15150K .......... .......... .......... .......... ..........  1% 3.17M 5m35s
>      15200K .......... .......... .......... .......... ..........  1% 3.20M 5m35s
>      15250K .......... .......... .......... .......... ..........  1% 3.68M 5m35s
>      15300K .......... .......... .......... .......... ..........  1%  185M 5m34s
>      15350K .......... .......... .......... .......... ..........  1% 4.88M 5m34s
>      15400K .......... .......... .......... .......... ..........  1% 4.48M 5m34s
>      15450K .......... .......... .......... .......... ..........  1% 3.80M 5m34s
>      15500K .......... .......... .......... .......... ..........  1%  165M 5m33s
>      15550K .......... .......... .......... .......... ..........  1% 4.16M 5m33s
>
>     *** WARNING: skipped 2292666 bytes of output ***
>
>     1474000K .......... .......... .......... .......... .......... 98% 4.32M 4s
>     1474050K .......... .......... .......... .......... .......... 98%  140M 4s
>     1474100K .......... .......... .......... .......... .......... 98% 4.11M 4s
>     1474150K .......... .......... .......... .......... .......... 98% 3.05M 4s
>     1474200K .......... .......... .......... .......... .......... 98% 3.47M 4s
>     1474250K .......... .......... .......... .......... .......... 98%  124M 4s
>     1474300K .......... .......... .......... .......... .......... 98% 3.21M 4s
>     1474350K .......... .......... .......... .......... .......... 98% 3.16M 4s
>     1474400K .......... .......... .......... .......... .......... 98% 3.18M 4s
>     1474450K .......... .......... .......... .......... .......... 98% 3.65M 3s
>     1474500K .......... .......... .......... .......... .......... 98%  127M 3s
>     1474550K .......... .......... .......... .......... .......... 98% 3.19M 3s
>     1474600K .......... .......... .......... .......... .......... 98% 3.24M 3s
>     1474650K .......... .......... .......... .......... .......... 98% 3.23M 3s
>     1474700K .......... .......... .......... .......... .......... 98%  157M 3s
>     1474750K .......... .......... .......... .......... .......... 98% 3.24M 3s
>     1474800K .......... .......... .......... .......... .......... 98% 3.26M 3s
>     1474850K .......... .......... .......... .......... .......... 98% 3.28M 3s
>     1474900K .......... .......... .......... .......... .......... 98% 3.28M 3s
>     1474950K .......... .......... .......... .......... .......... 98%  126M 3s
>     1475000K .......... .......... .......... .......... .......... 98% 3.54M 3s
>     1475050K .......... .......... .......... .......... .......... 98% 3.57M 3s
>     1475100K .......... .......... .......... .......... .......... 98% 4.01M 3s
>     1475150K .......... .......... .......... .......... .......... 98% 2.86M 3s
>     1475200K .......... .......... .......... .......... .......... 99%  142M 3s
>     1475250K .......... .......... .......... .......... .......... 99% 3.38M 3s
>     1475300K .......... .......... .......... .......... .......... 99% 3.02M 3s
>     1475350K .......... .......... .......... .......... .......... 99% 3.74M 3s
>     1475400K .......... .......... .......... .......... .......... 99%  108M 3s
>     1475450K .......... .......... .......... .......... .......... 99% 4.64M 3s
>     1475500K .......... .......... .......... .......... .......... 99% 3.24M 3s
>     1475550K .......... .......... .......... .......... .......... 99% 2.98M 3s
>     1475600K .......... .......... .......... .......... .......... 99% 3.06M 3s
>     1475650K .......... .......... .......... .......... .......... 99%  134M 3s
>     1475700K .......... .......... .......... .......... .......... 99% 4.15M 3s
>     1475750K .......... .......... .......... .......... .......... 99% 3.80M 3s
>     1475800K .......... .......... .......... .......... .......... 99% 4.16M 3s
>     1475850K .......... .......... .......... .......... .......... 99%  154M 3s
>     1475900K .......... .......... .......... .......... .......... 99% 3.88M 3s
>     1475950K .......... .......... .......... .......... .......... 99% 4.28M 3s
>     1476000K .......... .......... .......... .......... .......... 99% 3.10M 3s
>     1476050K .......... .......... .......... .......... .......... 99% 4.47M 3s
>     1476100K .......... .......... .......... .......... .......... 99%  165M 3s
>     1476150K .......... .......... .......... .......... .......... 99% 3.14M 3s
>     1476200K .......... .......... .......... .......... .......... 99% 3.62M 3s
>     1476250K .......... .......... .......... .......... .......... 99% 3.84M 3s
>     1476300K .......... .......... .......... .......... .......... 99%  209M 3s
>     1476350K .......... .......... .......... .......... .......... 99% 4.15M 3s
>     1476400K .......... .......... .......... .......... .......... 99% 3.13M 3s
>     1476450K .......... .......... .......... .......... .......... 99% 4.19M 3s
>     1476500K .......... .......... .......... .......... .......... 99% 3.02M 3s
>     1476550K .......... .......... .......... .......... .......... 99%  114M 3s
>     1476600K .......... .......... .......... .......... .......... 99% 2.70M 3s
>     1476650K .......... .......... .......... .......... .......... 99% 2.84M 3s
>     1476700K .......... .......... .......... .......... .......... 99% 3.62M 3s
>     1476750K .......... .......... .......... .......... .......... 99% 2.98M 3s
>     1476800K .......... .......... .......... .......... .......... 99%  123M 3s
>     1476850K .......... .......... .......... .......... .......... 99% 2.98M 3s
>     1476900K .......... .......... .......... .......... .......... 99% 2.98M 3s
>     1476950K .......... .......... .......... .......... .......... 99% 4.13M 3s
>     1477000K .......... .......... .......... .......... .......... 99%  144M 3s
>     1477050K .......... .......... .......... .......... .......... 99% 4.27M 3s
>     1477100K .......... .......... .......... .......... .......... 99% 3.65M 3s
>     1477150K .......... .......... .......... .......... .......... 99% 3.99M 3s
>     1477200K .......... .......... .......... .......... .......... 99% 2.76M 3s
>     1477250K .......... .......... .......... .......... .......... 99%  145M 3s
>     1477300K .......... .......... .......... .......... .......... 99% 4.00M 3s
>     1477350K .......... .......... .......... .......... .......... 99% 5.87M 3s
>     1477400K .......... .......... .......... .......... .......... 99% 3.71M 3s
>     1477450K .......... .......... .......... .......... .......... 99%  194M 3s
>     1477500K .......... .......... .......... .......... .......... 99% 3.75M 3s
>     1477550K .......... .......... .......... .......... .......... 99% 4.40M 3s
>     1477600K .......... .......... .......... .......... .......... 99% 3.71M 3s
>     1477650K .......... .......... .......... .......... .......... 99% 3.71M 3s
>     1477700K .......... .......... .......... .......... .......... 99%  140M 3s
>     1477750K .......... .......... .......... .......... .......... 99% 4.33M 3s
>     1477800K .......... .......... .......... .......... .......... 99% 3.01M 3s
>     1477850K .......... .......... .......... .......... .......... 99% 4.95M 3s
>     1477900K .......... .......... .......... .......... .......... 99%  149M 3s
>     1477950K .......... .......... .......... .......... .......... 99% 3.02M 3s
>     1478000K .......... .......... .......... .......... .......... 99% 3.08M 3s
>     1478050K .......... .......... .......... .......... .......... 99% 3.11M 3s
>     1478100K .......... .......... .......... .......... .......... 99% 3.03M 3s
>     1478150K .......... .......... .......... .......... .......... 99%  123M 3s
>     1478200K .......... .......... .......... .......... .......... 99% 3.01M 3s
>     1478250K .......... .......... .......... .......... .......... 99% 3.18M 3s
>     1478300K .......... .......... .......... .......... .......... 99% 2.93M 3s
>     1478350K .......... .......... .......... .......... .......... 99% 3.03M 3s
>     1478400K .......... .......... .......... .......... .......... 99%  144M 3s
>     1478450K .......... .......... .......... .......... .......... 99% 3.81M 3s
>     1478500K .......... .......... .......... .......... .......... 99% 3.10M 3s
>     1478550K .......... .......... .......... .......... .......... 99% 3.08M 3s
>     1478600K .......... .......... .......... .......... .......... 99%  132M 3s
>     1478650K .......... .......... .......... .......... .......... 99% 3.18M 3s
>     1478700K .......... .......... .......... .......... .......... 99% 3.14M 3s
>     1478750K .......... .......... .......... .......... .......... 99% 3.16M 3s
>     1478800K .......... .......... .......... .......... .......... 99% 3.06M 3s
>     1478850K .......... .......... .......... .......... .......... 99%  167M 3s
>     1478900K .......... .......... .......... .......... .......... 99% 4.10M 3s
>     1478950K .......... .......... .......... .......... .......... 99% 2.89M 2s
>     1479000K .......... .......... .......... .......... .......... 99% 2.96M 2s
>     1479050K .......... .......... .......... .......... .......... 99%  143M 2s
>     1479100K .......... .......... .......... .......... .......... 99% 3.61M 2s
>     1479150K .......... .......... .......... .......... .......... 99% 3.86M 2s
>     1479200K .......... .......... .......... .......... .......... 99% 4.52M 2s
>     1479250K .......... .......... .......... .......... .......... 99% 3.61M 2s
>     1479300K .......... .......... .......... .......... .......... 99%  143M 2s
>     1479350K .......... .......... .......... .......... .......... 99% 4.01M 2s
>     1479400K .......... .......... .......... .......... .......... 99% 3.20M 2s
>     1479450K .......... .......... .......... .......... .......... 99% 3.88M 2s
>     1479500K .......... .......... .......... .......... .......... 99%  208M 2s
>     1479550K .......... .......... .......... .......... .......... 99% 3.73M 2s
>     1479600K .......... .......... .......... .......... .......... 99% 3.11M 2s
>     1479650K .......... .......... .......... .......... .......... 99% 4.10M 2s
>     1479700K .......... .......... .......... .......... .......... 99% 3.15M 2s
>     1479750K .......... .......... .......... .......... .......... 99%  140M 2s
>     1479800K .......... .......... .......... .......... .......... 99% 3.02M 2s
>     1479850K .......... .......... .......... .......... .......... 99% 3.05M 2s
>     1479900K .......... .......... .......... .......... .......... 99% 3.01M 2s
>     1479950K .......... .......... .......... .......... .......... 99% 3.58M 2s
>     1480000K .......... .......... .......... .......... .......... 99%  148M 2s
>     1480050K .......... .......... .......... .......... .......... 99% 2.95M 2s
>     1480100K .......... .......... .......... .......... .......... 99% 3.23M 2s
>     1480150K .......... .......... .......... .......... .......... 99% 3.26M 2s
>     1480200K .......... .......... .......... .......... .......... 99%  216M 2s
>     1480250K .......... .......... .......... .......... .......... 99% 4.17M 2s
>     1480300K .......... .......... .......... .......... .......... 99% 4.15M 2s
>     1480350K .......... .......... .......... .......... .......... 99% 3.64M 2s
>     1480400K .......... .......... .......... .......... .......... 99% 2.49M 2s
>     1480450K .......... .......... .......... .......... .......... 99%  131M 2s
>     1480500K .......... .......... .......... .......... .......... 99% 3.71M 2s
>     1480550K .......... .......... .......... .......... .......... 99% 3.16M 2s
>     1480600K .......... .......... .......... .......... .......... 99% 3.05M 2s
>     1480650K .......... .......... .......... .......... .......... 99%  160M 2s
>     1480700K .......... .......... .......... .......... .......... 99% 3.54M 2s
>     1480750K .......... .......... .......... .......... .......... 99% 2.00M 2s
>     1480800K .......... .......... .......... .......... .......... 99% 3.49M 2s
>     1480850K .......... .......... .......... .......... .......... 99% 2.98M 2s
>     1480900K .......... .......... .......... .......... .......... 99%  137M 2s
>     1480950K .......... .......... .......... .......... .......... 99% 3.82M 2s
>     1481000K .......... .......... .......... .......... .......... 99% 4.41M 2s
>     1481050K .......... .......... .......... .......... .......... 99% 5.45M 2s
>     1481100K .......... .......... .......... .......... .......... 99%  189M 2s
>     1481150K .......... .......... .......... .......... .......... 99% 3.35M 2s
>     1481200K .......... .......... .......... .......... .......... 99% 4.40M 2s
>     1481250K .......... .......... .......... .......... .......... 99% 4.99M 2s
>     1481300K .......... .......... .......... .......... .......... 99% 3.61M 2s
>     1481350K .......... .......... .......... .......... .......... 99%  123M 2s
>     1481400K .......... .......... .......... .......... .......... 99% 4.13M 2s
>     1481450K .......... .......... .......... .......... .......... 99% 4.15M 2s
>     1481500K .......... .......... .......... .......... .......... 99% 4.17M 2s
>     1481550K .......... .......... .......... .......... .......... 99% 2.08M 2s
>     1481600K .......... .......... .......... .......... .......... 99%  146M 2s
>     1481650K .......... .......... .......... .......... .......... 99% 3.60M 2s
>     1481700K .......... .......... .......... .......... .......... 99% 3.55M 2s
>     1481750K .......... .......... .......... .......... .......... 99% 4.10M 2s
>     1481800K .......... .......... .......... .......... .......... 99%  135M 2s
>     1481850K .......... .......... .......... .......... .......... 99% 3.69M 2s
>     1481900K .......... .......... .......... .......... .......... 99% 3.60M 2s
>     1481950K .......... .......... .......... .......... .......... 99% 3.05M 2s
>     1482000K .......... .......... .......... .......... .......... 99% 3.03M 2s
>     1482050K .......... .......... .......... .......... .......... 99%  122M 2s
>     1482100K .......... .......... .......... .......... .......... 99% 2.85M 2s
>     1482150K .......... .......... .......... .......... .......... 99% 2.97M 2s
>     1482200K .......... .......... .......... .......... .......... 99% 2.93M 2s
>     1482250K .......... .......... .......... .......... .......... 99%  177M 2s
>     1482300K .......... .......... .......... .......... .......... 99% 4.90M 2s
>     1482350K .......... .......... .......... .......... .......... 99% 3.44M 2s
>     1482400K .......... .......... .......... .......... .......... 99% 3.91M 2s
>     1482450K .......... .......... .......... .......... .......... 99% 2.90M 2s
>     1482500K .......... .......... .......... .......... .......... 99%  139M 2s
>     1482550K .......... .......... .......... .......... .......... 99% 2.97M 2s
>     1482600K .......... .......... .......... .......... .......... 99% 3.02M 2s
>     1482650K .......... .......... .......... .......... .......... 99% 3.40M 2s
>     1482700K .......... .......... .......... .......... .......... 99%  147M 2s
>     1482750K .......... .......... .......... .......... .......... 99% 3.87M 2s
>     1482800K .......... .......... .......... .......... .......... 99% 5.92M 2s
>     1482850K .......... .......... .......... .......... .......... 99% 2.90M 2s
>     1482900K .......... .......... .......... .......... .......... 99% 3.98M 2s
>     1482950K .......... .......... .......... .......... .......... 99%  134M 2s
>     1483000K .......... .......... .......... .......... .......... 99% 4.82M 2s
>     1483050K .......... .......... .......... .......... .......... 99% 4.05M 2s
>     1483100K .......... .......... .......... .......... .......... 99% 3.63M 2s
>     1483150K .......... .......... .......... .......... .......... 99% 3.52M 2s
>     1483200K .......... .......... .......... .......... .......... 99%  154M 2s
>     1483250K .......... .......... .......... .......... .......... 99% 3.95M 2s
>     1483300K .......... .......... .......... .......... .......... 99% 3.99M 2s
>     1483350K .......... .......... .......... .......... .......... 99% 3.32M 2s
>     1483400K .......... .......... .......... .......... .......... 99%  147M 1s
>     1483450K .......... .......... .......... .......... .......... 99% 5.69M 1s
>     1483500K .......... .......... .......... .......... .......... 99% 3.62M 1s
>     1483550K .......... .......... .......... .......... .......... 99% 2.80M 1s
>     1483600K .......... .......... .......... .......... .......... 99% 4.37M 1s
>     1483650K .......... .......... .......... .......... .......... 99%  124M 1s
>     1483700K .......... .......... .......... .......... .......... 99% 3.03M 1s
>     1483750K .......... .......... .......... .......... .......... 99% 2.96M 1s
>     1483800K .......... .......... .......... .......... .......... 99% 3.96M 1s
>     1483850K .......... .......... .......... .......... .......... 99%  161M 1s
>     1483900K .......... .......... .......... .......... .......... 99% 3.05M 1s
>     1483950K .......... .......... .......... .......... .......... 99% 3.07M 1s
>     1484000K .......... .......... .......... .......... .......... 99% 3.14M 1s
>     1484050K .......... .......... .......... .......... .......... 99% 3.14M 1s
>     1484100K .......... .......... .......... .......... .......... 99%  119M 1s
>     1484150K .......... .......... .......... .......... .......... 99% 2.96M 1s
>     1484200K .......... .......... .......... .......... .......... 99% 3.05M 1s
>     1484250K .......... .......... .......... .......... .......... 99% 3.05M 1s
>     1484300K .......... .......... .......... .......... .......... 99%  154M 1s
>     1484350K .......... .......... .......... .......... .......... 99% 2.74M 1s
>     1484400K .......... .......... .......... .......... .......... 99% 3.04M 1s
>     1484450K .......... .......... .......... .......... .......... 99% 3.06M 1s
>     1484500K .......... .......... .......... .......... .......... 99% 2.99M 1s
>     1484550K .......... .......... .......... .......... .......... 99%  110M 1s
>     1484600K .......... .......... .......... .......... .......... 99% 2.99M 1s
>     1484650K .......... .......... .......... .......... .......... 99% 3.04M 1s
>     1484700K .......... .......... .......... .......... .......... 99% 4.33M 1s
>     1484750K .......... .......... .......... .......... .......... 99% 3.58M 1s
>     1484800K .......... .......... .......... .......... .......... 99%  142M 1s
>     1484850K .......... .......... .......... .......... .......... 99% 3.95M 1s
>     1484900K .......... .......... .......... .......... .......... 99% 4.10M 1s
>     1484950K .......... .......... .......... .......... .......... 99% 3.15M 1s
>     1485000K .......... .......... .......... .......... .......... 99%  128M 1s
>     1485050K .......... .......... .......... .......... .......... 99% 4.71M 1s
>     1485100K .......... .......... .......... .......... .......... 99% 3.68M 1s
>     1485150K .......... .......... .......... .......... .......... 99% 2.96M 1s
>     1485200K .......... .......... .......... .......... .......... 99% 2.82M 1s
>     1485250K .......... .......... .......... .......... .......... 99%  133M 1s
>     1485300K .......... .......... .......... .......... .......... 99% 2.92M 1s
>     1485350K .......... .......... .......... .......... .......... 99% 2.75M 1s
>     1485400K .......... .......... .......... .......... .......... 99% 2.80M 1s
>     1485450K .......... .......... .......... .......... .......... 99%  156M 1s
>     1485500K .......... .......... .......... .......... .......... 99% 2.96M 1s
>     1485550K .......... .......... .......... .......... .......... 99% 3.49M 1s
>     1485600K .......... .......... .......... .......... .......... 99% 2.66M 1s
>     1485650K .......... .......... .......... .......... .......... 99% 3.72M 1s
>     1485700K .......... .......... .......... .......... .......... 99%  142M 1s
>     1485750K .......... .......... .......... .......... .......... 99% 4.77M 1s
>     1485800K .......... .......... .......... .......... .......... 99% 3.85M 1s
>     1485850K .......... .......... .......... .......... .......... 99% 3.88M 1s
>     1485900K .......... .......... .......... .......... .......... 99%  150M 1s
>     1485950K .......... .......... .......... .......... .......... 99% 2.85M 1s
>     1486000K .......... .......... .......... .......... .......... 99% 3.84M 1s
>     1486050K .......... .......... .......... .......... .......... 99% 3.78M 1s
>     1486100K .......... .......... .......... .......... .......... 99% 3.81M 1s
>     1486150K .......... .......... .......... .......... .......... 99%  136M 1s
>     1486200K .......... .......... .......... .......... .......... 99% 3.05M 1s
>     1486250K .......... .......... .......... .......... .......... 99% 2.79M 1s
>     1486300K .......... .......... .......... .......... .......... 99% 3.68M 1s
>     1486350K .......... .......... .......... .......... .......... 99% 3.85M 1s
>     1486400K .......... .......... .......... .......... .......... 99%  119M 1s
>     1486450K .......... .......... .......... .......... .......... 99% 2.22M 1s
>     1486500K .......... .......... .......... .......... .......... 99% 3.81M 1s
>     1486550K .......... .......... .......... .......... .......... 99% 3.67M 1s
>     1486600K .......... .......... .......... .......... .......... 99%  133M 1s
>     1486650K .......... .......... .......... .......... .......... 99% 3.52M 1s
>     1486700K .......... .......... .......... .......... .......... 99% 3.78M 1s
>     1486750K .......... .......... .......... .......... .......... 99% 4.63M 1s
>     1486800K .......... .......... .......... .......... .......... 99% 3.46M 1s
>     1486850K .......... .......... .......... .......... .......... 99%  195M 1s
>     1486900K .......... .......... .......... .......... .......... 99% 2.67M 1s
>     1486950K .......... .......... .......... .......... .......... 99% 3.46M 1s
>     1487000K .......... .......... .......... .......... .......... 99% 2.87M 1s
>     1487050K .......... .......... .......... .......... .......... 99%  191M 1s
>     1487100K .......... .......... .......... .......... .......... 99% 1.45M 1s
>     1487150K .......... .......... .......... .......... .......... 99% 2.74M 1s
>     1487200K .......... .......... .......... .......... .......... 99% 2.76M 1s
>     1487250K .......... .......... .......... .......... .......... 99% 2.95M 1s
>     1487300K .......... .......... .......... .......... .......... 99%  194M 1s
>     1487350K .......... .......... .......... .......... .......... 99% 2.55M 1s
>     1487400K .......... .......... .......... .......... .......... 99% 2.77M 1s
>     1487450K .......... .......... .......... .......... .......... 99% 2.86M 1s
>     1487500K .......... .......... .......... .......... .......... 99%  220M 1s
>     1487550K .......... .......... .......... .......... .......... 99% 3.23M 1s
>     1487600K .......... .......... .......... .......... .......... 99% 2.77M 1s
>     1487650K .......... .......... .......... .......... .......... 99% 2.73M 1s
>     1487700K .......... .......... .......... .......... .......... 99% 2.87M 1s
>     1487750K .......... .......... .......... .......... .......... 99%  133M 1s
>     1487800K .......... .......... .......... .......... .......... 99% 2.82M 1s
>     1487850K .......... .......... .......... .......... .......... 99% 2.93M 1s
>     1487900K .......... .......... .......... .......... .......... 99% 2.79M 0s
>     1487950K .......... .......... .......... .......... .......... 99% 3.66M 0s
>     1488000K .......... .......... .......... .......... .......... 99%  144M 0s
>     1488050K .......... .......... .......... .......... .......... 99% 3.04M 0s
>     1488100K .......... .......... .......... .......... .......... 99% 3.02M 0s
>     1488150K .......... .......... .......... .......... .......... 99% 2.92M 0s
>     1488200K .......... .......... .......... .......... .......... 99%  174M 0s
>     1488250K .......... .......... .......... .......... .......... 99% 2.79M 0s
>     1488300K .......... .......... .......... .......... .......... 99% 3.82M 0s
>     1488350K .......... .......... .......... .......... .......... 99% 2.41M 0s
>     1488400K .......... .......... .......... .......... .......... 99% 3.82M 0s
>     1488450K .......... .......... .......... .......... .......... 99%  135M 0s
>     1488500K .......... .......... .......... .......... .......... 99% 3.01M 0s
>     1488550K .......... .......... .......... .......... .......... 99% 3.92M 0s
>     1488600K .......... .......... .......... .......... .......... 99% 3.94M 0s
>     1488650K .......... .......... .......... .......... .......... 99%  152M 0s
>     1488700K .......... .......... .......... .......... .......... 99% 4.03M 0s
>     1488750K .......... .......... .......... .......... .......... 99% 2.93M 0s
>     1488800K .......... .......... .......... .......... .......... 99% 2.79M 0s
>     1488850K .......... .......... .......... .......... .......... 99% 3.55M 0s
>     1488900K .......... .......... .......... .......... .......... 99%  138M 0s
>     1488950K .......... .......... .......... .......... .......... 99% 2.89M 0s
>     1489000K .......... .......... .......... .......... .......... 99% 4.48M 0s
>     1489050K .......... .......... .......... .......... .......... 99% 3.68M 0s
>     1489100K .......... .......... .......... .......... .......... 99%  174M 0s
>     1489150K .......... .......... .......... .......... .......... 99% 4.37M 0s
>     1489200K .......... .......... .......... .......... .......... 99% 3.65M 0s
>     1489250K .......... .......... .......... .......... .......... 99% 3.78M 0s
>     1489300K .......... .......... .......... .......... .......... 99% 3.01M 0s
>     1489350K .......... .......... .......... .......... .......... 99%  119M 0s
>     1489400K .......... .......... .......... .......... .......... 99% 2.86M 0s
>     1489450K .......... .......... .......... .......... .......... 99% 2.98M 0s
>     1489500K .......... .......... .......... .......... .......... 99% 3.00M 0s
>     1489550K .......... .......... .......... .......... .......... 99% 2.97M 0s
>     1489600K .......... .......... .......... .......... .......... 99%  151M 0s
>     1489650K .......... .......... .......... .......... .......... 99% 2.00M 0s
>     1489700K .......... .......... .......... .......... .......... 99% 4.04M 0s
>     1489750K .......... .......... .......... .......... .......... 99% 2.82M 0s
>     1489800K .......... .......... .......... .......... .......... 99%  161M 0s
>     1489850K .......... .......... .......... .......... .......... 99% 3.14M 0s
>     1489900K .......... .......... .......... .......... .......... 99% 3.00M 0s
>     1489950K .......... .......... .......... .......... .......... 99% 3.03M 0s
>     1490000K .......... .......... .......... .......... .......... 99% 2.93M 0s
>     1490050K .......... .......... .......... .......... .......... 99%  135M 0s
>     1490100K .......... .......... .......... .........            100% 2.35M=5m33s
>
>     2020-11-30 09:22:07 (4.36 MB/s) - ‘svwiki-20201120-pages-articles.xml.bz2’ saved [1525903265/1525903265]

  

Extract the .xml-file

In [ ]:
ls
bzip2 -dk svwiki-20201120-pages-articles.xml.bz2

  

>     conf
>     derby.log
>     eventlogs
>     ganglia
>     logs
>     svwiki-20201120-pages-articles.xml.bz2

  

Copy file to DBFS cluster

In [ ]:
val localpath="file:/databricks/driver/svwiki-20201120-pages-articles.xml"

dbutils.fs.mkdirs("dbfs:/datasets/")

dbutils.fs.cp(localpath, "dbfs:/datasets/")

  

We verify that we now have an xml file with some contents.

In [ ]:
display(dbutils.fs.ls("dbfs:/datasets/svwiki-20201120-pages-articles.xml"))

  

[TABLE]

  

Extracting and Filtering
========================

Before we can try to build an LDA model, we need to split the contents
into separate pages, and anything that is not relevant article content.

Extracting Pages
----------------

The full wikipedia dump is a very large xml files, each page (with
metadata) is enclosed in &lt;page&gt; &lt;/page&gt; tags, so we split it
into an rdd.

In [ ]:
// ---------------spark_import
import org.apache.spark.SparkContext
import org.apache.spark.SparkConf
import org.apache.spark.sql.SQLContext

// ----------------xml_loader_import
import org.apache.hadoop.io.LongWritable
import org.apache.hadoop.io.Text
import org.apache.hadoop.conf.Configuration
import org.apache.hadoop.io.{ LongWritable, Text }
import com.databricks.spark.xml._

import org.apache.hadoop.conf.Configuration

// ---- Mathing
import scala.util.matching.UnanchoredRegex


  

>     import org.apache.spark.SparkContext
>     import org.apache.spark.SparkConf
>     import org.apache.spark.sql.SQLContext
>     import org.apache.hadoop.io.LongWritable
>     import org.apache.hadoop.io.Text
>     import org.apache.hadoop.conf.Configuration
>     import org.apache.hadoop.io.{LongWritable, Text}
>     import com.databricks.spark.xml._
>     import org.apache.hadoop.conf.Configuration
>     import scala.util.matching.UnanchoredRegex

  

Our function to read the data uses a library call to extract pages into
an RDD.

In [ ]:
def readWikiDump(sc: SparkContext, file: String) : RDD[String] = {
  val conf = new Configuration()
  conf.set(XmlInputFormat.START_TAG_KEY, "<page>")
  conf.set(XmlInputFormat.END_TAG_KEY, "</page>")
  val rdd = sc.newAPIHadoopFile(file, 
                                classOf[XmlInputFormat], 
                                classOf[LongWritable], 
                                classOf[Text], 
                                conf)
  rdd.map{case (k,v) => (new String(v.copyBytes()))}
}

  

>     readWikiDump: (sc: org.apache.spark.SparkContext, file: String)org.apache.spark.rdd.RDD[String]

  

We look at a small sample to see that we succeeded in extracting pages,
but we see that the pages still contain a lof of uninteresting bits in
their text.

In [ ]:
// Read the wiki dump
val dump = readWikiDump(sc, "dbfs:/datasets/svwiki-20201120-pages-articles.xml")

val n_wp_pages = dump.count()
dump.take(5)

  

>     dump: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[993] at map at command-2629358840275831:10
>     n_wp_pages: Long = 6412168
>     res0: Array[String] =
>     Array(<page>
>         <title>Amager</title>
>         <ns>0</ns>
>         <id>1</id>
>         <revision>
>           <id>48450724</id>
>           <parentid>48155628</parentid>
>           <timestamp>2020-11-15T15:50:09Z</timestamp>
>           <contributor>
>             <username>Nalle&amp;Lisa</username>
>             <id>348723</id>
>           </contributor>
>           <comment>Länkfix</comment>
>           <model>wikitext</model>
>           <format>text/x-wiki</format>
>           <text bytes="7485" xml:space="preserve">{{källor|datum=2013-05}}
>
>     {{Geobox|Island
>     &lt;!-- *** Mallhuvud *** --&gt;
>     | name                  = Amager
>     | native_name           =
>     | other_name            =
>     | category              = Ö
>     &lt;!-- *** Namn **** --&gt;
>     | etymology             =
>     | nickname              =
>     &lt;!-- *** Bild *** --&gt;
>     | image                 = Amager.jpg
>     | image_caption         =
>     &lt;!-- *** Länder, regioner mm *** --&gt;
>     | country               = Danmark
>     | country_flag          = true
>     | state                 = | state_type =
>     | region                = [[Region Hovedstaden]]| region_type =
>     | district              = | district_type =
>     | municipality          = [[Dragørs kommun]]| municipality_type =
>     | municipality1         = [[Köpenhamns kommun]]
>     | municipality2         = [[Tårnby kommun]]
>     &lt;!-- *** Tillhörighet, föremål inom området mm *** --&gt;
>     | parent                =
>     | range                 = | range_type = Arkipelag
>     | border                =
>     | part                  =
>     | city                  =
>     | capital               =
>     | landmark              =
>     | river                 =
>     &lt;!-- *** Läge *** --&gt;
>     | location              = [[Öresund]]
>     | lat_d                 = 55| lat_m = 37| lat_s = 0| lat_NS = N
>     | long_d                = 12| long_m = 37| long_s = 0| long_EW = E
>     | highest               =
>     | highest_location      = | highest_region = | highest_country =
>     | highest_elevation     =
>     | highest_lat_d         = | highest_lat_m = | highest_lat_s = | highest_lat_NS =
>     | highest_long_d        = | highest_long_m = | highest_long_s = | highest_long_EW =
>     &lt;!-- *** Dimensioner *** --&gt;
>     | length                = 16.1| length_orientation =
>     | width                 = 11.1| width_orientation =
>     | height                = 4
>     | area                  = 96.29
>     &lt;!-- *** Befolkning *** --&gt;
>     | population            = 196047| population_date = 1 januari 2018 | population_note =
>     | population_density    = auto| population_density_note =
>     &lt;!-- *** Tillgänglighet *** --&gt;
>     | public                =
>     | access                =
>     &lt;!-- *** Koder ***  --&gt;
>     | timezone              = CET| utc_offset = +1
>     | timezone_DST          = CEST| utc_offset_DST = +2
>     | postal_code           =
>     | area_code             =
>     | code                  =
>     &lt;!-- *** Fria fält *** --&gt;
>     | free                  =
>     | free_type             =
>     &lt;!-- *** Kartor *** --&gt;
>     | map                   = Denmark physical map.svg
>     | map_caption           = Amagers läge i Danmark.
>     | map_locator           = Danmark
>     | map_locator_x         =
>     | map_locator_y         =
>     &lt;!-- *** Fotnoter *** --&gt;
>     | footnotes             =
>     }}
>
>     '''Amager''' är en [[Danmark|dansk]] [[ö (landområde)|ö]] i [[Öresund]]. Öns norra och västra delar tillhör [[Köpenhamn]], medan övriga delar upptas av [[Tårnby kommun]] och [[Dragørs kommun]].
>     Amager har en yta på 96,29 km² och befolkningen uppgår till 196&amp;nbsp;047 personer (1/1 2018). På den östra delen av ön finns [[Kastrups flygplats]]. Amager har också flera militära anläggningar (t.ex. Kastrup fort och Dragör fort), strandparksområdena [[Amager Strandpark]] och Kastrup Strandpark, flera restauranger i världsklass (t.ex. Noma), ett stort naturområde ([[Amager Fælled]]), ett av Nordens största köpcentra (Field's), och mycket annat. Amager förkortas ofta Ama'r för att återspegla det danska uttalet.
>
>     Amager är delvis en [[konstgjord ö]], delvis en naturlig sådan. T.ex. så anlades en konstgjord ö i anslutning till det redan existerande Amager Strandpark, när denna rustades upp väsentligen 2005. Ön är mycket låg och vissa delar ligger under havsytan, framför allt det genom fördämning och utfyllnad skapade området [[Kalvebod Fælled]] (Vestamager). Ön började uppodlas i större skala under 1500-talets första hälft då kung [[Kristian II]] bjöd in nederländska bönder till att odla upp ön för kronans räkning. Flertalet slog sig ned i fiskeläget [[Dragør]] där många än idag har namn med nederländskt ursprung som Neels och Tønnes.
>
>     Den stora utvecklingsfasen började på 1800-talet då flera fabriker, bostadsområden och militär verksamhet utvecklade sig på ön. Det fanns ett antal stora och kända fabriker på ön och den industriella prägeln höll sig väldigt tydlig långt in på 1900-talet. Det förekommer också områden som tidigare varit sommarstugeområden och där det än idag finns kolonilottområden, t.ex. i Sundbyöster ut mot kusten. Under 1900-talet började militären att utveckla stora övningsområden på ön vilket resulterade i Vestamager som idag innefattar ett stort naturområde endast några kilometer från Köpenhamns centrum. Under senare delen av 1900-talet har flygplatsen [[Kastrups flygplats|Kastrup]] som ligger på ön expanderat kraftigt och i samband med byggandet av [[Öresundsförbindelsen]] började även ett stort område byggas som kallas för [[Ørestad]].
>
>     Ön, som är förbunden med övriga Köpenhamn genom flera broar, [[järnväg]] och sedan 2017 även [[Köpenhamns tunnelbana|Metro]], har under de senaste 10-20 åren förändrats mycket. Dels har flera områden exploaterats med nya moderna kvarter lägenheter, t.ex. i områdena Örestad, [[Islands Brygge]] och Amager Strand, och dels så har Amagers närhet till Köpenhamn raderat dess tidigare rykte som en industriell förstad till Köpenhamn. Idag finns det områden av alla sorter på ön - tack vare flygplatsen finns ett aktivt näringsliv kvar, samtidigt som de nya områdena och inte minst upprustningen av t.ex. Amager Strand har förändrat bilden av ön. Idag är flera delar av ön mycket attraktiva bostadsområden.
>
>     Ön återfick i samband med Öresundsbron även en järnvägsförbindelse vilket inte funnits sedan 1960-talet. En [[motorväg]] byggdes samtidigt över öns mellersta del. [[Köpenhamns metro]] går till Kastrup och Vestamager med ett antal stationer på ön. Kring metro- och regionaltågsstationen Ørestad har ett stort kontors-, handels- och bostadsområde uppförts under [[2000-talet (decennium)|00-talet]]. Detta sträcker sig från [[Bella Center]] i norr till metrostationen [[Vestamager station|Vestamager]] i söder. Även om området inte ligger vid kusten har Ørestad delvis fått karaktären av en hamnstad genom anläggandet av konstgjorda kanaler och dammar. Vid [[Islands Brygge]] finns en &quot;äkta&quot; sjöstad.
>
>     Före [[Öresundsförbindelsen]]s uppförande hade småstaden [[Dragør]] en färjeförbindelse med [[Limhamn]] i södra [[Malmö]]. På grund av denna fanns i Dragør tidigare en betydande gränshandel. Denna har under 2000-talet helt upphört liksom den genomgående trafiken.
>
>     Öns västra del, som utvanns från Öresund under 1920- och 30-talen, är med undantag för Bella Center, Ørestad samt ett koloniområde, avsatt som natur- och rekreationsområde. Marken är ängsmark (odlad mark finns dock på den naturliga delen av öns södra del).
>     Längst i söder finns den flerhundraåriga skogen [[Kongelunden]]. På öns östra del, vid Öresundskusten, finns [[Amager Strandpark]] med en populär sandstrand. Området har omgestaltats, med en [[konstgjord ö]] och en [[lagun]] innanför. Nyinvigningen av parken, som funnits sedan 1934, ägde rum 2005.
>
>     Amager är den tätast befolkade ön i Danmark. Den näst mest tätbefolkade är [[Thurø]].
>
>     == Se även ==
>     *[[Amagerbro]]
>     *[[Amagerport]]
>     *[[Amagerbroderi]]
>
>     ==Externa länkar==
>     {{commonscat|Amager}}
>
>     {{Öar i Danmark}}
>
>     {{auktoritetsdata}}
>
>     [[Kategori:Amager| ]]
>     [[Kategori:Öar i Region Hovedstaden]]</text>
>           <sha1>qom8qpk70qa0alvfb4b4ha9or4ftohh</sha1>
>         </revision>
>       </page>, <page>
>         <title>Afrika</title>
>         <ns>0</ns>
>         <id>2</id>
>         <revision>
>           <id>48459142</id>
>           <parentid>48374249</parentid>
>           <timestamp>2020-11-18T06:30:03Z</timestamp>
>           <contributor>
>             <username>LarskeBot</username>
>             <id>310602</id>
>           </contributor>
>           <minor />
>           <comment>/* Geografi */mellan X - Y → mellan X och Y med [[Project:AWB|AWB]]</comment>
>           <model>wikitext</model>
>           <format>text/x-wiki</format>
>           <text bytes="37806" xml:space="preserve">{{templateStyles-tagg Faktamall}}
>     {| class=&quot;infobox&quot; style=&quot;font-size:90%;&quot; width=&quot;300&quot;
>     |+ &lt;big&gt;'''Afrika'''&lt;/big&gt;
>     |-
>     | align=&quot;center&quot; colspan=&quot;2&quot; style=&quot;background:#f9f9f9;&quot; | [[Fil:Africa (orthographic projection).svg|280px|Afrika i världen.]]
>     ----
>     |-
>     | align=&quot;center&quot; colspan=&quot;2&quot; style=&quot;background:#f9f9f9;&quot; | [[Fil:Afrika politisk.png|280px|Politisk karta.]]
>     ----
>     |-
>     | colspan=&quot;2&quot; |
>     |-
>     | '''Area:''' &lt;br /&gt; '''Folkmängd:''' &lt;br /&gt; '''Antal stater:''' &lt;br /&gt; '''Autonoma regioner:''' &lt;br /&gt; '''Högsta punkt:''' &lt;br /&gt; '''Lägsta punkt:''' || 30&amp;nbsp;221&amp;nbsp;532&amp;nbsp;km² &lt;br /&gt; 1&amp;nbsp;022&amp;nbsp;234&amp;nbsp;000&lt;ref name=&quot;esa.un.org&quot;/&gt; (2010) &lt;br /&gt; 54 &lt;br /&gt; 8 &lt;br /&gt; [[Kilimanjaro]]&amp;nbsp;5&amp;nbsp;895&amp;nbsp;[[m ö.h.|m&amp;nbsp;ö.h.]] &lt;br /&gt; [[Assalsjön]]&amp;nbsp;156&amp;nbsp;[[Meter över havet|m&amp;nbsp;u.h.]]
>     |-
>     | colspan=&quot;2&quot; |
>     ----
>     |-
>     | colspan=&quot;2&quot; | '''Stater'''
>     |-
>     | {{flaggbild2|Algeriet}} &lt;br /&gt; {{flaggbild2|Angola}} &lt;br /&gt; {{flaggbild2|Benin}} &lt;br /&gt; {{flaggbild2|Botswana}} &lt;br /&gt; {{flaggbild2|Burkina Faso}} &lt;br /&gt; {{flaggbild2|Burundi}} &lt;br /&gt; {{flaggbild2|Centralafrikanska republiken}} &lt;br /&gt; {{flaggbild2|Djibouti}} &lt;br /&gt; {{flaggbild2|Egypten}} &lt;br /&gt; {{flaggbild2|Ekvatorialguinea}} &lt;br /&gt; {{flaggbild2|Elfenbenskusten}} &lt;br /&gt; {{flaggbild2|Eritrea}} &lt;br /&gt; {{flaggbild2|Etiopien}} &lt;br /&gt; {{flaggbild2|Gabon}} &lt;br /&gt; {{flaggbild2|Gambia}} &lt;br /&gt; {{flaggbild2|Ghana}} &lt;br /&gt; {{flaggbild2|Guinea}} &lt;br /&gt; {{flaggbild2|Guinea-Bissau}} &lt;br /&gt; {{flaggbild2|Kamerun}} &lt;br /&gt; {{flaggbild2|Kap Verde}} &lt;br /&gt; {{flaggbild2|Kenya}} &lt;br /&gt; {{flaggbild2|Komorerna}} &lt;br /&gt; {{flaggbild2|Kongo-Brazzaville}} &lt;br /&gt; {{flaggbild2|Kongo-Kinshasa}} &lt;br /&gt; {{flaggbild2|Lesotho}} &lt;br /&gt; {{flaggbild2|Liberia}} &lt;br /&gt;{{flaggbild2|Libyen}} ||{{flaggbild2|Madagaskar}} &lt;br /&gt; {{flaggbild2|Malawi}} &lt;br /&gt; {{flaggbild2|Mali}} &lt;br /&gt; {{flaggbild2|Marocko}} &lt;br /&gt; {{flaggbild2|Mauretanien}} &lt;br /&gt; {{flaggbild2|Mauritius}} &lt;br /&gt; {{flaggbild2|Moçambique}} &lt;br /&gt; {{flaggbild2|Namibia}} &lt;br /&gt; {{flaggbild2|Niger}} &lt;br /&gt; {{flaggbild2|Nigeria}} &lt;br /&gt; {{flaggbild2|Rwanda}} &lt;br /&gt; {{flaggbild2|São Tomé och Príncipe}} &lt;br /&gt; {{flaggbild2|Senegal}} &lt;br /&gt; {{flaggbild2|Seychellerna}} &lt;br /&gt; {{flaggbild2|Sierra Leone}} &lt;br /&gt; {{flaggbild2|Somalia}} &lt;br /&gt; {{flaggbild2|Sudan}} &lt;br /&gt; {{flaggbild2|Sydafrika}} &lt;br /&gt; {{flaggbild2|Sydsudan}} &lt;br /&gt; {{flaggbild2|Swaziland}} &lt;br /&gt; {{flaggbild2|Tanzania}} &lt;br /&gt; {{flaggbild2|Tchad}} &lt;br /&gt; {{flaggbild2|Togo}} &lt;br /&gt; {{flaggbild2|Tunisien}} &lt;br /&gt; {{flaggbild2|Uganda}} &lt;br /&gt; {{flaggbild2|Zambia}} &lt;br /&gt; {{flaggbild2|Zimbabwe}}
>     |-
>     | colspan=&quot;2&quot; |
>     ----
>     '''Stater som har delar av ytan i Afrika'''
>     |-
>     | {{flaggbild2|Egypten}} &lt;br /&gt; {{flaggbild2|Frankrike}} &lt;br /&gt; {{flaggbild2|Jemen}} &lt;br /&gt; {{flaggbild2|Portugal}} &lt;br /&gt; {{flaggbild2|Spanien}}
>     |-
>     | colspan=&quot;2&quot; |
>     ----
>     '''Särskilda områden'''
>     &lt;br /&gt; {{flaggbild|Ceuta}} [[Ceuta]] ([[Spanien]])
>
>     {{flaggbild|Kanarieöarna}} [[Kanarieöarna]] ([[Spanien]])
>
>     {{flaggbild|Madeira}} [[Madeira]] ([[Portugal]])
>
>     {{flaggbild|Frankrike}} [[Mayotte]] ([[Frankrike]])
>
>     {{flaggbild|Melilla}} [[Melilla]] ([[Spanien]])
>
>     {{flaggbild|Frankrike}} [[Réunion]] ([[Frankrike]])
>
>     {{flaggbild|Sankta Helena}} [[Sankta Helena]] ([[Storbritannien]])
>
>     [[Fil:Flag of Somaliland.svg|22px|Somaliland]] [[Somaliland]]
>
>     {{flaggbild2|Västsahara}} ([[Marocko]])
>     ----
>     '''Största städer (över 10 miljoner invånare med förorter):'''
>     &lt;br /&gt;{{flaggbild|Egypten}} [[Kairo]]
>     &lt;br /&gt;{{flaggbild|Kongo-Kinshasa}} [[Kinshasa]]
>     &lt;br /&gt;{{flaggbild|Nigeria}} [[Lagos]]
>     ----
>     [[Fil:Africa satellite.jpg|280px|Satellitfoto]]
>     |}
>     '''Afrika''' är [[jorden]]s näst största [[kontinent]] (efter [[Eurasien]]) och även jordens näst största världsdel efter [[Asien]], både vad gäller [[areal]] och [[folkmängd]]. Med världsdelens öar inräknade mäter Afrika 30 244 050&amp;nbsp;km², vilket motsvarar 20,3 % av jordens landmassa eller cirka 6 % av jordens totala area. Omkring 22 miljoner km² av dessa ligger i [[tropikerna]] vilket gör den afrikanska kontinenten till världens varmaste kontinent. I Afrika bor det ungefär 1 miljard människor i 55 länder – en sjundedel av jordens befolkning. Dess längd i nordlig-sydlig riktning är omkring 8 000&amp;nbsp;km och dess största bredd omkring 7&amp;nbsp;800&amp;nbsp;km.
>
>     == Etymologi ==
>     Ordet &quot;Afrika&quot;s [[etymologi]]ska bakgrund är dunkel, och ordet har dessutom haft olika betydelse under olika [[epok]]er.
>
>     I äldre tider kallades allt land väster om [[Egypten]] för &quot;[[Libyen]]&quot;. Enligt den [[Grekland|grek]]iska historieskrivaren [[Herodotos]] berättelse seglade [[fenicisk]]a sjömän på uppdrag av den egyptiska kungen Neko runt Libyen, det vill säga [[kontinent]]en Afrika redan omkring 600 f.Kr.
>
>     När [[Romerska riket|romarna]] först använde termen ''[[Africa terra]]'' avsåg de före [[andra puniska kriget]] endast trakten kring [[Karthago]], det vill säga motsvarande ungefär dagens Tunisien där själva staden Karthago låg samt stora delar av Libyen och Algeriet. Romarna uppfattade först [[Egypten]] som en del av [[Asien]], och det var först geografen [[Klaudios Ptolemaios|Ptolemaios]] som drog gränsen vid [[Suez]]. Under kejsartiden kom ''Africa'' att avse hela det området romarna kände till av kontinenten, och allt eftersom européerna har upptäckt mer och mer av kontinenten har termen avsett ett allt större område. Namnet ska ha avsett &quot;landet där ''afri''&quot; (singularis: ''afer'') bodde. ''Afer'' avsåg namnet på ett folk som kan motsvara dagens [[berber]]. [[Viking]]arna under 800- och 900-talen kallade norra Afrika för &quot;Blåland&quot;.
>
>     == Geografi ==
>     {{Fördjupning|Afrikas geografi}}
>     Geologiskt är Afrika en mycket gammal kontinent, i huvudsak uppbyggd av [[urberg]]. Under jordens forntid var den en del av jättekontinenten [[Gondwana]]land. Denna gled isär under [[Jura (period)|juraperioden]] och [[Krita (geologi)|kritaperioden]] för 100-150 miljoner år sedan, och så fick Afrika sin nuvarande form.
>
>     Inom sina regelbundna kuster rymmer den en total yta på omkring 30 244 050&amp;nbsp;km² inklusive öarna. Afrika åtskiljs från [[Europa]] av [[Medelhavet]] och förenas med [[Asien]] i sitt nordöstra hörn genom [[näs]]et vid [[Suez]], omkring 130&amp;nbsp;km i bredd. Omkring 8 000&amp;nbsp;km skiljer den nordligaste punkten, [[Ras ben Sakka]] i [[Tunisien]], från dess sydligaste, [[Kap Agulhas]] i [[Sydafrika]]. Från [[Kap Verde]], dess västligaste punkt, till [[Ras Hafun]] i [[Somalia]] är det omkring 7 400&amp;nbsp;km. Afrikas kuster mäter omkring 28 000&amp;nbsp;km (1&amp;nbsp;km kust på 1 057&amp;nbsp;km²) varav [[Medelhavet]] står för 5 400&amp;nbsp;km, [[Atlanten]] 11 000&amp;nbsp;km, [[Indiska oceanen]] 8 700&amp;nbsp;km och [[Röda havet]] 2 900&amp;nbsp;km, och avsaknaden av djupare [[bukt]]er och betydande [[halvö]]ar blir uppenbar om man jämför med Europas 9 700 000&amp;nbsp;km² som omgärdas av 32 000&amp;nbsp;km kust (1&amp;nbsp;km kust på 278&amp;nbsp;km² land), eller för det [[fjord]]rika [[Norge]], där motsvarande siffra 385 000&amp;nbsp;km² land och en kuststräcka på 20 000&amp;nbsp;km (1&amp;nbsp;km kust per 19&amp;nbsp;km² land).
>
>     Afrikas genomsnittliga höjd över havet är omkring 600 m, vilket motsvarar de båda amerikanska kontinenternas höjd men är lägre än den asiatiska på omkring 950 m. Jämfört med de andra kontinenterna utmärker sig Afrika med en liten andel landområden på mycket hög respektive mycket låg höjd. Landområdena under 180 [[Meter över havet|m ö.h.]] upptar en osedvanligt liten del av den totala landmassan, och även de högsta delarna, som är betydligt lägre än motsvarande i Asien, utgörs av isolerade berg och bergskedjor. Generellt ligger de högre [[platå]]erna i öst och söder och de lägre delarna åt väst och norr och med en linje från ungefär vid [[Röda havet]]s mitt till [[Kongofloden]]s mynning kan kontinenten delas upp i ett lågland i norr och ett högland i söder. Man kan således dela upp Afrika i fyra övergripande landområden:
>     # Slättlanden vid kusterna – ofta med [[Mangrove|mangroveträsk]] insprängda – når aldrig långt in från kusterna utom vid vattendragens nedre delar. Slamrika slättland finns egentligen bara vid de större flodernas deltan, och annars utgör kustområdena det lägsta steget i en serie terrasser som bildar stigningen upp till de inre platåerna.
>     # [[Atlasbergen]] i norr som ligger isolerat av [[Sahara]]öknen som bitvis ligger under havsnivån.
>     # De södra och östra höglanden som ligger på mellan 600 och 1 000 m ö.h.
>     # De nordliga och västra slättlanden, inklusive Sahara, som kantas och genomfars av stråk med högra landområden, men i allmänhet håller sig under 600 m ö.h.
>
>     Skog täcker cirka en femtedel av Afrikas yta.
>
>     Hela Afrika har ett [[Tropiskt klimat|tropiskt]] eller [[Subtropiskt klimat]].
>
>     === Fauna ===
>
>     {{Fördjupning|Afrikas fauna}}
>
>     Afrika är huvudsakligen känt för den öppna [[savann]]en med väldiga hjordar av hovdjur som [[afrikansk buffel]], gnuer, [[antiloper]], [[giraff]]er och [[zebror]]. Dessa jagas av stora rovdjur som [[lejon]], [[hyenor]] och [[Gepard|geparder]]. Den afrikanska djungeln befolkas av [[ormar]] och [[primater]] samt av vattenlevande djur som [[krokodiler]] och [[groddjur]]. Afrika har även det största antalet arter som kan räknas till [[megafauna]]n, bland annat [[afrikanska elefanter]], [[trubbnoshörning]] och [[flodhäst]], på grund av att kontinenten bara i mindre utsträckning var påverkad av det [[massutdöende]] som skedde under [[pleistocen]].
>
>     == Historia ==
>     {{Fördjupning|Afrikas historia}}
>     Den första mänskliga migrationen, från kontinenten men även inom Afrikas kuster, kan spåras [[lingvistisk]]t, kulturellt och genom datoranalyser av [[genetik|genetiskt]] material. Någonstans i [[Östafrika]] tror dagens forskare att de första [[hominider]] uppkom. I [[Etiopien]] har man hittat ett två miljoner år gammalt skelett som uppkallades [[Australopithecus afarensis|Lucy]], den första kända mänskliga [[primater|primaten]].
>
>     === Tidigare civilisationer och handel ===
>     [[Fil:African-civilizations-map-pre-colonial.svg|miniatyr|höger|Några av de mest kända afrikanska civilisationerna. Civilisationer från olika tidsepoker visas parallellt.]]Lingvistiska spår tyder på att [[Bantu]]folk emigrerade från [[Västafrika]] in i [[khoisan]]folkets (de som tidigare kallades [[bushmän]] och [[hottentott]]er) område och undanträngde dem. Bantufolken använde en uppsättning grödor, bland annat [[kassava]] och [[jams]], som var mer lämpade för jordbruk i det [[tropik|tropiska]] Afrika, och med sitt jordbruk kunde bantu försörja en betydligt större folkmängd än de undanträngde jägare eller samlare. Bantufolkens traditionella bosättningsområde sträcker sig från [[Sahara]]öknen till de [[Tempererat klimat|tempererade regionerna]] i söder. Historiskt har deras vapen varit spjut, sköld, pil och båge. Det för [[khoisanspråk]]ens karaktäristiska klickljuden återfinns i [[nguni (språk)|nguni]]- och [[bantu]]språk som till exempel [[xhosa]] och [[zulu]]. Khoisanspråken talas idag främst av utspridda minoritetsgrupper i södra Afrika i länderna kring [[Kalahariöknen]].
>
>     Afrika söder om Sahara har bestått olika stater och riken, mindre stadsstater samt [[stam (samhällsvetenskap)|stammar]] och [[klan]]er. Det forna egyptiska riket, är en av världens äldsta stater, och den mest kända av de äldre afrikanska [[civilisation]]erna. Det fanns även statsbildningar i [[Nubien]], [[Ghana]], [[Timbuktu]], [[Kanem-Bornu]] och i nuvarande [[Zimbabwe]].
>
>     [[Nordafrika]] kom från och med 600-talet under arabiskt inflytande – en tid av kulturell blomstring – och med kamelens hjälp spreds [[Islam]] och den arabiska kulturen genom Sahara och vidare söderut. Arabiska och persiska handelsmän spred likaså Islam längs med den östafrikanska kusten parallellt med en omfattande svart slavhandel. Detta östafrikanska handelsutbyte med araber och perser sträckte sig ända bort till Java, därmed den etniska, kulturella och språkliga mångfalden på Östafrikas kust, [[Madagaskar]] och [[Komorerna]].
>
>     === Arabvärlden och Europa utforskar kontinenten ===
>     Det arabiska intresset för Afrika går långt tillbaka, men handel med slavar från centra som Lamu och Zanzibar vid östkusten har pågått åtminstone sedan senare delen av 1000-talet e.Kr. De miljoner slavar som exporterades till arabvärlden och österut under den [[arabiska slavhandeln]] finns idag kvar som en stor etnisk grupp där, i stor utsträckning integrerad med den ursprungliga arabiska befolkningen. Slavhandel med svarta pågår fortfarande i vissa arabländer, till exempel Sudan.&lt;ref&gt;[http://www.hrw.org/backgrounder/africa/sudanupdate.htm Human Rights Watch] [http://news.bbc.co.uk/2/hi/programmes/this_world/6458377.stm BBC World]&lt;/ref&gt;
>
>     Under [[1300-talet]] började vissa europeiska handelsexpeditioner få kontakt med Nord- och Västafrika.
>
>     Från 1500-talets mitt fram till förbudet mot slavhandel i Storbritanniens parlament 1807, som kom att få full kraft först något årtionde senare, handlades slavar från lokala stamledare i främst Väst- och Centralafrika. Flera miljoner afrikaner såldes på detta vis som slavar till främst de Europeiska kolonierna i Amerika.
>
>     Tidiga kolonisatörer var [[araber]], [[Portugal|portugiser]] och [[Spanien|spanjorer]]. Därefter koloniserades kontinenten av framförallt av [[Storbritannien]], [[Frankrike]], [[Spanien]], [[Italien]], [[Belgien]], [[Nederländerna]], [[Tyskland]] och [[Portugal]].
>
>     === Koloniseringen ===
>     {{Fördjupning|Kapplöpningen om Afrika}}
>     [[Fil:Map_of_Africa_from_Encyclopaedia_Britannica_1890.jpg|miniatyr|Politisk karta över Afrika från 1890.]] Från slutet av 1000-talet koloniserade arabiska köpmän den afrikanska östkusten. Holländarna koloniserade på 1600-talet den då relativt folktomma södra udden av Afrika. Portugiserna grundade flera kolonier i västra Afrika och vid nuvarande Moçambique. I mitten av [[1800-talet]] leddes flera expeditioner in i de centrala delarna av Afrika, bland annat av Livingstone och Speke, men också den svenske upptäcktsresanden Andersson. Denna kolonisering förändrade i grunden det ekonomiska livet på kontinenten, och innebar i flera fall ensidig export av naturresurser. [[Leopold II av Belgien|Kung Leopolds]] rovdrift på sin privata koloni [[Kongostaten]] är ökänd och kostade miljontals afrikanska liv.
>
>     I Afrika fanns efter första världskriget endast tre självständiga stater nämligen konungariket Egypten, republiken Liberia och konungariket Abessinien. Tanger bildade med sitt område en internationell zon. För övrigt var hela världsdelen uppdelad på kolonier åt europeiska makter.&lt;ref&gt;{{runeberg.org|nfcn|0245.html Supplement Afrika}}&lt;/ref&gt;
>
>     Flertalet kolonier erhöll [[självständighet]] under slutet av 1950-talet och början av 1960-talet. Avkoloniseringen resulterade i flertalet fall med katastrof, där diktatur och etnisk rensning vanligen blev följden, till exempel i Rhodesia (nuvarande Zimbabwe), Rwanda, Uganda, Nigeria m.fl. stater.
>
>     == Politik ==
>     {{Fördjupning|Afrikas politik}}
>     [[Fil:Electoral_Democracies_in_Africa,_Freedom_House_2007.png|miniatyr|De blå länderna är demokratier enligt tankesmedjan Freedom House.]]
>     Det postkoloniala Afrikas politik har i hög grad präglats av diktatur, statskupper och brist på [[mänskliga rättigheter]].
>
>     [[Nation]]erna i Afrika har ofta visat sig vara instabila [[stat]]er som efter självständigheten hemsökts av korruption, våldsamma [[statskupp]]er, [[despotism]] och [[dikatur|militärdiktaturer]]. Under perioden [[1960]]-[[1989]] genomfördes över 70 kupper och 13 [[mord]] på [[president]]er. Under det [[kalla kriget]] kämpade [[USA]] och [[Sovjetunionen]] om makten i Afrika, och tillsammans med länder som [[Frankrike]] och [[Kuba]], stödde de aktivt diktaturer och iscensatte flera av dessa kupper och illgärningar mot demokratiskt valda regeringar. Enligt en spridd uppfattning beror många [[militär]]a konflikter om [[territorium|territorier]] och [[gräns]]er under Afrikas moderna historia på de gränser från kolonial tid som huvudsakligen drogs upp efter [[Kongokonferensen]], men detta kan inte visas eftersom något jämförelseområde inte finns. Konflikter i Afrika har ofta drivits av tillgången till kontinentens rika [[råvara|råvarutillgångar]] – [[diamant]]er och [[metall]]er som [[guld]], [[platina]], [[uran]] och [[koppar]].
>
>     Korruption och havererade politiska program har lett till utbredd [[undernäring]] och undermålig infrastruktur orsakar alltjämt brist på mat och rent vatten. Sanitetsproblemen har också orsakat en akut spridning av bland annat [[HIV]]-viruset som leder till [[sjukdom]]en [[AIDS]].
>
>     Trots nöd och umbäranden finns vissa positiva tecken – [[demokrati]]skt valda [[regering]]ar växer i antal även om de inte utgör någon majoritet ännu. Genom påtryckningar från till exempel [[Internationella valutafonden]] (IMF) har många afrikanska stater lyckats vända en nedåtgående ekonomisk spiral som varat i flera decennier till en positiv tillväxt. Om man beaktar IMF:s katastrofala misslyckanden i länder som [[Argentina]] och [[Filippinerna]] är det dock oklart i vilken omfattning dessa konventionella ekonomiska kriterier ger en rättvis bild av vad Afrika faktiskt behöver.
>
>     De afrikanska ledarna har i olika omgångar sökt former för samarbete sinsemellan. Hittills har dock detta inte lyckats särskilt väl. Under [[inbördeskrig]]et i [[Kongo-Kinshasa|Kongo]] var det grannstaterna som spädde på konflikten snarare än utomafrikanska stater, och motivet var främst att skaffa kortsiktiga vinster genom att utvinna diamanter och andra mineraler. Antalet döda i konflikten har uppskattats till 3,5 miljoner under en femårsperiod.
>
>     De politiska samarbetsorganisationer man skapat har inte kunnat leva upp till förväntningarna. Organisationen för afrikansk enighet (OAU 1963-2002) kallades till exempel diktatorsklubben för dess tendens att alltid stödja de sittande politiska ledarna och ägna väldigt lite energi åt att främja den vanlige afrikanens mänskliga rättigheter. 2002 skapades i stället ''[[Afrikanska unionen]]'', som är tänkt att underlätta ett framtida samarbete mellan kontinentens stater. Hittills har denna organisation inte klarat att hantera till exempel utrotningskriget i Darfur-provinsen i Sudan, där en arabisk statsledning tillåtit slakt på hundratusentals av bantubefolkningen i sydväst. En avgörande skillnad mellan såväl OAU som AU å ena sidan och EU å den andra, är att EU startades av en grupp länder som satte demokrati och mänskliga rättigheter i fokus för den egna politiken, och som krävt detta av tillkommande medlemsstater. OAU och AU har från början i stället varit territoriellt inriktade, och visserligen haft demokrati och mänskliga rättigheter som skrivelser i sina stadgar, men ytterst sällan satt kraft bakom dessa ord. På så sätt liknar AU mer FN än EU.
>
>     == Ekonomi ==
>     {{Fördjupning|Afrikas ekonomi}}
>     [[Fil:Bakweri cocoyam farmer from Cameroon.jpg|250px|miniatyr|Jordbrukare på en [[jams]]odling i Kamerun]]
>     Afrikas ekonomi omfattar ungefär 887 miljoner människor i 54 olika stater ([[Juli 2005]]). Afrika är världens fattigaste kontinent, och den är, i medeltal, fattigare än den var för 25 år sedan. På [[FN]]:s Human Development Report från [[2003]] (som rankade världens 175 länder enligt en rad kriterier) var plats mellan 151 och 175 intagna av länder i Afrika.
>
>     Det finns dock länder som lyckas bra ekonomiskt, bland andra [[Botswana]], [[Ghana]] och [[Sydafrika]]. [[Etiopien]] har också en relativ bra ekonomi tack vare den höga tillväxten i landet på cirka 11-15% per år. Sydafrika har en väl fungerande inhemsk aktiehandel och har stora naturtillgångar i bland annat [[guld]] och [[diamanter]] och ett väl fungerande rättsväsen. Det finns i Sydafrika även tillgång till investeringskapital, stora marknader och utbildad arbetskraft. Det finns även andra afrikanska länder, exempelvis [[Egypten]], som länge haft en väl fungerande handelssektor och ekonomi.
>
>     Kontinentens kust mot [[Atlanten]] återfinns stora fyndigheter av [[Petroleum|olja]]. I [[Nigeria]] finns en av världens mest betydande oljereserver och landets ekonomi är bland de snabbast växande i världen.
>
>     Afrikas jordbruk sysselsätter drygt hälften av Afrikas befolkning och är den viktigaste näringen. Cirka 6 % (ungefär 1 814 600&amp;nbsp;km²) av världsdelens yta är uppodlad, mest i savannområdena. I stora delar av de brukningsbara områdena är jordens bördighet ganska låg. De bästa jordarna återfinns i begränsade höglandsområden som i till exempel [[Kamerun]] och [[Östafrika]].
>
>     == Demografi ==
>     {{Fördjupning|Afrikas demografi}}
>     ===Befolkning===
>     Människan, ''[[Homo sapiens|Homo sapiens sapiens]]'', antas härstamma från Afrika och spred sig härifrån till de omgivande kontinenterna. Den svarta befolkningen söder om Sahara tillhör den ursprungliga befolkningen som hela tiden levat i Afrika.
>
>     Norr om Sahara präglar samröret med övriga [[Medelhavet|Medelhavsområdet]] och inte minst sydvästra [[Asien]] både befolkning, språk och kultur.
>
>     På ön [[Madagaskar]], som länge var obebodd, härstammar majoriteten av befolkningen från människor som emigrerat från [[Indonesien]]. Under kolonialtiden skedde inflyttning från [[Europa]] och [[Asien]] till södra Afrika, där den icke-svarta befolkningen idag utgör betydande minoritetsgrupper.
>
>     Afrikas befolkning uppvisar många för utvecklingsregioner typiska drag, som höga [[födelsetal]] i kombination med sjunkande [[Mortalitet (demografi)|dödstal]] med snabb befolkningsökning och låg medelålder (45 % är under 15 år) som följd. Även om befolkningstätheten varierar kraftigt mellan olika regioner är Afrika generellt en glestbefolkad kontinent, med i genomsnitt 27,8 invånare per km².
>
>     Afrika är den kontinent som har den lägsta urbaniseringsgraden, med endast 25% boende i städer. Även här är variationerna stora, och i norra och östra Afrika finns en lång urban tradition.
>
>     === Språk ===
>     {{Fördjupning|Afrikas språk}}
>     [[Fil:African languages SV.png|miniatyr|300px|Språkfamiljer i Afrika]]
>     Uppskattningsvis finns det ungefär 1800 talade [[språk]] i Afrika idag. Vissa av dessa, till exempel [[Swahili]], [[Hausa]], och [[Yoruba (språk)|Yoruba]], talas av många miljoner människor, medan det finns andra som bara talas av några hundra eller ännu färre. Afrika har även många olika [[teckenspråk]]. De flesta afrikaner talar som modersmål ett språk tillhörande någon av de fyra språkfamiljerna [[afroasiatiska språk]], [[nilo-sahariska språk]], [[Niger-Kongospråk]] eller [[khoisanspråk]].
>     Med några få nämnvärda undantag i östra Afrika, har nästan alla afrikanska länder språk som har sitt ursprung utanför världsdelen som officiellt språk, på grund av kolonialismen och mänsklig migration. Till exempel används i ett stort antal afrikanska länder [[franska]] eller [[engelska]] som administrativt språk för stat, handel, utbildning och media. [[Arabiska]], [[portugisiska]], [[afrikaans]] och [[malagassiska]] är andra exempel på språk med ursprung utanför Afrika som idag används av miljoner afrikaner, både i offentliga och privata sammanhang.
>
>     === Religion ===
>     {{Fördjupning|Afrikas religion}}{{Se även|Kristendomen i Afrika}}
>     [[Kristendom]] och [[islam]] är de största religionerna i Afrika, 46,3 % av alla afrikaner är kristna och 40,5 % är muslimer. Resten 11,8 % tillhör olika lokala afrikanska religioner som funnits före den europeiska kolonisationen. Ett mindre antal afrikaner är [[Hinduism|hinduer]], eller har tro från [[Judendom|judisk]] tradition. Exempel på [[Afrikanska judar|afrikanska folk som tillhör judendomen]] är [[Beta Israel]], [[Lemba]]folk och [[Abayudaya]] i östra [[Uganda]]. Sydafrika och Angola har över 55% kristna katoliker.
>
>     == Kultur ==
>     {{Fördjupning|Afrikas kultur}}
>     Afrika har ett stort antal överlappande kulturer, och flera tusen olika etniska grupper. Den mest konventionella distinktionen är den mellan området söder om Sahara och området norr om öknen. De nordliga länderna associerar sig främst med arabkulturen, medan de sydliga länderna består av i sin tur många olika kulturella områden. En av de största kulturella identiteterna är den som innefattar alla folk som talar något [[bantuspråk]].
>
>     Det går också att göra en indelning i gamla franska Västafrika å ena sidan, och resten av Afrika (mer specifikt de brittiska eller portugisiska kolonierna i södra och östra Afrika) å andra sidan. En annan kulturell linje går mellan de afrikaner som håller kvar vid sina gamla traditionella livsstilar, och de som har anpassat sig till en mer modern livsstil. Det går även att dela upp traditionalisterna i sådana som ägnar sig åt uppfödning av boskap, och sådana som ägnar sig åt jordbruk.
>
>     [[Afrikansk konst]] och [[afrikansk arkitektur]] reflekterar de afrikanska kulturernas diversitet. De äldsta exemplen på afrikansk konst är 75 000 år gamla pärlsmycken gjorda av [[nassarius]]skal, dessa hittades i [[Blombosgrottan]]. [[Pyramiderna i Giza]] var världens högsta byggnader under 4 000 år. [[Klippkyrkorna i Lalibela]] i [[Etiopien]], bland vilka ''[[Bete Giyorgis]]'' (&quot;St. Göranskyrkan&quot;) är mest välkänd, är också kända exempel på afrikansk arkitektur.
>
>     == Regioner och territorier ==
>     [[Fil:Africa-regions.png|miniatyr|Regioner i Afrika:
>     {{legend|#0000ff|[[Nordafrika]]}}
>     {{legend|#00ff00|[[Västafrika]]}}
>     {{legend|#ff00ff|[[Centralafrika]]}}
>     {{legend|#ffa500|[[Östafrika]]}}
>     {{legend|#ff0000|[[Södra Afrika]]}}]]
>     {| class=&quot;wikitable sortable&quot;
>     |+
>     |- bgcolor=&quot;#ECECEC&quot;
>     ! Regioners namn&lt;ref&gt;Enligt [[:Fil:United Nations geographical subregions.png|FN:s regionindelning]] av världen.&lt;br /&gt;&lt;/ref&gt; och&lt;br /&gt; territorier, med flagga
>     ! data-sort-type=number | [[Area]]&lt;br /&gt;(km²)
>     ! data-sort-type=number |[[Folkmängd]]&lt;br /&gt;(2010)&lt;ref name=&quot;esa.un.org1&quot;/&gt;
>     ! data-sort-type=number |[[Befolkningstäthet]]&lt;br /&gt;(per km²)
>     ! [[Huvudstad]]
>     |-
>     | colspan=5 style=&quot;background:#eee;&quot; | '''[[Centralafrika]]''':
>     |-
>     | {{flaggbild2|Angola}}
>     | align=&quot;right&quot; | 1 246 700
>     | align=&quot;right&quot; | 19&amp;nbsp;081&amp;nbsp;912
>     | align=&quot;right&quot; | 15,3
>     | [[Luanda]]
>     |-
>     | {{flaggbild2|Kamerun}}
>     | align=&quot;right&quot; | 475 440
>     | align=&quot;right&quot; | 19 598 889
>     | align=&quot;right&quot; | 41,2
>     | [[Yaoundé]]
>     |-
>     | {{flaggbild2|Centralafrikanska republiken}}
>     | align=&quot;right&quot; | 622 984
>     | align=&quot;right&quot; | 4 401 051
>     | align=&quot;right&quot; | 7,1
>     | [[Bangui]]
>     |-
>     | {{flaggbild2|Tchad}}
>     | align=&quot;right&quot; | 1 284 000
>     | align=&quot;right&quot; | 11 227 208
>     | align=&quot;right&quot; | 8,7
>     | [[N'Djamena]]
>     |-
>     | {{flaggbild2|Kongo-Brazzaville}}
>     | align=&quot;right&quot; | 342 000
>     | align=&quot;right&quot; | 4 042 899
>     | align=&quot;right&quot; | 12,9
>     | [[Brazzaville]]
>     |-
>     | {{flaggbild2|Kongo-Kinshasa}}
>     | align=&quot;right&quot; | 2 345 410
>     | align=&quot;right&quot; | 65 965 795
>     | align=&quot;right&quot; | 28,1
>     | [[Kinshasa]]
>     |-
>     | {{flaggbild2|Ekvatorialguinea}}
>     | align=&quot;right&quot; | 28 051
>     | align=&quot;right&quot; | 700 401
>     | align=&quot;right&quot; | 24,9
>     | [[Malabo]]
>     |-
>     | {{flaggbild2|Gabon}}
>     | align=&quot;right&quot; | 267 667
>     | align=&quot;right&quot; | 1 505 463
>     | align=&quot;right&quot; | 5,6
>     | [[Libreville]]
>     |-
>     | {{flaggbild2|São Tomé och Príncipe}}
>     | align=&quot;right&quot; | 1 001
>     | align=&quot;right&quot; | 165 397
>     | align=&quot;right&quot; | 165,0
>     | [[São Tomé]]
>     |-
>     | colspan=5 style=&quot;background:#eee;&quot; | '''[[Nordafrika]]''':
>     |-
>     | {{flaggbild2|Algeriet}}
>     | align=&quot;right&quot; | 2 381 740
>     | align=&quot;right&quot; | 35 468 000
>     | align=&quot;right&quot; | 14,9
>     | [[Alger (stad)|Alger]]
>     |-
>     | {{flaggbild2|Egypten}}&lt;ref&gt;[[Egypten]] brukar anses ligga i två världsdelar, Nordafrika (enligt FN:s regionindelning) och Västasien; befolknings- och arealuppgifterna avser endast den afrikanska delen, väster om [[Suezkanalen]].&lt;/small&gt;&lt;br /&gt;&lt;/ref&gt;
>     | align=&quot;right&quot; | 1 001 450
>     | align=&quot;right&quot; | 81 121 000
>     | align=&quot;right&quot; | 81,0
>     | [[Kairo]]
>     |-
>     | {{flaggbild2|Libyen}}
>     | align=&quot;right&quot; | 1 759 540
>     | align=&quot;right&quot; | 6 355 112
>     | align=&quot;right&quot; | 3,6
>     | [[Tripoli]]
>     |-
>     | {{flaggbild2|Marocko}}
>     | align=&quot;right&quot; | 446 550
>     | align=&quot;right&quot; | 31 951 412
>     | align=&quot;right&quot; | 71,5
>     | [[Rabat]]
>     |-
>     | {{flaggbild2|Sudan}}
>     | align=&quot;right&quot; | 1 886 068
>     | align=&quot;right&quot; | 33 551 941
>     | align=&quot;right&quot; | 17,8
>     | [[Khartoum]]
>     |-
>     | {{flaggbild2|Tunisien}}
>     | align=&quot;right&quot; | 163 610
>     | align=&quot;right&quot; | 10 480 934
>     | align=&quot;right&quot; | 64,1
>     | [[Tunis]]
>     |-
>     | {{flaggbild2|Västsahara}} ([[Marocko]])&lt;ref&gt;[[Västsahara]] är ockuperat av [[Marocko]], som också har anspråk på området.&lt;/small&gt;&lt;br /&gt;&lt;/ref&gt;
>     | align=&quot;right&quot; | 266 000
>     | align=&quot;right&quot; | 530 500
>     | align=&quot;right&quot; | 2,0
>     | [[El Aaiún]]
>     |-
>     | colspan=5 | ''Områden som är likställda delar av europeiska stater'':
>     |-
>     | [[Fil:Flag of the Canary Islands.svg|22x20px|border|Kanarieöarna]] [[Kanarieöarna]] ([[Spanien]])&lt;ref&gt;De till [[Spanien]] hörande [[Kanarieöarna]] brukar geografiskt räknas till Nordafrika på grund av deras relativa närhet till [[Marocko]] och [[Västsahara]]; befolknings- och arealuppgifterna är från år 2001.&lt;br /&gt;&lt;/ref&gt;
>     | align=&quot;right&quot; | 7 492
>     | align=&quot;right&quot; | 2 117 519
>     | align=&quot;right&quot; | 282,6
>     | [[Las Palmas de Gran Canaria]],&lt;br /&gt;[[Santa Cruz de Tenerife]]
>     |-
>     | [[Fil:Flag Ceuta.svg|22x20px|border|Ceuta]] [[Ceuta]] ([[Spanien]])&lt;ref&gt;Den [[Spanien|spanska]] [[exklav]]en [[Ceuta]] omges på landsidan av Marocko i Nordafrika; befolknings- och arealuppgifterna är från år 2001.&lt;br /&gt;&lt;/ref&gt;
>     | align=&quot;right&quot; | 20
>     | align=&quot;right&quot; | 78 674
>     | align=&quot;right&quot; | 3933,7
>     | –
>     |-
>     | [[Fil:Flag of Madeira.svg|22x20px|Madeira]] [[Madeira]] ([[Portugal]])&lt;ref&gt;Det till [[Portugal]] hörande [[Madeira]] brukar geografiskt räknas till Nordafrika på grund av deras relativa närhet till [[Marocko]]; befolknings- och arealuppgifterna är från år 2001.&lt;br /&gt;&lt;/ref&gt;
>     | align=&quot;right&quot; | 797
>     | align=&quot;right&quot; | 267 302
>     | align=&quot;right&quot; | 335,4
>     | [[Funchal]]
>     |-
>     | [[Fil:Flag of Melilla.svg|22x20px|Melilla]] [[Melilla]] ([[Spanien]])&lt;ref&gt;Den [[Spanien|spanska]] [[exklav]]en [[Melilla]] omges på landsidan av Marocko i Nordafrika; befolknings- och arealuppgifterna är från år 2001.&lt;br /&gt;&lt;/ref&gt;
>     | align=&quot;right&quot; | 12
>     | align=&quot;right&quot; | 73 460
>     | align=&quot;right&quot; | 6121,6
>     | –
>     |-
>     | colspan=5 style=&quot;background:#eee;&quot; | '''[[Södra Afrika]]''':
>     |-
>     | {{flaggbild2|Botswana}}
>     | align=&quot;right&quot; | 600 370
>     | align=&quot;right&quot; | 2 006 945
>     | align=&quot;right&quot; | 3,3
>     | [[Gaborone]]
>     |-
>     | {{flaggbild2|Lesotho}}
>     | align=&quot;right&quot; | 30 355
>     | align=&quot;right&quot; | 2 171 318
>     | align=&quot;right&quot; | 71,5
>     | [[Maseru]]
>     |-
>     | {{flaggbild2|Namibia}}
>     | align=&quot;right&quot; | 825 418
>     | align=&quot;right&quot; | 2 283 289
>     | align=&quot;right&quot; | 2,7
>     | [[Windhoek]]
>     |-
>     | {{flaggbild2|Sydafrika}}
>     | align=&quot;right&quot; | 1 219 912
>     | align=&quot;right&quot; | 50 132 817
>     | align=&quot;right&quot; | 41,1
>     | [[Bloemfontein]], [[Kapstaden]], [[Pretoria]] (Tshwane)&lt;ref&gt;[[Bloemfontein]] är [[Sydafrika]]s judiciella huvudstad, medan [[Kapstaden]] är säte för den lagstiftande församlingen och [[Pretoria]] är administrativ huvudstad. Staden Pretoria har inlett en process för att eventuellt byta namn till Tshwane.&lt;br /&gt;&lt;/ref&gt;
>     |-
>     | {{flaggbild2|Swaziland}}
>     | align=&quot;right&quot; | 17 363
>     | align=&quot;right&quot; | 1 186 056
>     | align=&quot;right&quot; | 68,3
>     | [[Mbabane]]
>     |-
>     | colspan=5 style=&quot;background:#eee;&quot; | '''[[Västafrika]]''':
>     |-
>     | {{flaggbild2|Benin}}
>     | align=&quot;right&quot; | 112 620
>     | align=&quot;right&quot; | 8 849 892
>     | align=&quot;right&quot; | 78,6
>     | [[Porto-Novo]]
>     |-
>     | {{flaggbild2|Burkina Faso}}
>     | align=&quot;right&quot; | 274 200
>     | align=&quot;right&quot; | 16 468 714
>     | align=&quot;right&quot; | 60,1
>     | [[Ouagadougou]]
>     |-
>     | {{flaggbild2|Kap Verde}}
>     | align=&quot;right&quot; | 4 033
>     | align=&quot;right&quot; | 495 999
>     | align=&quot;right&quot; | 123,0
>     | [[Praia]]
>     |-
>     | {{flaggbild2|Elfenbenskusten}}
>     | align=&quot;right&quot; | 322 460
>     | align=&quot;right&quot; | 19 737 800
>     | align=&quot;right&quot; | 61,2
>     | [[Abidjan]], [[Yamoussoukro]]&lt;ref&gt;[[Yamoussoukro]] är [[Elfenbenskusten]]s officiella huvudstad, men [[Abidjan]] fungerar ''[[de facto]]'' som huvudstad.&lt;br /&gt;&lt;/ref&gt;
>     |-
>     | {{flaggbild2|Gambia}}
>     | align=&quot;right&quot; | 11 300
>     | align=&quot;right&quot; | 1 728 394
>     | align=&quot;right&quot; | 152,9
>     | [[Banjul]]
>     |-
>     | {{flaggbild2|Ghana}}
>     | align=&quot;right&quot; | 239 460
>     | align=&quot;right&quot; | 24 391 823
>     | align=&quot;right&quot; | 101,9
>     | [[Accra]]
>     |-
>     | {{flaggbild2|Guinea}}
>     | align=&quot;right&quot; | 245 857
>     | align=&quot;right&quot; | 9 981 590
>     | align=&quot;right&quot; | 40,6
>     | [[Conakry]]
>     |-
>     | {{flaggbild2|Guinea-Bissau}}
>     | align=&quot;right&quot; | 36 120
>     | align=&quot;right&quot; | 1 515 224
>     | align=&quot;right&quot; | 41,9
>     | [[Bissau]]
>     |-
>     | {{flaggbild2|Liberia}}
>     | align=&quot;right&quot; | 111 370
>     | align=&quot;right&quot; | 3 994 122
>     | align=&quot;right&quot; | 35,9
>     | [[Monrovia]]
>     |-
>     | {{flaggbild2|Mali}}
>     | align=&quot;right&quot; | 1 240 000
>     | align=&quot;right&quot; | 15 369 809
>     | align=&quot;right&quot; | 9,1
>     | [[Bamako]]
>     |-
>     | {{flaggbild2|Mauretanien}}
>     | align=&quot;right&quot; | 1 030 700
>     | align=&quot;right&quot; | 3 459 773
>     | align=&quot;right&quot; | 3,4
>     | [[Nouakchott]]
>     |-
>     | {{flaggbild2|Niger}}
>     | align=&quot;right&quot; | 1 267 000
>     | align=&quot;right&quot; | 15 511 953
>     | align=&quot;right&quot; | 12,2
>     | [[Niamey]]
>     |-
>     | {{flaggbild2|Nigeria}}
>     | align=&quot;right&quot; | 923 768
>     | align=&quot;right&quot; | 158 423 182
>     | align=&quot;right&quot; | 171,5
>     | [[Abuja]]
>     |-
>     | [[Fil:Flag of Saint Helena.svg|22x20px|Sankta Helena]] [[Sankta Helena (ö)|Sankta Helena]] ([[Storbritannien]])
>     | align=&quot;right&quot; | 410
>     | align=&quot;right&quot; | 4 118
>     | align=&quot;right&quot; | 10,0
>     | [[Jamestown, Sankta Helena|Jamestown]]
>     |-
>     | {{flaggbild2|Senegal}}
>     | align=&quot;right&quot; | 196 190
>     | align=&quot;right&quot; | 12 433 728
>     | align=&quot;right&quot; | 63,4
>     | [[Dakar]]
>     |-
>     | {{flaggbild2|Sierra Leone}}
>     | align=&quot;right&quot; | 71 740
>     | align=&quot;right&quot; | 5 867 536
>     | align=&quot;right&quot; | 81,8
>     | [[Freetown]]
>     |-
>     | {{flaggbild2|Togo}}
>     | align=&quot;right&quot; | 56 785
>     | align=&quot;right&quot; | 6 027 798
>     | align=&quot;right&quot; | 106,1
>     | [[Lomé]]
>     |-
>     | colspan=5 style=&quot;background:#eee;&quot; | '''[[Östafrika]]''':
>     |-
>     | {{flaggbild2|Burundi}}
>     | align=&quot;right&quot; | 27 830
>     | align=&quot;right&quot; | 8 382 849
>     | align=&quot;right&quot; | 301,2
>     | [[Bujumbura]]
>     |-
>     | {{flaggbild2|Komorerna}}
>     | align=&quot;right&quot; | 2 170
>     | align=&quot;right&quot; | 734 750
>     | align=&quot;right&quot; | 338,6
>     | [[Moroni]]
>     |-
>     | {{flaggbild2|Djibouti}}
>     | align=&quot;right&quot; | 23 000
>     | align=&quot;right&quot; | 888 716
>     | align=&quot;right&quot; | 38,6
>     | [[Djibouti (stad)|Djibouti]]
>     |-
>     | {{flaggbild2|Eritrea}}
>     | align=&quot;right&quot; | 121 320
>     | align=&quot;right&quot; | 5 253 676
>     | align=&quot;right&quot; | 43,3
>     | [[Asmara]]
>     |-
>     | {{flaggbild2|Etiopien}}
>     | align=&quot;right&quot; | 1 127 127
>     | align=&quot;right&quot; | 82 949 541
>     | align=&quot;right&quot; | 73,6
>     | [[Addis Abeba]]
>     |-
>     | {{flaggbild2|Kenya}}
>     | align=&quot;right&quot; | 582 650
>     | align=&quot;right&quot; | 40 512 682
>     | align=&quot;right&quot; | 69,5
>     | [[Nairobi]]
>     |-
>     |{{flaggbild2|Madagaskar}}
>     | align=&quot;right&quot; | 587 040
>     | align=&quot;right&quot; | 20 713 819
>     | align=&quot;right&quot; | 35,3
>     | [[Antananarivo]]
>     |-
>     | {{flaggbild2|Malawi}}
>     | align=&quot;right&quot; | 118 480
>     | align=&quot;right&quot; | 14 900 841
>     | align=&quot;right&quot; | 90,3
>     | [[Lilongwe]]
>     |-
>     | {{flaggbild2|Mauritius}}
>     | align=&quot;right&quot; | 2 040
>     | align=&quot;right&quot; | 1 299 172
>     | align=&quot;right&quot; | 636,8
>     | [[Port Louis]]
>     |-
>     | {{flaggbild|Frankrike}} [[Mayotte]] ([[Frankrike]])
>     | align=&quot;right&quot; | 374
>     | align=&quot;right&quot; | 204 114
>     | align=&quot;right&quot; | 545,8
>     | [[Mamoudzou]]
>     |-
>     | {{flaggbild|Moçambique}} [[Moçambique]]
>     | align=&quot;right&quot; | 801 590
>     | align=&quot;right&quot; | 23 390 765
>     | align=&quot;right&quot; | 29,2
>     | [[Maputo]]
>     |-
>     | {{flaggbild|Frankrike}} [[Réunion]] ([[Frankrike]])
>     | align=&quot;right&quot; | 2 512
>     | align=&quot;right&quot; | 846 068
>     | align=&quot;right&quot; | 336,8
>     | [[Saint-Denis (Réunion)|Saint-Denis]]
>     |-
>     | {{flaggbild2|Rwanda}}
>     | align=&quot;right&quot; | 26 338
>     | align=&quot;right&quot; | 10 624 005
>     | align=&quot;right&quot; | 403,4
>     | [[Kigali]]
>     |-
>     | {{flaggbild2|Seychellerna}}
>     | align=&quot;right&quot; | 455
>     | align=&quot;right&quot; | 86 518
>     | align=&quot;right&quot; | 190,1
>     | [[Victoria, Seychellerna|Victoria]]
>     |-
>     | {{flaggbild2|Somalia}}
>     | align=&quot;right&quot; | 637 657
>     | align=&quot;right&quot; | 9 330 872
>     | align=&quot;right&quot; | 14,6
>     | [[Mogadishu]]
>     |-
>     | {{flaggbild2|Sydsudan}}
>     | align=&quot;right&quot; | 633 783
>     | align=&quot;right&quot; | 8 207 607
>     | align=&quot;right&quot; | 13
>     | [[Juba]]
>     |-
>     | {{flaggbild2|Tanzania}}
>     | align=&quot;right&quot; | 945 087
>     | align=&quot;right&quot; | 44 841 226
>     | align=&quot;right&quot; | 47,4
>     | [[Dodoma]]
>     |-
>     | {{flaggbild2|Uganda}}
>     | align=&quot;right&quot; | 236 040
>     | align=&quot;right&quot; | 33 424 683
>     | align=&quot;right&quot; | 141,6
>     | [[Kampala]]
>     |-
>     | {{flaggbild2|Zambia}}
>     | align=&quot;right&quot; | 752 614
>     | align=&quot;right&quot; | 13 088 570
>     | align=&quot;right&quot; | 17,4
>     | [[Lusaka]]
>     |-
>     | {{flaggbild2|Zimbabwe}}
>     | align=&quot;right&quot; | 390 580
>     | align=&quot;right&quot; | 12 571 454
>     | align=&quot;right&quot; | 32,2
>     | [[Harare]]
>     |- style=&quot; font-weight:bold; &quot;
>     | Total
>     | align=&quot;right&quot; | 30&amp;nbsp;305&amp;nbsp;053
>     | align=&quot;right&quot; | 1&amp;nbsp;022&amp;nbsp;234&amp;nbsp;400
>     | align=&quot;right&quot; | 33,7
>     |}
>
>     == Se även ==
>     * [[Nordafrika]]
>     * [[Västafrika]]
>     * [[Östafrika]]
>     * [[Centralafrika]]
>     * [[Södra Afrika]]
>     * [[Afrikas klimat]]
>
>     == Referenser ==
>
>     === Noter ===
>     &lt;references&gt;
>     &lt;ref name=&quot;esa.un.org&quot;&gt;[http://esa.un.org/unpd/wpp/Excel-Data/population.htm/ &quot;World Population Prospects: The 2010 Revision&quot;] {{Wayback|url=http://esa.un.org/unpd/wpp/Excel-Data/population.htm/ |date=20120606170530 }} [[Förenta nationerna|United Nations]] (Department of Economic and Social Affairs, population division)&lt;/ref&gt;
>     &lt;ref name=&quot;esa.un.org1&quot;&gt;[http://esa.un.org/unpd/wpp/Excel-Data/population.htm/ &quot;World Population Prospects: The 2010 Revision&quot;] {{Wayback|url=http://esa.un.org/unpd/wpp/Excel-Data/population.htm/ |date=20120606170530 }} [[United Nations]] (Department of Economic and Social Affairs, population division)&lt;/ref&gt;
>     &lt;/references&gt;
>
>     === Källor ===
>     {{enwp|url=https://en.wikipedia.org/w/index.php?title=Africa&amp;oldid=81702686|artikel=Africa}}
>     * [https://www.courses.psu.edu/test/test100_hkr/AFIM/Main_HTML/Intro_1.html Pennsylvania State University – ''African Ethnic Groups'' och andra kartor]
>     * [http://www.pygmies.info/ Pygméerna i Afrika] Kultur och musik av de första urinvånarna i Afrika
>     * Afrika i skolböckerna, M. Palmberg 1987
>
>     == Externa länkar ==
>     {{commonscat|Africa}}
>     {{Wikinews|Afrika}}
>     {{wikiatlas|Africa}}
>
>     {{Afrika}}
>     {{Världens regioner}}
>     {{auktoritetsdata}}
>
>     [[Kategori:Afrika| ]]
>     [[Kategori:Wikipedia:Basartiklar]]</text>
>           <sha1>1ueywtz2sm89v81ujno96unvkkmxpoa</sha1>
>         </revision>
>       </page>, <page>
>         <title>Amerika</title>
>         <ns>0</ns>
>         <id>3</id>
>         <revision>
>           <id>48343172</id>
>           <parentid>48187017</parentid>
>           <timestamp>2020-10-11T13:36:09Z</timestamp>
>           <contributor>
>             <username>Olympicbandy</username>
>             <id>564328</id>
>           </contributor>
>           <model>wikitext</model>
>           <format>text/x-wiki</format>
>           <text bytes="2728" xml:space="preserve">{{för|Amerika som kortnamn på '''Amerikas förenta stater'''|USA|andra betydelser|Amerika (olika betydelser)}}
>     [[Fil:Americas (orthographic projection).svg|miniatyr|250px|Karta som visar Nord- och Sydamerika, tillsammans benämnda Amerika]]
>     '''Amerika''' (engelska: ''the Americas'') består av landmassorna mellan [[Atlanten]] och [[Stilla havet]] på [[västra halvklotet]], namngivna efter den italienske upptäcktsresanden [[Amerigo Vespucci]]. Amerika uppfattas vanligen idag som två världsdelar, [[Nordamerika]] och [[Sydamerika]].&lt;ref&gt;[https://snl.no/Amerika Store norske leksikon/ Amerika]&lt;/ref&gt; Amerika indelas geografiskt i Nordamerika (med [[Centralamerika]]) och Sydamerika. [[Grönland]] ingår geografiskt i Nordamerika, men räknas politiskt och kulturellt till Europa. Kulturgeografiskt avgränsas också ofta [[Latinamerika]], då inklusive både Central- och Sydamerika...

  

Before we begin filtering the contents of pages, we want to discard 1.
Pages that are redirects, i.e., pages containing a
`<redirect_title="...">` tag 1. Non-articles, i.e., pages not in
namespace 0 (not containing `<ns>0</ns>`) 1. Any page containing the
macro tag stating that it was generated by a robot.

In [ ]:
val redirection = "<redirect title=".r
def is_not_redirect(source : String) = { redirection.findFirstIn(source) == None }

val ns0 = "<ns>0</ns>".r
def is_article(source : String) = { ns0.findFirstIn(source) != None }

val autogenerated = "robotskapad|Robotskapad".r
def is_not_autogenerated(source : String) = { autogenerated.findFirstIn(source) == None }

val articles = dump.filter(is_not_redirect).filter(is_article).filter(is_not_autogenerated)

val n_articles = articles.count()
articles.take(5)

  

>     redirection: scala.util.matching.Regex = <redirect title=
>     is_not_redirect: (source: String)Boolean
>     ns0: scala.util.matching.Regex = <ns>0</ns>
>     is_article: (source: String)Boolean
>     autogenerated: scala.util.matching.Regex = robotskapad|Robotskapad
>     is_not_autogenerated: (source: String)Boolean
>     articles: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[1010] at filter at command-2629358840275842:10
>     n_articles: Long = 813245
>     res9: Array[String] =
>     Array(<page>
>         <title>Amager</title>
>         <ns>0</ns>
>         <id>1</id>
>         <revision>
>           <id>48450724</id>
>           <parentid>48155628</parentid>
>           <timestamp>2020-11-15T15:50:09Z</timestamp>
>           <contributor>
>             <username>Nalle&amp;Lisa</username>
>             <id>348723</id>
>           </contributor>
>           <comment>Länkfix</comment>
>           <model>wikitext</model>
>           <format>text/x-wiki</format>
>           <text bytes="7485" xml:space="preserve">{{källor|datum=2013-05}}
>
>     {{Geobox|Island
>     &lt;!-- *** Mallhuvud *** --&gt;
>     | name                  = Amager
>     | native_name           =
>     | other_name            =
>     | category              = Ö
>     &lt;!-- *** Namn **** --&gt;
>     | etymology             =
>     | nickname              =
>     &lt;!-- *** Bild *** --&gt;
>     | image                 = Amager.jpg
>     | image_caption         =
>     &lt;!-- *** Länder, regioner mm *** --&gt;
>     | country               = Danmark
>     | country_flag          = true
>     | state                 = | state_type =
>     | region                = [[Region Hovedstaden]]| region_type =
>     | district              = | district_type =
>     | municipality          = [[Dragørs kommun]]| municipality_type =
>     | municipality1         = [[Köpenhamns kommun]]
>     | municipality2         = [[Tårnby kommun]]
>     &lt;!-- *** Tillhörighet, föremål inom området mm *** --&gt;
>     | parent                =
>     | range                 = | range_type = Arkipelag
>     | border                =
>     | part                  =
>     | city                  =
>     | capital               =
>     | landmark              =
>     | river                 =
>     &lt;!-- *** Läge *** --&gt;
>     | location              = [[Öresund]]
>     | lat_d                 = 55| lat_m = 37| lat_s = 0| lat_NS = N
>     | long_d                = 12| long_m = 37| long_s = 0| long_EW = E
>     | highest               =
>     | highest_location      = | highest_region = | highest_country =
>     | highest_elevation     =
>     | highest_lat_d         = | highest_lat_m = | highest_lat_s = | highest_lat_NS =
>     | highest_long_d        = | highest_long_m = | highest_long_s = | highest_long_EW =
>     &lt;!-- *** Dimensioner *** --&gt;
>     | length                = 16.1| length_orientation =
>     | width                 = 11.1| width_orientation =
>     | height                = 4
>     | area                  = 96.29
>     &lt;!-- *** Befolkning *** --&gt;
>     | population            = 196047| population_date = 1 januari 2018 | population_note =
>     | population_density    = auto| population_density_note =
>     &lt;!-- *** Tillgänglighet *** --&gt;
>     | public                =
>     | access                =
>     &lt;!-- *** Koder ***  --&gt;
>     | timezone              = CET| utc_offset = +1
>     | timezone_DST          = CEST| utc_offset_DST = +2
>     | postal_code           =
>     | area_code             =
>     | code                  =
>     &lt;!-- *** Fria fält *** --&gt;
>     | free                  =
>     | free_type             =
>     &lt;!-- *** Kartor *** --&gt;
>     | map                   = Denmark physical map.svg
>     | map_caption           = Amagers läge i Danmark.
>     | map_locator           = Danmark
>     | map_locator_x         =
>     | map_locator_y         =
>     &lt;!-- *** Fotnoter *** --&gt;
>     | footnotes             =
>     }}
>
>     '''Amager''' är en [[Danmark|dansk]] [[ö (landområde)|ö]] i [[Öresund]]. Öns norra och västra delar tillhör [[Köpenhamn]], medan övriga delar upptas av [[Tårnby kommun]] och [[Dragørs kommun]].
>     Amager har en yta på 96,29 km² och befolkningen uppgår till 196&amp;nbsp;047 personer (1/1 2018). På den östra delen av ön finns [[Kastrups flygplats]]. Amager har också flera militära anläggningar (t.ex. Kastrup fort och Dragör fort), strandparksområdena [[Amager Strandpark]] och Kastrup Strandpark, flera restauranger i världsklass (t.ex. Noma), ett stort naturområde ([[Amager Fælled]]), ett av Nordens största köpcentra (Field's), och mycket annat. Amager förkortas ofta Ama'r för att återspegla det danska uttalet.
>
>     Amager är delvis en [[konstgjord ö]], delvis en naturlig sådan. T.ex. så anlades en konstgjord ö i anslutning till det redan existerande Amager Strandpark, när denna rustades upp väsentligen 2005. Ön är mycket låg och vissa delar ligger under havsytan, framför allt det genom fördämning och utfyllnad skapade området [[Kalvebod Fælled]] (Vestamager). Ön började uppodlas i större skala under 1500-talets första hälft då kung [[Kristian II]] bjöd in nederländska bönder till att odla upp ön för kronans räkning. Flertalet slog sig ned i fiskeläget [[Dragør]] där många än idag har namn med nederländskt ursprung som Neels och Tønnes.
>
>     Den stora utvecklingsfasen började på 1800-talet då flera fabriker, bostadsområden och militär verksamhet utvecklade sig på ön. Det fanns ett antal stora och kända fabriker på ön och den industriella prägeln höll sig väldigt tydlig långt in på 1900-talet. Det förekommer också områden som tidigare varit sommarstugeområden och där det än idag finns kolonilottområden, t.ex. i Sundbyöster ut mot kusten. Under 1900-talet började militären att utveckla stora övningsområden på ön vilket resulterade i Vestamager som idag innefattar ett stort naturområde endast några kilometer från Köpenhamns centrum. Under senare delen av 1900-talet har flygplatsen [[Kastrups flygplats|Kastrup]] som ligger på ön expanderat kraftigt och i samband med byggandet av [[Öresundsförbindelsen]] började även ett stort område byggas som kallas för [[Ørestad]].
>
>     Ön, som är förbunden med övriga Köpenhamn genom flera broar, [[järnväg]] och sedan 2017 även [[Köpenhamns tunnelbana|Metro]], har under de senaste 10-20 åren förändrats mycket. Dels har flera områden exploaterats med nya moderna kvarter lägenheter, t.ex. i områdena Örestad, [[Islands Brygge]] och Amager Strand, och dels så har Amagers närhet till Köpenhamn raderat dess tidigare rykte som en industriell förstad till Köpenhamn. Idag finns det områden av alla sorter på ön - tack vare flygplatsen finns ett aktivt näringsliv kvar, samtidigt som de nya områdena och inte minst upprustningen av t.ex. Amager Strand har förändrat bilden av ön. Idag är flera delar av ön mycket attraktiva bostadsområden.
>
>     Ön återfick i samband med Öresundsbron även en järnvägsförbindelse vilket inte funnits sedan 1960-talet. En [[motorväg]] byggdes samtidigt över öns mellersta del. [[Köpenhamns metro]] går till Kastrup och Vestamager med ett antal stationer på ön. Kring metro- och regionaltågsstationen Ørestad har ett stort kontors-, handels- och bostadsområde uppförts under [[2000-talet (decennium)|00-talet]]. Detta sträcker sig från [[Bella Center]] i norr till metrostationen [[Vestamager station|Vestamager]] i söder. Även om området inte ligger vid kusten har Ørestad delvis fått karaktären av en hamnstad genom anläggandet av konstgjorda kanaler och dammar. Vid [[Islands Brygge]] finns en &quot;äkta&quot; sjöstad.
>
>     Före [[Öresundsförbindelsen]]s uppförande hade småstaden [[Dragør]] en färjeförbindelse med [[Limhamn]] i södra [[Malmö]]. På grund av denna fanns i Dragør tidigare en betydande gränshandel. Denna har under 2000-talet helt upphört liksom den genomgående trafiken.
>
>     Öns västra del, som utvanns från Öresund under 1920- och 30-talen, är med undantag för Bella Center, Ørestad samt ett koloniområde, avsatt som natur- och rekreationsområde. Marken är ängsmark (odlad mark finns dock på den naturliga delen av öns södra del).
>     Längst i söder finns den flerhundraåriga skogen [[Kongelunden]]. På öns östra del, vid Öresundskusten, finns [[Amager Strandpark]] med en populär sandstrand. Området har omgestaltats, med en [[konstgjord ö]] och en [[lagun]] innanför. Nyinvigningen av parken, som funnits sedan 1934, ägde rum 2005.
>
>     Amager är den tätast befolkade ön i Danmark. Den näst mest tätbefolkade är [[Thurø]].
>
>     == Se även ==
>     *[[Amagerbro]]
>     *[[Amagerport]]
>     *[[Amagerbroderi]]
>
>     ==Externa länkar==
>     {{commonscat|Amager}}
>
>     {{Öar i Danmark}}
>
>     {{auktoritetsdata}}
>
>     [[Kategori:Amager| ]]
>     [[Kategori:Öar i Region Hovedstaden]]</text>
>           <sha1>qom8qpk70qa0alvfb4b4ha9or4ftohh</sha1>
>         </revision>
>       </page>, <page>
>         <title>Afrika</title>
>         <ns>0</ns>
>         <id>2</id>
>         <revision>
>           <id>48459142</id>
>           <parentid>48374249</parentid>
>           <timestamp>2020-11-18T06:30:03Z</timestamp>
>           <contributor>
>             <username>LarskeBot</username>
>             <id>310602</id>
>           </contributor>
>           <minor />
>           <comment>/* Geografi */mellan X - Y → mellan X och Y med [[Project:AWB|AWB]]</comment>
>           <model>wikitext</model>
>           <format>text/x-wiki</format>
>           <text bytes="37806" xml:space="preserve">{{templateStyles-tagg Faktamall}}
>     {| class=&quot;infobox&quot; style=&quot;font-size:90%;&quot; width=&quot;300&quot;
>     |+ &lt;big&gt;'''Afrika'''&lt;/big&gt;
>     |-
>     | align=&quot;center&quot; colspan=&quot;2&quot; style=&quot;background:#f9f9f9;&quot; | [[Fil:Africa (orthographic projection).svg|280px|Afrika i världen.]]
>     ----
>     |-
>     | align=&quot;center&quot; colspan=&quot;2&quot; style=&quot;background:#f9f9f9;&quot; | [[Fil:Afrika politisk.png|280px|Politisk karta.]]
>     ----
>     |-
>     | colspan=&quot;2&quot; |
>     |-
>     | '''Area:''' &lt;br /&gt; '''Folkmängd:''' &lt;br /&gt; '''Antal stater:''' &lt;br /&gt; '''Autonoma regioner:''' &lt;br /&gt; '''Högsta punkt:''' &lt;br /&gt; '''Lägsta punkt:''' || 30&amp;nbsp;221&amp;nbsp;532&amp;nbsp;km² &lt;br /&gt; 1&amp;nbsp;022&amp;nbsp;234&amp;nbsp;000&lt;ref name=&quot;esa.un.org&quot;/&gt; (2010) &lt;br /&gt; 54 &lt;br /&gt; 8 &lt;br /&gt; [[Kilimanjaro]]&amp;nbsp;5&amp;nbsp;895&amp;nbsp;[[m ö.h.|m&amp;nbsp;ö.h.]] &lt;br /&gt; [[Assalsjön]]&amp;nbsp;156&amp;nbsp;[[Meter över havet|m&amp;nbsp;u.h.]]
>     |-
>     | colspan=&quot;2&quot; |
>     ----
>     |-
>     | colspan=&quot;2&quot; | '''Stater'''
>     |-
>     | {{flaggbild2|Algeriet}} &lt;br /&gt; {{flaggbild2|Angola}} &lt;br /&gt; {{flaggbild2|Benin}} &lt;br /&gt; {{flaggbild2|Botswana}} &lt;br /&gt; {{flaggbild2|Burkina Faso}} &lt;br /&gt; {{flaggbild2|Burundi}} &lt;br /&gt; {{flaggbild2|Centralafrikanska republiken}} &lt;br /&gt; {{flaggbild2|Djibouti}} &lt;br /&gt; {{flaggbild2|Egypten}} &lt;br /&gt; {{flaggbild2|Ekvatorialguinea}} &lt;br /&gt; {{flaggbild2|Elfenbenskusten}} &lt;br /&gt; {{flaggbild2|Eritrea}} &lt;br /&gt; {{flaggbild2|Etiopien}} &lt;br /&gt; {{flaggbild2|Gabon}} &lt;br /&gt; {{flaggbild2|Gambia}} &lt;br /&gt; {{flaggbild2|Ghana}} &lt;br /&gt; {{flaggbild2|Guinea}} &lt;br /&gt; {{flaggbild2|Guinea-Bissau}} &lt;br /&gt; {{flaggbild2|Kamerun}} &lt;br /&gt; {{flaggbild2|Kap Verde}} &lt;br /&gt; {{flaggbild2|Kenya}} &lt;br /&gt; {{flaggbild2|Komorerna}} &lt;br /&gt; {{flaggbild2|Kongo-Brazzaville}} &lt;br /&gt; {{flaggbild2|Kongo-Kinshasa}} &lt;br /&gt; {{flaggbild2|Lesotho}} &lt;br /&gt; {{flaggbild2|Liberia}} &lt;br /&gt;{{flaggbild2|Libyen}} ||{{flaggbild2|Madagaskar}} &lt;br /&gt; {{flaggbild2|Malawi}} &lt;br /&gt; {{flaggbild2|Mali}} &lt;br /&gt; {{flaggbild2|Marocko}} &lt;br /&gt; {{flaggbild2|Mauretanien}} &lt;br /&gt; {{flaggbild2|Mauritius}} &lt;br /&gt; {{flaggbild2|Moçambique}} &lt;br /&gt; {{flaggbild2|Namibia}} &lt;br /&gt; {{flaggbild2|Niger}} &lt;br /&gt; {{flaggbild2|Nigeria}} &lt;br /&gt; {{flaggbild2|Rwanda}} &lt;br /&gt; {{flaggbild2|São Tomé och Príncipe}} &lt;br /&gt; {{flaggbild2|Senegal}} &lt;br /&gt; {{flaggbild2|Seychellerna}} &lt;br /&gt; {{flaggbild2|Sierra Leone}} &lt;br /&gt; {{flaggbild2|Somalia}} &lt;br /&gt; {{flaggbild2|Sudan}} &lt;br /&gt; {{flaggbild2|Sydafrika}} &lt;br /&gt; {{flaggbild2|Sydsudan}} &lt;br /&gt; {{flaggbild2|Swaziland}} &lt;br /&gt; {{flaggbild2|Tanzania}} &lt;br /&gt; {{flaggbild2|Tchad}} &lt;br /&gt; {{flaggbild2|Togo}} &lt;br /&gt; {{flaggbild2|Tunisien}} &lt;br /&gt; {{flaggbild2|Uganda}} &lt;br /&gt; {{flaggbild2|Zambia}} &lt;br /&gt; {{flaggbild2|Zimbabwe}}
>     |-
>     | colspan=&quot;2&quot; |
>     ----
>     '''Stater som har delar av ytan i Afrika'''
>     |-
>     | {{flaggbild2|Egypten}} &lt;br /&gt; {{flaggbild2|Frankrike}} &lt;br /&gt; {{flaggbild2|Jemen}} &lt;br /&gt; {{flaggbild2|Portugal}} &lt;br /&gt; {{flaggbild2|Spanien}}
>     |-
>     | colspan=&quot;2&quot; |
>     ----
>     '''Särskilda områden'''
>     &lt;br /&gt; {{flaggbild|Ceuta}} [[Ceuta]] ([[Spanien]])
>
>     {{flaggbild|Kanarieöarna}} [[Kanarieöarna]] ([[Spanien]])
>
>     {{flaggbild|Madeira}} [[Madeira]] ([[Portugal]])
>
>     {{flaggbild|Frankrike}} [[Mayotte]] ([[Frankrike]])
>
>     {{flaggbild|Melilla}} [[Melilla]] ([[Spanien]])
>
>     {{flaggbild|Frankrike}} [[Réunion]] ([[Frankrike]])
>
>     {{flaggbild|Sankta Helena}} [[Sankta Helena]] ([[Storbritannien]])
>
>     [[Fil:Flag of Somaliland.svg|22px|Somaliland]] [[Somaliland]]
>
>     {{flaggbild2|Västsahara}} ([[Marocko]])
>     ----
>     '''Största städer (över 10 miljoner invånare med förorter):'''
>     &lt;br /&gt;{{flaggbild|Egypten}} [[Kairo]]
>     &lt;br /&gt;{{flaggbild|Kongo-Kinshasa}} [[Kinshasa]]
>     &lt;br /&gt;{{flaggbild|Nigeria}} [[Lagos]]
>     ----
>     [[Fil:Africa satellite.jpg|280px|Satellitfoto]]
>     |}
>     '''Afrika''' är [[jorden]]s näst största [[kontinent]] (efter [[Eurasien]]) och även jordens näst största världsdel efter [[Asien]], både vad gäller [[areal]] och [[folkmängd]]. Med världsdelens öar inräknade mäter Afrika 30 244 050&amp;nbsp;km², vilket motsvarar 20,3 % av jordens landmassa eller cirka 6 % av jordens totala area. Omkring 22 miljoner km² av dessa ligger i [[tropikerna]] vilket gör den afrikanska kontinenten till världens varmaste kontinent. I Afrika bor det ungefär 1 miljard människor i 55 länder – en sjundedel av jordens befolkning. Dess längd i nordlig-sydlig riktning är omkring 8 000&amp;nbsp;km och dess största bredd omkring 7&amp;nbsp;800&amp;nbsp;km.
>
>     == Etymologi ==
>     Ordet &quot;Afrika&quot;s [[etymologi]]ska bakgrund är dunkel, och ordet har dessutom haft olika betydelse under olika [[epok]]er.
>
>     I äldre tider kallades allt land väster om [[Egypten]] för &quot;[[Libyen]]&quot;. Enligt den [[Grekland|grek]]iska historieskrivaren [[Herodotos]] berättelse seglade [[fenicisk]]a sjömän på uppdrag av den egyptiska kungen Neko runt Libyen, det vill säga [[kontinent]]en Afrika redan omkring 600 f.Kr.
>
>     När [[Romerska riket|romarna]] först använde termen ''[[Africa terra]]'' avsåg de före [[andra puniska kriget]] endast trakten kring [[Karthago]], det vill säga motsvarande ungefär dagens Tunisien där själva staden Karthago låg samt stora delar av Libyen och Algeriet. Romarna uppfattade först [[Egypten]] som en del av [[Asien]], och det var först geografen [[Klaudios Ptolemaios|Ptolemaios]] som drog gränsen vid [[Suez]]. Under kejsartiden kom ''Africa'' att avse hela det området romarna kände till av kontinenten, och allt eftersom européerna har upptäckt mer och mer av kontinenten har termen avsett ett allt större område. Namnet ska ha avsett &quot;landet där ''afri''&quot; (singularis: ''afer'') bodde. ''Afer'' avsåg namnet på ett folk som kan motsvara dagens [[berber]]. [[Viking]]arna under 800- och 900-talen kallade norra Afrika för &quot;Blåland&quot;.
>
>     == Geografi ==
>     {{Fördjupning|Afrikas geografi}}
>     Geologiskt är Afrika en mycket gammal kontinent, i huvudsak uppbyggd av [[urberg]]. Under jordens forntid var den en del av jättekontinenten [[Gondwana]]land. Denna gled isär under [[Jura (period)|juraperioden]] och [[Krita (geologi)|kritaperioden]] för 100-150 miljoner år sedan, och så fick Afrika sin nuvarande form.
>
>     Inom sina regelbundna kuster rymmer den en total yta på omkring 30 244 050&amp;nbsp;km² inklusive öarna. Afrika åtskiljs från [[Europa]] av [[Medelhavet]] och förenas med [[Asien]] i sitt nordöstra hörn genom [[näs]]et vid [[Suez]], omkring 130&amp;nbsp;km i bredd. Omkring 8 000&amp;nbsp;km skiljer den nordligaste punkten, [[Ras ben Sakka]] i [[Tunisien]], från dess sydligaste, [[Kap Agulhas]] i [[Sydafrika]]. Från [[Kap Verde]], dess västligaste punkt, till [[Ras Hafun]] i [[Somalia]] är det omkring 7 400&amp;nbsp;km. Afrikas kuster mäter omkring 28 000&amp;nbsp;km (1&amp;nbsp;km kust på 1 057&amp;nbsp;km²) varav [[Medelhavet]] står för 5 400&amp;nbsp;km, [[Atlanten]] 11 000&amp;nbsp;km, [[Indiska oceanen]] 8 700&amp;nbsp;km och [[Röda havet]] 2 900&amp;nbsp;km, och avsaknaden av djupare [[bukt]]er och betydande [[halvö]]ar blir uppenbar om man jämför med Europas 9 700 000&amp;nbsp;km² som omgärdas av 32 000&amp;nbsp;km kust (1&amp;nbsp;km kust på 278&amp;nbsp;km² land), eller för det [[fjord]]rika [[Norge]], där motsvarande siffra 385 000&amp;nbsp;km² land och en kuststräcka på 20 000&amp;nbsp;km (1&amp;nbsp;km kust per 19&amp;nbsp;km² land).
>
>     Afrikas genomsnittliga höjd över havet är omkring 600 m, vilket motsvarar de båda amerikanska kontinenternas höjd men är lägre än den asiatiska på omkring 950 m. Jämfört med de andra kontinenterna utmärker sig Afrika med en liten andel landområden på mycket hög respektive mycket låg höjd. Landområdena under 180 [[Meter över havet|m ö.h.]] upptar en osedvanligt liten del av den totala landmassan, och även de högsta delarna, som är betydligt lägre än motsvarande i Asien, utgörs av isolerade berg och bergskedjor. Generellt ligger de högre [[platå]]erna i öst och söder och de lägre delarna åt väst och norr och med en linje från ungefär vid [[Röda havet]]s mitt till [[Kongofloden]]s mynning kan kontinenten delas upp i ett lågland i norr och ett högland i söder. Man kan således dela upp Afrika i fyra övergripande landområden:
>     # Slättlanden vid kusterna – ofta med [[Mangrove|mangroveträsk]] insprängda – når aldrig långt in från kusterna utom vid vattendragens nedre delar. Slamrika slättland finns egentligen bara vid de större flodernas deltan, och annars utgör kustområdena det lägsta steget i en serie terrasser som bildar stigningen upp till de inre platåerna.
>     # [[Atlasbergen]] i norr som ligger isolerat av [[Sahara]]öknen som bitvis ligger under havsnivån.
>     # De södra och östra höglanden som ligger på mellan 600 och 1 000 m ö.h.
>     # De nordliga och västra slättlanden, inklusive Sahara, som kantas och genomfars av stråk med högra landområden, men i allmänhet håller sig under 600 m ö.h.
>
>     Skog täcker cirka en femtedel av Afrikas yta.
>
>     Hela Afrika har ett [[Tropiskt klimat|tropiskt]] eller [[Subtropiskt klimat]].
>
>     === Fauna ===
>
>     {{Fördjupning|Afrikas fauna}}
>
>     Afrika är huvudsakligen känt för den öppna [[savann]]en med väldiga hjordar av hovdjur som [[afrikansk buffel]], gnuer, [[antiloper]], [[giraff]]er och [[zebror]]. Dessa jagas av stora rovdjur som [[lejon]], [[hyenor]] och [[Gepard|geparder]]. Den afrikanska djungeln befolkas av [[ormar]] och [[primater]] samt av vattenlevande djur som [[krokodiler]] och [[groddjur]]. Afrika har även det största antalet arter som kan räknas till [[megafauna]]n, bland annat [[afrikanska elefanter]], [[trubbnoshörning]] och [[flodhäst]], på grund av att kontinenten bara i mindre utsträckning var påverkad av det [[massutdöende]] som skedde under [[pleistocen]].
>
>     == Historia ==
>     {{Fördjupning|Afrikas historia}}
>     Den första mänskliga migrationen, från kontinenten men även inom Afrikas kuster, kan spåras [[lingvistisk]]t, kulturellt och genom datoranalyser av [[genetik|genetiskt]] material. Någonstans i [[Östafrika]] tror dagens forskare att de första [[hominider]] uppkom. I [[Etiopien]] har man hittat ett två miljoner år gammalt skelett som uppkallades [[Australopithecus afarensis|Lucy]], den första kända mänskliga [[primater|primaten]].
>
>     === Tidigare civilisationer och handel ===
>     [[Fil:African-civilizations-map-pre-colonial.svg|miniatyr|höger|Några av de mest kända afrikanska civilisationerna. Civilisationer från olika tidsepoker visas parallellt.]]Lingvistiska spår tyder på att [[Bantu]]folk emigrerade från [[Västafrika]] in i [[khoisan]]folkets (de som tidigare kallades [[bushmän]] och [[hottentott]]er) område och undanträngde dem. Bantufolken använde en uppsättning grödor, bland annat [[kassava]] och [[jams]], som var mer lämpade för jordbruk i det [[tropik|tropiska]] Afrika, och med sitt jordbruk kunde bantu försörja en betydligt större folkmängd än de undanträngde jägare eller samlare. Bantufolkens traditionella bosättningsområde sträcker sig från [[Sahara]]öknen till de [[Tempererat klimat|tempererade regionerna]] i söder. Historiskt har deras vapen varit spjut, sköld, pil och båge. Det för [[khoisanspråk]]ens karaktäristiska klickljuden återfinns i [[nguni (språk)|nguni]]- och [[bantu]]språk som till exempel [[xhosa]] och [[zulu]]. Khoisanspråken talas idag främst av utspridda minoritetsgrupper i södra Afrika i länderna kring [[Kalahariöknen]].
>
>     Afrika söder om Sahara har bestått olika stater och riken, mindre stadsstater samt [[stam (samhällsvetenskap)|stammar]] och [[klan]]er. Det forna egyptiska riket, är en av världens äldsta stater, och den mest kända av de äldre afrikanska [[civilisation]]erna. Det fanns även statsbildningar i [[Nubien]], [[Ghana]], [[Timbuktu]], [[Kanem-Bornu]] och i nuvarande [[Zimbabwe]].
>
>     [[Nordafrika]] kom från och med 600-talet under arabiskt inflytande – en tid av kulturell blomstring – och med kamelens hjälp spreds [[Islam]] och den arabiska kulturen genom Sahara och vidare söderut. Arabiska och persiska handelsmän spred likaså Islam längs med den östafrikanska kusten parallellt med en omfattande svart slavhandel. Detta östafrikanska handelsutbyte med araber och perser sträckte sig ända bort till Java, därmed den etniska, kulturella och språkliga mångfalden på Östafrikas kust, [[Madagaskar]] och [[Komorerna]].
>
>     === Arabvärlden och Europa utforskar kontinenten ===
>     Det arabiska intresset för Afrika går långt tillbaka, men handel med slavar från centra som Lamu och Zanzibar vid östkusten har pågått åtminstone sedan senare delen av 1000-talet e.Kr. De miljoner slavar som exporterades till arabvärlden och österut under den [[arabiska slavhandeln]] finns idag kvar som en stor etnisk grupp där, i stor utsträckning integrerad med den ursprungliga arabiska befolkningen. Slavhandel med svarta pågår fortfarande i vissa arabländer, till exempel Sudan.&lt;ref&gt;[http://www.hrw.org/backgrounder/africa/sudanupdate.htm Human Rights Watch] [http://news.bbc.co.uk/2/hi/programmes/this_world/6458377.stm BBC World]&lt;/ref&gt;
>
>     Under [[1300-talet]] började vissa europeiska handelsexpeditioner få kontakt med Nord- och Västafrika.
>
>     Från 1500-talets mitt fram till förbudet mot slavhandel i Storbritanniens parlament 1807, som kom att få full kraft först något årtionde senare, handlades slavar från lokala stamledare i främst Väst- och Centralafrika. Flera miljoner afrikaner såldes på detta vis som slavar till främst de Europeiska kolonierna i Amerika.
>
>     Tidiga kolonisatörer var [[araber]], [[Portugal|portugiser]] och [[Spanien|spanjorer]]. Därefter koloniserades kontinenten av framförallt av [[Storbritannien]], [[Frankrike]], [[Spanien]], [[Italien]], [[Belgien]], [[Nederländerna]], [[Tyskland]] och [[Portugal]].
>
>     === Koloniseringen ===
>     {{Fördjupning|Kapplöpningen om Afrika}}
>     [[Fil:Map_of_Africa_from_Encyclopaedia_Britannica_1890.jpg|miniatyr|Politisk karta över Afrika från 1890.]] Från slutet av 1000-talet koloniserade arabiska köpmän den afrikanska östkusten. Holländarna koloniserade på 1600-talet den då relativt folktomma södra udden av Afrika. Portugiserna grundade flera kolonier i västra Afrika och vid nuvarande Moçambique. I mitten av [[1800-talet]] leddes flera expeditioner in i de centrala delarna av Afrika, bland annat av Livingstone och Speke, men också den svenske upptäcktsresanden Andersson. Denna kolonisering förändrade i grunden det ekonomiska livet på kontinenten, och innebar i flera fall ensidig export av naturresurser. [[Leopold II av Belgien|Kung Leopolds]] rovdrift på sin privata koloni [[Kongostaten]] är ökänd och kostade miljontals afrikanska liv.
>
>     I Afrika fanns efter första världskriget endast tre självständiga stater nämligen konungariket Egypten, republiken Liberia och konungariket Abessinien. Tanger bildade med sitt område en internationell zon. För övrigt var hela världsdelen uppdelad på kolonier åt europeiska makter.&lt;ref&gt;{{runeberg.org|nfcn|0245.html Supplement Afrika}}&lt;/ref&gt;
>
>     Flertalet kolonier erhöll [[självständighet]] under slutet av 1950-talet och början av 1960-talet. Avkoloniseringen resulterade i flertalet fall med katastrof, där diktatur och etnisk rensning vanligen blev följden, till exempel i Rhodesia (nuvarande Zimbabwe), Rwanda, Uganda, Nigeria m.fl. stater.
>
>     == Politik ==
>     {{Fördjupning|Afrikas politik}}
>     [[Fil:Electoral_Democracies_in_Africa,_Freedom_House_2007.png|miniatyr|De blå länderna är demokratier enligt tankesmedjan Freedom House.]]
>     Det postkoloniala Afrikas politik har i hög grad präglats av diktatur, statskupper och brist på [[mänskliga rättigheter]].
>
>     [[Nation]]erna i Afrika har ofta visat sig vara instabila [[stat]]er som efter självständigheten hemsökts av korruption, våldsamma [[statskupp]]er, [[despotism]] och [[dikatur|militärdiktaturer]]. Under perioden [[1960]]-[[1989]] genomfördes över 70 kupper och 13 [[mord]] på [[president]]er. Under det [[kalla kriget]] kämpade [[USA]] och [[Sovjetunionen]] om makten i Afrika, och tillsammans med länder som [[Frankrike]] och [[Kuba]], stödde de aktivt diktaturer och iscensatte flera av dessa kupper och illgärningar mot demokratiskt valda regeringar. Enligt en spridd uppfattning beror många [[militär]]a konflikter om [[territorium|territorier]] och [[gräns]]er under Afrikas moderna historia på de gränser från kolonial tid som huvudsakligen drogs upp efter [[Kongokonferensen]], men detta kan inte visas eftersom något jämförelseområde inte finns. Konflikter i Afrika har ofta drivits av tillgången till kontinentens rika [[råvara|råvarutillgångar]] – [[diamant]]er och [[metall]]er som [[guld]], [[platina]], [[uran]] och [[koppar]].
>
>     Korruption och havererade politiska program har lett till utbredd [[undernäring]] och undermålig infrastruktur orsakar alltjämt brist på mat och rent vatten. Sanitetsproblemen har också orsakat en akut spridning av bland annat [[HIV]]-viruset som leder till [[sjukdom]]en [[AIDS]].
>
>     Trots nöd och umbäranden finns vissa positiva tecken – [[demokrati]]skt valda [[regering]]ar växer i antal även om de inte utgör någon majoritet ännu. Genom påtryckningar från till exempel [[Internationella valutafonden]] (IMF) har många afrikanska stater lyckats vända en nedåtgående ekonomisk spiral som varat i flera decennier till en positiv tillväxt. Om man beaktar IMF:s katastrofala misslyckanden i länder som [[Argentina]] och [[Filippinerna]] är det dock oklart i vilken omfattning dessa konventionella ekonomiska kriterier ger en rättvis bild av vad Afrika faktiskt behöver.
>
>     De afrikanska ledarna har i olika omgångar sökt former för samarbete sinsemellan. Hittills har dock detta inte lyckats särskilt väl. Under [[inbördeskrig]]et i [[Kongo-Kinshasa|Kongo]] var det grannstaterna som spädde på konflikten snarare än utomafrikanska stater, och motivet var främst att skaffa kortsiktiga vinster genom att utvinna diamanter och andra mineraler. Antalet döda i konflikten har uppskattats till 3,5 miljoner under en femårsperiod.
>
>     De politiska samarbetsorganisationer man skapat har inte kunnat leva upp till förväntningarna. Organisationen för afrikansk enighet (OAU 1963-2002) kallades till exempel diktatorsklubben för dess tendens att alltid stödja de sittande politiska ledarna och ägna väldigt lite energi åt att främja den vanlige afrikanens mänskliga rättigheter. 2002 skapades i stället ''[[Afrikanska unionen]]'', som är tänkt att underlätta ett framtida samarbete mellan kontinentens stater. Hittills har denna organisation inte klarat att hantera till exempel utrotningskriget i Darfur-provinsen i Sudan, där en arabisk statsledning tillåtit slakt på hundratusentals av bantubefolkningen i sydväst. En avgörande skillnad mellan såväl OAU som AU å ena sidan och EU å den andra, är att EU startades av en grupp länder som satte demokrati och mänskliga rättigheter i fokus för den egna politiken, och som krävt detta av tillkommande medlemsstater. OAU och AU har från början i stället varit territoriellt inriktade, och visserligen haft demokrati och mänskliga rättigheter som skrivelser i sina stadgar, men ytterst sällan satt kraft bakom dessa ord. På så sätt liknar AU mer FN än EU.
>
>     == Ekonomi ==
>     {{Fördjupning|Afrikas ekonomi}}
>     [[Fil:Bakweri cocoyam farmer from Cameroon.jpg|250px|miniatyr|Jordbrukare på en [[jams]]odling i Kamerun]]
>     Afrikas ekonomi omfattar ungefär 887 miljoner människor i 54 olika stater ([[Juli 2005]]). Afrika är världens fattigaste kontinent, och den är, i medeltal, fattigare än den var för 25 år sedan. På [[FN]]:s Human Development Report från [[2003]] (som rankade världens 175 länder enligt en rad kriterier) var plats mellan 151 och 175 intagna av länder i Afrika.
>
>     Det finns dock länder som lyckas bra ekonomiskt, bland andra [[Botswana]], [[Ghana]] och [[Sydafrika]]. [[Etiopien]] har också en relativ bra ekonomi tack vare den höga tillväxten i landet på cirka 11-15% per år. Sydafrika har en väl fungerande inhemsk aktiehandel och har stora naturtillgångar i bland annat [[guld]] och [[diamanter]] och ett väl fungerande rättsväsen. Det finns i Sydafrika även tillgång till investeringskapital, stora marknader och utbildad arbetskraft. Det finns även andra afrikanska länder, exempelvis [[Egypten]], som länge haft en väl fungerande handelssektor och ekonomi.
>
>     Kontinentens kust mot [[Atlanten]] återfinns stora fyndigheter av [[Petroleum|olja]]. I [[Nigeria]] finns en av världens mest betydande oljereserver och landets ekonomi är bland de snabbast växande i världen.
>
>     Afrikas jordbruk sysselsätter drygt hälften av Afrikas befolkning och är den viktigaste näringen. Cirka 6 % (ungefär 1 814 600&amp;nbsp;km²) av världsdelens yta är uppodlad, mest i savannområdena. I stora delar av de brukningsbara områdena är jordens bördighet ganska låg. De bästa jordarna återfinns i begränsade höglandsområden som i till exempel [[Kamerun]] och [[Östafrika]].
>
>     == Demografi ==
>     {{Fördjupning|Afrikas demografi}}
>     ===Befolkning===
>     Människan, ''[[Homo sapiens|Homo sapiens sapiens]]'', antas härstamma från Afrika och spred sig härifrån till de omgivande kontinenterna. Den svarta befolkningen söder om Sahara tillhör den ursprungliga befolkningen som hela tiden levat i Afrika.
>
>     Norr om Sahara präglar samröret med övriga [[Medelhavet|Medelhavsområdet]] och inte minst sydvästra [[Asien]] både befolkning, språk och kultur.
>
>     På ön [[Madagaskar]], som länge var obebodd, härstammar majoriteten av befolkningen från människor som emigrerat från [[Indonesien]]. Under kolonialtiden skedde inflyttning från [[Europa]] och [[Asien]] till södra Afrika, där den icke-svarta befolkningen idag utgör betydande minoritetsgrupper.
>
>     Afrikas befolkning uppvisar många för utvecklingsregioner typiska drag, som höga [[födelsetal]] i kombination med sjunkande [[Mortalitet (demografi)|dödstal]] med snabb befolkningsökning och låg medelålder (45 % är under 15 år) som följd. Även om befolkningstätheten varierar kraftigt mellan olika regioner är Afrika generellt en glestbefolkad kontinent, med i genomsnitt 27,8 invånare per km².
>
>     Afrika är den kontinent som har den lägsta urbaniseringsgraden, med endast 25% boende i städer. Även här är variationerna stora, och i norra och östra Afrika finns en lång urban tradition.
>
>     === Språk ===
>     {{Fördjupning|Afrikas språk}}
>     [[Fil:African languages SV.png|miniatyr|300px|Språkfamiljer i Afrika]]
>     Uppskattningsvis finns det ungefär 1800 talade [[språk]] i Afrika idag. Vissa av dessa, till exempel [[Swahili]], [[Hausa]], och [[Yoruba (språk)|Yoruba]], talas av många miljoner människor, medan det finns andra som bara talas av några hundra eller ännu färre. Afrika har även många olika [[teckenspråk]]. De flesta afrikaner talar som modersmål ett språk tillhörande någon av de fyra språkfamiljerna [[afroasiatiska språk]], [[nilo-sahariska språk]], [[Niger-Kongospråk]] eller [[khoisanspråk]].
>     Med några få nämnvärda undantag i östra Afrika, har nästan alla afrikanska länder språk som har sitt ursprung utanför världsdelen som officiellt språk, på grund av kolonialismen och mänsklig migration. Till exempel används i ett stort antal afrikanska länder [[franska]] eller [[engelska]] som administrativt språk för stat, handel, utbildning och media. [[Arabiska]], [[portugisiska]], [[afrikaans]] och [[malagassiska]] är andra exempel på språk med ursprung utanför Afrika som idag används av miljoner afrikaner, både i offentliga och privata sammanhang.
>
>     === Religion ===
>     {{Fördjupning|Afrikas religion}}{{Se även|Kristendomen i Afrika}}
>     [[Kristendom]] och [[islam]] är de största religionerna i Afrika, 46,3 % av alla afrikaner är kristna och 40,5 % är muslimer. Resten 11,8 % tillhör olika lokala afrikanska religioner som funnits före den europeiska kolonisationen. Ett mindre antal afrikaner är [[Hinduism|hinduer]], eller har tro från [[Judendom|judisk]] tradition. Exempel på [[Afrikanska judar|afrikanska folk som tillhör judendomen]] är [[Beta Israel]], [[Lemba]]folk och [[Abayudaya]] i östra [[Uganda]]. Sydafrika och Angola har över 55% kristna katoliker.
>
>     == Kultur ==
>     {{Fördjupning|Afrikas kultur}}
>     Afrika har ett stort antal överlappande kulturer, och flera tusen olika etniska grupper. Den mest konventionella distinktionen är den mellan området söder om Sahara och området norr om öknen. De nordliga länderna associerar sig främst med arabkulturen, medan de sydliga länderna består av i sin tur många olika kulturella områden. En av de största kulturella identiteterna är den som innefattar alla folk som talar något [[bantuspråk]].
>
>     Det går också att göra en indelning i gamla franska Västafrika å ena sidan, och resten av Afrika (mer specifikt de brittiska eller portugisiska kolonierna i södra och östra Afrika) å andra sidan. En annan kulturell linje går mellan de afrikaner som håller kvar vid sina gamla traditionella livsstilar, och de som har anpassat sig till en mer modern livsstil. Det går även att dela upp traditionalisterna i sådana som ägnar sig åt uppfödning av boskap, och sådana som ägnar sig åt jordbruk.
>
>     [[Afrikansk konst]] och [[afrikansk arkitektur]] reflekterar de afrikanska kulturernas diversitet. De äldsta exemplen på afrikansk konst är 75 000 år gamla pärlsmycken gjorda av [[nassarius]]skal, dessa hittades i [[Blombosgrottan]]. [[Pyramiderna i Giza]] var världens högsta byggnader under 4 000 år. [[Klippkyrkorna i Lalibela]] i [[Etiopien]], bland vilka ''[[Bete Giyorgis]]'' (&quot;St. Göranskyrkan&quot;) är mest välkänd, är också kända exempel på afrikansk arkitektur.
>
>     == Regioner och territorier ==
>     [[Fil:Africa-regions.png|miniatyr|Regioner i Afrika:
>     {{legend|#0000ff|[[Nordafrika]]}}
>     {{legend|#00ff00|[[Västafrika]]}}
>     {{legend|#ff00ff|[[Centralafrika]]}}
>     {{legend|#ffa500|[[Östafrika]]}}
>     {{legend|#ff0000|[[Södra Afrika]]}}]]
>     {| class=&quot;wikitable sortable&quot;
>     |+
>     |- bgcolor=&quot;#ECECEC&quot;
>     ! Regioners namn&lt;ref&gt;Enligt [[:Fil:United Nations geographical subregions.png|FN:s regionindelning]] av världen.&lt;br /&gt;&lt;/ref&gt; och&lt;br /&gt; territorier, med flagga
>     ! data-sort-type=number | [[Area]]&lt;br /&gt;(km²)
>     ! data-sort-type=number |[[Folkmängd]]&lt;br /&gt;(2010)&lt;ref name=&quot;esa.un.org1&quot;/&gt;
>     ! data-sort-type=number |[[Befolkningstäthet]]&lt;br /&gt;(per km²)
>     ! [[Huvudstad]]
>     |-
>     | colspan=5 style=&quot;background:#eee;&quot; | '''[[Centralafrika]]''':
>     |-
>     | {{flaggbild2|Angola}}
>     | align=&quot;right&quot; | 1 246 700
>     | align=&quot;right&quot; | 19&amp;nbsp;081&amp;nbsp;912
>     | align=&quot;right&quot; | 15,3
>     | [[Luanda]]
>     |-
>     | {{flaggbild2|Kamerun}}
>     | align=&quot;right&quot; | 475 440
>     | align=&quot;right&quot; | 19 598 889
>     | align=&quot;right&quot; | 41,2
>     | [[Yaoundé]]
>     |-
>     | {{flaggbild2|Centralafrikanska republiken}}
>     | align=&quot;right&quot; | 622 984
>     | align=&quot;right&quot; | 4 401 051
>     | align=&quot;right&quot; | 7,1
>     | [[Bangui]]
>     |-
>     | {{flaggbild2|Tchad}}
>     | align=&quot;right&quot; | 1 284 000
>     | align=&quot;right&quot; | 11 227 208
>     | align=&quot;right&quot; | 8,7
>     | [[N'Djamena]]
>     |-
>     | {{flaggbild2|Kongo-Brazzaville}}
>     | align=&quot;right&quot; | 342 000
>     | align=&quot;right&quot; | 4 042 899
>     | align=&quot;right&quot; | 12,9
>     | [[Brazzaville]]
>     |-
>     | {{flaggbild2|Kongo-Kinshasa}}
>     | align=&quot;right&quot; | 2 345 410
>     | align=&quot;right&quot; | 65 965 795
>     | align=&quot;right&quot; | 28,1
>     | [[Kinshasa]]
>     |-
>     | {{flaggbild2|Ekvatorialguinea}}
>     | align=&quot;right&quot; | 28 051
>     | align=&quot;right&quot; | 700 401
>     | align=&quot;right&quot; | 24,9
>     | [[Malabo]]
>     |-
>     | {{flaggbild2|Gabon}}
>     | align=&quot;right&quot; | 267 667
>     | align=&quot;right&quot; | 1 505 463
>     | align=&quot;right&quot; | 5,6
>     | [[Libreville]]
>     |-
>     | {{flaggbild2|São Tomé och Príncipe}}
>     | align=&quot;right&quot; | 1 001
>     | align=&quot;right&quot; | 165 397
>     | align=&quot;right&quot; | 165,0
>     | [[São Tomé]]
>     |-
>     | colspan=5 style=&quot;background:#eee;&quot; | '''[[Nordafrika]]''':
>     |-
>     | {{flaggbild2|Algeriet}}
>     | align=&quot;right&quot; | 2 381 740
>     | align=&quot;right&quot; | 35 468 000
>     | align=&quot;right&quot; | 14,9
>     | [[Alger (stad)|Alger]]
>     |-
>     | {{flaggbild2|Egypten}}&lt;ref&gt;[[Egypten]] brukar anses ligga i två världsdelar, Nordafrika (enligt FN:s regionindelning) och Västasien; befolknings- och arealuppgifterna avser endast den afrikanska delen, väster om [[Suezkanalen]].&lt;/small&gt;&lt;br /&gt;&lt;/ref&gt;
>     | align=&quot;right&quot; | 1 001 450
>     | align=&quot;right&quot; | 81 121 000
>     | align=&quot;right&quot; | 81,0
>     | [[Kairo]]
>     |-
>     | {{flaggbild2|Libyen}}
>     | align=&quot;right&quot; | 1 759 540
>     | align=&quot;right&quot; | 6 355 112
>     | align=&quot;right&quot; | 3,6
>     | [[Tripoli]]
>     |-
>     | {{flaggbild2|Marocko}}
>     | align=&quot;right&quot; | 446 550
>     | align=&quot;right&quot; | 31 951 412
>     | align=&quot;right&quot; | 71,5
>     | [[Rabat]]
>     |-
>     | {{flaggbild2|Sudan}}
>     | align=&quot;right&quot; | 1 886 068
>     | align=&quot;right&quot; | 33 551 941
>     | align=&quot;right&quot; | 17,8
>     | [[Khartoum]]
>     |-
>     | {{flaggbild2|Tunisien}}
>     | align=&quot;right&quot; | 163 610
>     | align=&quot;right&quot; | 10 480 934
>     | align=&quot;right&quot; | 64,1
>     | [[Tunis]]
>     |-
>     | {{flaggbild2|Västsahara}} ([[Marocko]])&lt;ref&gt;[[Västsahara]] är ockuperat av [[Marocko]], som också har anspråk på området.&lt;/small&gt;&lt;br /&gt;&lt;/ref&gt;
>     | align=&quot;right&quot; | 266 000
>     | align=&quot;right&quot; | 530 500
>     | align=&quot;right&quot; | 2,0
>     | [[El Aaiún]]
>     |-
>     | colspan=5 | ''Områden som är likställda delar av europeiska stater'':
>     |-
>     | [[Fil:Flag of the Canary Islands.svg|22x20px|border|Kanarieöarna]] [[Kanarieöarna]] ([[Spanien]])&lt;ref&gt;De till [[Spanien]] hörande [[Kanarieöarna]] brukar geografiskt räknas till Nordafrika på grund av deras relativa närhet till [[Marocko]] och [[Västsahara]]; befolknings- och arealuppgifterna är från år 2001.&lt;br /&gt;&lt;/ref&gt;
>     | align=&quot;right&quot; | 7 492
>     | align=&quot;right&quot; | 2 117 519
>     | align=&quot;right&quot; | 282,6
>     | [[Las Palmas de Gran Canaria]],&lt;br /&gt;[[Santa Cruz de Tenerife]]
>     |-
>     | [[Fil:Flag Ceuta.svg|22x20px|border|Ceuta]] [[Ceuta]] ([[Spanien]])&lt;ref&gt;Den [[Spanien|spanska]] [[exklav]]en [[Ceuta]] omges på landsidan av Marocko i Nordafrika; befolknings- och arealuppgifterna är från år 2001.&lt;br /&gt;&lt;/ref&gt;
>     | align=&quot;right&quot; | 20
>     | align=&quot;right&quot; | 78 674
>     | align=&quot;right&quot; | 3933,7
>     | –
>     |-
>     | [[Fil:Flag of Madeira.svg|22x20px|Madeira]] [[Madeira]] ([[Portugal]])&lt;ref&gt;Det till [[Portugal]] hörande [[Madeira]] brukar geografiskt räknas till Nordafrika på grund av deras relativa närhet till [[Marocko]]; befolknings- och arealuppgifterna är från år 2001.&lt;br /&gt;&lt;/ref&gt;
>     | align=&quot;right&quot; | 797
>     | align=&quot;right&quot; | 267 302
>     | align=&quot;right&quot; | 335,4
>     | [[Funchal]]
>     |-
>     | [[Fil:Flag of Melilla.svg|22x20px|Melilla]] [[Melilla]] ([[Spanien]])&lt;ref&gt;Den [[Spanien|spanska]] [[exklav]]en [[Melilla]] omges på landsidan av Marocko i Nordafrika; befolknings- och arealuppgifterna är från år 2001.&lt;br /&gt;&lt;/ref&gt;
>     | align=&quot;right&quot; | 12
>     | align=&quot;right&quot; | 73 460
>     | align=&quot;right&quot; | 6121,6
>     | –
>     |-
>     | colspan=5 style=&quot;background:#eee;&quot; | '''[[Södra Afrika]]''':
>     |-
>     | {{flaggbild2|Botswana}}
>     | align=&quot;right&quot; | 600 370
>     | align=&quot;right&quot; | 2 006 945
>     | align=&quot;right&quot; | 3,3
>     | [[Gaborone]]
>     |-
>     | {{flaggbild2|Lesotho}}
>     | align=&quot;right&quot; | 30 355
>     | align=&quot;right&quot; | 2 171 318
>     | align=&quot;right&quot; | 71,5
>     | [[Maseru]]
>     |-
>     | {{flaggbild2|Namibia}}
>     | align=&quot;right&quot; | 825 418
>     | align=&quot;right&quot; | 2 283 289
>     | align=&quot;right&quot; | 2,7
>     | [[Windhoek]]
>     |-
>     | {{flaggbild2|Sydafrika}}
>     | align=&quot;right&quot; | 1 219 912
>     | align=&quot;right&quot; | 50 132 817
>     | align=&quot;right&quot; | 41,1
>     | [[Bloemfontein]], [[Kapstaden]], [[Pretoria]] (Tshwane)&lt;ref&gt;[[Bloemfontein]] är [[Sydafrika]]s judiciella huvudstad, medan [[Kapstaden]] är säte för den lagstiftande församlingen och [[Pretoria]] är administrativ huvudstad. Staden Pretoria har inlett en process för att eventuellt byta namn till Tshwane.&lt;br /&gt;&lt;/ref&gt;
>     |-
>     | {{flaggbild2|Swaziland}}
>     | align=&quot;right&quot; | 17 363
>     | align=&quot;right&quot; | 1 186 056
>     | align=&quot;right&quot; | 68,3
>     | [[Mbabane]]
>     |-
>     | colspan=5 style=&quot;background:#eee;&quot; | '''[[Västafrika]]''':
>     |-
>     | {{flaggbild2|Benin}}
>     | align=&quot;right&quot; | 112 620
>     | align=&quot;right&quot; | 8 849 892
>     | align=&quot;right&quot; | 78,6
>     | [[Porto-Novo]]
>     |-
>     | {{flaggbild2|Burkina Faso}}
>     | align=&quot;right&quot; | 274 200
>     | align=&quot;right&quot; | 16 468 714
>     | align=&quot;right&quot; | 60,1
>     | [[Ouagadougou]]
>     |-
>     | {{flaggbild2|Kap Verde}}
>     | align=&quot;right&quot; | 4 033
>     | align=&quot;right&quot; | 495 999
>     | align=&quot;right&quot; | 123,0
>     | [[Praia]]
>     |-
>     | {{flaggbild2|Elfenbenskusten}}
>     | align=&quot;right&quot; | 322 460
>     | align=&quot;right&quot; | 19 737 800
>     | align=&quot;right&quot; | 61,2
>     | [[Abidjan]], [[Yamoussoukro]]&lt;ref&gt;[[Yamoussoukro]] är [[Elfenbenskusten]]s officiella huvudstad, men [[Abidjan]] fungerar ''[[de facto]]'' som huvudstad.&lt;br /&gt;&lt;/ref&gt;
>     |-
>     | {{flaggbild2|Gambia}}
>     | align=&quot;right&quot; | 11 300
>     | align=&quot;right&quot; | 1 728 394
>     | align=&quot;right&quot; | 152,9
>     | [[Banjul]]
>     |-
>     | {{flaggbild2|Ghana}}
>     | align=&quot;right&quot; | 239 460
>     | align=&quot;right&quot; | 24 391 823
>     | align=&quot;right&quot; | 101,9
>     | [[Accra]]
>     |-
>     | {{flaggbild2|Guinea}}
>     | align=&quot;right&quot; | 245 857
>     | align=&quot;right&quot; | 9 981 590
>     | align=&quot;right&quot; | 40,6
>     | [[Conakry]]
>     |-
>     | {{flaggbild2|Guinea-Bissau}}
>     | align=&quot;right&quot; | 36 120
>     | align=&quot;right&quot; | 1 515 224
>     | align=&quot;right&quot; | 41,9
>     | [[Bissau]]
>     |-
>     | {{flaggbild2|Liberia}}
>     | align=&quot;right&quot; | 111 370
>     | align=&quot;right&quot; | 3 994 122
>     | align=&quot;right&quot; | 35,9
>     | [[Monrovia]]
>     |-
>     | {{flaggbild2|Mali}}
>     | align=&quot;right&quot; | 1 240 000
>     | align=&quot;right&quot; | 15 369 809
>     | align=&quot;right&quot; | 9,1
>     | [[Bamako]]
>     |-
>     | {{flaggbild2|Mauretanien}}
>     | align=&quot;right&quot; | 1 030 700
>     | align=&quot;right&quot; | 3 459 773
>     | align=&quot;right&quot; | 3,4
>     | [[Nouakchott]]
>     |-
>     | {{flaggbild2|Niger}}
>     | align=&quot;right&quot; | 1 267 000
>     | align=&quot;right&quot; | 15 511 953
>     | align=&quot;right&quot; | 12,2
>     | [[Niamey]]
>     |-
>     | {{flaggbild2|Nigeria}}
>     | align=&quot;right&quot; | 923 768
>     | align=&quot;right&quot; | 158 423 182
>     | align=&quot;right&quot; | 171,5
>     | [[Abuja]]
>     |-
>     | [[Fil:Flag of Saint Helena.svg|22x20px|Sankta Helena]] [[Sankta Helena (ö)|Sankta Helena]] ([[Storbritannien]])
>     | align=&quot;right&quot; | 410
>     | align=&quot;right&quot; | 4 118
>     | align=&quot;right&quot; | 10,0
>     | [[Jamestown, Sankta Helena|Jamestown]]
>     |-
>     | {{flaggbild2|Senegal}}
>     | align=&quot;right&quot; | 196 190
>     | align=&quot;right&quot; | 12 433 728
>     | align=&quot;right&quot; | 63,4
>     | [[Dakar]]
>     |-
>     | {{flaggbild2|Sierra Leone}}
>     | align=&quot;right&quot; | 71 740
>     | align=&quot;right&quot; | 5 867 536
>     | align=&quot;right&quot; | 81,8
>     | [[Freetown]]
>     |-
>     | {{flaggbild2|Togo}}
>     | align=&quot;right&quot; | 56 785
>     | align=&quot;right&quot; | 6 027 798
>     | align=&quot;right&quot; | 106,1
>     | [[Lomé]]
>     |-
>     | colspan=5 style=&quot;background:#eee;&quot; | '''[[Östafrika]]''':
>     |-
>     | {{flaggbild2|Burundi}}
>     | align=&quot;right&quot; | 27 830
>     | align=&quot;right&quot; | 8 382 849
>     | align=&quot;right&quot; | 301,2
>     | [[Bujumbura]]
>     |-
>     | {{flaggbild2|Komorerna}}
>     | align=&quot;right&quot; | 2 170
>     | align=&quot;right&quot; | 734 750
>     | align=&quot;right&quot; | 338,6
>     | [[Moroni]]
>     |-
>     | {{flaggbild2|Djibouti}}
>     | align=&quot;right&quot; | 23 000
>     | align=&quot;right&quot; | 888 716
>     | align=&quot;right&quot; | 38,6
>     | [[Djibouti (stad)|Djibouti]]
>     |-
>     | {{flaggbild2|Eritrea}}
>     | align=&quot;right&quot; | 121 320
>     | align=&quot;right&quot; | 5 253 676
>     | align=&quot;right&quot; | 43,3
>     | [[Asmara]]
>     |-
>     | {{flaggbild2|Etiopien}}
>     | align=&quot;right&quot; | 1 127 127
>     | align=&quot;right&quot; | 82 949 541
>     | align=&quot;right&quot; | 73,6
>     | [[Addis Abeba]]
>     |-
>     | {{flaggbild2|Kenya}}
>     | align=&quot;right&quot; | 582 650
>     | align=&quot;right&quot; | 40 512 682
>     | align=&quot;right&quot; | 69,5
>     | [[Nairobi]]
>     |-
>     |{{flaggbild2|Madagaskar}}
>     | align=&quot;right&quot; | 587 040
>     | align=&quot;right&quot; | 20 713 819
>     | align=&quot;right&quot; | 35,3
>     | [[Antananarivo]]
>     |-
>     | {{flaggbild2|Malawi}}
>     | align=&quot;right&quot; | 118 480
>     | align=&quot;right&quot; | 14 900 841
>     | align=&quot;right&quot; | 90,3
>     | [[Lilongwe]]
>     |-
>     | {{flaggbild2|Mauritius}}
>     | align=&quot;right&quot; | 2 040
>     | align=&quot;right&quot; | 1 299 172
>     | align=&quot;right&quot; | 636,8
>     | [[Port Louis]]
>     |-
>     | {{flaggbild|Frankrike}} [[Mayotte]] ([[Frankrike]])
>     | align=&quot;right&quot; | 374
>     | align=&quot;right&quot; | 204 114
>     | align=&quot;right&quot; | 545,8
>     | [[Mamoudzou]]
>     |-
>     | {{flaggbild|Moçambique}} [[Moçambique]]
>     | align=&quot;right&quot; | 801 590
>     | align=&quot;right&quot; | 23 390 765
>     | align=&quot;right&quot; | 29,2
>     | [[Maputo]]
>     |-
>     | {{flaggbild|Frankrike}} [[Réunion]] ([[Frankrike]])
>     | align=&quot;right&quot; | 2 512
>     | align=&quot;right&quot; | 846 068
>     | align=&quot;right&quot; | 336,8
>     | [[Saint-Denis (Réunion)|Saint-Denis]]
>     |-
>     | {{flaggbild2|Rwanda}}
>     | align=&quot;right&quot; | 26 338
>     | align=&quot;right&quot; | 10 624 005
>     | align=&quot;right&quot; | 403,4
>     | [[Kigali]]
>     |-
>     | {{flaggbild2|Seychellerna}}
>     | align=&quot;right&quot; | 455
>     | align=&quot;right&quot; | 86 518
>     | align=&quot;right&quot; | 190,1
>     | [[Victoria, Seychellerna|Victoria]]
>     |-
>     | {{flaggbild2|Somalia}}
>     | align=&quot;right&quot; | 637 657
>     | align=&quot;right&quot; | 9 330 872
>     | align=&quot;right&quot; | 14,6
>     | [[Mogadishu]]
>     |-
>     | {{flaggbild2|Sydsudan}}
>     | align=&quot;right&quot; | 633 783
>     | align=&quot;right&quot; | 8 207 607
>     | align=&quot;right&quot; | 13
>     | [[Juba]]
>     |-
>     | {{flaggbild2|Tanzania}}
>     | align=&quot;right&quot; | 945 087
>     | align=&quot;right&quot; | 44 841 226
>     | align=&quot;right&quot; | 47,4
>     | [[Dodoma]]
>     |-
>     | {{flaggbild2|Uganda}}
>     | align=&quot;right&quot; | 236 040
>     | align=&quot;right&quot; | 33 424 683
>     | align=&quot;right&quot; | 141,6
>     | [[Kampala]]
>     |-
>     | {{flaggbild2|Zambia}}
>     | align=&quot;right&quot; | 752 614
>     | align=&quot;right&quot; | 13 088 570
>     | align=&quot;right&quot; | 17,4
>     | [[Lusaka]]
>     |-
>     | {{flaggbild2|Zimbabwe}}
>     | align=&quot;right&quot; | 390 580
>     | align=&quot;right&quot; | 12 571 454
>     | align=&quot;right&quot; | 32,2
>     | [[Harare]]
>     |- style=&quot; font-weight:bold; &quot;
>     | Total
>     | align=&quot;right&quot; | 30&amp;nbsp;305&amp;nbsp;053
>     | align=&quot;right&quot; | 1&amp;nbsp;022&amp;nbsp;234&amp;nbsp;400
>     | align=&quot;right&quot; | 33,7
>     |}
>
>     == Se även ==
>     * [[Nordafrika]]
>     * [[Västafrika]]
>     * [[Östafrika]]
>     * [[Centralafrika]]
>     * [[Södra Afrika]]
>     * [[Afrikas klimat]]
>
>     == Referenser ==
>
>     === Noter ===
>     &lt;references&gt;
>     &lt;ref name=&quot;esa.un.org&quot;&gt;[http://esa.un.org/unpd/wpp/Excel-Data/population.htm/ &quot;World Population Prospects: The 2010 Revision&quot;] {{Wayback|url=http://esa.un.org/unpd/wpp/Excel-Data/population.htm/ |date=20120606170530 }} [[Förenta nationerna|United Nations]] (Department of Economic and Social Affairs, population division)&lt;/ref&gt;
>     &lt;ref name=&quot;esa.un.org1&quot;&gt;[http://esa.un.org/unpd/wpp/Excel-Data/population.htm/ &quot;World Population Prospects: The 2010 Revision&quot;] {{Wayback|url=http://esa.un.org/unpd/wpp/Excel-Data/population.htm/ |date=20120606170530 }} [[United Nations]] (Department of Economic and Social Affairs, population division)&lt;/ref&gt;
>     &lt;/references&gt;
>
>     === Källor ===
>     {{enwp|url=https://en.wikipedia.org/w/index.php?title=Africa&amp;oldid=81702686|artikel=Africa}}
>     * [https://www.courses.psu.edu/test/test100_hkr/AFIM/Main_HTML/Intro_1.html Pennsylvania State University – ''African Ethnic Groups'' och andra kartor]
>     * [http://www.pygmies.info/ Pygméerna i Afrika] Kultur och musik av de första urinvånarna i Afrika
>     * Afrika i skolböckerna, M. Palmberg 1987
>
>     == Externa länkar ==
>     {{commonscat|Africa}}
>     {{Wikinews|Afrika}}
>     {{wikiatlas|Africa}}
>
>     {{Afrika}}
>     {{Världens regioner}}
>     {{auktoritetsdata}}
>
>     [[Kategori:Afrika| ]]
>     [[Kategori:Wikipedia:Basartiklar]]</text>
>           <sha1>1ueywtz2sm89v81ujno96unvkkmxpoa</sha1>
>         </revision>
>       </page>, <page>
>         <title>Amerika</title>
>         <ns>0</ns>
>         <id>3</id>
>         <revision>
>           <id>48343172</id>
>           <parentid>48187017</parentid>
>           <timestamp>2020-10-11T13:36:09Z</timestamp>
>           <contributor>
>             <username>Olympicbandy</username>
>             <id>564328</id>
>           </contributor>
>           <model>wikitext</model>
>           <format>text/x-wiki</format>
>           <text bytes="2728" xml:space="preserve">{{för|Amerika som kortnamn på '''Amerikas förenta stater'''|USA|andra betydelser|Amerika (olika betydelser)}}
>     [[Fil:Americas (orthographic projection).svg|miniatyr|250px|Karta som visar Nord- och Sydamerika, tillsammans benämnda Amerika]]
>     '''Amerika''' (engelska: ''the Americas'') består av landmassorna mellan [[Atlanten]] och [[Stilla havet]] på [[västra halvklotet]], namngivna efter den italienske upptäcktsresanden [[Amerigo Vespucci]]. Amerika uppfattas vanligen idag som två världsdelar, [[Nordamerika]] och [[Sydamerika]].&lt;ref&gt;[https://snl.no/Amerika Store norske leksikon...

  

We note that we have filtered away around 80% of the pages. Our initial
results were mostly returning autogenerated articles, and those are not
very interesting to our topic analysis pursuits, so we decided to remove
them.

Labels
------

Before we clean the xml and markup, we extract the article id and title,
as we may want to use them when we analyze the resulting LDA model.

In [ ]:
val id_regexp = raw"<id>([0-9]*)</id>".r
def extract_id(source : String) = {
  val m = id_regexp.findFirstMatchIn(source).get
  m.group(1).toLong
}

val title_regexp = raw"<title>([^\<\>]*)</title>".r
def extract_title(source : String) = {
  val m = title_regexp.findFirstMatchIn(source).get
  m.group(1)
}

val labelled_articles = articles.map(source => (extract_id(source), extract_title(source), source))

labelled_articles.take(5).map({case(id, title, contents) => title})

  

>     id_regexp: scala.util.matching.Regex = <id>([0-9]*)</id>
>     extract_id: (source: String)Long
>     title_regexp: scala.util.matching.Regex = <title>([^\<\>]*)</title>
>     extract_title: (source: String)String
>     labelled_articles: org.apache.spark.rdd.RDD[(Long, String, String)] = MapPartitionsRDD[1021] at map at command-2629358840275853:13
>     res11: Array[String] = Array(Amager, Afrika, Amerika, Abbekås, Alingsås)

  

Clean xml and markup
--------------------

We now consider xml structure. We perform the following steps

1.  Replace html-strings `&amp;`, `&quot;`, `&nbsp;`, `&lt;` and `&gt;`
    by "&", "", " ", "&lt;" and "&gt;", repsectively.

-   This is performed first, as
    -   The "&" from `&amp;` is used as the "&" in `&nbsp;` and other
        sequences.
    -   The "&lt;" and "&gt;" from `&lt;` and `&gt` are used in tags.

1.  First, we remove the following start- and stop-tags, and the
    contents in between

-   id
-   ns
-   parentid
-   timestamp
-   contributor
-   comment
-   model
-   format
-   sha1

1.  We remove any tag, i.e., anything of the form `"<A>"` for any string
    `A`.
2.  We remove anything enclosed in double-curly-braces, i.e., anything
    on the form `{{A}}` for any `A`.

In [ ]:
def filter_xml(source : String) = {
  val noAmp = raw"&amp;".r.replaceAllIn(source, "&")
  val noNbsp = raw"&nbsp;".r.replaceAllIn(noAmp, " ")
  val noQuot = raw"&quot;".r.replaceAllIn(noNbsp, "")
  val noLt = raw"&lt;".r.replaceAllIn(noQuot, "<")
  val noGt = raw"&gt;".r.replaceAllIn(noLt, ">")
  
    
  
  val tags = Seq("id", "ns", "parentid", "timestamp", "contributor", "comment", "model", "format", "sha1")
  var current = noGt
  for (tag <- tags) {
    val start = s"<$tag>".r
    val end = s"</$tag>".r
    while (start.findAllIn(current).hasNext) {
      val a = start.findAllIn(current).start
      val b = end.findAllIn(current).start
      val len = current.length
      current = current.substring(0, a) + current.substring(b+tag.length+3, current.length)
      assert (current.length < len)
    }
  }
  current = raw"<[^<>]*>".r.replaceAllIn(current, "")

  // A loop to remove innermost curly braces, then outer such things.
  val curly = raw"\{[^\{\}]*\}".r
  while (curly.findAllIn(current).hasNext)
  {
     current = curly.replaceAllIn(current, "")
  }

  current
  }

  
val filtered_xml = labelled_articles.map({ case(id, title, source) => (id, title, filter_xml(source)) })
  
filtered_xml.take(5)


  

>     filter_xml: (source: String)String
>     filtered_xml: org.apache.spark.rdd.RDD[(Long, String, String)] = MapPartitionsRDD[1025] at map at command-2629358840275841:36
>     res14: Array[(Long, String, String)] =
>     Array((1,Amager,"
>         Amager
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>     '''Amager''' är en [[Danmark|dansk]] [[ö (landområde)|ö]] i [[Öresund]]. Öns norra och västra delar tillhör [[Köpenhamn]], medan övriga delar upptas av [[Tårnby kommun]] och [[Dragørs kommun]].
>     Amager har en yta på 96,29 km² och befolkningen uppgår till 196 047 personer (1/1 2018). På den östra delen av ön finns [[Kastrups flygplats]]. Amager har också flera militära anläggningar (t.ex. Kastrup fort och Dragör fort), strandparksområdena [[Amager Strandpark]] och Kastrup Strandpark, flera restauranger i världsklass (t.ex. Noma), ett stort naturområde ([[Amager Fælled]]), ett av Nordens största köpcentra (Field's), och mycket annat. Amager förkortas ofta Ama'r för att återspegla det danska uttalet.
>
>     Amager är delvis en [[konstgjord ö]], delvis en naturlig sådan. T.ex. så anlades en konstgjord ö i anslutning till det redan existerande Amager Strandpark, när denna rustades upp väsentligen 2005. Ön är mycket låg och vissa delar ligger under havsytan, framför allt det genom fördämning och utfyllnad skapade området [[Kalvebod Fælled]] (Vestamager). Ön började uppodlas i större skala under 1500-talets första hälft då kung [[Kristian II]] bjöd in nederländska bönder till att odla upp ön för kronans räkning. Flertalet slog sig ned i fiskeläget [[Dragør]] där många än idag har namn med nederländskt ursprung som Neels och Tønnes.
>
>     Den stora utvecklingsfasen började på 1800-talet då flera fabriker, bostadsområden och militär verksamhet utvecklade sig på ön. Det fanns ett antal stora och kända fabriker på ön och den industriella prägeln höll sig väldigt tydlig långt in på 1900-talet. Det förekommer också områden som tidigare varit sommarstugeområden och där det än idag finns kolonilottområden, t.ex. i Sundbyöster ut mot kusten. Under 1900-talet började militären att utveckla stora övningsområden på ön vilket resulterade i Vestamager som idag innefattar ett stort naturområde endast några kilometer från Köpenhamns centrum. Under senare delen av 1900-talet har flygplatsen [[Kastrups flygplats|Kastrup]] som ligger på ön expanderat kraftigt och i samband med byggandet av [[Öresundsförbindelsen]] började även ett stort område byggas som kallas för [[Ørestad]].
>
>     Ön, som är förbunden med övriga Köpenhamn genom flera broar, [[järnväg]] och sedan 2017 även [[Köpenhamns tunnelbana|Metro]], har under de senaste 10-20 åren förändrats mycket. Dels har flera områden exploaterats med nya moderna kvarter lägenheter, t.ex. i områdena Örestad, [[Islands Brygge]] och Amager Strand, och dels så har Amagers närhet till Köpenhamn raderat dess tidigare rykte som en industriell förstad till Köpenhamn. Idag finns det områden av alla sorter på ön - tack vare flygplatsen finns ett aktivt näringsliv kvar, samtidigt som de nya områdena och inte minst upprustningen av t.ex. Amager Strand har förändrat bilden av ön. Idag är flera delar av ön mycket attraktiva bostadsområden.
>
>     Ön återfick i samband med Öresundsbron även en järnvägsförbindelse vilket inte funnits sedan 1960-talet. En [[motorväg]] byggdes samtidigt över öns mellersta del. [[Köpenhamns metro]] går till Kastrup och Vestamager med ett antal stationer på ön. Kring metro- och regionaltågsstationen Ørestad har ett stort kontors-, handels- och bostadsområde uppförts under [[2000-talet (decennium)|00-talet]]. Detta sträcker sig från [[Bella Center]] i norr till metrostationen [[Vestamager station|Vestamager]] i söder. Även om området inte ligger vid kusten har Ørestad delvis fått karaktären av en hamnstad genom anläggandet av konstgjorda kanaler och dammar. Vid [[Islands Brygge]] finns en äkta sjöstad.
>
>     Före [[Öresundsförbindelsen]]s uppförande hade småstaden [[Dragør]] en färjeförbindelse med [[Limhamn]] i södra [[Malmö]]. På grund av denna fanns i Dragør tidigare en betydande gränshandel. Denna har under 2000-talet helt upphört liksom den genomgående trafiken.
>
>     Öns västra del, som utvanns från Öresund under 1920- och 30-talen, är med undantag för Bella Center, Ørestad samt ett koloniområde, avsatt som natur- och rekreationsområde. Marken är ängsmark (odlad mark finns dock på den naturliga delen av öns södra del).
>     Längst i söder finns den flerhundraåriga skogen [[Kongelunden]]. På öns östra del, vid Öresundskusten, finns [[Amager Strandpark]] med en populär sandstrand. Området har omgestaltats, med en [[konstgjord ö]] och en [[lagun]] innanför. Nyinvigningen av parken, som funnits sedan 1934, ägde rum 2005.
>
>     Amager är den tätast befolkade ön i Danmark. Den näst mest tätbefolkade är [[Thurø]].
>
>     == Se även ==
>     *[[Amagerbro]]
>     *[[Amagerport]]
>     *[[Amagerbroderi]]
>
>     ==Externa länkar==
>
>
>
>
>
>
>     [[Kategori:Amager| ]]
>     [[Kategori:Öar i Region Hovedstaden]]
>
>
>       "), (2,Afrika,"
>         Afrika
>
>
>
>
>
>
>
>
>
>
>
>
>
>     '''Afrika''' är [[jorden]]s näst största [[kontinent]] (efter [[Eurasien]]) och även jordens näst största världsdel efter [[Asien]], både vad gäller [[areal]] och [[folkmängd]]. Med världsdelens öar inräknade mäter Afrika 30 244 050 km², vilket motsvarar 20,3 % av jordens landmassa eller cirka 6 % av jordens totala area. Omkring 22 miljoner km² av dessa ligger i [[tropikerna]] vilket gör den afrikanska kontinenten till världens varmaste kontinent. I Afrika bor det ungefär 1 miljard människor i 55 länder – en sjundedel av jordens befolkning. Dess längd i nordlig-sydlig riktning är omkring 8 000 km och dess största bredd omkring 7 800 km.
>
>     == Etymologi ==
>     Ordet Afrikas [[etymologi]]ska bakgrund är dunkel, och ordet har dessutom haft olika betydelse under olika [[epok]]er.
>
>     I äldre tider kallades allt land väster om [[Egypten]] för [[Libyen]]. Enligt den [[Grekland|grek]]iska historieskrivaren [[Herodotos]] berättelse seglade [[fenicisk]]a sjömän på uppdrag av den egyptiska kungen Neko runt Libyen, det vill säga [[kontinent]]en Afrika redan omkring 600 f.Kr.
>
>     När [[Romerska riket|romarna]] först använde termen ''[[Africa terra]]'' avsåg de före [[andra puniska kriget]] endast trakten kring [[Karthago]], det vill säga motsvarande ungefär dagens Tunisien där själva staden Karthago låg samt stora delar av Libyen och Algeriet. Romarna uppfattade först [[Egypten]] som en del av [[Asien]], och det var först geografen [[Klaudios Ptolemaios|Ptolemaios]] som drog gränsen vid [[Suez]]. Under kejsartiden kom ''Africa'' att avse hela det området romarna kände till av kontinenten, och allt eftersom européerna har upptäckt mer och mer av kontinenten har termen avsett ett allt större område. Namnet ska ha avsett landet där ''afri'' (singularis: ''afer'') bodde. ''Afer'' avsåg namnet på ett folk som kan motsvara dagens [[berber]]. [[Viking]]arna under 800- och 900-talen kallade norra Afrika för Blåland.
>
>     == Geografi ==
>
>     Geologiskt är Afrika en mycket gammal kontinent, i huvudsak uppbyggd av [[urberg]]. Under jordens forntid var den en del av jättekontinenten [[Gondwana]]land. Denna gled isär under [[Jura (period)|juraperioden]] och [[Krita (geologi)|kritaperioden]] för 100-150 miljoner år sedan, och så fick Afrika sin nuvarande form.
>
>     Inom sina regelbundna kuster rymmer den en total yta på omkring 30 244 050 km² inklusive öarna. Afrika åtskiljs från [[Europa]] av [[Medelhavet]] och förenas med [[Asien]] i sitt nordöstra hörn genom [[näs]]et vid [[Suez]], omkring 130 km i bredd. Omkring 8 000 km skiljer den nordligaste punkten, [[Ras ben Sakka]] i [[Tunisien]], från dess sydligaste, [[Kap Agulhas]] i [[Sydafrika]]. Från [[Kap Verde]], dess västligaste punkt, till [[Ras Hafun]] i [[Somalia]] är det omkring 7 400 km. Afrikas kuster mäter omkring 28 000 km (1 km kust på 1 057 km²) varav [[Medelhavet]] står för 5 400 km, [[Atlanten]] 11 000 km, [[Indiska oceanen]] 8 700 km och [[Röda havet]] 2 900 km, och avsaknaden av djupare [[bukt]]er och betydande [[halvö]]ar blir uppenbar om man jämför med Europas 9 700 000 km² som omgärdas av 32 000 km kust (1 km kust på 278 km² land), eller för det [[fjord]]rika [[Norge]], där motsvarande siffra 385 000 km² land och en kuststräcka på 20 000 km (1 km kust per 19 km² land).
>
>     Afrikas genomsnittliga höjd över havet är omkring 600 m, vilket motsvarar de båda amerikanska kontinenternas höjd men är lägre än den asiatiska på omkring 950 m. Jämfört med de andra kontinenterna utmärker sig Afrika med en liten andel landområden på mycket hög respektive mycket låg höjd. Landområdena under 180 [[Meter över havet|m ö.h.]] upptar en osedvanligt liten del av den totala landmassan, och även de högsta delarna, som är betydligt lägre än motsvarande i Asien, utgörs av isolerade berg och bergskedjor. Generellt ligger de högre [[platå]]erna i öst och söder och de lägre delarna åt väst och norr och med en linje från ungefär vid [[Röda havet]]s mitt till [[Kongofloden]]s mynning kan kontinenten delas upp i ett lågland i norr och ett högland i söder. Man kan således dela upp Afrika i fyra övergripande landområden:
>     # Slättlanden vid kusterna – ofta med [[Mangrove|mangroveträsk]] insprängda – når aldrig långt in från kusterna utom vid vattendragens nedre delar. Slamrika slättland finns egentligen bara vid de större flodernas deltan, och annars utgör kustområdena det lägsta steget i en serie terrasser som bildar stigningen upp till de inre platåerna.
>     # [[Atlasbergen]] i norr som ligger isolerat av [[Sahara]]öknen som bitvis ligger under havsnivån.
>     # De södra och östra höglanden som ligger på mellan 600 och 1 000 m ö.h.
>     # De nordliga och västra slättlanden, inklusive Sahara, som kantas och genomfars av stråk med högra landområden, men i allmänhet håller sig under 600 m ö.h.
>
>     Skog täcker cirka en femtedel av Afrikas yta.
>
>     Hela Afrika har ett [[Tropiskt klimat|tropiskt]] eller [[Subtropiskt klimat]].
>
>     === Fauna ===
>
>
>
>     Afrika är huvudsakligen känt för den öppna [[savann]]en med väldiga hjordar av hovdjur som [[afrikansk buffel]], gnuer, [[antiloper]], [[giraff]]er och [[zebror]]. Dessa jagas av stora rovdjur som [[lejon]], [[hyenor]] och [[Gepard|geparder]]. Den afrikanska djungeln befolkas av [[ormar]] och [[primater]] samt av vattenlevande djur som [[krokodiler]] och [[groddjur]]. Afrika har även det största antalet arter som kan räknas till [[megafauna]]n, bland annat [[afrikanska elefanter]], [[trubbnoshörning]] och [[flodhäst]], på grund av att kontinenten bara i mindre utsträckning var påverkad av det [[massutdöende]] som skedde under [[pleistocen]].
>
>     == Historia ==
>
>     Den första mänskliga migrationen, från kontinenten men även inom Afrikas kuster, kan spåras [[lingvistisk]]t, kulturellt och genom datoranalyser av [[genetik|genetiskt]] material. Någonstans i [[Östafrika]] tror dagens forskare att de första [[hominider]] uppkom. I [[Etiopien]] har man hittat ett två miljoner år gammalt skelett som uppkallades [[Australopithecus afarensis|Lucy]], den första kända mänskliga [[primater|primaten]].
>
>     === Tidigare civilisationer och handel ===
>     [[Fil:African-civilizations-map-pre-colonial.svg|miniatyr|höger|Några av de mest kända afrikanska civilisationerna. Civilisationer från olika tidsepoker visas parallellt.]]Lingvistiska spår tyder på att [[Bantu]]folk emigrerade från [[Västafrika]] in i [[khoisan]]folkets (de som tidigare kallades [[bushmän]] och [[hottentott]]er) område och undanträngde dem. Bantufolken använde en uppsättning grödor, bland annat [[kassava]] och [[jams]], som var mer lämpade för jordbruk i det [[tropik|tropiska]] Afrika, och med sitt jordbruk kunde bantu försörja en betydligt större folkmängd än de undanträngde jägare eller samlare. Bantufolkens traditionella bosättningsområde sträcker sig från [[Sahara]]öknen till de [[Tempererat klimat|tempererade regionerna]] i söder. Historiskt har deras vapen varit spjut, sköld, pil och båge. Det för [[khoisanspråk]]ens karaktäristiska klickljuden återfinns i [[nguni (språk)|nguni]]- och [[bantu]]språk som till exempel [[xhosa]] och [[zulu]]. Khoisanspråken talas idag främst av utspridda minoritetsgrupper i södra Afrika i länderna kring [[Kalahariöknen]].
>
>     Afrika söder om Sahara har bestått olika stater och riken, mindre stadsstater samt [[stam (samhällsvetenskap)|stammar]] och [[klan]]er. Det forna egyptiska riket, är en av världens äldsta stater, och den mest kända av de äldre afrikanska [[civilisation]]erna. Det fanns även statsbildningar i [[Nubien]], [[Ghana]], [[Timbuktu]], [[Kanem-Bornu]] och i nuvarande [[Zimbabwe]].
>
>     [[Nordafrika]] kom från och med 600-talet under arabiskt inflytande – en tid av kulturell blomstring – och med kamelens hjälp spreds [[Islam]] och den arabiska kulturen genom Sahara och vidare söderut. Arabiska och persiska handelsmän spred likaså Islam längs med den östafrikanska kusten parallellt med en omfattande svart slavhandel. Detta östafrikanska handelsutbyte med araber och perser sträckte sig ända bort till Java, därmed den etniska, kulturella och språkliga mångfalden på Östafrikas kust, [[Madagaskar]] och [[Komorerna]].
>
>     === Arabvärlden och Europa utforskar kontinenten ===
>     Det arabiska intresset för Afrika går långt tillbaka, men handel med slavar från centra som Lamu och Zanzibar vid östkusten har pågått åtminstone sedan senare delen av 1000-talet e.Kr. De miljoner slavar som exporterades till arabvärlden och österut under den [[arabiska slavhandeln]] finns idag kvar som en stor etnisk grupp där, i stor utsträckning integrerad med den ursprungliga arabiska befolkningen. Slavhandel med svarta pågår fortfarande i vissa arabländer, till exempel Sudan.[http://www.hrw.org/backgrounder/africa/sudanupdate.htm Human Rights Watch] [http://news.bbc.co.uk/2/hi/programmes/this_world/6458377.stm BBC World]
>
>     Under [[1300-talet]] började vissa europeiska handelsexpeditioner få kontakt med Nord- och Västafrika.
>
>     Från 1500-talets mitt fram till förbudet mot slavhandel i Storbritanniens parlament 1807, som kom att få full kraft först något årtionde senare, handlades slavar från lokala stamledare i främst Väst- och Centralafrika. Flera miljoner afrikaner såldes på detta vis som slavar till främst de Europeiska kolonierna i Amerika.
>
>     Tidiga kolonisatörer var [[araber]], [[Portugal|portugiser]] och [[Spanien|spanjorer]]. Därefter koloniserades kontinenten av framförallt av [[Storbritannien]], [[Frankrike]], [[Spanien]], [[Italien]], [[Belgien]], [[Nederländerna]], [[Tyskland]] och [[Portugal]].
>
>     === Koloniseringen ===
>
>     [[Fil:Map_of_Africa_from_Encyclopaedia_Britannica_1890.jpg|miniatyr|Politisk karta över Afrika från 1890.]] Från slutet av 1000-talet koloniserade arabiska köpmän den afrikanska östkusten. Holländarna koloniserade på 1600-talet den då relativt folktomma södra udden av Afrika. Portugiserna grundade flera kolonier i västra Afrika och vid nuvarande Moçambique. I mitten av [[1800-talet]] leddes flera expeditioner in i de centrala delarna av Afrika, bland annat av Livingstone och Speke, men också den svenske upptäcktsresanden Andersson. Denna kolonisering förändrade i grunden det ekonomiska livet på kontinenten, och innebar i flera fall ensidig export av naturresurser. [[Leopold II av Belgien|Kung Leopolds]] rovdrift på sin privata koloni [[Kongostaten]] är ökänd och kostade miljontals afrikanska liv.
>
>     I Afrika fanns efter första världskriget endast tre självständiga stater nämligen konungariket Egypten, republiken Liberia och konungariket Abessinien. Tanger bildade med sitt område en internationell zon. För övrigt var hela världsdelen uppdelad på kolonier åt europeiska makter.
>
>     Flertalet kolonier erhöll [[självständighet]] under slutet av 1950-talet och början av 1960-talet. Avkoloniseringen resulterade i flertalet fall med katastrof, där diktatur och etnisk rensning vanligen blev följden, till exempel i Rhodesia (nuvarande Zimbabwe), Rwanda, Uganda, Nigeria m.fl. stater.
>
>     == Politik ==
>
>     [[Fil:Electoral_Democracies_in_Africa,_Freedom_House_2007.png|miniatyr|De blå länderna är demokratier enligt tankesmedjan Freedom House.]]
>     Det postkoloniala Afrikas politik har i hög grad präglats av diktatur, statskupper och brist på [[mänskliga rättigheter]].
>
>     [[Nation]]erna i Afrika har ofta visat sig vara instabila [[stat]]er som efter självständigheten hemsökts av korruption, våldsamma [[statskupp]]er, [[despotism]] och [[dikatur|militärdiktaturer]]. Under perioden [[1960]]-[[1989]] genomfördes över 70 kupper och 13 [[mord]] på [[president]]er. Under det [[kalla kriget]] kämpade [[USA]] och [[Sovjetunionen]] om makten i Afrika, och tillsammans med länder som [[Frankrike]] och [[Kuba]], stödde de aktivt diktaturer och iscensatte flera av dessa kupper och illgärningar mot demokratiskt valda regeringar. Enligt en spridd uppfattning beror många [[militär]]a konflikter om [[territorium|territorier]] och [[gräns]]er under Afrikas moderna historia på de gränser från kolonial tid som huvudsakligen drogs upp efter [[Kongokonferensen]], men detta kan inte visas eftersom något jämförelseområde inte finns. Konflikter i Afrika har ofta drivits av tillgången till kontinentens rika [[råvara|råvarutillgångar]] – [[diamant]]er och [[metall]]er som [[guld]], [[platina]], [[uran]] och [[koppar]].
>
>     Korruption och havererade politiska program har lett till utbredd [[undernäring]] och undermålig infrastruktur orsakar alltjämt brist på mat och rent vatten. Sanitetsproblemen har också orsakat en akut spridning av bland annat [[HIV]]-viruset som leder till [[sjukdom]]en [[AIDS]].
>
>     Trots nöd och umbäranden finns vissa positiva tecken – [[demokrati]]skt valda [[regering]]ar växer i antal även om de inte utgör någon majoritet ännu. Genom påtryckningar från till exempel [[Internationella valutafonden]] (IMF) har många afrikanska stater lyckats vända en nedåtgående ekonomisk spiral som varat i flera decennier till en positiv tillväxt. Om man beaktar IMF:s katastrofala misslyckanden i länder som [[Argentina]] och [[Filippinerna]] är det dock oklart i vilken omfattning dessa konventionella ekonomiska kriterier ger en rättvis bild av vad Afrika faktiskt behöver.
>
>     De afrikanska ledarna har i olika omgångar sökt former för samarbete sinsemellan. Hittills har dock detta inte lyckats särskilt väl. Under [[inbördeskrig]]et i [[Kongo-Kinshasa|Kongo]] var det grannstaterna som spädde på konflikten snarare än utomafrikanska stater, och motivet var främst att skaffa kortsiktiga vinster genom att utvinna diamanter och andra mineraler. Antalet döda i konflikten har uppskattats till 3,5 miljoner under en femårsperiod.
>
>     De politiska samarbetsorganisationer man skapat har inte kunnat leva upp till förväntningarna. Organisationen för afrikansk enighet (OAU 1963-2002) kallades till exempel diktatorsklubben för dess tendens att alltid stödja de sittande politiska ledarna och ägna väldigt lite energi åt att främja den vanlige afrikanens mänskliga rättigheter. 2002 skapades i stället ''[[Afrikanska unionen]]'', som är tänkt att underlätta ett framtida samarbete mellan kontinentens stater. Hittills har denna organisation inte klarat att hantera till exempel utrotningskriget i Darfur-provinsen i Sudan, där en arabisk statsledning tillåtit slakt på hundratusentals av bantubefolkningen i sydväst. En avgörande skillnad mellan såväl OAU som AU å ena sidan och EU å den andra, är att EU startades av en grupp länder som satte demokrati och mänskliga rättigheter i fokus för den egna politiken, och som krävt detta av tillkommande medlemsstater. OAU och AU har från början i stället varit territoriellt inriktade, och visserligen haft demokrati och mänskliga rättigheter som skrivelser i sina stadgar, men ytterst sällan satt kraft bakom dessa ord. På så sätt liknar AU mer FN än EU.
>
>     == Ekonomi ==
>
>     [[Fil:Bakweri cocoyam farmer from Cameroon.jpg|250px|miniatyr|Jordbrukare på en [[jams]]odling i Kamerun]]
>     Afrikas ekonomi omfattar ungefär 887 miljoner människor i 54 olika stater ([[Juli 2005]]). Afrika är världens fattigaste kontinent, och den är, i medeltal, fattigare än den var för 25 år sedan. På [[FN]]:s Human Development Report från [[2003]] (som rankade världens 175 länder enligt en rad kriterier) var plats mellan 151 och 175 intagna av länder i Afrika.
>
>     Det finns dock länder som lyckas bra ekonomiskt, bland andra [[Botswana]], [[Ghana]] och [[Sydafrika]]. [[Etiopien]] har också en relativ bra ekonomi tack vare den höga tillväxten i landet på cirka 11-15% per år. Sydafrika har en väl fungerande inhemsk aktiehandel och har stora naturtillgångar i bland annat [[guld]] och [[diamanter]] och ett väl fungerande rättsväsen. Det finns i Sydafrika även tillgång till investeringskapital, stora marknader och utbildad arbetskraft. Det finns även andra afrikanska länder, exempelvis [[Egypten]], som länge haft en väl fungerande handelssektor och ekonomi.
>
>     Kontinentens kust mot [[Atlanten]] återfinns stora fyndigheter av [[Petroleum|olja]]. I [[Nigeria]] finns en av världens mest betydande oljereserver och landets ekonomi är bland de snabbast växande i världen.
>
>     Afrikas jordbruk sysselsätter drygt hälften av Afrikas befolkning och är den viktigaste näringen. Cirka 6 % (ungefär 1 814 600 km²) av världsdelens yta är uppodlad, mest i savannområdena. I stora delar av de brukningsbara områdena är jordens bördighet ganska låg. De bästa jordarna återfinns i begränsade höglandsområden som i till exempel [[Kamerun]] och [[Östafrika]].
>
>     == Demografi ==
>
>     ===Befolkning===
>     Människan, ''[[Homo sapiens|Homo sapiens sapiens]]'', antas härstamma från Afrika och spred sig härifrån till de omgivande kontinenterna. Den svarta befolkningen söder om Sahara tillhör den ursprungliga befolkningen som hela tiden levat i Afrika.
>
>     Norr om Sahara präglar samröret med övriga [[Medelhavet|Medelhavsområdet]] och inte minst sydvästra [[Asien]] både befolkning, språk och kultur.
>
>     På ön [[Madagaskar]], som länge var obebodd, härstammar majoriteten av befolkningen från människor som emigrerat från [[Indonesien]]. Under kolonialtiden skedde inflyttning från [[Europa]] och [[Asien]] till södra Afrika, där den icke-svarta befolkningen idag utgör betydande minoritetsgrupper.
>
>     Afrikas befolkning uppvisar många för utvecklingsregioner typiska drag, som höga [[födelsetal]] i kombination med sjunkande [[Mortalitet (demografi)|dödstal]] med snabb befolkningsökning och låg medelålder (45 % är under 15 år) som följd. Även om befolkningstätheten varierar kraftigt mellan olika regioner är Afrika generellt en glestbefolkad kontinent, med i genomsnitt 27,8 invånare per km².
>
>     Afrika är den kontinent som har den lägsta urbaniseringsgraden, med endast 25% boende i städer. Även här är variationerna stora, och i norra och östra Afrika finns en lång urban tradition.
>
>     === Språk ===
>
>     [[Fil:African languages SV.png|miniatyr|300px|Språkfamiljer i Afrika]]
>     Uppskattningsvis finns det ungefär 1800 talade [[språk]] i Afrika idag. Vissa av dessa, till exempel [[Swahili]], [[Hausa]], och [[Yoruba (språk)|Yoruba]], talas av många miljoner människor, medan det finns andra som bara talas av några hundra eller ännu färre. Afrika har även många olika [[teckenspråk]]. De flesta afrikaner talar som modersmål ett språk tillhörande någon av de fyra språkfamiljerna [[afroasiatiska språk]], [[nilo-sahariska språk]], [[Niger-Kongospråk]] eller [[khoisanspråk]].
>     Med några få nämnvärda undantag i östra Afrika, har nästan alla afrikanska länder språk som har sitt ursprung utanför världsdelen som officiellt språk, på grund av kolonialismen och mänsklig migration. Till exempel används i ett stort antal afrikanska länder [[franska]] eller [[engelska]] som administrativt språk för stat, handel, utbildning och media. [[Arabiska]], [[portugisiska]], [[afrikaans]] och [[malagassiska]] är andra exempel på språk med ursprung utanför Afrika som idag används av miljoner afrikaner, både i offentliga och privata sammanhang.
>
>     === Religion ===
>
>     [[Kristendom]] och [[islam]] är de största religionerna i Afrika, 46,3 % av alla afrikaner är kristna och 40,5 % är muslimer. Resten 11,8 % tillhör olika lokala afrikanska religioner som funnits före den europeiska kolonisationen. Ett mindre antal afrikaner är [[Hinduism|hinduer]], eller har tro från [[Judendom|judisk]] tradition. Exempel på [[Afrikanska judar|afrikanska folk som tillhör judendomen]] är [[Beta Israel]], [[Lemba]]folk och [[Abayudaya]] i östra [[Uganda]]. Sydafrika och Angola har över 55% kristna katoliker.
>
>     == Kultur ==
>
>     Afrika har ett stort antal överlappande kulturer, och flera tusen olika etniska grupper. Den mest konventionella distinktionen är den mellan området söder om Sahara och området norr om öknen. De nordliga länderna associerar sig främst med arabkulturen, medan de sydliga länderna består av i sin tur många olika kulturella områden. En av de största kulturella identiteterna är den som innefattar alla folk som talar något [[bantuspråk]].
>
>     Det går också att göra en indelning i gamla franska Västafrika å ena sidan, och resten av Afrika (mer specifikt de brittiska eller portugisiska kolonierna i södra och östra Afrika) å andra sidan. En annan kulturell linje går mellan de afrikaner som håller kvar vid sina gamla traditionella livsstilar, och de som har anpassat sig till en mer modern livsstil. Det går även att dela upp traditionalisterna i sådana som ägnar sig åt uppfödning av boskap, och sådana som ägnar sig åt jordbruk.
>
>     [[Afrikansk konst]] och [[afrikansk arkitektur]] reflekterar de afrikanska kulturernas diversitet. De äldsta exemplen på afrikansk konst är 75 000 år gamla pärlsmycken gjorda av [[nassarius]]skal, dessa hittades i [[Blombosgrottan]]. [[Pyramiderna i Giza]] var världens högsta byggnader under 4 000 år. [[Klippkyrkorna i Lalibela]] i [[Etiopien]], bland vilka ''[[Bete Giyorgis]]'' (St. Göranskyrkan) är mest välkänd, är också kända exempel på afrikansk arkitektur.
>
>     == Regioner och territorier ==
>     [[Fil:Africa-regions.png|miniatyr|Regioner i Afrika:
>
>
>
>
>     ]]
>
>
>     == Se även ==
>     * [[Nordafrika]]
>     * [[Västafrika]]
>     * [[Östafrika]]
>     * [[Centralafrika]]
>     * [[Södra Afrika]]
>     * [[Afrikas klimat]]
>
>     == Referenser ==
>
>     === Noter ===
>
>     [http://esa.un.org/unpd/wpp/Excel-Data/population.htm/ World Population Prospects: The 2010 Revision]  [[Förenta nationerna|United Nations]] (Department of Economic and Social Affairs, population division)
>     [http://esa.un.org/unpd/wpp/Excel-Data/population.htm/ World Population Prospects: The 2010 Revision]  [[United Nations]] (Department of Economic and Social Affairs, population division)
>
>
>     === Källor ===
>
>     * [https://www.courses.psu.edu/test/test100_hkr/AFIM/Main_HTML/Intro_1.html Pennsylvania State University – ''African Ethnic Groups'' och andra kartor]
>     * [http://www.pygmies.info/ Pygméerna i Afrika] Kultur och musik av de första urinvånarna i Afrika
>     * Afrika i skolböckerna, M. Palmberg 1987
>
>     == Externa länkar ==
>
>
>
>
>
>
>
>
>     [[Kategori:Afrika| ]]
>     [[Kategori:Wikipedia:Basartiklar]]
>
>
>       "), (3,Amerika,"
>         Amerika
>
>
>
>
>
>
>
>
>
>
>     [[Fil:Americas (orthographic projection).svg|miniatyr|250px|Karta som visar Nord- och Sydamerika, tillsammans benämnda Amerika]]
>     '''Amerika''' (engelska: ''the Americas'') består av landmassorna mellan [[Atlanten]] och [[Stilla havet]] på [[västra halvklotet]], namngivna efter den italienske upptäcktsresanden [[Amerigo Vespucci]]. Amerika uppfattas vanligen idag som två världsdelar, [[Nordamerika]] och [[Sydamerika]].[https://snl.no/Amerika Store norske leksikon/ Amerika] Amerika indelas geografiskt i Nordamerika (med [[Centralamerika]]) och Sydamerika. [[Grönland]] ingår geografiskt i Nordamerika, men räknas politiskt och kulturellt till Europa. Kulturgeografiskt avgränsas också ofta [[Latinamerika]], då inklusive både Central- och Sydamerika samt Mexiko, och [[Angloamerika]], som är USA och Kanada. Områdets totala landyta är ungefär 42 miljoner kvadratkilometer och är således något mindre än [[Asien]]s.
>
>     Ofta används ''Amerika'' och ''amerikansk'' för enbart landet [[USA]], Amerikas förenta stater, vilket medför en förväxlingsrisk med dubbelkontinenten som helhet men är jämförbart med hur andra landsnamn förkortas i dagligt tal (exempelvis [[Mexiko]], vars fullständiga namn översatt till svenska är Mexikos förenta stater). På engelska, portugisiska och spanska kommer man ofta runt problemet genom att de två världsdelarna kallas ''the Americas'' respektive ''las Américas'' ([de två] Amerikorna, ett uttryck som dock aldrig har etablerats på svenska).
>
>     == Namnet ==
>     Amerika namngavs efter [[Amerigo Vespucci]], som var den första som insåg att Amerika var en kontinent som nyligen kommit till kännedom för européer, som inte var [[Östindien]]. Namnet används 1507 för första gången i en världskarta som skapades av den tyske geografen [[Martin Waldseemüller]] och beskriver där [[Sydamerika]]. 1511 omnämnes det i ett engelskt skådespel och 1516 används det av [[Leonardo da Vinci]] på hans världskarta och vann alltså snabb utbredning. Spanjorerna brukade i början ha andra namn. De kallade landet för [[Nya världen|Den nya världen]] eller ''las Indias occidentales'' (västra Indien). För Sydamerika behöll de under hela 1500-talet namnen ''Peruana'' och ''Brasilia''. Först på 1600-talet blev namnet Amerika allmänt antaget.
>
>     == Historia ==
>     * se [[Amerikas historia]]
>
>     == Källor ==
>
>     *
>
>
>     == Externa länkar ==
>
>
>
>
>
>
>
>     [[Kategori:Amerika]]
>     [[Kategori:Kontinenter]]
>
>
>       "), (5,Abbekås,"
>         Abbekås
>
>
>
>
>
>
>
>
>
>
>
>
>     '''Abbekås''' (uttalas abbekå-s)[http://runeberg.org/ortnamn/0008.html Runeberg.org: Svenska ortnamn med uttalsuppgifter] sida 4 är en [[tätort]] i [[Skivarps distrikt]] ([[Skivarps socken]]) på [[Söderslätt]] i [[Skurups kommun]].
>
>     == Historia ==
>     Abbekås var från början ett fiskeläge på [[Skåne]]s sydkust mellan [[Ystad]] och [[Trelleborg]], omtalad redan på medeltiden. Onsdagen den 13 november 1872 förstördes stora delar av byn av en stormflod kallad [[Stormfloden 1872|Backafloden]], framkallad av en orkan som drog in över sydkusten från nordost. 34 hus förstördes, alla fiskeredskap, båtar och befolkningens vinterförråd. Vattenståndet ökade till 3,6 meter över det normala, vilket fortfarande är svenskt rekord. Redan året därpå inleddes med hjälp av statsbidrag arbetet med en riktig hamn, och fisket och kusthandeln kunde åter komma igång. En ny storhetstid för fisket inträffade under andra världskriget.
>
>     2011 renoverades pirar och kajer, och inloppet gjordes om för att öka säkerheten i yttre hamnbassängen.
>
>     === Befolkningsutveckling ===
>
>
>     ==Namnet==
>     Namnet skrevs 1536 ''Abbekassz''. Efterleden är ''kås'', 'båtplats, mindre hamn'. Förleden innehåller förmodligen mansnamnet ''Abbi''..
>
>     ==Samhället==
>     Abbekås hamn består av en yttre och en inre hamnbassäng, där den inre används av fiskarna och den yttre är till för fritidsseglare och gästbåtar. Här finns såväl hamnkrog och hotell som ett modernt tvätt/dusch/toaletthus för besökare. Längre upp i byn finns ytterligare en restaurang och en livsmedelsaffär, ett konstgalleri och två Bed&Breakfast-ställen. Strax öster om byn ligger [[Abbekås GK|Abbekås Golfklubb]].
>
>     ==Abbekås i musiken==
>     Abbekås är omsjunget av [[Edvard Persson]] i sången ''Abbekåsagåsen Joakim''.
>
>     == Kända profiler ==
>     *[[Lorens Brolin]], spelman
>
>     == Bilder ==
>
>     Abbekås 2.jpg|Abbekås hamn
>     Hamnen Abbekås.jpg|Abbekås hamn
>     Magasinet Abbekås.jpg|Magasinet vid Abbekås hamn
>     Coast line Abbekås.jpg|Strandlinjen väster om Abbekås hamn
>     Abbekås 3.jpg|Abbekåsagåsen Joakim
>
>
>     ==Omgivande orter==
>
>
>     == Noter ==
>
>
>     == Externa länkar ==
>     *[https://web.archive.org/web/20170517190948/http://abbekas.org/ Abbekås Byalag]
>
>
>
>     [[Kategori:Orter i Skurups kommun]]
>     [[Kategori:Fiskelägen]]
>     [[Kategori:Tätorter i Sverige]]
>
>
>       "), (7,Alingsås,"
>         Alingsås
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>     [[Fil:Suecia 3-045 , Allings åås - Alingsås.jpg|miniatyr|höger|Alingsås omkring år 1700. Ur [[Suecia antiqua et hodierna]], och därmed troligen inte helt tillförlitlig.]]
>     [[File:Bronsskulpturen Våren i Museiparken i Alingsås, skapad av skulpturprofessorn Eric Grate under första hälften av 1900-talet, fotograferad den 7 sept 2006.JPG|mini|Bronsskulpturen Våren i Museiparken i Alingsås, skapad av skulpturprofessorn [[Eric Grate]].]]
>
>     '''Alingsås''' () är en [[tätort]] i [[Västergötland]] i [[Västra Götalands län]] med knappt 27 000 invånare (2016) och är centralort i [[Alingsås kommun]]. Hela kommunen har drygt  invånare ().
>
>     Orten ligger vid sjöarna [[Mjörn (sjö)|Mjörn]] och [[Gerdsken]] och genomskärs av [[Säveån]] och Gerdska ström (''Lillån'') och Färgen som ligger vid Hjälmared. Alingsås genomkorsas av [[E20 (Sverige)|E20]], [[Länsväg 180|väg 180]] och [[västra stambanan]].
>
>     Alingsås är känt för sina [[kafé]]er och [[Jonas Alströmer]] som populariserade [[potatis]]en i Sverige.
>
>     Alingsås kallas ibland för Potatisstaden, och varje år firas det med en potatisfestival.
>
>     == Historia ==
>     Ortens namn, belagt sedan 1357, kommer möjligen från sammansättningen av orden aling och ås, syftande på att en person från [[Ale härad|Ale]], kallad ''aling'', bosatt sig på platsen. Efterleden -ås kommer troligen av att personen bosatte sig på åsen som idag ligger vid stadsdelen Nolby. Alltså kallades platsen för Alings ås.
>
>     Alingsås fick [[stadsprivilegium|stadsrättigheter]] 21 september [[Gamla stilen och nya stilen|g.s.]] (1 oktober [[Gamla stilen och nya stilen|n.s.]]) år 1619 av [[Gustav II Adolf]], vilket gör Alingsås till en av de äldsta städerna i regionen, sedan den utgjort rekommenderad inflyttningsort för [[borgare]] som inte tilläts återuppbygga det i krig brända [[Nya Lödöse]].
>
>     1639 erhöll man privilegier och stadsvapen. Staden förde dock under de första åren en tynande tillvaro och hade i början av 1720-talet omkring 150 invånare, men fick ett uppsving från 1720-talet och framåt i samband med etableringen av [[Jonas Alströmer]]s manufakturverk. Verksamheten var dock beroende av [[statsstöd]] och drabbades hårt av manufakturkrisen i slutet av 1700-talet. Alingsås hade då omkring 1 000 invånare.''Nationalencyklopedin'' multimedia plus, 2000
>
>     1724 och 1779 drabbades Alingsås av stadsbränder som förstörde större delen av staden.
>
>     Manufakturverkets fortsatta tillbakagång och slutliga avveckling 1847 medförde en långsam utveckling för Alingsås. Efter att Alingsås anslutits till [[Västra stambanan]] 1859 och [[Alingsås bomullsväveri]] anlagts 1862 följde en period av uppgång och Alingsås utvecklades till ett viktigt textilindustricentrum. [[Tekokrisen]] på 1960- och 1970-talen slog hårt mot Alingsås.
>
>     ===Administrativa tillhörigheter===
>     Från att varit kyrkby i [[Alingsås socken]], utbröts orten 1643 till [[Alingsås stad]]. Efter [[kommunreformen 1862]] låg sedan bebyggelsen i [[stadskommun]]en Alingsås stad och med en mindre del före 1952 i Alingsås socken. Staden utökades 1952 och 1955 med grannsocknar, bland annat Alingsås socken, och orten utgjorde därefter, som dess centralort, en liten del av stadskommunens område. 1971 uppgick Alingsås stad i Alingsås kommun och orten är sedan dess [[centralort]] i kommunen.
>
>     Alingsås tillhör sedan 1967 [[Alingsås församling]] för att innan dess tillhört Alingsås stadsförsamling och med en del Alingsås landsförsamling.
>
>     Alingsås tillhörde judiciellt till 1965 [[Alingsås rådhusrätt]] och därefter till 1971 [[Vättle, Ale och Kullings tingslag]]. Sedan 1971 ingår orten i [[Alingsås tingsrätts domsaga]].
>
>     === Befolkningsutveckling ===
>     Staden hade 1950 12 572 invånare.
>
>     [[Fil:Alingsås, framför Lilla torget (ca 1972).jpg|miniatyr|Alingsås vid Säveån 1972.]]
>
>     == Stadsdelar ==
>
>     [[Fil:Jernlunden, Nolhaga, Alingsås, ca 1993.jpg|miniatyr|Utfärdsbåt  i [[Säveån]].]]
>
>     == Kommunikationer ==
>
>     === Biltrafik ===
>     [[E20 (Sverige)|E20]] går rakt igenom Alingsås och delar Alingsås mer eller mindre i två delar. [[E20 (Sverige)|E20]] går till Göteborg i sydvästlig riktning samt Stockholm i nordöst. [[Länsväg 180]] korsar Alingsås och når Borås i sydöst.
>
>     === Kollektivtrafik ===
>     [[Alingsås station]] trafikeras av [[Göteborgs pendeltåg]], regionaltåg samt fjärrtåg. Stationen öppnade redan år 1857.
>     Det finns också bussar till olika delar av staden, samt bussar till [[Borås]], [[Sollebrunn]], [[Vårgårda]], [[Ingared]] och [[Stora Lygnö]].
>
>     === Flygplats ===
>     Närmaste flygplats med reguljär flygtrafik är [[Göteborg Landvetter flygplats|Landvetter]].
>
>     == Utbildning ==
>
>     === Gymnasieskolor ===
>     I Alingsås finns den kommunala gymnasieskolan [[Alströmergymnasiet]]. Alströmergymnasiet är godkända av fotbolls- och handbollsförbundet att bedriva så kallad nationell godkänd idrottsutbildning (NIU), med namnet Alströmergymnasiet NIU elitdrottsgymnasium.
>
>     === Högskoleutbildning ===
>     På Campus Alingsås, inuti Utbildningens hus, erbjuds högskoleutbildningar där Campus Alingsås fungerar som lärcentrum. Exempel på utbildningar som givits eller ges är Butikschefsprogrammet, Förskollärarprogrammet och Grundlärarprogrammet. Det finns även utbildningar inom yrkeshögskola i Utbildningens hus.
>
>     == Kultur, evenemang och medier==
>     [[Alingsås museum]] är ett museum som var inrymt i Alingsås äldsta profana byggnad, det så kallade Alströmerska magasinet.
>
>     Det internationella [[Lights in Alingsås]] är ett evenemang där kommunen tillsammans med de kommunala bolagen och [[Sparbanken Alingsås]] bjuder in ljusdesigners från hela världen för att tillsammans med studenter ljussätta offentliga miljöer i staden.
>
>     ''Potatisfestivalen'' anordnas årligen i juni månad. Musikuppträdanden, restaurangtält och marknad lockar många människor till Alingsås.
>
>     Det treåriga projektet 10 000-projektet, som syftar till att stärka handeln i Alingsås, är ett samverkansprojekt mellan Alingsås näringsliv och kommun. Projektet innefattas av ett flertal evenemang som är utspridda över året:
>     *''Påskparaden ''leder påskparaden mellan Järtas park och Stora torget, där den avslutas med godisregn.
>     *[[Fil:Alingsås, Säveån nära Mjörn, ca 1995.jpg|miniatyr|Nolhagaparken vid Säveån]]''Fikafesten'' i maj månad visar Kafestaden Alingsås från sin bästa sida. Kaféer och butiker har specialerbjudanden och dessutom bjuds det på gatumusik och loppmarknader.
>     *Tomteparaden första advent är ett välbesökt evenemang där tomtar kastar kola till alla barn mellan 0–100 år. Alingsås luciakandidater åker även runt och promotar sig själva.
>     * Östlyckeskolans Kabaret Högstadieskolan Östlyckan sätter varje år upp en uppskattad kabaret på Palladium i Alingsås. Showen innehåller dans (a-dans, b-dans, c-dans och pardans), skådespeleri och musik.
>
>     I augusti månad bjuds det in till ''Alingsås Stora Barnkalas'' och hela innerstaden fylls med aktiviteter för alla barn mellan 0–100 år.
>
>     ''Julmys'' äger rum under hela december månad och många butiker och kaféer har specialerbjudanden.
>
>     Varje år anordnar [[Alingsås Motorveteraner]] sitt Chicken Race. På långfredagen samlas hundratals klassiska mopeder på ABB Kabeldons parkeringsplats för att sedan ge sig ut på en cirka 6 mil lång runda. År 2015 blev det rekord med hela 980 startande.
>
>     [[Nolhaga parkbad]] är ett badhus med äventyrsbad, spa och relaxavdelning i [[Nolhaga]]parken.
>
>     === Lokala medier ===
>     Alingsås har två lokaltidningar, morgontidningen ''[[Alingsås Tidning]]'' som utkommer tre gånger i veckan, och gratistidningen ''[[Alingsås kuriren|Alingsås Kuriren]]'' som utkommer en gång i veckan.
>
>     ==Näringsliv, handel och föreningar==
>
>     === Näringsliv ===
>     Som många andra svenska orter har Alingsås fått ställa om från ett producerande samhälle till ett tjänstesamhälle samt stödja sig allt mer på pendling till särskilt Göteborg. Det finns fortfarande industri i Alingsås, men på en mindre skala än tidigare. Exempel på nedläggningar är Electrolux, Arla, Karamellpojkarna/Cloetta samt Lindex som hade sitt huvudkontor i staden tills 2003. Utpendlingen till Göteborg var 3341 personer, mot inpendlingen från Göteborg med 691 personer (2012) . Utpendlingen till Borås var 613 personer (2013) . Varav 10919 personer arbetade inom Alingsås kommuns gränser och den totala utpendlingen från Alingsås var 4092 personer (2013).
>
>     Välkända industrier genom tiderna:
>     * [[Alströmers manufakturverk]]
>     * [[Alingsås bomullsväveri]]
>     * [[AB Alingsås keramik]]
>     * [[Karamellpojkarna]]
>     * [[ABB|ABB Kabeldon]]
>     * [[B&B Tools]]
>     * [[Strålfors]]
>
>     [[Fil:Gamla torget, Alingsås, ca 1974.jpg|miniatyr|Gamla torget i Alingsås]]
>
>     === Handel ===
>     Handeln i Alingsås har på 2000-talet utvecklat sig snabbt. Centrumhandeln är anrik och fylld med många äldre trähus, med ett flertal innergårdar med små butiker, kaféer och restauranger. 2008 öppnades centrumgallerian Storken på affärsgatan Kungsgatan, med ett 30-tal butiker och restauranger/kaféer. Under 2014 öppnades affärshuset Vimpeln samt Bolltorps handelsområde strax utanför stadskärnan.
>
>     Handelsområden och gallerior:
>     * Storken (öppnade i oktober 2008)
>     * Vimpeln (öppnade i mars 2014)
>     * Bolltorps handelsområde (öppnade i oktober 2014)
>
>     === Föreningsliv ===
>     Alingsås har cirka 350 föreningar av olika slag. Under 2013 öppnade den nya evenemangshallen Estrad Alingsås, som används av det lokala handbollslaget [[Alingsås handbollsklubb|AHK]] samt för diverse evenemang så som [[Lights in Alingsås]].
>
>     Scoutkåren Alströmer grundades 1923 som Alingsås Flickscoutkår. Efter bildandet av [[Svenska Scoutförbundet]] 1960 bytte kåren namn till Scoutkåren Alströmer. Samtidigt integrerades pojkar i kårens verksamhet. Kåren tillhör [[Älvsborgs Södra Scoutdistrikt]] av [[Svenska Scoutförbundet]].
>
>     ==Naturreservat==
>     Friluftsfrämjandet har en lokalavdelning i Alingsås. Föreningen har sitt klubbhus i nya naturreservatet [[Hjortmarka naturreservat|Hjortmarka]] som delas med [[Bälinge socken, Västergötland|Bälinge socken]]. (För fler tätortsnära naturreservat se [[Alingsås socken]].)
>
>     == Sport ==
>     ===Bandy===
>     Redan före 1921 spelades [[bandy]] under organiserade former. Alingsås IF var stadens dominerande lag. Under 1920-talet var AIF i DM-final. Mest framgångsrik blev Alingsås Sportklubb, bildad 1930 som Ingelunds IF, som tre gånger under 1940-talet kvalade till Allsvenskan (utan att nå dit). Alingsås SK sammanslogs 1966 med Stockslycke BK till BK Alingsås. (Alingsås Idrottshistoriska Sällskap)
>
>     Aktiva bandyklubbar i Alingsås är [[Hemsjö IF Bandy|Hemsjö IF]] och [[Vadsjöns BK]]. Bandyn saknar ännu konstfrusen bandybana i Alingsås.
>
>     === Handboll ===
>     Alingsås elitlag i Handboll, [[Alingsås HK]], spelar i [[Handbollsligan]], landets högsta division. Klubben blev svenska mästare 2009 och 2014.
>
>     Potatiscupen är en stor inomhuscup för ungdomslag som spelas årligen i många av Alingsås inomhushallar. Finalspelet går i Estrad Alingsås där Alingsås Handbollklubb spelar sina hemmamatcher.
>
>     === Ishockey ===
>     Ishockeyklubben [[Sörhaga/Alingsås Hockey]] spelade säsongen 2020/2021 i [[Hockeytrean]].
>
>     === Fotboll ===
>     Staden har ett antal lag i seriespel på seniornivå. De mest framgångsrika är [[Holmalunds IF]] (dam- och herrlag), [[Alingsås IF|Alingsås Idrottsförening]] (herrlag), [[Gerdskens BK|Gerdskens bollklubb]] (herrlag) och [[Alingsås Kvinnliga Idrottsklubb]] (damlag).
>
>     === Bowling ===
>     Bowlingklubben [[Team Alingsås BC]] har sedan säsongen 2011/12 spelat i landets högsta division för herrar, [[Elitserien i bowling för herrar|Elitserien]], efter att sedan starten 2007/08 ha kvalat sig upp en division varje år. Klubben spelar sedan första säsongen i Stures Bowling & Sportbar där även en utbredd korpverksamhet äger rum.
>
>     == Kända personer med anknytning till Alingsås ==
>
>     *[[Claes Adelsköld]], [[järnväg]]sbyggare, [[Riksdagsledamot|riksdagsman]], född på och senare i livet ägare av Nolhaga
>     *[[Jonas Alströmer]], en av den industriella revolutionens förgrundsfigurer i Sverige, född och verksam i Alingsås
>     *[[Karin Boye]], poet och författare, avled i Nolby i Alingsås
>     * [[Iwan Bratt]], läkare i Alingsås
>     *[[Christian Bäckman]], [[ishockeyspelare]], föddes och inledde sin karriär i Alingsås
>     *[[Johan Elmander]], [[fotboll]]sspelare, föddes och inledde sin karriär i Alingsås
>     *[[Lotta Engberg]], [[artist]] och [[programledare]], bodde i [[Norsesund]] utanför Alingsås i nära 20 år
>     *[[Markus Granseth]], [[programledare]], uppvuxen i Alingsås
>     *[[Christoffer Hiding]], [[Idol (Sverige)|Idol-deltagare]], uppvuxen i Alingsås
>     *[[Chris Härenstam]], [[sportjournalist]] på [[Sveriges Television|SVT]], uppvuxen i Alingsås
>     *[[Karl-Johan Karlsson]], journalist och författare, uppvuxen i Alingsås
>     * [[Peter Karlsson (friidrottare)|Peter Karlsson]], sprintermästare på 100 meter (innehar fortfarande svenska rekordet).
>     *[[Moa Lignell]], [[Idol (Sverige)|Idol-deltagare]], uppvuxen i Alingsås
>     *[[Hedda Lindahl]], [[politiker]], [[minister]], från Alingsås
>     *[[Peter Lorentzon]], [[skådespelare]], född och uppvuxen i Alingsås
>     *[[Daniel Nyhlén]], [[journalist]], inledde sin journalistkarriär på [[Alingsås Tidning]]
>     *[[Marianne Samuelsson]], [[politiker]], född i Alingsås
>     *[[Peter Weiss]], [[författare]], [[konstnär]], filmare, bodde en tid som krigsflykting i Alingsås, en tid som han beskrivit i sin produktion
>     *[[Jack Vreeswijk]], [[artist]], som vuxen boende i Alingsås
>
>     ==Se även==
>     *[[Lista över fasta fornminnen i Alingsås kommun#Alingsås|Lista över fasta fornminnen i Alingsås]] (för omfattning av detta område, se [[Alingsås stad#Sockenkod]])
>     *[[:Kategori:Personer från Alingsås|Personer från Alingsås]]
>
>     == Referenser ==
>     ===Noter===
>
>     ===Tryckta källor===
>     *Händelser man minns - en bokfilm, Harald Schiller 1970
>
>     == Externa länkar ==
>
>
>
>
>
>
>     [[Kategori:Alingsås| ]]
>     [[Kategori:Orter i Alingsås kommun]]
>     [[Kategori:Centralorter i Västra Götalands län]]
>     [[Kategori:Tätorter i Sverige]]
>     [[Kategori:Orter avbildade i Suecian]]
>
>
>       "))

  

The resulting strings contain a lot of markup structure, and we clean it
by

1.  Removing all `$` and `\`, as scala will otherwise try to help us by
    doing magic to them.
2.  Removing all of the sections "Se även", "Referenser", "Noter",
    "Källor" and "Externa länkar"
3.  Remove links
4.  Replace unnamed internal links `[[Link]]` by `Link`
5.  Replace named internal links `[[Link|Properties|Name]]` by `Name`
6.  Replace unnamed internal links again to clean up
7.  Replace named external links `[Link Name]` by `Name`
8.  Replace unnamed external links `[Link]` by `Link`
9.  Convert all text to lower case
10. Remove quotation marks `'`
11. Remove repeated whitespace, as the obtained results contain a lot of
    whitespace.

The last line in the cell below gets rendered as a string, as the
databricks notebook does not realize that the string `raw"\"` ends
there, but it guesses that `\"` is a quoted character and that the
string continues to the end of the cell.

In [ ]:
def sanitize(source: String) : String= {
  val noDollar = source.replaceAllLiterally("$", "")
  val noBackSlash = noDollar.replaceAllLiterally(raw"\", "")
  noBackSlash
}

val sanitized_xml = filtered_xml.map({case(id, title, source) => (id, title, sanitize(source))})

  

>     sanitize: (source: String)String
>     sanitized_xml: org.apache.spark.rdd.RDD[(Long, String, String)] = MapPartitionsRDD[1026] at map at command-2629358840275860:7

In [ ]:
def filterMarkup(source: String) : String = {
  
  // Loop to remove some specific sections
  val sections = Seq("Se även", "Referenser", "Noter", "Källor", "Externa länkar")
  var current = source
  for (section <- sections) {
    val regexp = s"[=]+ *$section *[=]+".r
    if (regexp.findAllIn(current).hasNext)
    {
      val a = regexp.findAllIn(current).start
      val pre = current.substring(0, a)
      var post = current.substring(a, current.length)
      post = regexp.replaceAllIn(post, "")
      if ("^=".r.findAllIn(post).hasNext) {
        val b = "^=".r.findAllIn(post).start
        post = post.substring(b, post.length)
      }
      else {
        post = ""        
      }
      current = pre + post
    }
  }
  
  // Unnamed internal links appear in figure captions, confusing later matching
  val noUnnamedInternalLinks = raw"\[\[([^\|\]]*)\]\]".r.replaceAllIn(current, m => m.group(1))

  // Loop to remove internal links and their properties
  current = noUnnamedInternalLinks
  val regexp = raw"\[\[[^\|\[]*\|([^\[\]]*)\]\]".r 
  while (regexp.findAllIn(current).hasNext) {
    val len = current.length
    current = regexp.replaceAllIn(current, m => "[[" + m.group(1) + "]]")
    assert (current.length < len)
  }
  val noNamedInternalLinks = current
  
  val noInternalLinks = raw"\[\[([^\]]*)\]\]".r.replaceAllIn(noNamedInternalLinks, m => m.group(1))
  val noNamedExternalLinks = raw"\[[^ \[]*\ ([^\]]*)\]".r.replaceAllIn(noInternalLinks, m => m.group(1))
  val noExternalLinks = raw"\[([^\]]*)\]".r.replaceAllIn(noNamedExternalLinks, m => m.group(1))
  
  
 val lowerCase = noExternalLinks.toLowerCase  

  val noQuoteChars = raw"'".r.replaceAllIn(lowerCase, "")
  // Loop to remove double whitespace characters
  current = noQuoteChars
  val doubleWhitespace = raw"\s(\s)".r
  while (doubleWhitespace.findAllIn(current).hasNext) {
    current = doubleWhitespace.replaceAllIn(current, m => m.group(1))
  }
  val noDoubleWhitespace = current
  noDoubleWhitespace
}


val filtered = sanitized_xml.map({case (id, title, source) => (id, title, filterMarkup(source))})

filtered.take(2)

  

>     filterMarkup: (source: String)String
>     filtered: org.apache.spark.rdd.RDD[(Long, String, String)] = MapPartitionsRDD[1037] at map at command-2629358840275850:57
>     res22: Array[(Long, String, String)] =
>     Array((1,Amager," amager
>     amager är en dansk ö i öresund. öns norra och västra delar tillhör köpenhamn, medan övriga delar upptas av tårnby kommun och dragørs kommun.
>     amager har en yta på 96,29 km² och befolkningen uppgår till 196 047 personer (1/1 2018). på den östra delen av ön finns kastrups flygplats. amager har också flera militära anläggningar (t.ex. kastrup fort och dragör fort), strandparksområdena amager strandpark och kastrup strandpark, flera restauranger i världsklass (t.ex. noma), ett stort naturområde (amager fælled), ett av nordens största köpcentra (fields), och mycket annat. amager förkortas ofta amar för att återspegla det danska uttalet.
>     amager är delvis en konstgjord ö, delvis en naturlig sådan. t.ex. så anlades en konstgjord ö i anslutning till det redan existerande amager strandpark, när denna rustades upp väsentligen 2005. ön är mycket låg och vissa delar ligger under havsytan, framför allt det genom fördämning och utfyllnad skapade området kalvebod fælled (vestamager). ön började uppodlas i större skala under 1500-talets första hälft då kung kristian ii bjöd in nederländska bönder till att odla upp ön för kronans räkning. flertalet slog sig ned i fiskeläget dragør där många än idag har namn med nederländskt ursprung som neels och tønnes.
>     den stora utvecklingsfasen började på 1800-talet då flera fabriker, bostadsområden och militär verksamhet utvecklade sig på ön. det fanns ett antal stora och kända fabriker på ön och den industriella prägeln höll sig väldigt tydlig långt in på 1900-talet. det förekommer också områden som tidigare varit sommarstugeområden och där det än idag finns kolonilottområden, t.ex. i sundbyöster ut mot kusten. under 1900-talet började militären att utveckla stora övningsområden på ön vilket resulterade i vestamager som idag innefattar ett stort naturområde endast några kilometer från köpenhamns centrum. under senare delen av 1900-talet har flygplatsen kastrup som ligger på ön expanderat kraftigt och i samband med byggandet av öresundsförbindelsen började även ett stort område byggas som kallas för ørestad.
>     ön, som är förbunden med övriga köpenhamn genom flera broar, järnväg och sedan 2017 även metro, har under de senaste 10-20 åren förändrats mycket. dels har flera områden exploaterats med nya moderna kvarter lägenheter, t.ex. i områdena örestad, islands brygge och amager strand, och dels så har amagers närhet till köpenhamn raderat dess tidigare rykte som en industriell förstad till köpenhamn. idag finns det områden av alla sorter på ön - tack vare flygplatsen finns ett aktivt näringsliv kvar, samtidigt som de nya områdena och inte minst upprustningen av t.ex. amager strand har förändrat bilden av ön. idag är flera delar av ön mycket attraktiva bostadsområden.
>     ön återfick i samband med öresundsbron även en järnvägsförbindelse vilket inte funnits sedan 1960-talet. en motorväg byggdes samtidigt över öns mellersta del. köpenhamns metro går till kastrup och vestamager med ett antal stationer på ön. kring metro- och regionaltågsstationen ørestad har ett stort kontors-, handels- och bostadsområde uppförts under 00-talet. detta sträcker sig från bella center i norr till metrostationen vestamager i söder. även om området inte ligger vid kusten har ørestad delvis fått karaktären av en hamnstad genom anläggandet av konstgjorda kanaler och dammar. vid islands brygge finns en äkta sjöstad.
>     före öresundsförbindelsens uppförande hade småstaden dragør en färjeförbindelse med limhamn i södra malmö. på grund av denna fanns i dragør tidigare en betydande gränshandel. denna har under 2000-talet helt upphört liksom den genomgående trafiken.
>     öns västra del, som utvanns från öresund under 1920- och 30-talen, är med undantag för bella center, ørestad samt ett koloniområde, avsatt som natur- och rekreationsområde. marken är ängsmark (odlad mark finns dock på den naturliga delen av öns södra del).
>     längst i söder finns den flerhundraåriga skogen kongelunden. på öns östra del, vid öresundskusten, finns amager strandpark med en populär sandstrand. området har omgestaltats, med en konstgjord ö och en lagun innanför. nyinvigningen av parken, som funnits sedan 1934, ägde rum 2005.
>     amager är den tätast befolkade ön i danmark. den näst mest tätbefolkade är thurø.
>     "), (2,Afrika," afrika
>     afrika är jordens näst största kontinent (efter eurasien) och även jordens näst största världsdel efter asien, både vad gäller areal och folkmängd. med världsdelens öar inräknade mäter afrika 30 244 050 km², vilket motsvarar 20,3 % av jordens landmassa eller cirka 6 % av jordens totala area. omkring 22 miljoner km² av dessa ligger i tropikerna vilket gör den afrikanska kontinenten till världens varmaste kontinent. i afrika bor det ungefär 1 miljard människor i 55 länder – en sjundedel av jordens befolkning. dess längd i nordlig-sydlig riktning är omkring 8 000 km och dess största bredd omkring 7 800 km.
>     == etymologi ==
>     ordet afrikas etymologiska bakgrund är dunkel, och ordet har dessutom haft olika betydelse under olika epoker.
>     i äldre tider kallades allt land väster om egypten för libyen. enligt den grekiska historieskrivaren herodotos berättelse seglade feniciska sjömän på uppdrag av den egyptiska kungen neko runt libyen, det vill säga kontinenten afrika redan omkring 600 f.kr.
>     när romarna först använde termen africa terra avsåg de före andra puniska kriget endast trakten kring karthago, det vill säga motsvarande ungefär dagens tunisien där själva staden karthago låg samt stora delar av libyen och algeriet. romarna uppfattade först egypten som en del av asien, och det var först geografen ptolemaios som drog gränsen vid suez. under kejsartiden kom africa att avse hela det området romarna kände till av kontinenten, och allt eftersom européerna har upptäckt mer och mer av kontinenten har termen avsett ett allt större område. namnet ska ha avsett landet där afri (singularis: afer) bodde. afer avsåg namnet på ett folk som kan motsvara dagens berber. vikingarna under 800- och 900-talen kallade norra afrika för blåland.
>     == geografi ==
>     geologiskt är afrika en mycket gammal kontinent, i huvudsak uppbyggd av urberg. under jordens forntid var den en del av jättekontinenten gondwanaland. denna gled isär under juraperioden och kritaperioden för 100-150 miljoner år sedan, och så fick afrika sin nuvarande form.
>     inom sina regelbundna kuster rymmer den en total yta på omkring 30 244 050 km² inklusive öarna. afrika åtskiljs från europa av medelhavet och förenas med asien i sitt nordöstra hörn genom näset vid suez, omkring 130 km i bredd. omkring 8 000 km skiljer den nordligaste punkten, ras ben sakka i tunisien, från dess sydligaste, kap agulhas i sydafrika. från kap verde, dess västligaste punkt, till ras hafun i somalia är det omkring 7 400 km. afrikas kuster mäter omkring 28 000 km (1 km kust på 1 057 km²) varav medelhavet står för 5 400 km, atlanten 11 000 km, indiska oceanen 8 700 km och röda havet 2 900 km, och avsaknaden av djupare bukter och betydande halvöar blir uppenbar om man jämför med europas 9 700 000 km² som omgärdas av 32 000 km kust (1 km kust på 278 km² land), eller för det fjordrika norge, där motsvarande siffra 385 000 km² land och en kuststräcka på 20 000 km (1 km kust per 19 km² land).
>     afrikas genomsnittliga höjd över havet är omkring 600 m, vilket motsvarar de båda amerikanska kontinenternas höjd men är lägre än den asiatiska på omkring 950 m. jämfört med de andra kontinenterna utmärker sig afrika med en liten andel landområden på mycket hög respektive mycket låg höjd. landområdena under 180 m ö.h. upptar en osedvanligt liten del av den totala landmassan, och även de högsta delarna, som är betydligt lägre än motsvarande i asien, utgörs av isolerade berg och bergskedjor. generellt ligger de högre platåerna i öst och söder och de lägre delarna åt väst och norr och med en linje från ungefär vid röda havets mitt till kongoflodens mynning kan kontinenten delas upp i ett lågland i norr och ett högland i söder. man kan således dela upp afrika i fyra övergripande landområden:
>     # slättlanden vid kusterna – ofta med mangroveträsk insprängda – når aldrig långt in från kusterna utom vid vattendragens nedre delar. slamrika slättland finns egentligen bara vid de större flodernas deltan, och annars utgör kustområdena det lägsta steget i en serie terrasser som bildar stigningen upp till de inre platåerna.
>     # atlasbergen i norr som ligger isolerat av saharaöknen som bitvis ligger under havsnivån.
>     # de södra och östra höglanden som ligger på mellan 600 och 1 000 m ö.h.
>     # de nordliga och västra slättlanden, inklusive sahara, som kantas och genomfars av stråk med högra landområden, men i allmänhet håller sig under 600 m ö.h.
>     skog täcker cirka en femtedel av afrikas yta.
>     hela afrika har ett tropiskt eller subtropiskt klimat.
>     === fauna ===
>     afrika är huvudsakligen känt för den öppna savannen med väldiga hjordar av hovdjur som afrikansk buffel, gnuer, antiloper, giraffer och zebror. dessa jagas av stora rovdjur som lejon, hyenor och geparder. den afrikanska djungeln befolkas av ormar och primater samt av vattenlevande djur som krokodiler och groddjur. afrika har även det största antalet arter som kan räknas till megafaunan, bland annat afrikanska elefanter, trubbnoshörning och flodhäst, på grund av att kontinenten bara i mindre utsträckning var påverkad av det massutdöende som skedde under pleistocen.
>     == historia ==
>     den första mänskliga migrationen, från kontinenten men även inom afrikas kuster, kan spåras lingvistiskt, kulturellt och genom datoranalyser av genetiskt material. någonstans i östafrika tror dagens forskare att de första hominider uppkom. i etiopien har man hittat ett två miljoner år gammalt skelett som uppkallades lucy, den första kända mänskliga primaten.
>     === tidigare civilisationer och handel ===
>     några av de mest kända afrikanska civilisationerna. civilisationer från olika tidsepoker visas parallellt.lingvistiska spår tyder på att bantufolk emigrerade från västafrika in i khoisanfolkets (de som tidigare kallades bushmän och hottentotter) område och undanträngde dem. bantufolken använde en uppsättning grödor, bland annat kassava och jams, som var mer lämpade för jordbruk i det tropiska afrika, och med sitt jordbruk kunde bantu försörja en betydligt större folkmängd än de undanträngde jägare eller samlare. bantufolkens traditionella bosättningsområde sträcker sig från saharaöknen till de tempererade regionerna i söder. historiskt har deras vapen varit spjut, sköld, pil och båge. det för khoisanspråkens karaktäristiska klickljuden återfinns i nguni- och bantuspråk som till exempel xhosa och zulu. khoisanspråken talas idag främst av utspridda minoritetsgrupper i södra afrika i länderna kring kalahariöknen.
>     afrika söder om sahara har bestått olika stater och riken, mindre stadsstater samt stammar och klaner. det forna egyptiska riket, är en av världens äldsta stater, och den mest kända av de äldre afrikanska civilisationerna. det fanns även statsbildningar i nubien, ghana, timbuktu, kanem-bornu och i nuvarande zimbabwe.
>     nordafrika kom från och med 600-talet under arabiskt inflytande – en tid av kulturell blomstring – och med kamelens hjälp spreds islam och den arabiska kulturen genom sahara och vidare söderut. arabiska och persiska handelsmän spred likaså islam längs med den östafrikanska kusten parallellt med en omfattande svart slavhandel. detta östafrikanska handelsutbyte med araber och perser sträckte sig ända bort till java, därmed den etniska, kulturella och språkliga mångfalden på östafrikas kust, madagaskar och komorerna.
>     === arabvärlden och europa utforskar kontinenten ===
>     det arabiska intresset för afrika går långt tillbaka, men handel med slavar från centra som lamu och zanzibar vid östkusten har pågått åtminstone sedan senare delen av 1000-talet e.kr. de miljoner slavar som exporterades till arabvärlden och österut under den arabiska slavhandeln finns idag kvar som en stor etnisk grupp där, i stor utsträckning integrerad med den ursprungliga arabiska befolkningen. slavhandel med svarta pågår fortfarande i vissa arabländer, till exempel sudan.human rights watch bbc world
>     under 1300-talet började vissa europeiska handelsexpeditioner få kontakt med nord- och västafrika.
>     från 1500-talets mitt fram till förbudet mot slavhandel i storbritanniens parlament 1807, som kom att få full kraft först något årtionde senare, handlades slavar från lokala stamledare i främst väst- och centralafrika. flera miljoner afrikaner såldes på detta vis som slavar till främst de europeiska kolonierna i amerika.
>     tidiga kolonisatörer var araber, portugiser och spanjorer. därefter koloniserades kontinenten av framförallt av storbritannien, frankrike, spanien, italien, belgien, nederländerna, tyskland och portugal.
>     === koloniseringen ===
>     politisk karta över afrika från 1890. från slutet av 1000-talet koloniserade arabiska köpmän den afrikanska östkusten. holländarna koloniserade på 1600-talet den då relativt folktomma södra udden av afrika. portugiserna grundade flera kolonier i västra afrika och vid nuvarande moçambique. i mitten av 1800-talet leddes flera expeditioner in i de centrala delarna av afrika, bland annat av livingstone och speke, men också den svenske upptäcktsresanden andersson. denna kolonisering förändrade i grunden det ekonomiska livet på kontinenten, och innebar i flera fall ensidig export av naturresurser. kung leopolds rovdrift på sin privata koloni kongostaten är ökänd och kostade miljontals afrikanska liv.
>     i afrika fanns efter första världskriget endast tre självständiga stater nämligen konungariket egypten, republiken liberia och konungariket abessinien. tanger bildade med sitt område en internationell zon. för övrigt var hela världsdelen uppdelad på kolonier åt europeiska makter.
>     flertalet kolonier erhöll självständighet under slutet av 1950-talet och början av 1960-talet. avkoloniseringen resulterade i flertalet fall med katastrof, där diktatur och etnisk rensning vanligen blev följden, till exempel i rhodesia (nuvarande zimbabwe), rwanda, uganda, nigeria m.fl. stater.
>     == politik ==
>     de blå länderna är demokratier enligt tankesmedjan freedom house.
>     det postkoloniala afrikas politik har i hög grad präglats av diktatur, statskupper och brist på mänskliga rättigheter.
>     nationerna i afrika har ofta visat sig vara instabila stater som efter självständigheten hemsökts av korruption, våldsamma statskupper, despotism och militärdiktaturer. under perioden 1960-1989 genomfördes över 70 kupper och 13 mord på presidenter. under det kalla kriget kämpade usa och sovjetunionen om makten i afrika, och tillsammans med länder som frankrike och kuba, stödde de aktivt diktaturer och iscensatte flera av dessa kupper och illgärningar mot demokratiskt valda regeringar. enligt en spridd uppfattning beror många militära konflikter om territorier och gränser under afrikas moderna historia på de gränser från kolonial tid som huvudsakligen drogs upp efter kongokonferensen, men detta kan inte visas eftersom något jämförelseområde inte finns. konflikter i afrika har ofta drivits av tillgången till kontinentens rika råvarutillgångar – diamanter och metaller som guld, platina, uran och koppar.
>     korruption och havererade politiska program har lett till utbredd undernäring och undermålig infrastruktur orsakar alltjämt brist på mat och rent vatten. sanitetsproblemen har också orsakat en akut spridning av bland annat hiv-viruset som leder till sjukdomen aids.
>     trots nöd och umbäranden finns vissa positiva tecken – demokratiskt valda regeringar växer i antal även om de inte utgör någon majoritet ännu. genom påtryckningar från till exempel internationella valutafonden (imf) har många afrikanska stater lyckats vända en nedåtgående ekonomisk spiral som varat i flera decennier till en positiv tillväxt. om man beaktar imf:s katastrofala misslyckanden i länder som argentina och filippinerna är det dock oklart i vilken omfattning dessa konventionella ekonomiska kriterier ger en rättvis bild av vad afrika faktiskt behöver.
>     de afrikanska ledarna har i olika omgångar sökt former för samarbete sinsemellan. hittills har dock detta inte lyckats särskilt väl. under inbördeskriget i kongo var det grannstaterna som spädde på konflikten snarare än utomafrikanska stater, och motivet var främst att skaffa kortsiktiga vinster genom att utvinna diamanter och andra mineraler. antalet döda i konflikten har uppskattats till 3,5 miljoner under en femårsperiod.
>     de politiska samarbetsorganisationer man skapat har inte kunnat leva upp till förväntningarna. organisationen för afrikansk enighet (oau 1963-2002) kallades till exempel diktatorsklubben för dess tendens att alltid stödja de sittande politiska ledarna och ägna väldigt lite energi åt att främja den vanlige afrikanens mänskliga rättigheter. 2002 skapades i stället afrikanska unionen, som är tänkt att underlätta ett framtida samarbete mellan kontinentens stater. hittills har denna organisation inte klarat att hantera till exempel utrotningskriget i darfur-provinsen i sudan, där en arabisk statsledning tillåtit slakt på hundratusentals av bantubefolkningen i sydväst. en avgörande skillnad mellan såväl oau som au å ena sidan och eu å den andra, är att eu startades av en grupp länder som satte demokrati och mänskliga rättigheter i fokus för den egna politiken, och som krävt detta av tillkommande medlemsstater. oau och au har från början i stället varit territoriellt inriktade, och visserligen haft demokrati och mänskliga rättigheter som skrivelser i sina stadgar, men ytterst sällan satt kraft bakom dessa ord. på så sätt liknar au mer fn än eu.
>     == ekonomi ==
>     jordbrukare på en jamsodling i kamerun
>     afrikas ekonomi omfattar ungefär 887 miljoner människor i 54 olika stater (juli 2005). afrika är världens fattigaste kontinent, och den är, i medeltal, fattigare än den var för 25 år sedan. på fn:s human development report från 2003 (som rankade världens 175 länder enligt en rad kriterier) var plats mellan 151 och 175 intagna av länder i afrika.
>     det finns dock länder som lyckas bra ekonomiskt, bland andra botswana, ghana och sydafrika. etiopien har också en relativ bra ekonomi tack vare den höga tillväxten i landet på cirka 11-15% per år. sydafrika har en väl fungerande inhemsk aktiehandel och har stora naturtillgångar i bland annat guld och diamanter och ett väl fungerande rättsväsen. det finns i sydafrika även tillgång till investeringskapital, stora marknader och utbildad arbetskraft. det finns även andra afrikanska länder, exempelvis egypten, som länge haft en väl fungerande handelssektor och ekonomi.
>     kontinentens kust mot atlanten återfinns stora fyndigheter av olja. i nigeria finns en av världens mest betydande oljereserver och landets ekonomi är bland de snabbast växande i världen.
>     afrikas jordbruk sysselsätter drygt hälften av afrikas befolkning och är den viktigaste näringen. cirka 6 % (ungefär 1 814 600 km²) av världsdelens yta är uppodlad, mest i savannområdena. i stora delar av de brukningsbara områdena är jordens bördighet ganska låg. de bästa jordarna återfinns i begränsade höglandsområden som i till exempel kamerun och östafrika.
>     == demografi ==
>     ===befolkning===
>     människan, homo sapiens sapiens, antas härstamma från afrika och spred sig härifrån till de omgivande kontinenterna. den svarta befolkningen söder om sahara tillhör den ursprungliga befolkningen som hela tiden levat i afrika.
>     norr om sahara präglar samröret med övriga medelhavsområdet och inte minst sydvästra asien både befolkning, språk och kultur.
>     på ön madagaskar, som länge var obebodd, härstammar majoriteten av befolkningen från människor som emigrerat från indonesien. under kolonialtiden skedde inflyttning från europa och asien till södra afrika, där den icke-svarta befolkningen idag utgör betydande minoritetsgrupper.
>     afrikas befolkning uppvisar många för utvecklingsregioner typiska drag, som höga födelsetal i kombination med sjunkande dödstal med snabb befolkningsökning och låg medelålder (45 % är under 15 år) som följd. även om befolkningstätheten varierar kraftigt mellan olika regioner är afrika generellt en glestbefolkad kontinent, med i genomsnitt 27,8 invånare per km².
>     afrika är den kontinent som har den lägsta urbaniseringsgraden, med endast 25% boende i städer. även här är variationerna stora, och i norra och östra afrika finns en lång urban tradition.
>     === språk ===
>     språkfamiljer i afrika
>     uppskattningsvis finns det ungefär 1800 talade språk i afrika idag. vissa av dessa, till exempel swahili, hausa, och yoruba, talas av många miljoner människor, medan det finns andra som bara talas av några hundra eller ännu färre. afrika har även många olika teckenspråk. de flesta afrikaner talar som modersmål ett språk tillhörande någon av de fyra språkfamiljerna afroasiatiska språk, nilo-sahariska språk, niger-kongospråk eller khoisanspråk.
>     med några få nämnvärda undantag i östra afrika, har nästan alla afrikanska länder språk som har sitt ursprung utanför världsdelen som officiellt språk, på grund av kolonialismen och mänsklig migration. till exempel används i ett stort antal afrikanska länder franska eller engelska som administrativt språk för stat, handel, utbildning och media. arabiska, portugisiska, afrikaans och malagassiska är andra exempel på språk med ursprung utanför afrika som idag används av miljoner afrikaner, både i offentliga och privata sammanhang.
>     === religion ===
>     kristendom och islam är de största religionerna i afrika, 46,3 % av alla afrikaner är kristna och 40,5 % är muslimer. resten 11,8 % tillhör olika lokala afrikanska religioner som funnits före den europeiska kolonisationen. ett mindre antal afrikaner är hinduer, eller har tro från judisk tradition. exempel på afrikanska folk som tillhör judendomen är beta israel, lembafolk och abayudaya i östra uganda. sydafrika och angola har över 55% kristna katoliker.
>     == kultur ==
>     afrika har ett stort antal överlappande kulturer, och flera tusen olika etniska grupper. den mest konventionella distinktionen är den mellan området söder om sahara och området norr om öknen. de nordliga länderna associerar sig främst med arabkulturen, medan de sydliga länderna består av i sin tur många olika kulturella områden. en av de största kulturella identiteterna är den som innefattar alla folk som talar något bantuspråk.
>     det går också att göra en indelning i gamla franska västafrika å ena sidan, och resten av afrika (mer specifikt de brittiska eller portugisiska kolonierna i södra och östra afrika) å andra sidan. en annan kulturell linje går mellan de afrikaner som håller kvar vid sina gamla traditionella livsstilar, och de som har anpassat sig till en mer modern livsstil. det går även att dela upp traditionalisterna i sådana som ägnar sig åt uppfödning av boskap, och sådana som ägnar sig åt jordbruk.
>     afrikansk konst och afrikansk arkitektur reflekterar de afrikanska kulturernas diversitet. de äldsta exemplen på afrikansk konst är 75 000 år gamla pärlsmycken gjorda av nassariusskal, dessa hittades i blombosgrottan. pyramiderna i giza var världens högsta byggnader under 4 000 år. klippkyrkorna i lalibela i etiopien, bland vilka bete giyorgis (st. göranskyrkan) är mest välkänd, är också kända exempel på afrikansk arkitektur.
>     == regioner och territorier ==
>     regioner i afrika:
>     "))

  

Before leaving this section, we may want to save our results, so that
the next section can reload it.

In [ ]:
// dbutils.fs.rm(filtered_articles_file, recurse=true)

  

>     res19: Boolean = true

In [ ]:
val outputs = filtered.map({case (id, title, contents) => (id, title.replaceAllLiterally(",", "**COMMA**"), contents.replaceAllLiterally("\n", "**NEWLINE**"))})
outputs.saveAsTextFile(filtered_articles_file);

  

>     outputs: org.apache.spark.rdd.RDD[(Long, String, String)] = MapPartitionsRDD[1038] at map at command-4443336225071544:1

  

Prepare for LDA
===============

We now turn to the spark library for our next set of preparatory steps,
and as the library work on dataframes, we do so as well.

Loading previous results
------------------------

If we want to continue from the result saved at the end of the previous
section, we can reload the data here.

In [ ]:
val format = "^\\(([^,]*),([^,]*),(.*)\\)$".r
def parse_line(s : String) = {
  assert (s(0) == '(')
  val firstComma = s.indexOf(",")
  val id = s.substring(1, firstComma).toLong
  val afterId = s.substring(firstComma+1, s.length)
  val nextComma = afterId.indexOf(",")
  val title = afterId.substring(0, nextComma)
  val contents = afterId.substring(nextComma+1, afterId.length-1)
          
  assert (afterId(afterId.length-1) == ')')
  (id, title.replaceAllLiterally("**COMMA**", ","), contents.replaceAllLiterally("**NEWLINE**", "\n"))
}

val filtered = spark.sparkContext.textFile(filtered_articles_file).map(parse_line)
val n_articles = filtered.count()

  

>     format: scala.util.matching.Regex = ^\(([^,]*),([^,]*),(.*)\)$
>     parse_line: (s: String)(Long, String, String)
>     filtered: org.apache.spark.rdd.RDD[(Long, String, String)] = MapPartitionsRDD[1042] at map at command-4443336225071727:15
>     n_articles: Long = 813245

  

DataFrame
---------

The tokenization and stopword removal work on dataframes, so we create
one, and look at some samples.

In [ ]:
val corpus_df = filtered.toDF("id", "title", "contents")

  

>     corpus_df: org.apache.spark.sql.DataFrame = [id: bigint, title: string ... 1 more field]

  

The command below is the first step that requires all of the above steps
to be completed for all the elements in the rdd, and it therefore takes
a few minutes.

In [ ]:
display(corpus_df.sample(0.0001))

  

[TABLE]

Truncated to 30 rows

  

Tokenization
------------

In order to feed the articles into the LDA model, we first need to
perform word tokenization. This is done by splitting the article into
regular expressions. In this case we ignore all non-alphabetic
characters and only keep words with a minimal length of 4 characters in
order to avoid short words that are not relevant to the subject.

In [ ]:
import org.apache.spark.ml.feature.RegexTokenizer

// Set params for RegexTokenizer
val tokenizer = new RegexTokenizer()
.setPattern("[^a-zåäö]+") // break remove anything but non-word character(s). 
.setMinTokenLength(4) // Filter away tokens with length < 4
.setInputCol("contents") // name of the input column
.setOutputCol("tokens") // name of the output column

// Tokenize document
val tokenized_df = tokenizer.transform(corpus_df)

  

>     import org.apache.spark.ml.feature.RegexTokenizer
>     tokenizer: org.apache.spark.ml.feature.RegexTokenizer = RegexTokenizer: uid=regexTok_5d41547188e0, minTokenLength=4, gaps=true, pattern=[^a-zåäö]+, toLowercase=true
>     tokenized_df: org.apache.spark.sql.DataFrame = [id: bigint, title: string ... 2 more fields]

In [ ]:
display(tokenized_df.sample(0.0001))

  

Remove Stopwords
----------------

Next, we remove the stopwords from the article. This should be done
iteratively, as the final model might find patterns in words that are
not relevant. In our first iteration we found several words being used
that were not relevant so we added them to the list of stopwords and
repeated the experiment, which lead to better results. This step can be
repeated several times in order to improve the model further.

We load the stopwords we downloaded near the beginning of this notebook,
and we add some custom stopwords that are relevant to wikipedia pages
and articles.

In [ ]:
val swedish_stopwords = sc.textFile(stopword_path).collect()
val wikipedia_stopwords = Array("label", "note", "area", "unit", "type", "mark", "long", "right", "kartposition", "quot", "text", "title", "page", "timestamp", "revision", "name", "username", "sha1", "format", "coord", "left", "center", "align", "region", "nasa", "source", "mouth", "species", "highest", "style", "kategori", "http", "wikipedia", "referenser", "källor", "noter")
// Combine newly identified stopwords to our exising list of stopwords
val stopwords = swedish_stopwords.union(wikipedia_stopwords)

  

>     swedish_stopwords: Array[String] = Array(word, all, alla, allas, allt, alltså, andra, andras, annan, annat, artonde, artonn, att, av, bakom, bara, bland, blev, bli, blir, blivit, båda, bådas, både, dag, dagar, dagarna, dagen, de, del, delen, dem, den, denna, deras, dess, dessa, det, detta, dig, din, dina, dit, ditt, dock, dom, du, där, därför, då, efter, eftersom, ej, elfte, eller, elva, en, er, era, ert, ett, fall, fanns, fast, fem, femte, femtio, femtionde, femton, femtonde, fick, fin, finnas, finns, fjorton, fjortonde, fjärde, fler, flera, flesta, fram, framför, från, fyra, fyrtio, fyrtionde, få, får, fått, för, före, förra, första, ge, genom, ger, gick, gjorde, gjort, gälla, gäller, gällt, gärna, gå, gång, går, gått, gör, göra, ha, hade, haft, han, hans, har, hela, heller, hellre, helst, helt, henne, hennes, heter, hit, hjälp, hon, honom, hundra, hundraen, hundraett, hur, här, i, ibland, icke, igen, in, inför, inga, ingen, inget, innan, inne, inom, inte, inuti, ja, jag, ju, jämfört, kan, kom, komma, kommer, kommit, kunde, kunna, kunnat, kvar, kör, legat, ligga, ligger, lägga, man, med, mellan, men, menar, mer, mera, mig, min, mina, mitt, mittemot, mot, ned, nederst, nedersta, nedre, nej, ner, ni, nio, nionde, nittio, nittionde, nitton, nittonde, nog, noll, nr, nu, nummer, nya, när, nästa, någon, någonting, något, några, nån, nåt, och, också, om, oss, på, rakt, redan, sade, sagt, samt, sedan, sen, ser, sex, sextio, sextionde, sexton, sextonde, sig, sin, sina, sist, sista, siste, sitt, sitta, sju, sjunde, sjuttio, sjuttionde, sjutton, sjuttonde, själv, sjätte, ska, skall, skriver, skulle, slutligen, snart, som, står, säga, säger, sätt, så, sådan, sådana, sådant, ta, tack, tar, till, tills, tio, tionde, tjugo, tjugoen, tjugoett, tjugonde, tjugotre, tjugotvå, tjungo, tog, tolfte, tolv, tre, tredje, trettio, trettionde, tretton, trettonde, tro, tror, två, tvåhundra, under, upp, ur, ut, utan, utanför, ute, vad, var, vara, varför, varifrån, varit, varje, varken, vars, varsågod, vart, vem, vems, verkligen, vet, vi, vid, vidare, vilka, vilkas, vilken, vilket, vill, visst, väl, värre, vår, våra, vårt, än, ändå, ännu, är, även, året, åt, åtminstone, åtta, åttio, åttionde, åttonde, över, övre)
>     wikipedia_stopwords: Array[String] = Array(label, note, area, unit, type, mark, long, right, kartposition, quot, text, title, page, timestamp, revision, name, username, sha1, format, coord, left, center, align, region, nasa, source, mouth, species, highest, style, kategori, http, wikipedia, referenser, källor, noter)
>     stopwords: Array[String] = Array(word, all, alla, allas, allt, alltså, andra, andras, annan, annat, artonde, artonn, att, av, bakom, bara, bland, blev, bli, blir, blivit, båda, bådas, både, dag, dagar, dagarna, dagen, de, del, delen, dem, den, denna, deras, dess, dessa, det, detta, dig, din, dina, dit, ditt, dock, dom, du, där, därför, då, efter, eftersom, ej, elfte, eller, elva, en, er, era, ert, ett, fall, fanns, fast, fem, femte, femtio, femtionde, femton, femtonde, fick, fin, finnas, finns, fjorton, fjortonde, fjärde, fler, flera, flesta, fram, framför, från, fyra, fyrtio, fyrtionde, få, får, fått, för, före, förra, första, ge, genom, ger, gick, gjorde, gjort, gälla, gäller, gällt, gärna, gå, gång, går, gått, gör, göra, ha, hade, haft, han, hans, har, hela, heller, hellre, helst, helt, henne, hennes, heter, hit, hjälp, hon, honom, hundra, hundraen, hundraett, hur, här, i, ibland, icke, igen, in, inför, inga, ingen, inget, innan, inne, inom, inte, inuti, ja, jag, ju, jämfört, kan, kom, komma, kommer, kommit, kunde, kunna, kunnat, kvar, kör, legat, ligga, ligger, lägga, man, med, mellan, men, menar, mer, mera, mig, min, mina, mitt, mittemot, mot, ned, nederst, nedersta, nedre, nej, ner, ni, nio, nionde, nittio, nittionde, nitton, nittonde, nog, noll, nr, nu, nummer, nya, när, nästa, någon, någonting, något, några, nån, nåt, och, också, om, oss, på, rakt, redan, sade, sagt, samt, sedan, sen, ser, sex, sextio, sextionde, sexton, sextonde, sig, sin, sina, sist, sista, siste, sitt, sitta, sju, sjunde, sjuttio, sjuttionde, sjutton, sjuttonde, själv, sjätte, ska, skall, skriver, skulle, slutligen, snart, som, står, säga, säger, sätt, så, sådan, sådana, sådant, ta, tack, tar, till, tills, tio, tionde, tjugo, tjugoen, tjugoett, tjugonde, tjugotre, tjugotvå, tjungo, tog, tolfte, tolv, tre, tredje, trettio, trettionde, tretton, trettonde, tro, tror, två, tvåhundra, under, upp, ur, ut, utan, utanför, ute, vad, var, vara, varför, varifrån, varit, varje, varken, vars, varsågod, vart, vem, vems, verkligen, vet, vi, vid, vidare, vilka, vilkas, vilken, vilket, vill, visst, väl, värre, vår, våra, vårt, än, ändå, ännu, är, även, året, åt, åtminstone, åtta, åttio, åttionde, åttonde, över, övre, label, note, area, unit, type, mark, long, right, kartposition, quot, text, title, page, timestamp, revision, name, username, sha1, format, coord, left, center, align, region, nasa, source, mouth, species, highest, style, kategori, http, wikipedia, referenser, källor, noter)

  

The spark ML library gives us functions to remove stopwords

In [ ]:
import org.apache.spark.ml.feature.StopWordsRemover

val remover = new StopWordsRemover()
.setStopWords(stopwords) 
.setInputCol("tokens")
.setOutputCol("filtered")

val filtered_df = remover.transform(tokenized_df)

  

>     import org.apache.spark.ml.feature.StopWordsRemover
>     remover: org.apache.spark.ml.feature.StopWordsRemover = StopWordsRemover: uid=stopWords_ecf00fa02fd5, numStopWords=366, locale=en, caseSensitive=false
>     filtered_df: org.apache.spark.sql.DataFrame = [id: bigint, title: string ... 3 more fields]

  

Vector of token counts
----------------------

The LDA model takes word counts as input, so the next step is to count
words that appear in the articles.

There are two hyperparameters to consider here:

-   vocabSize - this is total number of words used. The vectorizer will
    pick the words that appear most frequently in all articles.
-   minDF - the minimum number of articles that each word must appear
    in. This is used to discard words that are very frequent in a small
    number of articles, but less frequent in general.

We found that using a vocab size of 10 000 and a minDF of 5 worked well,
but these can be experimented with further in order to obtain even
better clusterings.

In [ ]:
import org.apache.spark.ml.feature.CountVectorizer

val vectorizer = new CountVectorizer()
.setInputCol("filtered")
.setOutputCol("features")
.setVocabSize(10000) 
.setMinDF(5) // the minimum number of different documents a term must appear in to be included in the vocabulary.
.fit(filtered_df)

  

>     import org.apache.spark.ml.feature.CountVectorizer
>     vectorizer: org.apache.spark.ml.feature.CountVectorizerModel = CountVectorizerModel: uid=cntVec_ad38aa4e1641, vocabularySize=10000

  

Next, we convert the dataframe to an RDD that contains the word counts

In [ ]:
import org.apache.spark.ml.linalg.Vector

val countVectors = vectorizer.transform(filtered_df).select("id", "features")
val lda_countVector = countVectors.map { case Row(id: Long, countVector: Vector) => (id, countVector) }
val lda_countVector_mllib = lda_countVector.map { case (id, vector) => (id, org.apache.spark.mllib.linalg.Vectors.fromML(vector)) }.rdd

lda_countVector_mllib.take(1)

  

>     import org.apache.spark.ml.linalg.Vector
>     countVectors: org.apache.spark.sql.DataFrame = [id: bigint, features: vector]
>     lda_countVector: org.apache.spark.sql.Dataset[(Long, org.apache.spark.ml.linalg.Vector)] = [_1: bigint, _2: vector]
>     lda_countVector_mllib: org.apache.spark.rdd.RDD[(Long, org.apache.spark.mllib.linalg.Vector)] = MapPartitionsRDD[1088] at rdd at command-4443336225071537:5
>     res41: Array[(Long, org.apache.spark.mllib.linalg.Vector)] = Array((1,(10000,[1,2,8,9,15,17,20,26,37,38,39,42,48,50,51,55,64,67,68,71,75,80,81,85,86,94,95,96,106,114,117,118,128,129,140,149,150,156,161,189,195,208,216,297,299,332,336,399,409,427,452,457,459,461,465,535,540,572,603,612,686,739,747,806,849,904,922,983,1002,1013,1037,1081,1128,1167,1170,1175,1272,1274,1295,1334,1374,1387,1474,1578,1593,1602,1707,1750,1764,1792,1835,1852,1927,2022,2027,2114,2182,2268,2358,2612,2726,2766,2929,3316,3386,3415,3468,3486,3588,3593,3634,3693,3735,4156,4184,4335,4339,4470,4579,4675,4691,4851,5046,5083,5118,5186,5355,5653,5712,5801,6051,6080,6140,6751,7381,7491,7602,8758,8892,8944,8990,9148,9185,9375,9416,9737,9779,9870],[7.0,2.0,1.0,3.0,3.0,1.0,1.0,4.0,4.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,5.0,2.0,2.0,1.0,4.0,2.0,1.0,4.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,3.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,4.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0])))

  

We can also save the countvector file for later use.

In [ ]:
println(lda_countVector_file)

// lda_countVector_mllib.saveAsTextFile(lda_countVector_file);

  

>     dbfs:/datasets/05_lda/lda_countvector_sv_20201120

  

LDA
===

It's now time to define our LDA model. We choose the online LDA due to
speed and simplicity, altough one could also try using the expectation
maximization algorithm here instead.

The online LDA takes two hyperparameters:

-   The number of topics to cluster - this should be set to som
    reasonable number depending on what the purpose of the clustering
    is. Sometimes we might have a-priori knowledge of what the number of
    topics ought to be, but for Wikipedia it is not obvious how this
    parameter should be determined.
-   The number of iterations - this can be increased in order to trade
    off between increased computational time and better clustering.

In this examples, we choose 50 topics since this makes it easy to
visualize the results. We use 20 iterations, which yields a reasonable
runtime (about 16 minutes).

In [ ]:
val numTopics = 50
val maxIterations = 20

  

>     numTopics: Int = 50
>     maxIterations: Int = 20

  

&lt;img src
='/files/shared*uploads/axel.berg@arm.com/Smoothed*LDA-1.png'&gt;

Let's review what the LDA model does.

According to the plane notation, we have

-   M - number of documents
-   N - the number of words per document (different for each document,
    of course)
-   theta\_i - the topic distribution for document i
-   phi\_k - the word distribution for topic k
-   z\_ij - the topic for the j-th word in document i
-   w\_ij - the j-th word in document i
-   alpha - Dirichlet prior on the per-document topic distributions
-   beta - Dirichlet prior on the per-topic word distribution

When performing online LDA, we are fitting the data to this model by
estimating values for alpha and beta iteratively. First, each word is
randomly assigned to a topic. Then, using variational inference, the
topics are reassigned in an iterative way using the empirical
distributions of words within each document.

This is can all be done using spark's build in LDA optimizer

In [ ]:
import org.apache.spark.mllib.clustering.{LDA, OnlineLDAOptimizer}

val lda = new LDA()
.setOptimizer(new OnlineLDAOptimizer().setMiniBatchFraction(0.8))
.setK(numTopics)
.setMaxIterations(maxIterations)
.setDocConcentration(-1) // use default values
.setTopicConcentration(-1) // use default values

  

>     import org.apache.spark.mllib.clustering.{LDA, OnlineLDAOptimizer}
>     lda: org.apache.spark.mllib.clustering.LDA = org.apache.spark.mllib.clustering.LDA@54e767e2

  

The next cell is where the magic happens. We invole the disitributed LDA
training functionality to obtain the model.

In [ ]:
val ldaModel = lda.run(lda_countVector_mllib)

  

>     ldaModel: org.apache.spark.mllib.clustering.LDAModel = org.apache.spark.mllib.clustering.LocalLDAModel@5c324bb0

In [ ]:
dbutils.fs.rm(lda_file, recurse=true)
ldaModel.save(sc, lda_file) 

  

  

Looking at the model
====================

Topics
------

We prepare by extracting a few relevant terms per topic, looking them up
in the vocabulary.

In [ ]:
val maxTermsPerTopic = 8
val topicIndices = ldaModel.describeTopics(maxTermsPerTopic)
val vocabList = vectorizer.vocabulary
val topics = topicIndices.map { case (terms, termWeights) =>
  terms.map(vocabList(_)).zip(termWeights)
}

  

>     maxTermsPerTopic: Int = 8
>     topicIndices: Array[(Array[Int], Array[Double])] = Array((Array(13, 7, 2, 54, 122, 43, 134, 81),Array(0.0965465174848558, 0.09561944647066932, 0.06123216019531808, 0.06047426356346239, 0.04133489674798034, 0.038999680148138156, 0.038645022485160795, 0.03709042755540236)), (Array(1, 6, 156, 82, 15, 85, 102, 9),Array(0.01860754472650629, 0.00800131987315067, 0.007185124511872375, 0.006562831220260255, 0.006140398920615785, 0.005472783411308425, 0.005347092400479047, 0.004738960299641082)), (Array(333, 3, 1188, 1116, 23, 24, 804, 612),Array(0.012891731862399829, 0.012216420059244167, 0.008799092539391087, 0.008170528757595195, 0.007589154082154804, 0.00753057351996227, 0.007501151936739348, 0.007441712973556016)), (Array(40, 118, 66, 103, 7, 109, 55, 140),Array(0.03550112124416654, 0.024239377096572767, 0.017880722746326707, 0.017534872470232804, 0.01572555256779709, 0.014243881447019926, 0.013291812684672874, 0.013255209377268314)), (Array(327, 903, 854, 1783, 3913, 7550, 5476, 570),Array(0.049154032038907364, 0.025424694972593722, 0.021006352382631138, 0.015146400328457767, 0.014384523267127552, 0.01237820887773258, 0.010986509101209276, 0.010348525497739654)), (Array(764, 324, 30, 144, 989, 147, 585, 506),Array(0.024609442134056914, 0.02313788863645436, 0.023099400549010995, 0.016486417015966634, 0.01631846978655288, 0.014208691182654495, 0.012902237189832062, 0.01089177248153098)), (Array(194, 300, 105, 19, 64, 89, 73, 20),Array(0.025131668033955827, 0.01897996595347449, 0.01864839205373941, 0.01754493324942852, 0.015642901268097295, 0.014806106220374982, 0.013746517306962325, 0.012193036368057153)), (Array(36, 33, 758, 433, 127, 1378, 12, 41),Array(0.02200110661389711, 0.016863821067931672, 0.011169441915583982, 0.009575257990621604, 0.007041018691775915, 0.006957959901751748, 0.006567438459526348, 0.006206311552289226)), (Array(10, 27, 56, 101, 104, 166, 169, 200),Array(0.09573192102472637, 0.07721821951064128, 0.05140930922690405, 0.038236453151551754, 0.037725195145277224, 0.027601289731431947, 0.026950628685089566, 0.02535800730760235)), (Array(6, 87, 16, 1194, 2, 99, 1762, 13),Array(0.040800430698927574, 0.0306444550609787, 0.02193657394829871, 0.019628310699189612, 0.018135004797691912, 0.015575322731524751, 0.014987081087118154, 0.01387372200507514)), (Array(78, 19, 7, 480, 342, 422, 1014, 35),Array(0.059935597281339396, 0.025518279914354958, 0.021135159839241465, 0.01909964708101696, 0.015705965232071534, 0.010849948320306425, 0.009618564510993638, 0.009077797952871827)), (Array(253, 312, 788, 49, 896, 76, 1001, 1379),Array(0.02947986672544381, 0.01634731925291766, 0.016230381323561027, 0.012766683820251106, 0.011650847462746833, 0.011556626328903735, 0.01020456193177741, 0.009873469854259966)), (Array(542, 509, 385, 1144, 46, 186, 786, 252),Array(0.019344695557099535, 0.012893474006120904, 0.011736207069720414, 0.01063631773660054, 0.010443829299695546, 0.0101881977731672, 0.009557330691273938, 0.008217466852727128)), (Array(4, 754, 0, 170, 187, 269, 667, 727),Array(0.020289726693646915, 0.016791173665524066, 0.016079171985534508, 0.01499395294114528, 0.013972428567201652, 0.012374270949374168, 0.011168740554371137, 0.011135322353134584)), (Array(89, 64, 266, 26, 73, 11, 39, 268),Array(0.010801612339730723, 0.009156920691378509, 0.007179320551697595, 0.006753345018760251, 0.006448937460397879, 0.006354325581171363, 0.006354067492091369, 0.005676295408387824)), (Array(2207, 3229, 5873, 10, 3788, 8036, 7, 1182),Array(0.015715771663330817, 0.010712777813893593, 0.009546931920178072, 0.009522518371431704, 0.008927422336395518, 0.008624625729438713, 0.007992126063379968, 0.006529834990708393)), (Array(159, 126, 0, 275, 135, 781, 705, 494),Array(0.02698330617838032, 0.023503577242053826, 0.018666434854939382, 0.012740783445882326, 0.010704005493014743, 0.008452521480672512, 0.006296868608240401, 0.005985625523382596)), (Array(372, 1703, 1170, 3034, 3740, 2726, 1392, 1622),Array(0.04218765174213785, 0.03227499394792523, 0.022093802114524035, 0.015475453980455287, 0.014964435315255199, 0.011712488574152705, 0.01093678433906292, 0.010620867322467302)), (Array(1153, 3, 1970, 327, 883, 1039, 795, 1445),Array(0.025822818151141222, 0.01673994393550959, 0.0162221055535359, 0.016147667279789293, 0.014719242344302535, 0.013540164643026688, 0.012889860747312692, 0.011859313412890347)), (Array(5, 186, 737, 567, 1098, 3, 1443, 277),Array(0.016659851631007156, 0.014638517969975177, 0.012174676630012262, 0.011931826860210994, 0.008614177921732406, 0.008571960123656082, 0.008218750193539872, 0.007263793397792697)), (Array(706, 11, 26, 45, 3250, 100, 2655, 1503),Array(0.009623406330442461, 0.006201796360980689, 0.005204262397916099, 0.005101125247310075, 0.004691523042876648, 0.004230887210594873, 0.004168868164663573, 0.004115098940215664)), (Array(61, 41, 90, 337, 1, 496, 821, 837),Array(0.04525180789717588, 0.044049400894926156, 0.03626793680282758, 0.01679474869841332, 0.015810735006045285, 0.014316742158067308, 0.008344858534169153, 0.007884142003735476)), (Array(618, 963, 1165, 1186, 853, 182, 1308, 118),Array(0.0319889335984476, 0.028638790480427122, 0.026047925982480766, 0.025022878764599684, 0.02455547250826605, 0.023942524658324686, 0.02388196435303055, 0.023865003269207676)), (Array(232, 1316, 245, 795, 3554, 1369, 1435, 2116),Array(0.11527587937013585, 0.03426338585615468, 0.027940989066922915, 0.02586919513913989, 0.019640603495146892, 0.01340066668262875, 0.012266502334040353, 0.011400107405277141)), (Array(139, 0, 145, 254, 43, 389, 952, 1164),Array(0.030487380264716278, 0.02295707332488064, 0.02185943462445832, 0.02158006308424395, 0.017951182756044196, 0.01381046893853163, 0.012060332643759336, 0.009283336277914061)), (Array(310, 396, 215, 274, 197, 392, 892, 55),Array(0.014224688659059093, 0.010376578312465355, 0.009450453832898244, 0.007866369511682956, 0.0070289603462744266, 0.006909192142816018, 0.005681334415927907, 0.004972023971847601)), (Array(11, 45, 70, 39, 17, 95, 114, 26),Array(0.013140118570301491, 0.012611570573629211, 0.010290002183836725, 0.008671598559755129, 0.006473174221460534, 0.0054923767122421665, 0.005095853907079055, 0.0046124218623229615)), (Array(0, 34, 62, 261, 137, 141, 215, 282),Array(0.018983731833461078, 0.01360170932960869, 0.01278403121933378, 0.008430126128419025, 0.00817341548678432, 0.007575625929221351, 0.006584504383062662, 0.006500762878735947)), (Array(1249, 2647, 1273, 1554, 909, 35, 1718, 1380),Array(0.011129385435448641, 0.010095432085198731, 0.009595460498682977, 0.009056951340934477, 0.00900833191441214, 0.007665375174231455, 0.007126281020184516, 0.0065320647933313205)), (Array(120, 0, 4, 170, 999, 6, 286, 126),Array(0.03225486403415035, 0.0262872943647285, 0.021836441055240335, 0.01540176402235281, 0.013025311479277617, 0.012171814994441733, 0.010439175079255476, 0.007551180380037851)), (Array(826, 1346, 621, 2675, 2692, 2683, 2910, 2579),Array(0.03904003795793799, 0.028399486781946183, 0.027765453259295036, 0.017873203876102778, 0.017712122104733583, 0.017593788827147818, 0.016728764710215706, 0.01643507684382941)), (Array(4, 0, 69, 34, 77, 88, 135, 74),Array(0.029891732209654676, 0.02919016668133009, 0.017438265220485975, 0.011715835460063146, 0.008035463938643697, 0.006715615901529757, 0.006647248632037367, 0.006490297499117551)), (Array(1, 1091, 1540, 1655, 1431, 1585, 183, 37),Array(0.008667614219590534, 0.0075079074864762485, 0.006178871682362424, 0.006155279388516955, 0.005976711316397548, 0.005824783570723379, 0.005311435196759531, 0.005241977916787085)), (Array(2, 36, 245, 305, 586, 9, 5, 13),Array(0.11416819164339527, 0.050324545368469545, 0.016818206760481916, 0.012752908710795822, 0.011537821982335002, 0.010719803791367678, 0.009862282038445082, 0.009030889894717564)), (Array(0, 225, 88, 103, 710, 838, 34, 456),Array(0.015562992356186932, 0.014361732260354492, 0.010996936575852071, 0.008898364089439226, 0.008510402443779293, 0.007835148639323315, 0.007822413125843019, 0.0076912002585314545)), (Array(418, 100, 30, 473, 144, 621, 593, 1288),Array(0.0490383779831419, 0.0451737236998185, 0.03860998978094461, 0.03313729008501857, 0.03243127474991183, 0.026676974831024446, 0.020572557981229155, 0.018627408143358587)), (Array(210, 271, 77, 474, 3, 386, 1085, 1520),Array(0.021506094077323432, 0.012642193617697675, 0.009856114103220653, 0.009640434831334053, 0.007416323493661734, 0.0070501796410253415, 0.00622283218509894, 0.004774487412967152)), (Array(91, 234, 30, 0, 198, 47, 351, 116),Array(0.049548964313542956, 0.02850432577285843, 0.028446571093525733, 0.025719194234325435, 0.019018029273716347, 0.01578734224357636, 0.013586482942791687, 0.012738997120474884)), (Array(76, 52, 155, 229, 207, 49, 279, 292),Array(0.01970360579518828, 0.01821370119779408, 0.017082344575046808, 0.013855092039910208, 0.013555870252636543, 0.013466257190504594, 0.011341706636945798, 0.010702386851864738)), (Array(3, 6, 0, 4, 5, 46, 63, 74),Array(0.01610236089988016, 0.01295793482297076, 0.012289878967059891, 0.0111242820604197, 0.00889158626143369, 0.007100839881414577, 0.006380591536520988, 0.005645343419116414)), (Array(566, 67, 2, 1454, 84, 670, 117, 355),Array(0.04161863388347664, 0.030796836222440446, 0.02320318810150786, 0.020750079375274618, 0.018812397851420987, 0.01751638950463774, 0.015932194624414184, 0.012446249856574871)), (Array(121, 148, 690, 1027, 1461, 1437, 304, 34),Array(0.015499573959212252, 0.013522719630244973, 0.012503594394591106, 0.012271030854027672, 0.009113928693704955, 0.008779880090479569, 0.008032186661676913, 0.007805528146684827)), (Array(665, 1231, 1386, 157, 878, 511, 626, 47),Array(0.025301447948363753, 0.024621884181644828, 0.022226305244453275, 0.018682292843571407, 0.015036155314197323, 0.012015630901632761, 0.01191022016715157, 0.011343585207302885)), (Array(420, 768, 531, 321, 1599, 2017, 2383, 1166),Array(0.03031565777266273, 0.018527306701743473, 0.01756508780594258, 0.011598225830002314, 0.010001306101007405, 0.007861442408790388, 0.0072026224398220665, 0.0067771252166904375)), (Array(123, 142, 130, 206, 235, 157, 202, 301),Array(0.012682775680940702, 0.011384803837954507, 0.010008416091887254, 0.00917243956825721, 0.008900390645680751, 0.008819728064668986, 0.006818054645865437, 0.006681524653828919)), (Array(6, 129, 3, 426, 428, 1, 0, 4),Array(0.021753727023331507, 0.01797498712789882, 0.01670602321448013, 0.015076049695498916, 0.014257718576193379, 0.013309556021258382, 0.01312978131727807, 0.012425290530620539)), (Array(1, 8, 243, 97, 17, 35, 348, 15),Array(0.005820074792683956, 0.004538831945186889, 0.004314856555291248, 0.004237430464162381, 0.003916465948809745, 0.003654453717041427, 0.003451672247594681, 0.0032575325582242503)), (Array(65, 119, 121, 338, 380, 124, 394, 415),Array(0.022218135305742865, 0.013080494313426295, 0.011768390685574703, 0.009349719304298362, 0.008560623310233247, 0.008474277648346488, 0.005869637229573973, 0.005341931887280892)), (Array(1, 298, 175, 8, 1497, 17, 50, 661),Array(0.014162207532755869, 0.010620395856144315, 0.0078990018349776, 0.005615073988152831, 0.005051451406521095, 0.004455989101595585, 0.004044387009852616, 0.004009099489131701)), (Array(1759, 979, 146, 1498, 1211, 1515, 3156, 2700),Array(0.017403057819584344, 0.016075336931558652, 0.013183116624561394, 0.011898752980852901, 0.011388104451927622, 0.01073784850568376, 0.010460610396011778, 0.009808156886054568)))
>     vocabList: Array[String] = Array(född, talet, kommun, svenska, svensk, sverige, stockholm, invånare, senare, tidigare, from, olika, samma, kommunen, januari, stora, tillsammans, många, mars, läst, namn, december, september, juli, juni, oktober, mycket, shift, april, augusti, meter, november, februari, namnet, amerikansk, enligt, socken, började, personer, ofta, staden, församling, norra, frankrike, stor, exempel, sveriges, plats, södra, vann, mest, större, spelade, början, regionen, största, value, därefter, mindre, cirka, historia, kyrka, john, carl, förekommer, filmen, omkring, området, grund, skådespelare, används, endast, antalet, familjen, johan, åren, säsongen, författare, county, först, medan, tillhör, gamla, känd, består, idag, antal, tyskland, politiker, arten, kyrkan, olympiska, barn, göteborg, västra, vissa, delar, tyska, karl, ingår, runt, linegrey, stockholms, finland, cobar, world, östra, slutet, amerikanska, stad, skrev, främst, erik, nära, kallas, tiden, deltog, stort, kilometer, serien, uppsala, film, departementet, album, bästa, gången, universitet, gift, samband, malmö, sång, befolkningsutveckling, version, landet, kantonen, tysk, nuvarande, york, peter, paris, norr, london, albumet, lång, havet, franska, gånger, maria, musik, kända, kung, saint, äldre, urval, egen, spelare, byggdes, låten, engelska, professor, följande, samtidigt, istället, biografi, arrondissementet, karriär, width, form, tillbaka, gray, ledamot, england, bildades, dotter, medlemmar, verk, ytterligare, anders, gustaf, europa, gruppen, sidan, ungefär, norge, italien, trots, grupp, ordförande, band, kring, dessutom, därmed, sida, enda, birds, söder, kallade, miljoner, guld, lars, plotdata, flyttade, släpptes, david, spelar, totalt, bandet, matcher, övriga, anna, översättning, studerade, black, storbritannien, period, university, danmark, fredrik, ledde, robert, längre, numera, särskilt, land, födda, helsingfors, världen, syfta, måste, klubben, världskriget, centrala, file, högsta, sommarspelen, gitarr, fortfarande, brittisk, arbetade, serie, ryska, kallad, världens, kriget, innehåller, landskommun, brittiska, internationella, aldrig, länder, slutade, thomas, poäng, kina, fransk, ursprungligen, martin, varav, göteborgs, provinsen, kort, william, användes, paul, visar, exempelvis, arter, kvinnor, släktet, andersson, medlem, boken, museum, nytt, grundades, berlin, bild, spelades, stod, division, fotbollsspelare, såsom, james, ryssland, människor, vatten, kungliga, bror, league, säsong, öster, population, laget, nästan, historik, dagens, nils, område, språk, liksom, clements, love, uppgick, michael, regi, distrikt, mitten, lämnade, gifte, roll, företaget, egna, grand, city, oftast, väster, liten, japan, start, gustav, ville, politiska, spela, fortsatte, sjön, huset, officiella, lista, förutom, fotboll, arbete, president, verksamhet, chef, svenskt, högre, områden, församlingen, spelet, lund, henrik, innebär, delstaten, sven, långa, george, year, lever, slaget, color, regeringen, silver, bort, slut, studier, europeiska, johansson, själva, verksam, procent, trummor, lyckades, titeln, skolan, musiker, spel, olof, varandra, charles, orten, lilla, iucn, årets, belägen, lika, anses, uppfördes, resultat, startade, magnus, avsnitt, förlag, tillhörde, låtar, richard, radio, böcker, respektive, nilsson, jean, förlorade, administrativ, international, utbredning, filmer, life, företag, bibliografi, rätt, danska, underarter, tidningen, vita, direkt, hand, spanien, familj, använda, distriktet, dels, sankt, avled, modern, gavs, medverkade, handling, användas, axel, island, national, partiet, föddes, united, ingick, flertal, längs, målare, dansk, konst, system, utgör, bestod, liknande, härad, använder, eget, gunnar, äldsta, listan, strax, records, bilder, vanligen, slott, live, sångare, samtliga, final, ålder, betyder, wilhelm, levande, ursprung, stöd, låtlista, uppdrag, finska, talets, gammal, minst, kallades, centrum, with, spelat, august, långt, person, örebro, handlar, konstnär, sommaren, norska, avlidna, berg, roman, tidigt, togs, late, börjar, byggnaden, census, https, hösten, svart, marie, troligen, referens, delas, adolf, länge, kungen, bildade, nådde, lärare, tekniska, lagen, stift, åter, österrike, fontsize, enbart, brons, alltid, ledare, norsk, problem, vägen, växte, increment, program, schweiz, finalen, visade, daniel, kronor, gridcolor, singel, dåvarande, ordet, betydelse, historiska, utveckling, bottom, främsta, kanada, stadens, omfattande, christian, skriven, huvudsakligen, colors, valet, gjordes, omfattar, html, ägde, sett, ihop, musikalbum, viss, köpenhamn, vertical, programmet, structure, orientation, röda, height, såväl, park, larsson, arkitekt, ekonomiska, gård, alexander, imagesize, plotarea, timeaxis, scalemajor, dateformat, förening, polen, alignbars, landets, yyyy, textcolor, information, naturreservat, herrar, ordningen, nordöstra, australien, volution, moderna, filmografi, vapen, återvände, föreningen, match, ursprungliga, status, platsen, georg, utbildning, kommuner, beskrevs, kvadratkilometer, lappland, yngre, spår, diskografi, bengt, henry, geografi, huvudstaden, johannes, räknas, utmärkelser, höga, skedde, general, inseechiffres, teater, stil, delvis, utseende, känt, livet, bättre, italienska, snabbt, valde, engelsk, militär, byggnader, miniatyr, wood, sten, debuterade, upptäcktes, medeltida, djur, trakten, sydvästra, music, platser, litteratur, tävlingen, månader, just, starkt, sydöstra, louis, tjänst, kontrakt, samarbete, spanska, bygga, nederländerna, kategoriserar, förste, hölls, kalifornien, drottning, beroende, sker, museet, japanska, pris, följd, unga, oscar, priset, nordiska, lite, skåne, bodde, mängd, brukar, bruk, projekt, arbeten, grekiska, allmänna, albert, stället, song, använde, läns, svarta, riket, hektar, skillnad, indien, rock, lades, sports, städer, press, våren, viktig, samman, gränsen, mästare, lokala, nordvästra, kultur, befolkningen, grundade, joseph, floden, piano, hustru, krig, bakgrund, sullivan, premiär, närheten, kalmar, kompositör, skapa, valdes, eriksson, american, material, checklist, forskning, jorden, emellertid, vanligt, olsson, finländsk, smith, sovjetunionen, innebar, skola, landskapet, ledning, vanliga, emot, journalist, otto, grupper, hockey, tilldelades, south, björn, brasilien, riksdagen, utnämndes, däremot, eric, skrivit, läkare, därpå, utgörs, star, north, damer, flyttades, slog, ganska, utsågs, leder, ernst, nyheter, italiensk, inrättades, senaste, lunds, fågel, följde, tidiga, cornell, studioalbum, umeå, produktion, livskraftig, invigdes, socknen, röst, aktiv, möjligt, rysk, kvinna, länsväg, herrarnas, verksamheten, nationella, parti, inklusive, exemplar, france, relativt, republiken, mexiko, jonas, angeles, samhället, hall, beslut, wien, universitetet, höger, dennes, gården, bytte, sändes, vanlig, prix, reference, västerås, författaren, download, utgiven, roller, jesus, baserad, roberson, befolkning, vunnit, schulenberg, iliff, statens, perioden, fartyg, fredericks, höll, kvinnliga, längd, west, denne, kraft, försök, försökte, ebird, ersattes, lägre, småland, upplagan, bygger, vice, kyrkans, arbetet, hertig, skrevs, johann, ford, skivan, staten, matchen, time, vänster, fågelarter, washington, baserat, politik, följer, orgel, finlands, öppna, folk, producent, ansåg, såldes, tävlade, clementschecklist, barcelona, bestående, guvernör, marken, erhöll, nord, antingen, diameter, omgången, tour, royal, resa, riktning, hålla, bäst, levde, oslo, tagit, rollista, titel, övrigt, turn, station, utgav, färg, tränare, göran, egenskaper, användning, byggde, lätt, släkte, bidrag, okänt, arbetar, syster, händelser, sånger, persson, rollen, frank, doktor, professionell, arbeta, moskva, inre, bolaget, datum, frågor, ansågs, kinesiska, slottet, viktigaste, drygt, veckor, egentligen, svårt, klass, betydande, regissör, plan, skriva, delarna, harry, priser, direktör, organisation, friedrich, märks, vetenskapliga, däribland, höjd, uppgift, your, motsvarar, utvecklades, skapade, romersk, ifrån, afrika, spelas, lämna, inledde, ungern, singlar, tjänstgjorde, framgång, orden, igenom, ludvig, förslag, högt, college, drog, alfred, kanske, prins, släppte, politisk, deltagare, artikel, spelaren, kritik, serier, karlsson, school, japansk, stefan, nordväst, singeln, seger, skiljer, andreas, pengar, verkade, team, köpte, asteroid, regering, säsonger, edward, kallat, hette, ljus, avlade, sydväst, klassen, svensson, minuter, tillkom, sedermera, ställde, officiellt, solen, texas, viktiga, visa, militära, verket, beskrivs, utöver, villa, johnson, santa, individer, mary, syfte, ishockeyspelare, catalogue, kristna, bildar, news, teatern, filosofie)
>     topics: Array[Array[(String, Double)]] = Array(Array((kommunen,0.0965465174848558), (invånare,0.09561944647066932), (kommun,0.06123216019531808), (regionen,0.06047426356346239), (departementet,0.04133489674798034), (frankrike,0.038999680148138156), (kantonen,0.038645022485160795), (tillhör,0.03709042755540236)), Array((talet,0.01860754472650629), (stockholm,0.00800131987315067), (byggdes,0.007185124511872375), (gamla,0.006562831220260255), (stora,0.006140398920615785), (idag,0.005472783411308425), (stockholms,0.005347092400479047), (tidigare,0.004738960299641082)), Array((chef,0.012891731862399829), (svenska,0.012216420059244167), (fartyget,0.008799092539391087), (regemente,0.008170528757595195), (juli,0.007589154082154804), (juni,0.00753057351996227), (fartyg,0.007501151936739348), (militär,0.007441712973556016)), Array((staden,0.03550112124416654), (kilometer,0.024239377096572767), (omkring,0.017880722746326707), (finland,0.017534872470232804), (invånare,0.01572555256779709), (stad,0.014243881447019926), (största,0.013291812684672874), (norr,0.013255209377268314)), Array((lista,0.049154032038907364), (klass,0.025424694972593722), (omgången,0.021006352382631138), (wales,0.015146400328457767), (bert,0.014384523267127552), (sjösatt,0.01237820887773258), (sektion,0.010986509101209276), (australien,0.010348525497739654)), Array((länsväg,0.024609442134056914), (sjön,0.02313788863645436), (meter,0.023099400549010995), (havet,0.016486417015966634), (santa,0.01631846978655288), (maria,0.014208691182654495), (kvadratkilometer,0.012902237189832062), (vägen,0.01089177248153098)), Array((birds,0.025131668033955827), (clements,0.01897996595347449), (world,0.01864839205373941), (läst,0.01754493324942852), (förekommer,0.015642901268097295), (arten,0.014806106220374982), (familjen,0.013746517306962325), (namn,0.012193036368057153)), Array((socken,0.02200110661389711), (namnet,0.016863821067931672), (socknen,0.011169441915583982), (härad,0.009575257990621604), (gift,0.007041018691775915), (ätten,0.006957959901751748), (samma,0.006567438459526348), (församling,0.006206311552289226)), Array((from,0.09573192102472637), (shift,0.07721821951064128), (value,0.05140930922690405), (linegrey,0.038236453151551754), (cobar,0.037725195145277224), (width,0.027601289731431947), (gray,0.026950628685089566), (plotdata,0.02535800730760235)), Array((stockholm,0.040800430698927574), (tyskland,0.0306444550609787), (tillsammans,0.02193657394829871), (förbundslandet,0.019628310699189612), (kommun,0.018135004797691912), (ingår,0.015575322731524751), (landkreis,0.014987081087118154), (kommunen,0.01387372200507514)), Array((county,0.059935597281339396), (läst,0.025518279914354958), (invånare,0.021135159839241465), (census,0.01909964708101696), (delstaten,0.015705965232071534), (united,0.010849948320306425), (states,0.009618564510993638), (enligt,0.009077797952871827)), Array((kina,0.02947986672544381), (grand,0.01634731925291766), (prix,0.016230381323561027), (vann,0.012766683820251106), (kinesiska,0.011650847462746833), (säsongen,0.011556626328903735), (lopp,0.01020456193177741), (loppet,0.009873469854259966)), Array((programmet,0.019344695557099535), (program,0.012893474006120904), (radio,0.011736207069720414), (programledare,0.01063631773660054), (sveriges,0.010443829299695546), (grupp,0.0101881977731672), (sändes,0.009557330691273938), (poäng,0.008217466852727128)), Array((svensk,0.020289726693646915), (umeå,0.016791173665524066), (född,0.016079171985534508), (ledamot,0.01499395294114528), (ordförande,0.013972428567201652), (andersson,0.012374270949374168), (läns,0.011168740554371137), (riksdagen,0.011135322353134584)), Array((arten,0.010801612339730723), (förekommer,0.009156920691378509), (arter,0.007179320551697595), (mycket,0.006753345018760251), (familjen,0.006448937460397879), (olika,0.006354325581171363), (ofta,0.006354067492091369), (släktet,0.005676295408387824)), Array((robin,0.015715771663330817), (bell,0.010712777813893593), (grey,0.009546931920178072), (from,0.009522518371431704), (castle,0.008927422336395518), (hood,0.008624625729438713), (invånare,0.007992126063379968), (arthur,0.006529834990708393)), Array((professor,0.02698330617838032), (universitet,0.023503577242053826), (född,0.018666434854939382), (berlin,0.012740783445882326), (tysk,0.010704005493014743), (universitetet,0.008452521480672512), (forskning,0.006296868608240401), (tekniska,0.005985625523382596)), Array((årets,0.04218765174213785), (mount,0.03227499394792523), (flygplats,0.022093802114524035), (airport,0.015475453980455287), (antarktis,0.014964435315255199), (flygplatsen,0.011712488574152705), (berget,0.01093678433906292), (chile,0.010620867322467302)), Array((rubriken,0.025822818151141222), (svenska,0.01673994393550959), (psalmer,0.0162221055535359), (lista,0.016147667279789293), (sånger,0.014719242344302535), (valkrets,0.013540164643026688), (jesus,0.012889860747312692), (publicerad,0.011859313412890347)), Array((sverige,0.016659851631007156), (grupp,0.014638517969975177), (damer,0.012174676630012262), (herrar,0.011931826860210994), (open,0.008614177921732406), (svenska,0.008571960123656082), (vinner,0.008218750193539872), (spelades,0.007263793397792697)), Array((jorden,0.009623406330442461), (olika,0.006201796360980689), (mycket,0.005204262397916099), (exempel,0.005101125247310075), (månen,0.004691523042876648), (runt,0.004230887210594873), (jordens,0.004168868164663573), (planet,0.004115098940215664)), Array((kyrka,0.04525180789717588), (församling,0.044049400894926156), (kyrkan,0.03626793680282758), (församlingen,0.01679474869841332), (talet,0.015810735006045285), (stift,0.014316742158067308), (kyrkans,0.008344858534169153), (orgel,0.007884142003735476)), Array((upptäcktes,0.0319889335984476), (asteroid,0.028638790480427122), (asteroiden,0.026047925982480766), (astronomen,0.025022878764599684), (diameter,0.02455547250826605), (ungefär,0.023942524658324686), (huvudbältet,0.02388196435303055), (kilometer,0.023865003269207676)), Array((file,0.11527587937013585), (image,0.03426338585615468), (landskommun,0.027940989066922915), (jesus,0.02586919513913989), (landsfiskalsdistrikt,0.019640603495146892), (street,0.01340066668262875), (church,0.012266502334040353), (line,0.011400107405277141)), Array((paris,0.030487380264716278), (född,0.02295707332488064), (franska,0.02185943462445832), (fransk,0.02158006308424395), (frankrike,0.017951182756044196), (jean,0.01381046893853163), (japansk,0.012060332643759336), (fran,0.009283336277914061)), Array((företaget,0.014224688659059093), (företag,0.010376578312465355), (university,0.009450453832898244), (grundades,0.007866369511682956), (miljoner,0.0070289603462744266), (international,0.006909192142816018), (bolaget,0.005681334415927907), (största,0.004972023971847601)), Array((olika,0.013140118570301491), (exempel,0.012611570573629211), (används,0.010290002183836725), (ofta,0.008671598559755129), (många,0.006473174221460534), (vissa,0.0054923767122421665), (kallas,0.005095853907079055), (mycket,0.0046124218623229615)), Array((född,0.018983731833461078), (amerikansk,0.01360170932960869), (john,0.01278403121933378), (william,0.008430126128419025), (york,0.00817341548678432), (london,0.007575625929221351), (university,0.006584504383062662), (james,0.006500762878735947)), Array((fängelse,0.011129385435448641), (mytologi,0.010095432085198731), (brott,0.009595460498682977), (dömdes,0.009056951340934477), (harry,0.00900833191441214), (enligt,0.007665375174231455), (bergman,0.007126281020184516), (polisen,0.0065320647933313205)), Array((uppsala,0.03225486403415035), (född,0.0262872943647285), (svensk,0.021836441055240335), (ledamot,0.01540176402235281), (filosofie,0.013025311479277617), (stockholm,0.012171814994441733), (kungliga,0.010439175079255476), (universitet,0.007551180380037851)), Array((ford,0.03904003795793799), (månaden,0.028399486781946183), (trakten,0.027765453259295036), (varmaste,0.017873203876102778), (kallaste,0.017712122104733583), (medeltemperaturen,0.017593788827147818), (årsmedeltemperaturen,0.016728764710215706), (nederbörd,0.01643507684382941)), Array((svensk,0.029891732209654676), (född,0.02919016668133009), (skådespelare,0.017438265220485975), (amerikansk,0.011715835460063146), (författare,0.008035463938643697), (politiker,0.006715615901529757), (tysk,0.006647248632037367), (johan,0.006490297499117551)), Array((talet,0.008667614219590534), (motor,0.0075079074864762485), (modellen,0.006178871682362424), (bilen,0.006155279388516955), (tillverkades,0.005976711316397548), (hästar,0.005824783570723379), (norge,0.005311435196759531), (började,0.005241977916787085)), Array((kommun,0.11416819164339527), (socken,0.050324545368469545), (landskommun,0.016818206760481916), (distrikt,0.012752908710795822), (lappland,0.011537821982335002), (tidigare,0.010719803791367678), (sverige,0.009862282038445082), (kommunen,0.009030889894717564)), Array((född,0.015562992356186932), (helsingfors,0.014361732260354492), (politiker,0.010996936575852071), (finland,0.008898364089439226), (finländsk,0.008510402443779293), (finlands,0.007835148639323315), (amerikansk,0.007822413125843019), (finska,0.0076912002585314545)), Array((island,0.0490383779831419), (runt,0.0451737236998185), (meter,0.03860998978094461), (berg,0.03313729008501857), (havet,0.03243127474991183), (trakten,0.026676974831024446), (huvudstaden,0.020572557981229155), (toppen,0.018627408143358587)), Array((översättning,0.021506094077323432), (boken,0.012642193617697675), (författare,0.009856114103220653), (roman,0.009640434831334053), (svenska,0.007416323493661734), (böcker,0.0070501796410253415), (romanen,0.00622283218509894), (bonnier,0.004774487412967152)), Array((olympiska,0.049548964313542956), (sommarspelen,0.02850432577285843), (meter,0.028446571093525733), (född,0.025719194234325435), (guld,0.019018029273716347), (plats,0.01578734224357636), (silver,0.013586482942791687), (deltog,0.012738997120474884)), Array((säsongen,0.01970360579518828), (spelade,0.01821370119779408), (spelare,0.017082344575046808), (klubben,0.013855092039910208), (matcher,0.013555870252636543), (vann,0.013466257190504594), (division,0.011341706636945798), (laget,0.010702386851864738)), Array((svenska,0.01610236089988016), (stockholm,0.01295793482297076), (född,0.012289878967059891), (svensk,0.0111242820604197), (sverige,0.00889158626143369), (sveriges,0.007100839881414577), (carl,0.006380591536520988), (johan,0.005645343419116414)), Array((naturreservat,0.04161863388347664), (området,0.030796836222440446), (kommun,0.02320318810150786), (reservatet,0.020750079375274618), (består,0.018812397851420987), (hektar,0.01751638950463774), (stort,0.015932194624414184), (europeiska,0.012446249856574871)), Array((film,0.015499573959212252), (musik,0.013522719630244973), (piano,0.012503594394591106), (opera,0.012271030854027672), (operan,0.009113928693704955), (orkester,0.008779880090479569), (regi,0.008032186661676913), (amerikansk,0.007805528146684827)), Array((song,0.025301447948363753), (eurovision,0.024621884181644828), (contest,0.022226305244453275), (låten,0.018682292843571407), (bidrag,0.015036155314197323), (finalen,0.012015630901632761), (tävlingen,0.01191022016715157), (plats,0.011343585207302885)), Array((partiet,0.03031565777266273), (parti,0.018527306701743473), (valet,0.01756508780594258), (politiska,0.011598225830002314), (partier,0.010001306101007405), (mandat,0.007861442408790388), (partiets,0.0072026224398220665), (politiskt,0.0067771252166904375)), Array((album,0.012682775680940702), (albumet,0.011384803837954507), (sång,0.010008416091887254), (bandet,0.00917243956825721), (gitarr,0.008900390645680751), (låten,0.008819728064668986), (släpptes,0.006818054645865437), (love,0.006681524653828919)), Array((stockholm,0.021753727023331507), (malmö,0.01797498712789882), (svenska,0.01670602321448013), (målare,0.015076049695498916), (konst,0.014257718576193379), (talet,0.013309556021258382), (född,0.01312978131727807), (svensk,0.012425290530620539)), Array((talet,0.005820074792683956), (senare,0.004538831945186889), (kriget,0.004314856555291248), (tyska,0.004237430464162381), (många,0.003916465948809745), (enligt,0.003654453717041427), (slaget,0.003451672247594681), (stora,0.0032575325582242503)), Array((filmen,0.022218135305742865), (serien,0.013080494313426295), (film,0.011768390685574703), (spelet,0.009349719304298362), (avsnitt,0.008560623310233247), (bästa,0.008474277648346488), (filmer,0.005869637229573973), (handling,0.005341931887280892)), Array((talet,0.014162207532755869), (språk,0.010620395856144315), (verk,0.0078990018349776), (senare,0.005615073988152831), (strauss,0.005051451406521095), (många,0.004455989101595585), (mest,0.004044387009852616), (grekiska,0.004009099489131701)), Array((bibcode,0.017403057819584344), (solen,0.016075336931558652), (gånger,0.013183116624561394), (stars,0.011898752980852901), (stjärna,0.011388104451927622), (massa,0.01073784850568376), (arxiv,0.010460610396011778), (solens,0.009808156886054568)))

In [ ]:
println(s"$numTopics topics:")
topics.zipWithIndex.foreach { case (topic, i) =>
  println(s"TOPIC $i")
  topic.foreach { case (term, weight) => println(s"$term\t$weight") }
  println(s"==========")
}

  

>     50 topics:
>     TOPIC 0
>     kommunen	0.0965465174848558
>     invånare	0.09561944647066932
>     kommun	0.06123216019531808
>     regionen	0.06047426356346239
>     departementet	0.04133489674798034
>     frankrike	0.038999680148138156
>     kantonen	0.038645022485160795
>     tillhör	0.03709042755540236
>     ==========
>     TOPIC 1
>     talet	0.01860754472650629
>     stockholm	0.00800131987315067
>     byggdes	0.007185124511872375
>     gamla	0.006562831220260255
>     stora	0.006140398920615785
>     idag	0.005472783411308425
>     stockholms	0.005347092400479047
>     tidigare	0.004738960299641082
>     ==========
>     TOPIC 2
>     chef	0.012891731862399829
>     svenska	0.012216420059244167
>     fartyget	0.008799092539391087
>     regemente	0.008170528757595195
>     juli	0.007589154082154804
>     juni	0.00753057351996227
>     fartyg	0.007501151936739348
>     militär	0.007441712973556016
>     ==========
>     TOPIC 3
>     staden	0.03550112124416654
>     kilometer	0.024239377096572767
>     omkring	0.017880722746326707
>     finland	0.017534872470232804
>     invånare	0.01572555256779709
>     stad	0.014243881447019926
>     största	0.013291812684672874
>     norr	0.013255209377268314
>     ==========
>     TOPIC 4
>     lista	0.049154032038907364
>     klass	0.025424694972593722
>     omgången	0.021006352382631138
>     wales	0.015146400328457767
>     bert	0.014384523267127552
>     sjösatt	0.01237820887773258
>     sektion	0.010986509101209276
>     australien	0.010348525497739654
>     ==========
>     TOPIC 5
>     länsväg	0.024609442134056914
>     sjön	0.02313788863645436
>     meter	0.023099400549010995
>     havet	0.016486417015966634
>     santa	0.01631846978655288
>     maria	0.014208691182654495
>     kvadratkilometer	0.012902237189832062
>     vägen	0.01089177248153098
>     ==========
>     TOPIC 6
>     birds	0.025131668033955827
>     clements	0.01897996595347449
>     world	0.01864839205373941
>     läst	0.01754493324942852
>     förekommer	0.015642901268097295
>     arten	0.014806106220374982
>     familjen	0.013746517306962325
>     namn	0.012193036368057153
>     ==========
>     TOPIC 7
>     socken	0.02200110661389711
>     namnet	0.016863821067931672
>     socknen	0.011169441915583982
>     härad	0.009575257990621604
>     gift	0.007041018691775915
>     ätten	0.006957959901751748
>     samma	0.006567438459526348
>     församling	0.006206311552289226
>     ==========
>     TOPIC 8
>     from	0.09573192102472637
>     shift	0.07721821951064128
>     value	0.05140930922690405
>     linegrey	0.038236453151551754
>     cobar	0.037725195145277224
>     width	0.027601289731431947
>     gray	0.026950628685089566
>     plotdata	0.02535800730760235
>     ==========
>     TOPIC 9
>     stockholm	0.040800430698927574
>     tyskland	0.0306444550609787
>     tillsammans	0.02193657394829871
>     förbundslandet	0.019628310699189612
>     kommun	0.018135004797691912
>     ingår	0.015575322731524751
>     landkreis	0.014987081087118154
>     kommunen	0.01387372200507514
>     ==========
>     TOPIC 10
>     county	0.059935597281339396
>     läst	0.025518279914354958
>     invånare	0.021135159839241465
>     census	0.01909964708101696
>     delstaten	0.015705965232071534
>     united	0.010849948320306425
>     states	0.009618564510993638
>     enligt	0.009077797952871827
>     ==========
>     TOPIC 11
>     kina	0.02947986672544381
>     grand	0.01634731925291766
>     prix	0.016230381323561027
>     vann	0.012766683820251106
>     kinesiska	0.011650847462746833
>     säsongen	0.011556626328903735
>     lopp	0.01020456193177741
>     loppet	0.009873469854259966
>     ==========
>     TOPIC 12
>     programmet	0.019344695557099535
>     program	0.012893474006120904
>     radio	0.011736207069720414
>     programledare	0.01063631773660054
>     sveriges	0.010443829299695546
>     grupp	0.0101881977731672
>     sändes	0.009557330691273938
>     poäng	0.008217466852727128
>     ==========
>     TOPIC 13
>     svensk	0.020289726693646915
>     umeå	0.016791173665524066
>     född	0.016079171985534508
>     ledamot	0.01499395294114528
>     ordförande	0.013972428567201652
>     andersson	0.012374270949374168
>     läns	0.011168740554371137
>     riksdagen	0.011135322353134584
>     ==========
>     TOPIC 14
>     arten	0.010801612339730723
>     förekommer	0.009156920691378509
>     arter	0.007179320551697595
>     mycket	0.006753345018760251
>     familjen	0.006448937460397879
>     olika	0.006354325581171363
>     ofta	0.006354067492091369
>     släktet	0.005676295408387824
>     ==========
>     TOPIC 15
>     robin	0.015715771663330817
>     bell	0.010712777813893593
>     grey	0.009546931920178072
>     from	0.009522518371431704
>     castle	0.008927422336395518
>     hood	0.008624625729438713
>     invånare	0.007992126063379968
>     arthur	0.006529834990708393
>     ==========
>     TOPIC 16
>     professor	0.02698330617838032
>     universitet	0.023503577242053826
>     född	0.018666434854939382
>     berlin	0.012740783445882326
>     tysk	0.010704005493014743
>     universitetet	0.008452521480672512
>     forskning	0.006296868608240401
>     tekniska	0.005985625523382596
>     ==========
>     TOPIC 17
>     årets	0.04218765174213785
>     mount	0.03227499394792523
>     flygplats	0.022093802114524035
>     airport	0.015475453980455287
>     antarktis	0.014964435315255199
>     flygplatsen	0.011712488574152705
>     berget	0.01093678433906292
>     chile	0.010620867322467302
>     ==========
>     TOPIC 18
>     rubriken	0.025822818151141222
>     svenska	0.01673994393550959
>     psalmer	0.0162221055535359
>     lista	0.016147667279789293
>     sånger	0.014719242344302535
>     valkrets	0.013540164643026688
>     jesus	0.012889860747312692
>     publicerad	0.011859313412890347
>     ==========
>     TOPIC 19
>     sverige	0.016659851631007156
>     grupp	0.014638517969975177
>     damer	0.012174676630012262
>     herrar	0.011931826860210994
>     open	0.008614177921732406
>     svenska	0.008571960123656082
>     vinner	0.008218750193539872
>     spelades	0.007263793397792697
>     ==========
>     TOPIC 20
>     jorden	0.009623406330442461
>     olika	0.006201796360980689
>     mycket	0.005204262397916099
>     exempel	0.005101125247310075
>     månen	0.004691523042876648
>     runt	0.004230887210594873
>     jordens	0.004168868164663573
>     planet	0.004115098940215664
>     ==========
>     TOPIC 21
>     kyrka	0.04525180789717588
>     församling	0.044049400894926156
>     kyrkan	0.03626793680282758
>     församlingen	0.01679474869841332
>     talet	0.015810735006045285
>     stift	0.014316742158067308
>     kyrkans	0.008344858534169153
>     orgel	0.007884142003735476
>     ==========
>     TOPIC 22
>     upptäcktes	0.0319889335984476
>     asteroid	0.028638790480427122
>     asteroiden	0.026047925982480766
>     astronomen	0.025022878764599684
>     diameter	0.02455547250826605
>     ungefär	0.023942524658324686
>     huvudbältet	0.02388196435303055
>     kilometer	0.023865003269207676
>     ==========
>     TOPIC 23
>     file	0.11527587937013585
>     image	0.03426338585615468
>     landskommun	0.027940989066922915
>     jesus	0.02586919513913989
>     landsfiskalsdistrikt	0.019640603495146892
>     street	0.01340066668262875
>     church	0.012266502334040353
>     line	0.011400107405277141
>     ==========
>     TOPIC 24
>     paris	0.030487380264716278
>     född	0.02295707332488064
>     franska	0.02185943462445832
>     fransk	0.02158006308424395
>     frankrike	0.017951182756044196
>     jean	0.01381046893853163
>     japansk	0.012060332643759336
>     fran	0.009283336277914061
>     ==========
>     TOPIC 25
>     företaget	0.014224688659059093
>     företag	0.010376578312465355
>     university	0.009450453832898244
>     grundades	0.007866369511682956
>     miljoner	0.0070289603462744266
>     international	0.006909192142816018
>     bolaget	0.005681334415927907
>     största	0.004972023971847601
>     ==========
>     TOPIC 26
>     olika	0.013140118570301491
>     exempel	0.012611570573629211
>     används	0.010290002183836725
>     ofta	0.008671598559755129
>     många	0.006473174221460534
>     vissa	0.0054923767122421665
>     kallas	0.005095853907079055
>     mycket	0.0046124218623229615
>     ==========
>     TOPIC 27
>     född	0.018983731833461078
>     amerikansk	0.01360170932960869
>     john	0.01278403121933378
>     william	0.008430126128419025
>     york	0.00817341548678432
>     london	0.007575625929221351
>     university	0.006584504383062662
>     james	0.006500762878735947
>     ==========
>     TOPIC 28
>     fängelse	0.011129385435448641
>     mytologi	0.010095432085198731
>     brott	0.009595460498682977
>     dömdes	0.009056951340934477
>     harry	0.00900833191441214
>     enligt	0.007665375174231455
>     bergman	0.007126281020184516
>     polisen	0.0065320647933313205
>     ==========
>     TOPIC 29
>     uppsala	0.03225486403415035
>     född	0.0262872943647285
>     svensk	0.021836441055240335
>     ledamot	0.01540176402235281
>     filosofie	0.013025311479277617
>     stockholm	0.012171814994441733
>     kungliga	0.010439175079255476
>     universitet	0.007551180380037851
>     ==========
>     TOPIC 30
>     ford	0.03904003795793799
>     månaden	0.028399486781946183
>     trakten	0.027765453259295036
>     varmaste	0.017873203876102778
>     kallaste	0.017712122104733583
>     medeltemperaturen	0.017593788827147818
>     årsmedeltemperaturen	0.016728764710215706
>     nederbörd	0.01643507684382941
>     ==========
>     TOPIC 31
>     svensk	0.029891732209654676
>     född	0.02919016668133009
>     skådespelare	0.017438265220485975
>     amerikansk	0.011715835460063146
>     författare	0.008035463938643697
>     politiker	0.006715615901529757
>     tysk	0.006647248632037367
>     johan	0.006490297499117551
>     ==========
>     TOPIC 32
>     talet	0.008667614219590534
>     motor	0.0075079074864762485
>     modellen	0.006178871682362424
>     bilen	0.006155279388516955
>     tillverkades	0.005976711316397548
>     hästar	0.005824783570723379
>     norge	0.005311435196759531
>     började	0.005241977916787085
>     ==========
>     TOPIC 33
>     kommun	0.11416819164339527
>     socken	0.050324545368469545
>     landskommun	0.016818206760481916
>     distrikt	0.012752908710795822
>     lappland	0.011537821982335002
>     tidigare	0.010719803791367678
>     sverige	0.009862282038445082
>     kommunen	0.009030889894717564
>     ==========
>     TOPIC 34
>     född	0.015562992356186932
>     helsingfors	0.014361732260354492
>     politiker	0.010996936575852071
>     finland	0.008898364089439226
>     finländsk	0.008510402443779293
>     finlands	0.007835148639323315
>     amerikansk	0.007822413125843019
>     finska	0.0076912002585314545
>     ==========
>     TOPIC 35
>     island	0.0490383779831419
>     runt	0.0451737236998185
>     meter	0.03860998978094461
>     berg	0.03313729008501857
>     havet	0.03243127474991183
>     trakten	0.026676974831024446
>     huvudstaden	0.020572557981229155
>     toppen	0.018627408143358587
>     ==========
>     TOPIC 36
>     översättning	0.021506094077323432
>     boken	0.012642193617697675
>     författare	0.009856114103220653
>     roman	0.009640434831334053
>     svenska	0.007416323493661734
>     böcker	0.0070501796410253415
>     romanen	0.00622283218509894
>     bonnier	0.004774487412967152
>     ==========
>     TOPIC 37
>     olympiska	0.049548964313542956
>     sommarspelen	0.02850432577285843
>     meter	0.028446571093525733
>     född	0.025719194234325435
>     guld	0.019018029273716347
>     plats	0.01578734224357636
>     silver	0.013586482942791687
>     deltog	0.012738997120474884
>     ==========
>     TOPIC 38
>     säsongen	0.01970360579518828
>     spelade	0.01821370119779408
>     spelare	0.017082344575046808
>     klubben	0.013855092039910208
>     matcher	0.013555870252636543
>     vann	0.013466257190504594
>     division	0.011341706636945798
>     laget	0.010702386851864738
>     ==========
>     TOPIC 39
>     svenska	0.01610236089988016
>     stockholm	0.01295793482297076
>     född	0.012289878967059891
>     svensk	0.0111242820604197
>     sverige	0.00889158626143369
>     sveriges	0.007100839881414577
>     carl	0.006380591536520988
>     johan	0.005645343419116414
>     ==========
>     TOPIC 40
>     naturreservat	0.04161863388347664
>     området	0.030796836222440446
>     kommun	0.02320318810150786
>     reservatet	0.020750079375274618
>     består	0.018812397851420987
>     hektar	0.01751638950463774
>     stort	0.015932194624414184
>     europeiska	0.012446249856574871
>     ==========
>     TOPIC 41
>     film	0.015499573959212252
>     musik	0.013522719630244973
>     piano	0.012503594394591106
>     opera	0.012271030854027672
>     operan	0.009113928693704955
>     orkester	0.008779880090479569
>     regi	0.008032186661676913
>     amerikansk	0.007805528146684827
>     ==========
>     TOPIC 42
>     song	0.025301447948363753
>     eurovision	0.024621884181644828
>     contest	0.022226305244453275
>     låten	0.018682292843571407
>     bidrag	0.015036155314197323
>     finalen	0.012015630901632761
>     tävlingen	0.01191022016715157
>     plats	0.011343585207302885
>     ==========
>     TOPIC 43
>     partiet	0.03031565777266273
>     parti	0.018527306701743473
>     valet	0.01756508780594258
>     politiska	0.011598225830002314
>     partier	0.010001306101007405
>     mandat	0.007861442408790388
>     partiets	0.0072026224398220665
>     politiskt	0.0067771252166904375
>     ==========
>     TOPIC 44
>     album	0.012682775680940702
>     albumet	0.011384803837954507
>     sång	0.010008416091887254
>     bandet	0.00917243956825721
>     gitarr	0.008900390645680751
>     låten	0.008819728064668986
>     släpptes	0.006818054645865437
>     love	0.006681524653828919
>     ==========
>     TOPIC 45
>     stockholm	0.021753727023331507
>     malmö	0.01797498712789882
>     svenska	0.01670602321448013
>     målare	0.015076049695498916
>     konst	0.014257718576193379
>     talet	0.013309556021258382
>     född	0.01312978131727807
>     svensk	0.012425290530620539
>     ==========
>     TOPIC 46
>     talet	0.005820074792683956
>     senare	0.004538831945186889
>     kriget	0.004314856555291248
>     tyska	0.004237430464162381
>     många	0.003916465948809745
>     enligt	0.003654453717041427
>     slaget	0.003451672247594681
>     stora	0.0032575325582242503
>     ==========
>     TOPIC 47
>     filmen	0.022218135305742865
>     serien	0.013080494313426295
>     film	0.011768390685574703
>     spelet	0.009349719304298362
>     avsnitt	0.008560623310233247
>     bästa	0.008474277648346488
>     filmer	0.005869637229573973
>     handling	0.005341931887280892
>     ==========
>     TOPIC 48
>     talet	0.014162207532755869
>     språk	0.010620395856144315
>     verk	0.0078990018349776
>     senare	0.005615073988152831
>     strauss	0.005051451406521095
>     många	0.004455989101595585
>     mest	0.004044387009852616
>     grekiska	0.004009099489131701
>     ==========
>     TOPIC 49
>     bibcode	0.017403057819584344
>     solen	0.016075336931558652
>     gånger	0.013183116624561394
>     stars	0.011898752980852901
>     stjärna	0.011388104451927622
>     massa	0.01073784850568376
>     arxiv	0.010460610396011778
>     solens	0.009808156886054568
>     ==========

  

Visualization
-------------

In [ ]:
// Zip topic terms with topic IDs
val termArray = topics.zipWithIndex

  

>     termArray: Array[(Array[(String, Double)], Int)] = Array((Array((kommunen,0.0965465174848558), (invånare,0.09561944647066932), (kommun,0.06123216019531808), (regionen,0.06047426356346239), (departementet,0.04133489674798034), (frankrike,0.038999680148138156), (kantonen,0.038645022485160795), (tillhör,0.03709042755540236)),0), (Array((talet,0.01860754472650629), (stockholm,0.00800131987315067), (byggdes,0.007185124511872375), (gamla,0.006562831220260255), (stora,0.006140398920615785), (idag,0.005472783411308425), (stockholms,0.005347092400479047), (tidigare,0.004738960299641082)),1), (Array((chef,0.012891731862399829), (svenska,0.012216420059244167), (fartyget,0.008799092539391087), (regemente,0.008170528757595195), (juli,0.007589154082154804), (juni,0.00753057351996227), (fartyg,0.007501151936739348), (militär,0.007441712973556016)),2), (Array((staden,0.03550112124416654), (kilometer,0.024239377096572767), (omkring,0.017880722746326707), (finland,0.017534872470232804), (invånare,0.01572555256779709), (stad,0.014243881447019926), (största,0.013291812684672874), (norr,0.013255209377268314)),3), (Array((lista,0.049154032038907364), (klass,0.025424694972593722), (omgången,0.021006352382631138), (wales,0.015146400328457767), (bert,0.014384523267127552), (sjösatt,0.01237820887773258), (sektion,0.010986509101209276), (australien,0.010348525497739654)),4), (Array((länsväg,0.024609442134056914), (sjön,0.02313788863645436), (meter,0.023099400549010995), (havet,0.016486417015966634), (santa,0.01631846978655288), (maria,0.014208691182654495), (kvadratkilometer,0.012902237189832062), (vägen,0.01089177248153098)),5), (Array((birds,0.025131668033955827), (clements,0.01897996595347449), (world,0.01864839205373941), (läst,0.01754493324942852), (förekommer,0.015642901268097295), (arten,0.014806106220374982), (familjen,0.013746517306962325), (namn,0.012193036368057153)),6), (Array((socken,0.02200110661389711), (namnet,0.016863821067931672), (socknen,0.011169441915583982), (härad,0.009575257990621604), (gift,0.007041018691775915), (ätten,0.006957959901751748), (samma,0.006567438459526348), (församling,0.006206311552289226)),7), (Array((from,0.09573192102472637), (shift,0.07721821951064128), (value,0.05140930922690405), (linegrey,0.038236453151551754), (cobar,0.037725195145277224), (width,0.027601289731431947), (gray,0.026950628685089566), (plotdata,0.02535800730760235)),8), (Array((stockholm,0.040800430698927574), (tyskland,0.0306444550609787), (tillsammans,0.02193657394829871), (förbundslandet,0.019628310699189612), (kommun,0.018135004797691912), (ingår,0.015575322731524751), (landkreis,0.014987081087118154), (kommunen,0.01387372200507514)),9), (Array((county,0.059935597281339396), (läst,0.025518279914354958), (invånare,0.021135159839241465), (census,0.01909964708101696), (delstaten,0.015705965232071534), (united,0.010849948320306425), (states,0.009618564510993638), (enligt,0.009077797952871827)),10), (Array((kina,0.02947986672544381), (grand,0.01634731925291766), (prix,0.016230381323561027), (vann,0.012766683820251106), (kinesiska,0.011650847462746833), (säsongen,0.011556626328903735), (lopp,0.01020456193177741), (loppet,0.009873469854259966)),11), (Array((programmet,0.019344695557099535), (program,0.012893474006120904), (radio,0.011736207069720414), (programledare,0.01063631773660054), (sveriges,0.010443829299695546), (grupp,0.0101881977731672), (sändes,0.009557330691273938), (poäng,0.008217466852727128)),12), (Array((svensk,0.020289726693646915), (umeå,0.016791173665524066), (född,0.016079171985534508), (ledamot,0.01499395294114528), (ordförande,0.013972428567201652), (andersson,0.012374270949374168), (läns,0.011168740554371137), (riksdagen,0.011135322353134584)),13), (Array((arten,0.010801612339730723), (förekommer,0.009156920691378509), (arter,0.007179320551697595), (mycket,0.006753345018760251), (familjen,0.006448937460397879), (olika,0.006354325581171363), (ofta,0.006354067492091369), (släktet,0.005676295408387824)),14), (Array((robin,0.015715771663330817), (bell,0.010712777813893593), (grey,0.009546931920178072), (from,0.009522518371431704), (castle,0.008927422336395518), (hood,0.008624625729438713), (invånare,0.007992126063379968), (arthur,0.006529834990708393)),15), (Array((professor,0.02698330617838032), (universitet,0.023503577242053826), (född,0.018666434854939382), (berlin,0.012740783445882326), (tysk,0.010704005493014743), (universitetet,0.008452521480672512), (forskning,0.006296868608240401), (tekniska,0.005985625523382596)),16), (Array((årets,0.04218765174213785), (mount,0.03227499394792523), (flygplats,0.022093802114524035), (airport,0.015475453980455287), (antarktis,0.014964435315255199), (flygplatsen,0.011712488574152705), (berget,0.01093678433906292), (chile,0.010620867322467302)),17), (Array((rubriken,0.025822818151141222), (svenska,0.01673994393550959), (psalmer,0.0162221055535359), (lista,0.016147667279789293), (sånger,0.014719242344302535), (valkrets,0.013540164643026688), (jesus,0.012889860747312692), (publicerad,0.011859313412890347)),18), (Array((sverige,0.016659851631007156), (grupp,0.014638517969975177), (damer,0.012174676630012262), (herrar,0.011931826860210994), (open,0.008614177921732406), (svenska,0.008571960123656082), (vinner,0.008218750193539872), (spelades,0.007263793397792697)),19), (Array((jorden,0.009623406330442461), (olika,0.006201796360980689), (mycket,0.005204262397916099), (exempel,0.005101125247310075), (månen,0.004691523042876648), (runt,0.004230887210594873), (jordens,0.004168868164663573), (planet,0.004115098940215664)),20), (Array((kyrka,0.04525180789717588), (församling,0.044049400894926156), (kyrkan,0.03626793680282758), (församlingen,0.01679474869841332), (talet,0.015810735006045285), (stift,0.014316742158067308), (kyrkans,0.008344858534169153), (orgel,0.007884142003735476)),21), (Array((upptäcktes,0.0319889335984476), (asteroid,0.028638790480427122), (asteroiden,0.026047925982480766), (astronomen,0.025022878764599684), (diameter,0.02455547250826605), (ungefär,0.023942524658324686), (huvudbältet,0.02388196435303055), (kilometer,0.023865003269207676)),22), (Array((file,0.11527587937013585), (image,0.03426338585615468), (landskommun,0.027940989066922915), (jesus,0.02586919513913989), (landsfiskalsdistrikt,0.019640603495146892), (street,0.01340066668262875), (church,0.012266502334040353), (line,0.011400107405277141)),23), (Array((paris,0.030487380264716278), (född,0.02295707332488064), (franska,0.02185943462445832), (fransk,0.02158006308424395), (frankrike,0.017951182756044196), (jean,0.01381046893853163), (japansk,0.012060332643759336), (fran,0.009283336277914061)),24), (Array((företaget,0.014224688659059093), (företag,0.010376578312465355), (university,0.009450453832898244), (grundades,0.007866369511682956), (miljoner,0.0070289603462744266), (international,0.006909192142816018), (bolaget,0.005681334415927907), (största,0.004972023971847601)),25), (Array((olika,0.013140118570301491), (exempel,0.012611570573629211), (används,0.010290002183836725), (ofta,0.008671598559755129), (många,0.006473174221460534), (vissa,0.0054923767122421665), (kallas,0.005095853907079055), (mycket,0.0046124218623229615)),26), (Array((född,0.018983731833461078), (amerikansk,0.01360170932960869), (john,0.01278403121933378), (william,0.008430126128419025), (york,0.00817341548678432), (london,0.007575625929221351), (university,0.006584504383062662), (james,0.006500762878735947)),27), (Array((fängelse,0.011129385435448641), (mytologi,0.010095432085198731), (brott,0.009595460498682977), (dömdes,0.009056951340934477), (harry,0.00900833191441214), (enligt,0.007665375174231455), (bergman,0.007126281020184516), (polisen,0.0065320647933313205)),28), (Array((uppsala,0.03225486403415035), (född,0.0262872943647285), (svensk,0.021836441055240335), (ledamot,0.01540176402235281), (filosofie,0.013025311479277617), (stockholm,0.012171814994441733), (kungliga,0.010439175079255476), (universitet,0.007551180380037851)),29), (Array((ford,0.03904003795793799), (månaden,0.028399486781946183), (trakten,0.027765453259295036), (varmaste,0.017873203876102778), (kallaste,0.017712122104733583), (medeltemperaturen,0.017593788827147818), (årsmedeltemperaturen,0.016728764710215706), (nederbörd,0.01643507684382941)),30), (Array((svensk,0.029891732209654676), (född,0.02919016668133009), (skådespelare,0.017438265220485975), (amerikansk,0.011715835460063146), (författare,0.008035463938643697), (politiker,0.006715615901529757), (tysk,0.006647248632037367), (johan,0.006490297499117551)),31), (Array((talet,0.008667614219590534), (motor,0.0075079074864762485), (modellen,0.006178871682362424), (bilen,0.006155279388516955), (tillverkades,0.005976711316397548), (hästar,0.005824783570723379), (norge,0.005311435196759531), (började,0.005241977916787085)),32), (Array((kommun,0.11416819164339527), (socken,0.050324545368469545), (landskommun,0.016818206760481916), (distrikt,0.012752908710795822), (lappland,0.011537821982335002), (tidigare,0.010719803791367678), (sverige,0.009862282038445082), (kommunen,0.009030889894717564)),33), (Array((född,0.015562992356186932), (helsingfors,0.014361732260354492), (politiker,0.010996936575852071), (finland,0.008898364089439226), (finländsk,0.008510402443779293), (finlands,0.007835148639323315), (amerikansk,0.007822413125843019), (finska,0.0076912002585314545)),34), (Array((island,0.0490383779831419), (runt,0.0451737236998185), (meter,0.03860998978094461), (berg,0.03313729008501857), (havet,0.03243127474991183), (trakten,0.026676974831024446), (huvudstaden,0.020572557981229155), (toppen,0.018627408143358587)),35), (Array((översättning,0.021506094077323432), (boken,0.012642193617697675), (författare,0.009856114103220653), (roman,0.009640434831334053), (svenska,0.007416323493661734), (böcker,0.0070501796410253415), (romanen,0.00622283218509894), (bonnier,0.004774487412967152)),36), (Array((olympiska,0.049548964313542956), (sommarspelen,0.02850432577285843), (meter,0.028446571093525733), (född,0.025719194234325435), (guld,0.019018029273716347), (plats,0.01578734224357636), (silver,0.013586482942791687), (deltog,0.012738997120474884)),37), (Array((säsongen,0.01970360579518828), (spelade,0.01821370119779408), (spelare,0.017082344575046808), (klubben,0.013855092039910208), (matcher,0.013555870252636543), (vann,0.013466257190504594), (division,0.011341706636945798), (laget,0.010702386851864738)),38), (Array((svenska,0.01610236089988016), (stockholm,0.01295793482297076), (född,0.012289878967059891), (svensk,0.0111242820604197), (sverige,0.00889158626143369), (sveriges,0.007100839881414577), (carl,0.006380591536520988), (johan,0.005645343419116414)),39), (Array((naturreservat,0.04161863388347664), (området,0.030796836222440446), (kommun,0.02320318810150786), (reservatet,0.020750079375274618), (består,0.018812397851420987), (hektar,0.01751638950463774), (stort,0.015932194624414184), (europeiska,0.012446249856574871)),40), (Array((film,0.015499573959212252), (musik,0.013522719630244973), (piano,0.012503594394591106), (opera,0.012271030854027672), (operan,0.009113928693704955), (orkester,0.008779880090479569), (regi,0.008032186661676913), (amerikansk,0.007805528146684827)),41), (Array((song,0.025301447948363753), (eurovision,0.024621884181644828), (contest,0.022226305244453275), (låten,0.018682292843571407), (bidrag,0.015036155314197323), (finalen,0.012015630901632761), (tävlingen,0.01191022016715157), (plats,0.011343585207302885)),42), (Array((partiet,0.03031565777266273), (parti,0.018527306701743473), (valet,0.01756508780594258), (politiska,0.011598225830002314), (partier,0.010001306101007405), (mandat,0.007861442408790388), (partiets,0.0072026224398220665), (politiskt,0.0067771252166904375)),43), (Array((album,0.012682775680940702), (albumet,0.011384803837954507), (sång,0.010008416091887254), (bandet,0.00917243956825721), (gitarr,0.008900390645680751), (låten,0.008819728064668986), (släpptes,0.006818054645865437), (love,0.006681524653828919)),44), (Array((stockholm,0.021753727023331507), (malmö,0.01797498712789882), (svenska,0.01670602321448013), (målare,0.015076049695498916), (konst,0.014257718576193379), (talet,0.013309556021258382), (född,0.01312978131727807), (svensk,0.012425290530620539)),45), (Array((talet,0.005820074792683956), (senare,0.004538831945186889), (kriget,0.004314856555291248), (tyska,0.004237430464162381), (många,0.003916465948809745), (enligt,0.003654453717041427), (slaget,0.003451672247594681), (stora,0.0032575325582242503)),46), (Array((filmen,0.022218135305742865), (serien,0.013080494313426295), (film,0.011768390685574703), (spelet,0.009349719304298362), (avsnitt,0.008560623310233247), (bästa,0.008474277648346488), (filmer,0.005869637229573973), (handling,0.005341931887280892)),47), (Array((talet,0.014162207532755869), (språk,0.010620395856144315), (verk,0.0078990018349776), (senare,0.005615073988152831), (strauss,0.005051451406521095), (många,0.004455989101595585), (mest,0.004044387009852616), (grekiska,0.004009099489131701)),48), (Array((bibcode,0.017403057819584344), (solen,0.016075336931558652), (gånger,0.013183116624561394), (stars,0.011898752980852901), (stjärna,0.011388104451927622), (massa,0.01073784850568376), (arxiv,0.010460610396011778), (solens,0.009808156886054568)),49))

In [ ]:
// Transform data into the form (term, probability, topicId)
val termRDD = sc.parallelize(termArray)
val termRDD2 = termRDD.flatMap( (x: (Array[(String, Double)], Int)) => {
  val arrayOfTuple = x._1
  val topicId = x._2
  arrayOfTuple.map(el => (el._1, el._2, topicId))
})

  

>     termRDD: org.apache.spark.rdd.RDD[(Array[(String, Double)], Int)] = ParallelCollectionRDD[1881] at parallelize at command-4443336225072171:2
>     termRDD2: org.apache.spark.rdd.RDD[(String, Double, Int)] = MapPartitionsRDD[1882] at flatMap at command-4443336225072171:3

In [ ]:
// Create DF with proper column names
val termDF = termRDD2.toDF.withColumnRenamed("_1", "term").withColumnRenamed("_2", "probability").withColumnRenamed("_3", "topicId")

  

>     termDF: org.apache.spark.sql.DataFrame = [term: string, probability: double ... 1 more field]

In [ ]:
display(termDF)

  

[TABLE]

Truncated to 30 rows

In [ ]:
// Create JSON data
val rawJson = termDF.toJSON.collect().mkString(",\n")

  

>     rawJson: String =
>     {"term":"kommunen","probability":0.0965465174848558,"topicId":0},
>     {"term":"invånare","probability":0.09561944647066932,"topicId":0},
>     {"term":"kommun","probability":0.06123216019531808,"topicId":0},
>     {"term":"regionen","probability":0.06047426356346239,"topicId":0},
>     {"term":"departementet","probability":0.04133489674798034,"topicId":0},
>     {"term":"frankrike","probability":0.038999680148138156,"topicId":0},
>     {"term":"kantonen","probability":0.038645022485160795,"topicId":0},
>     {"term":"tillhör","probability":0.03709042755540236,"topicId":0},
>     {"term":"talet","probability":0.01860754472650629,"topicId":1},
>     {"term":"stockholm","probability":0.00800131987315067,"topicId":1},
>     {"term":"byggdes","probability":0.007185124511872375,"topicId":1},
>     {"term":"gamla","probability":0.006562831220260255,"topicId":1},
>     {"term":"stora","probability":0.006140398920615785,"topicId":1},
>     {"term":"idag","probability":0.005472783411308425,"topicId":1},
>     {"term":"stockholms","probability":0.005347092400479047,"topicId":1},
>     {"term":"tidigare","probability":0.004738960299641082,"topicId":1},
>     {"term":"chef","probability":0.012891731862399829,"topicId":2},
>     {"term":"svenska","probability":0.012216420059244167,"topicId":2},
>     {"term":"fartyget","probability":0.008799092539391087,"topicId":2},
>     {"term":"regemente","probability":0.008170528757595195,"topicId":2},
>     {"term":"juli","probability":0.007589154082154804,"topicId":2},
>     {"term":"juni","probability":0.00753057351996227,"topicId":2},
>     {"term":"fartyg","probability":0.007501151936739348,"topicId":2},
>     {"term":"militär","probability":0.007441712973556016,"topicId":2},
>     {"term":"staden","probability":0.03550112124416654,"topicId":3},
>     {"term":"kilometer","probability":0.024239377096572767,"topicId":3},
>     {"term":"omkring","probability":0.017880722746326707,"topicId":3},
>     {"term":"finland","probability":0.017534872470232804,"topicId":3},
>     {"term":"invånare","probability":0.01572555256779709,"topicId":3},
>     {"term":"stad","probability":0.014243881447019926,"topicId":3},
>     {"term":"största","probability":0.013291812684672874,"topicId":3},
>     {"term":"norr","probability":0.013255209377268314,"topicId":3},
>     {"term":"lista","probability":0.049154032038907364,"topicId":4},
>     {"term":"klass","probability":0.025424694972593722,"topicId":4},
>     {"term":"omgången","probability":0.021006352382631138,"topicId":4},
>     {"term":"wales","probability":0.015146400328457767,"topicId":4},
>     {"term":"bert","probability":0.014384523267127552,"topicId":4},
>     {"term":"sjösatt","probability":0.01237820887773258,"topicId":4},
>     {"term":"sektion","probability":0.010986509101209276,"topicId":4},
>     {"term":"australien","probability":0.010348525497739654,"topicId":4},
>     {"term":"länsväg","probability":0.024609442134056914,"topicId":5},
>     {"term":"sjön","probability":0.02313788863645436,"topicId":5},
>     {"term":"meter","probability":0.023099400549010995,"topicId":5},
>     {"term":"havet","probability":0.016486417015966634,"topicId":5},
>     {"term":"santa","probability":0.01631846978655288,"topicId":5},
>     {"term":"maria","probability":0.014208691182654495,"topicId":5},
>     {"term":"kvadratkilometer","probability":0.012902237189832062,"topicId":5},
>     {"term":"vägen","probability":0.01089177248153098,"topicId":5},
>     {"term":"birds","probability":0.025131668033955827,"topicId":6},
>     {"term":"clements","probability":0.01897996595347449,"topicId":6},
>     {"term":"world","probability":0.01864839205373941,"topicId":6},
>     {"term":"läst","probability":0.01754493324942852,"topicId":6},
>     {"term":"förekommer","probability":0.015642901268097295,"topicId":6},
>     {"term":"arten","probability":0.014806106220374982,"topicId":6},
>     {"term":"familjen","probability":0.013746517306962325,"topicId":6},
>     {"term":"namn","probability":0.012193036368057153,"topicId":6},
>     {"term":"socken","probability":0.02200110661389711,"topicId":7},
>     {"term":"namnet","probability":0.016863821067931672,"topicId":7},
>     {"term":"socknen","probability":0.011169441915583982,"topicId":7},
>     {"term":"härad","probability":0.009575257990621604,"topicId":7},
>     {"term":"gift","probability":0.007041018691775915,"topicId":7},
>     {"term":"ätten","probability":0.006957959901751748,"topicId":7},
>     {"term":"samma","probability":0.006567438459526348,"topicId":7},
>     {"term":"församling","probability":0.006206311552289226,"topicId":7},
>     {"term":"from","probability":0.09573192102472637,"topicId":8},
>     {"term":"shift","probability":0.07721821951064128,"topicId":8},
>     {"term":"value","probability":0.05140930922690405,"topicId":8},
>     {"term":"linegrey","probability":0.038236453151551754,"topicId":8},
>     {"term":"cobar","probability":0.037725195145277224,"topicId":8},
>     {"term":"width","probability":0.027601289731431947,"topicId":8},
>     {"term":"gray","probability":0.026950628685089566,"topicId":8},
>     {"term":"plotdata","probability":0.02535800730760235,"topicId":8},
>     {"term":"stockholm","probability":0.040800430698927574,"topicId":9},
>     {"term":"tyskland","probability":0.0306444550609787,"topicId":9},
>     {"term":"tillsammans","probability":0.02193657394829871,"topicId":9},
>     {"term":"förbundslandet","probability":0.019628310699189612,"topicId":9},
>     {"term":"kommun","probability":0.018135004797691912,"topicId":9},
>     {"term":"ingår","probability":0.015575322731524751,"topicId":9},
>     {"term":"landkreis","probability":0.014987081087118154,"topicId":9},
>     {"term":"kommunen","probability":0.01387372200507514,"topicId":9},
>     {"term":"county","probability":0.059935597281339396,"topicId":10},
>     {"term":"läst","probability":0.025518279914354958,"topicId":10},
>     {"term":"invånare","probability":0.021135159839241465,"topicId":10},
>     {"term":"census","probability":0.01909964708101696,"topicId":10},
>     {"term":"delstaten","probability":0.015705965232071534,"topicId":10},
>     {"term":"united","probability":0.010849948320306425,"topicId":10},
>     {"term":"states","probability":0.009618564510993638,"topicId":10},
>     {"term":"enligt","probability":0.009077797952871827,"topicId":10},
>     {"term":"kina","probability":0.02947986672544381,"topicId":11},
>     {"term":"grand","probability":0.01634731925291766,"topicId":11},
>     {"term":"prix","probability":0.016230381323561027,"topicId":11},
>     {"term":"vann","probability":0.012766683820251106,"topicId":11},
>     {"term":"kinesiska","probability":0.011650847462746833,"topicId":11},
>     {"term":"säsongen","probability":0.011556626328903735,"topicId":11},
>     {"term":"lopp","probability":0.01020456193177741,"topicId":11},
>     {"term":"loppet","probability":0.009873469854259966,"topicId":11},
>     {"term":"programmet","probability":0.019344695557099535,"topicId":12},
>     {"term":"program","probability":0.012893474006120904,"topicId":12},
>     {"term":"radio","probability":0.011736207069720414,"topicId":12},
>     {"term":"programledare","probability":0.01063631773660054,"topicId":12},
>     {"term":"sveriges","probability":0.010443829299695546,"topicId":12},
>     {"term":"grupp","probability":0.0101881977731672,"topicId":12},
>     {"term":"sändes","probability":0.009557330691273938,"topicId":12},
>     {"term":"poäng","probability":0.008217466852727128,"topicId":12},
>     {"term":"svensk","probability":0.020289726693646915,"topicId":13},
>     {"term":"umeå","probability":0.016791173665524066,"topicId":13},
>     {"term":"född","probability":0.016079171985534508,"topicId":13},
>     {"term":"ledamot","probability":0.01499395294114528,"topicId":13},
>     {"term":"ordförande","probability":0.013972428567201652,"topicId":13},
>     {"term":"andersson","probability":0.012374270949374168,"topicId":13},
>     {"term":"läns","probability":0.011168740554371137,"topicId":13},
>     {"term":"riksdagen","probability":0.011135322353134584,"topicId":13},
>     {"term":"arten","probability":0.010801612339730723,"topicId":14},
>     {"term":"förekommer","probability":0.009156920691378509,"topicId":14},
>     {"term":"arter","probability":0.007179320551697595,"topicId":14},
>     {"term":"mycket","probability":0.006753345018760251,"topicId":14},
>     {"term":"familjen","probability":0.006448937460397879,"topicId":14},
>     {"term":"olika","probability":0.006354325581171363,"topicId":14},
>     {"term":"ofta","probability":0.006354067492091369,"topicId":14},
>     {"term":"släktet","probability":0.005676295408387824,"topicId":14},
>     {"term":"robin","probability":0.015715771663330817,"topicId":15},
>     {"term":"bell","probability":0.010712777813893593,"topicId":15},
>     {"term":"grey","probability":0.009546931920178072,"topicId":15},
>     {"term":"from","probability":0.009522518371431704,"topicId":15},
>     {"term":"castle","probability":0.008927422336395518,"topicId":15},
>     {"term":"hood","probability":0.008624625729438713,"topicId":15},
>     {"term":"invånare","probability":0.007992126063379968,"topicId":15},
>     {"term":"arthur","probability":0.006529834990708393,"topicId":15},
>     {"term":"professor","probability":0.02698330617838032,"topicId":16},
>     {"term":"universitet","probability":0.023503577242053826,"topicId":16},
>     {"term":"född","probability":0.018666434854939382,"topicId":16},
>     {"term":"berlin","probability":0.012740783445882326,"topicId":16},
>     {"term":"tysk","probability":0.010704005493014743,"topicId":16},
>     {"term":"universitetet","probability":0.008452521480672512,"topicId":16},
>     {"term":"forskning","probability":0.006296868608240401,"topicId":16},
>     {"term":"tekniska","probability":0.005985625523382596,"topicId":16},
>     {"term":"årets","probability":0.04218765174213785,"topicId":17},
>     {"term":"mount","probability":0.03227499394792523,"topicId":17},
>     {"term":"flygplats","probability":0.022093802114524035,"topicId":17},
>     {"term":"airport","probability":0.015475453980455287,"topicId":17},
>     {"term":"antarktis","probability":0.014964435315255199,"topicId":17},
>     {"term":"flygplatsen","probability":0.011712488574152705,"topicId":17},
>     {"term":"berget","probability":0.01093678433906292,"topicId":17},
>     {"term":"chile","probability":0.010620867322467302,"topicId":17},
>     {"term":"rubriken","probability":0.025822818151141222,"topicId":18},
>     {"term":"svenska","probability":0.01673994393550959,"topicId":18},
>     {"term":"psalmer","probability":0.0162221055535359,"topicId":18},
>     {"term":"lista","probability":0.016147667279789293,"topicId":18},
>     {"term":"sånger","probability":0.014719242344302535,"topicId":18},
>     {"term":"valkrets","probability":0.013540164643026688,"topicId":18},
>     {"term":"jesus","probability":0.012889860747312692,"topicId":18},
>     {"term":"publicerad","probability":0.011859313412890347,"topicId":18},
>     {"term":"sverige","probability":0.016659851631007156,"topicId":19},
>     {"term":"grupp","probability":0.014638517969975177,"topicId":19},
>     {"term":"damer","probability":0.012174676630012262,"topicId":19},
>     {"term":"herrar","probability":0.011931826860210994,"topicId":19},
>     {"term":"open","probability":0.008614177921732406,"topicId":19},
>     {"term":"svenska","probability":0.008571960123656082,"topicId":19},
>     {"term":"vinner","probability":0.008218750193539872,"topicId":19},
>     {"term":"spelades","probability":0.007263793397792697,"topicId":19},
>     {"term":"jorden","probability":0.009623406330442461,"topicId":20},
>     {"term":"olika","probability":0.006201796360980689,"topicId":20},
>     {"term":"mycket","probability":0.005204262397916099,"topicId":20},
>     {"term":"exempel","probability":0.005101125247310075,"topicId":20},
>     {"term":"månen","probability":0.004691523042876648,"topicId":20},
>     {"term":"runt","probability":0.004230887210594873,"topicId":20},
>     {"term":"jordens","probability":0.004168868164663573,"topicId":20},
>     {"term":"planet","probability":0.004115098940215664,"topicId":20},
>     {"term":"kyrka","probability":0.04525180789717588,"topicId":21},
>     {"term":"församling","probability":0.044049400894926156,"topicId":21},
>     {"term":"kyrkan","probability":0.03626793680282758,"topicId":21},
>     {"term":"församlingen","probability":0.01679474869841332,"topicId":21},
>     {"term":"talet","probability":0.015810735006045285,"topicId":21},
>     {"term":"stift","probability":0.014316742158067308,"topicId":21},
>     {"term":"kyrkans","probability":0.008344858534169153,"topicId":21},
>     {"term":"orgel","probability":0.007884142003735476,"topicId":21},
>     {"term":"upptäcktes","probability":0.0319889335984476,"topicId":22},
>     {"term":"asteroid","probability":0.028638790480427122,"topicId":22},
>     {"term":"asteroiden","probability":0.026047925982480766,"topicId":22},
>     {"term":"astronomen","probability":0.025022878764599684,"topicId":22},
>     {"term":"diameter","probability":0.02455547250826605,"topicId":22},
>     {"term":"ungefär","probability":0.023942524658324686,"topicId":22},
>     {"term":"huvudbältet","probability":0.02388196435303055,"topicId":22},
>     {"term":"kilometer","probability":0.023865003269207676,"topicId":22},
>     {"term":"file","probability":0.11527587937013585,"topicId":23},
>     {"term":"image","probability":0.03426338585615468,"topicId":23},
>     {"term":"landskommun","probability":0.027940989066922915,"topicId":23},
>     {"term":"jesus","probability":0.02586919513913989,"topicId":23},
>     {"term":"landsfiskalsdistrikt","probability":0.019640603495146892,"topicId":23},
>     {"term":"street","probability":0.01340066668262875,"topicId":23},
>     {"term":"church","probability":0.012266502334040353,"topicId":23},
>     {"term":"line","probability":0.011400107405277141,"topicId":23},
>     {"term":"paris","probability":0.030487380264716278,"topicId":24},
>     {"term":"född","probability":0.02295707332488064,"topicId":24},
>     {"term":"franska","probability":0.02185943462445832,"topicId":24},
>     {"term":"fransk","probability":0.02158006308424395,"topicId":24},
>     {"term":"frankrike","probability":0.017951182756044196,"topicId":24},
>     {"term":"jean","probability":0.01381046893853163,"topicId":24},
>     {"term":"japansk","probability":0.012060332643759336,"topicId":24},
>     {"term":"fran","probability":0.009283336277914061,"topicId":24},
>     {"term":"företaget","probability":0.014224688659059093,"topicId":25},
>     {"term":"företag","probability":0.010376578312465355,"topicId":25},
>     {"term":"university","probability":0.009450453832898244,"topicId":25},
>     {"term":"grundades","probability":0.007866369511682956,"topicId":25},
>     {"term":"miljoner","probability":0.0070289603462744266,"topicId":25},
>     {"term":"international","probability":0.006909192142816018,"topicId":25},
>     {"term":"bolaget","probability":0.005681334415927907,"topicId":25},
>     {"term":"största","probability":0.004972023971847601,"topicId":25},
>     {"term":"olika","probability":0.013140118570301491,"topicId":26},
>     {"term":"exempel","probability":0.012611570573629211,"topicId":26},
>     {"term":"används","probability":0.010290002183836725,"topicId":26},
>     {"term":"ofta","probability":0.008671598559755129,"topicId":26},
>     {"term":"många","probability":0.006473174221460534,"topicId":26},
>     {"term":"vissa","probability":0.0054923767122421665,"topicId":26},
>     {"term":"kallas","probability":0.005095853907079055,"topicId":26},
>     {"term":"mycket","probability":0.0046124218623229615,"topicId":26},
>     {"term":"född","probability":0.018983731833461078,"topicId":27},
>     {"term":"amerikansk","probability":0.01360170932960869,"topicId":27},
>     {"term":"john","probability":0.01278403121933378,"topicId":27},
>     {"term":"william","probability":0.008430126128419025,"topicId":27},
>     {"term":"york","probability":0.00817341548678432,"topicId":27},
>     {"term":"london","probability":0.007575625929221351,"topicId":27},
>     {"term":"university","probability":0.006584504383062662,"topicId":27},
>     {"term":"james","probability":0.006500762878735947,"topicId":27},
>     {"term":"fängelse","probability":0.011129385435448641,"topicId":28},
>     {"term":"mytologi","probability":0.010095432085198731,"topicId":28},
>     {"term":"brott","probability":0.009595460498682977,"topicId":28},
>     {"term":"dömdes","probability":0.009056951340934477,"topicId":28},
>     {"term":"harry","probability":0.00900833191441214,"topicId":28},
>     {"term":"enligt","probability":0.007665375174231455,"topicId":28},
>     {"term":"bergman","probability":0.007126281020184516,"topicId":28},
>     {"term":"polisen","probability":0.0065320647933313205,"topicId":28},
>     {"term":"uppsala","probability":0.03225486403415035,"topicId":29},
>     {"term":"född","probability":0.0262872943647285,"topicId":29},
>     {"term":"svensk","probability":0.021836441055240335,"topicId":29},
>     {"term":"ledamot","probability":0.01540176402235281,"topicId":29},
>     {"term":"filosofie","probability":0.013025311479277617,"topicId":29},
>     {"term":"stockholm","probability":0.012171814994441733,"topicId":29},
>     {"term":"kungliga","probability":0.010439175079255476,"topicId":29},
>     {"term":"universitet","probability":0.007551180380037851,"topicId":29},
>     {"term":"ford","probability":0.03904003795793799,"topicId":30},
>     {"term":"månaden","probability":0.028399486781946183,"topicId":30},
>     {"term":"trakten","probability":0.027765453259295036,"topicId":30},
>     {"term":"varmaste","probability":0.017873203876102778,"topicId":30},
>     {"term":"kallaste","probability":0.017712122104733583,"topicId":30},
>     {"term":"medeltemperaturen","probability":0.017593788827147818,"topicId":30},
>     {"term":"årsmedeltemperaturen","probability":0.016728764710215706,"topicId":30},
>     {"term":"nederbörd","probability":0.01643507684382941,"topicId":30},
>     {"term":"svensk","probability":0.029891732209654676,"topicId":31},
>     {"term":"född","probability":0.02919016668133009,"topicId":31},
>     {"term":"skådespelare","probability":0.017438265220485975,"topicId":31},
>     {"term":"amerikansk","probability":0.011715835460063146,"topicId":31},
>     {"term":"författare","probability":0.008035463938643697,"topicId":31},
>     {"term":"politiker","probability":0.006715615901529757,"topicId":31},
>     {"term":"tysk","probability":0.006647248632037367,"topicId":31},
>     {"term":"johan","probability":0.006490297499117551,"topicId":31},
>     {"term":"talet","probability":0.008667614219590534,"topicId":32},
>     {"term":"motor","probability":0.0075079074864762485,"topicId":32},
>     {"term":"modellen","probability":0.006178871682362424,"topicId":32},
>     {"term":"bilen","probability":0.006155279388516955,"topicId":32},
>     {"term":"tillverkades","probability":0.005976711316397548,"topicId":32},
>     {"term":"hästar","probability":0.005824783570723379,"topicId":32},
>     {"term":"norge","probability":0.005311435196759531,"topicId":32},
>     {"term":"började","probability":0.005241977916787085,"topicId":32},
>     {"term":"kommun","probability":0.11416819164339527,"topicId":33},
>     {"term":"socken","probability":0.050324545368469545,"topicId":33},
>     {"term":"landskommun","probability":0.016818206760481916,"topicId":33},
>     {"term":"distrikt","probability":0.012752908710795822,"topicId":33},
>     {"term":"lappland","probability":0.011537821982335002,"topicId":33},
>     {"term":"tidigare","probability":0.010719803791367678,"topicId":33},
>     {"term":"sverige","probability":0.009862282038445082,"topicId":33},
>     {"term":"kommunen","probability":0.009030889894717564,"topicId":33},
>     {"term":"född","probability":0.015562992356186932,"topicId":34},
>     {"term":"helsingfors","probability":0.014361732260354492,"topicId":34},
>     {"term":"politiker","probability":0.010996936575852071,"topicId":34},
>     {"term":"finland","probability":0.008898364089439226,"topicId":34},
>     {"term":"finländsk","probability":0.008510402443779293,"topicId":34},
>     {"term":"finlands","probability":0.007835148639323315,"topicId":34},
>     {"term":"amerikansk","probability":0.007822413125843019,"topicId":34},
>     {"term":"finska","probability":0.0076912002585314545,"topicId":34},
>     {"term":"island","probability":0.0490383779831419,"topicId":35},
>     {"term":"runt","probability":0.0451737236998185,"topicId":35},
>     {"term":"meter","probability":0.03860998978094461,"topicId":35},
>     {"term":"berg","probability":0.03313729008501857,"topicId":35},
>     {"term":"havet","probability":0.03243127474991183,"topicId":35},
>     {"term":"trakten","probability":0.026676974831024446,"topicId":35},
>     {"term":"huvudstaden","probability":0.020572557981229155,"topicId":35},
>     {"term":"toppen","probability":0.018627408143358587,"topicId":35},
>     {"term":"översättning","probability":0.021506094077323432,"topicId":36},
>     {"term":"boken","probability":0.012642193617697675,"topicId":36},
>     {"term":"författare","probability":0.009856114103220653,"topicId":36},
>     {"term":"roman","probability":0.009640434831334053,"topicId":36},
>     {"term":"svenska","probability":0.007416323493661734,"topicId":36},
>     {"term":"böcker","probability":0.0070501796410253415,"topicId":36},
>     {"term":"romanen","probability":0.00622283218509894,"topicId":36},
>     {"term":"bonnier","probability":0.004774487412967152,"topicId":36},
>     {"term":"olympiska","probability":0.049548964313542956,"topicId":37},
>     {"term":"sommarspelen","probability":0.02850432577285843,"topicId":37},
>     {"term":"meter","probability":0.028446571093525733,"topicId":37},
>     {"term":"född","probability":0.025719194234325435,"topicId":37},
>     {"term":"guld","probability":0.019018029273716347,"topicId":37},
>     {"term":"plats","probability":0.01578734224357636,"topicId":37},
>     {"term":"silver","probability":0.013586482942791687,"topicId":37},
>     {"term":"deltog","probability":0.012738997120474884,"topicId":37},
>     {"term":"säsongen","probability":0.01970360579518828,"topicId":38},
>     {"term":"spelade","probability":0.01821370119779408,"topicId":38},
>     {"term":"spelare","probability":0.017082344575046808,"topicId":38},
>     {"term":"klubben","probability":0.013855092039910208,"topicId":38},
>     {"term":"matcher","probability":0.013555870252636543,"topicId":38},
>     {"term":"vann","probability":0.013466257190504594,"topicId":38},
>     {"term":"division","probability":0.011341706636945798,"topicId":38},
>     {"term":"laget","probability":0.010702386851864738,"topicId":38},
>     {"term":"svenska","probability":0.01610236089988016,"topicId":39},
>     {"term":"stockholm","probability":0.01295793482297076,"topicId":39},
>     {"term":"född","probability":0.012289878967059891,"topicId":39},
>     {"term":"svensk","probability":0.0111242820604197,"topicId":39},
>     {"term":"sverige","probability":0.00889158626143369,"topicId":39},
>     {"term":"sveriges","probability":0.007100839881414577,"topicId":39},
>     {"term":"carl","probability":0.006380591536520988,"topicId":39},
>     {"term":"johan","probability":0.005645343419116414,"topicId":39},
>     {"term":"naturreservat","probability":0.04161863388347664,"topicId":40},
>     {"term":"området","probability":0.030796836222440446,"topicId":40},
>     {"term":"kommun","probability":0.02320318810150786,"topicId":40},
>     {"term":"reservatet","probability":0.020750079375274618,"topicId":40},
>     {"term":"består","probability":0.018812397851420987,"topicId":40},
>     {"term":"hektar","probability":0.01751638950463774,"topicId":40},
>     {"term":"stort","probability":0.015932194624414184,"topicId":40},
>     {"term":"europeiska","probability":0.012446249856574871,"topicId":40},
>     {"term":"film","probability":0.015499573959212252,"topicId":41},
>     {"term":"musik","probability":0.013522719630244973,"topicId":41},
>     {"term":"piano","probability":0.012503594394591106,"topicId":41},
>     {"term":"opera","probability":0.012271030854027672,"topicId":41},
>     {"term":"operan","probability":0.009113928693704955,"topicId":41},
>     {"term":"orkester","probability":0.008779880090479569,"topicId":41},
>     {"term":"regi","probability":0.008032186661676913,"topicId":41},
>     {"term":"amerikansk","probability":0.007805528146684827,"topicId":41},
>     {"term":"song","probability":0.025301447948363753,"topicId":42},
>     {"term":"eurovision","probability":0.024621884181644828,"topicId":42},
>     {"term":"contest","probability":0.022226305244453275,"topicId":42},
>     {"term":"låten","probability":0.018682292843571407,"topicId":42},
>     {"term":"bidrag","probability":0.015036155314197323,"topicId":42},
>     {"term":"finalen","probability":0.012015630901632761,"topicId":42},
>     {"term":"tävlingen","probability":0.01191022016715157,"topicId":42},
>     {"term":"plats","probability":0.011343585207302885,"topicId":42},
>     {"term":"partiet","probability":0.03031565777266273,"topicId":43},
>     {"term":"parti","probability":0.018527306701743473,"topicId":43},
>     {"term":"valet","probability":0.01756508780594258,"topicId":43},
>     {"term":"politiska","probability":0.011598225830002314,"topicId":43},
>     {"term":"partier","probability":0.010001306101007405,"topicId":43},
>     {"term":"mandat","probability":0.007861442408790388,"topicId":43},
>     {"term":"partiets","probability":0.0072026224398220665,"topicId":43},
>     {"term":"politiskt","probability":0.0067771252166904375,"topicId":43},
>     {"term":"album","probability":0.012682775680940702,"topicId":44},
>     {"term":"albumet","probability":0.011384803837954507,"topicId":44},
>     {"term":"sång","probability":0.010008416091887254,"topicId":44},
>     {"term":"bandet","probability":0.00917243956825721,"topicId":44},
>     {"term":"gitarr","probability":0.008900390645680751,"topicId":44},
>     {"term":"låten","probability":0.008819728064668986,"topicId":44},
>     {"term":"släpptes","probability":0.006818054645865437,"topicId":44},
>     {"term":"love","probability":0.006681524653828919,"topicId":44},
>     {"term":"stockholm","probability":0.021753727023331507,"topicId":45},
>     {"term":"malmö","probability":0.01797498712789882,"topicId":45},
>     {"term":"svenska","probability":0.01670602321448013,"topicId":45},
>     {"term":"målare","probability":0.015076049695498916,"topicId":45},
>     {"term":"konst","probability":0.014257718576193379,"topicId":45},
>     {"term":"talet","probability":0.013309556021258382,"topicId":45},
>     {"term":"född","probability":0.01312978131727807,"topicId":45},
>     {"term":"svensk","probability":0.012425290530620539,"topicId":45},
>     {"term":"talet","probability":0.005820074792683956,"topicId":46},
>     {"term":"senare","probability":0.004538831945186889,"topicId":46},
>     {"term":"kriget","probability":0.004314856555291248,"topicId":46},
>     {"term":"tyska","probability":0.004237430464162381,"topicId":46},
>     {"term":"många","probability":0.003916465948809745,"topicId":46},
>     {"term":"enligt","probability":0.003654453717041427,"topicId":46},
>     {"term":"slaget","probability":0.003451672247594681,"topicId":46},
>     {"term":"stora","probability":0.0032575325582242503,"topicId":46},
>     {"term":"filmen","probability":0.022218135305742865,"topicId":47},
>     {"term":"serien","probability":0.013080494313426295,"topicId":47},
>     {"term":"film","probability":0.011768390685574703,"topicId":47},
>     {"term":"spelet","probability":0.009349719304298362,"topicId":47},
>     {"term":"avsnitt","probability":0.008560623310233247,"topicId":47},
>     {"term":"bästa","probability":0.008474277648346488,"topicId":47},
>     {"term":"filmer","probability":0.005869637229573973,"topicId":47},
>     {"term":"handling","probability":0.005341931887280892,"topicId":47},
>     {"term":"talet","probability":0.014162207532755869,"topicId":48},
>     {"term":"språk","probability":0.010620395856144315,"topicId":48},
>     {"term":"verk","probability":0.0078990018349776,"topicId":48},
>     {"term":"senare","probability":0.005615073988152831,"topicId":48},
>     {"term":"strauss","probability":0.005051451406521095,"topicId":48},
>     {"term":"många","probability":0.004455989101595585,"topicId":48},
>     {"term":"mest","probability":0.004044387009852616,"topicId":48},
>     {"term":"grekiska","probability":0.004009099489131701,"topicId":48},
>     {"term":"bibcode","probability":0.017403057819584344,"topicId":49},
>     {"term":"solen","probability":0.016075336931558652,"topicId":49},
>     {"term":"gånger","probability":0.013183116624561394,"topicId":49},
>     {"term":"stars","probability":0.011898752980852901,"topicId":49},
>     {"term":"stjärna","probability":0.011388104451927622,"topicId":49},
>     {"term":"massa","probability":0.01073784850568376,"topicId":49},
>     {"term":"arxiv","probability":0.010460610396011778,"topicId":49},
>     {"term":"solens","probability":0.009808156886054568,"topicId":49}

  

Plot visualization

In [ ]:
displayHTML(s"""
<!DOCTYPE html>
<meta charset="utf-8">
<style>

circle {
  fill: rgb(31, 119, 180);
  fill-opacity: 0.5;
  stroke: rgb(31, 119, 180);
  stroke-width: 1px;
}

.leaf circle {
  fill: #ff7f0e;
  fill-opacity: 1;
}

text {
  font: 14px sans-serif;
}

</style>
<body>
<script src="https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min.js"></script>
<script>

var json = {
 "name": "data",
 "children": [
  {
     "name": "topics",
     "children": [
      ${rawJson}
     ]
    }
   ]
};

var r = 1000,
    format = d3.format(",d"),
    fill = d3.scale.category20c();

var bubble = d3.layout.pack()
    .sort(null)
    .size([r, r])
    .padding(1.5);

var vis = d3.select("body").append("svg")
    .attr("width", r)
    .attr("height", r)
    .attr("class", "bubble");

  
var node = vis.selectAll("g.node")
    .data(bubble.nodes(classes(json))
    .filter(function(d) { return !d.children; }))
    .enter().append("g")
    .attr("class", "node")
    .attr("transform", function(d) { return "translate(" + d.x + "," + d.y + ")"; })
    color = d3.scale.category20();
  
  node.append("title")
      .text(function(d) { return d.className + ": " + format(d.value); });

  node.append("circle")
      .attr("r", function(d) { return d.r; })
      .style("fill", function(d) {return color(d.topicName);});

var text = node.append("text")
    .attr("text-anchor", "middle")
    .attr("dy", ".3em")
    .text(function(d) { return d.className.substring(0, d.r / 3)});
  
  text.append("tspan")
      .attr("dy", "1.2em")
      .attr("x", 0)
      .text(function(d) {return Math.ceil(d.value * 10000) /10000; });

// Returns a flattened hierarchy containing all leaf nodes under the root.
function classes(root) {
  var classes = [];

  function recurse(term, node) {
    if (node.children) node.children.forEach(function(child) { recurse(node.term, child); });
    else classes.push({topicName: node.topicId, className: node.term, value: node.probability});
  }

  recurse(null, root);
  return {children: classes};
}
</script>
""")

  

  

Finding Articles
----------------

If we are reading an article, we can use our LDA model to find other
articles on the same topic.

When we created our model earlier, we did it as an `LDA`, but we need to
be more specific, so we reload it as a `LocalLDAModel`.

In [ ]:
// Load the model
import org.apache.spark.mllib.clustering.LocalLDAModel
val model = LocalLDAModel.load(sc, lda_file)

  

>     import org.apache.spark.mllib.clustering.LocalLDAModel
>     model: org.apache.spark.mllib.clustering.LocalLDAModel = org.apache.spark.mllib.clustering.LocalLDAModel@6067f4d5

In [ ]:
// Load the countvector
// val lda_countVector_mllib_loaded = sc.textFile("lda_countvector_sv_20201208") // This doesn't work, need to figure out how to parse the loaded rdd
// print(lda_countVector_mllib_loaded)

  

  

We sample some articles.

In [ ]:
val numsample = 20
val sample = lda_countVector_mllib.takeSample(false, numsample)

  

>     numsample: Int = 20
>     sample: Array[(Long, org.apache.spark.mllib.linalg.Vector)] = Array((2779114,(10000,[0,8,34,37,57,83,85,111,117,127,163,211,215,230,238,241,333,419,889,931,938,951,970,1033,1073,1143,1623,1689,1709,1981,2020,2082,2130,2608,2718,3102,3172,3590,4103,4151,4492,4574,4612,4736,4876,4881,5008,5031,6656,7222,7225,7453,8484,8637,8933,9340,9712,9912],[1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])), (226060,(10000,[7,13,21,22,28,30,40,55,61,67,72,84,100,109,133,140,144,205,231,233,313,315,342,461,525,583,585,593,621,696,740,774,777,1156,1284,1346,1745,2128,2162,2235,2355,2572,2579,2675,2683,2692,2734,2862,2910,3976,4059,4138,4149,4278,4654,5819,6139,6561,6694,6872,8226,9230],[4.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])), (1613204,(10000,[2,7,10,13,27,43,48,54,56,72,81,101,104,122,131,134,164,166,169,200,212,214,291,318,346,349,477,486,499,508,515,522,530,541,543,544,546,554,555,556,557,558,561,563,564,571,600,2448],[1.0,2.0,7.0,2.0,6.0,1.0,1.0,1.0,4.0,1.0,1.0,3.0,3.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])), (4970370,(10000,[3,19,20,39,64,73,105,132,194,219,242,261,272,300,326,371,393,438,500,559,568,579,611,615,694,704,749,752,792,797,800,801,805,814,832,845,846,915,1022,1075,1131,1229,1309,1333,1424,1994,2103,2115,2119,2142,2261,2323,2372,2401,2690,2881,2898,2906,3012,3821,4076,4143,4666,5598,6187,6233,6254,6541,6900,7163],[1.0,2.0,2.0,1.0,1.0,1.0,3.0,1.0,3.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])), (700470,(10000,[5,12,24,26,47,75,100,106,113,120,125,126,156,241,255,273,320,335,354,582,598,874,1243,1343,1431,1689,1882,1965,1987,1994,2030,2033,2054,2071,2116,2327,2345,2403,2625,2717,2732,3044,3201,3310,3405,3694,3889,4259,4287,4335,4738,4781,4961,5069,5319,5394,5524,5876,6771,7737,8854,9372],[1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])), (636341,(10000,[23,32,62,123,142,277,301,328,383,413,444,446,454,538,665,997,1107,1430,1494,1568,1603,1742,2056,2187,2338,2549,2731,3242,3460,3476,3803,3849,4320,4640,4899,5518,6115,6130,6227,6534,7323,7408,8461,9064,9194,9362,9964],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])), (4987605,(10000,[123,589,753,930,1892,2980],[1.0,2.0,1.0,1.0,1.0,1.0])), (766457,(10000,[32,55,160,321,419,420,491,531,562,768,1013,1166,1222,1599,1643,1826,2017,2464,3628,3793,7461],[1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,3.0,7.0,1.0,1.0,1.0,2.0,7.0,1.0])), (4965484,(10000,[5,28,174,187,233,278,286,431,442,476,894,936,1388,1997,2031,2354,2495,3013,3076,4498,5108,7037,9062],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0])), (82392,(10000,[1,5,8,9,11,15,17,37,43,45,50,53,58,59,60,68,80,82,90,95,106,107,114,115,141,143,145,149,150,152,158,171,179,182,192,196,197,218,239,241,250,273,278,281,283,284,306,320,335,370,377,387,512,519,520,536,587,606,607,679,693,706,721,732,739,744,750,783,798,803,806,901,920,928,982,983,1008,1141,1178,1296,1399,1459,1511,1538,1542,1571,1598,1611,1636,1723,1750,1756,1760,1812,1881,1930,1950,1978,2022,2081,2161,2212,2235,2240,2321,2354,2557,2581,2603,2652,2664,2752,2754,2894,2984,3095,3108,3160,3222,3239,3410,3560,3581,3657,3700,3774,3822,3832,3922,3938,3962,4128,4184,4294,4304,4654,4680,4862,4950,4974,5255,5289,5369,5427,5461,5466,5591,5689,5813,5833,5921,6089,6114,6247,6316,6606,6617,6682,6740,7256,7310,7376,7746,8579,9141,9276,9365,9397,9536,9584,9918],[13.0,1.0,3.0,1.0,2.0,1.0,1.0,7.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,3.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,7.0,2.0,1.0,1.0,2.0,3.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,4.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,3.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0])), (7982574,(10000,[0,14,19,28,32,34,62,88,170,211,261,282,358,411,422,548,599,1014,1143,1300,1482,1541,1670,1813,1922,1934,2056,2260,2266,2848,2853,3869,4140,4266,4468,4489,4877,4899,5663,6158,6452,7076],[1.0,5.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,6.0,3.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,4.0,1.0,3.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])), (7973374,(10000,[8,14,16,18,22,31,32,44,53,60,66,70,80,107,154,176,189,192,193,196,214,231,247,248,260,265,331,336,355,429,430,503,505,517,533,562,579,612,699,718,767,769,773,780,811,819,835,842,895,906,927,928,936,964,1074,1149,1180,1197,1221,1223,1315,1318,1336,1389,1393,1429,1493,1561,1601,1796,1821,2017,2065,2076,2087,2167,2193,2239,2314,2349,2354,2415,2418,2423,2450,2488,2635,2676,2680,2681,2738,2761,2813,2838,2889,2931,2993,3062,3067,3085,3088,3091,3168,3263,3279,3286,3294,3660,3673,3757,3773,3872,4045,4070,4206,4529,4613,4631,4672,4859,5073,5103,5117,5287,5384,5385,5557,5825,5842,5967,5974,5996,6108,6263,6309,6703,6746,6836,6887,6904,6955,7016,7064,7089,7162,7238,7328,7985,8119,8120,8128,8214,8301,8323,8396,8596,8610,8751,9214,9339],[1.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,6.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,7.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,9.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0])), (367933,(10000,[11,16,17,26,39,44,45,68,70,114,158,185,191,208,314,316,367,404,505,636,761,787,852,1121,1275,1460,1544,1571,1573,1713,1796,1890,2080,2558,2637,2689,3003,3358,3552,3710,3980,3997,4151,5137,5499,6067,6658,6737,6755,7029,7108,7254,7446,7910,8255,8611,8914,8943,9344,9584],[1.0,1.0,1.0,1.0,1.0,1.0,4.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])), (1204225,(10000,[1,34,255,495,1025,1088,1276,1657,1760,2278,3616,4475,5320,5476,6914,7977,9032,9449],[1.0,1.0,1.0,3.0,3.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,4.0,1.0,1.0,1.0,1.0,3.0])), (1225253,(10000,[17,26,44,47,48,50,55,58,66,71,85,113,185,196,208,221,248,295,316,398,449,470,475,485,526,552,603,613,616,740,777,821,849,898,901,923,937,1046,1065,1270,1299,1372,1427,1815,1987,2012,2087,2124,2246,2297,2459,2475,2663,2747,3063,3693,3798,3804,4112,4925,4961,4989,4992,5437,5493,5667,6130,6212,8047,8437,9555],[2.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0])), (503655,(10000,[0,4,6,18,120,138,203,209,296,364,379,405,507,509,513,1068,1189,1207,1224,1394,1506,1512,1988,2591,2730,3151,4124,4908,5016,6305,6403,6575,6621,7057,7839,9567],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,1.0])), (821159,(10000,[8,11,20,32,33,49,51,52,55,60,62,67,76,79,96,125,136,141,151,158,162,168,171,172,185,187,191,201,226,229,250,274,277,279,288,289,303,306,313,314,322,353,362,375,390,403,466,537,580,611,674,682,700,707,713,717,724,738,809,812,854,863,865,900,932,939,951,962,965,1037,1077,1078,1158,1239,1282,1293,1354,1356,1407,1412,1470,1472,1536,1610,1682,1847,1859,1878,1911,1944,1986,2000,2058,2098,2166,2255,2272,2342,2507,2520,2558,2607,2657,2826,2836,3004,3010,3047,3205,3214,3259,3563,3685,3750,3959,3966,4275,4362,4370,4416,4457,4665,4899,5038,5116,5298,5376,5876,6031,6079,6097,6620,6726,7782,7987,8432,8457,8582,9481,9640],[1.0,1.0,1.0,1.0,1.0,8.0,1.0,4.0,1.0,1.0,1.0,1.0,10.0,1.0,1.0,4.0,1.0,4.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,1.0,1.0,1.0,15.0,13.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,11.0,2.0,1.0,1.0,7.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,11.0,1.0,1.0,3.0,3.0,1.0,1.0,1.0,2.0,1.0,1.0,3.0,6.0,1.0,1.0,1.0,3.0,1.0,1.0])), (8299631,(10000,[160,227,570,624,1703,3740],[1.0,1.0,2.0,1.0,4.0,2.0])), (4684122,(10000,[0,8,14,16,24,29,32,49,52,75,76,165,207,261,292,368,392,577,599,610,722,730,739,799,856,888,993,1055,1077,1216,1265,1318,1354,1413,1414,1417,1421,1488,1574,1703,2390,2830,2909,4123,4768,5085,5352,5583,5798,6880,7018,9269,9442],[1.0,1.0,3.0,1.0,1.0,1.0,1.0,2.0,4.0,2.0,3.0,1.0,2.0,2.0,5.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,5.0,1.0,1.0,1.0,1.0,1.0,12.0,1.0,1.0,4.0,1.0,5.0,1.0,1.0,4.0,1.0])), (1695311,(10000,[11,12,17,20,64,70,266,520,559,760,873,874,876,1103,1220,1329,1534,1616,1648,1899,1958,1966,2223,2536,2797,3196,3405,3480,3780,4279,6439,6983,7940,7976],[1.0,1.0,1.0,1.0,3.0,3.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])))

  

We compute topic distributions, and extract some additional information
about them.

In [ ]:
val sampleRdd = sc.makeRDD(sample)
val topicDistributions = model.topicDistributions(sampleRdd)
val MLTopics = topicDistributions.map({case(id, vs) => (id, vs.argmax)})
val ids = MLTopics.map({case(id, vs)=>id}).collect()
val selection = filtered.filter({case(id, title, contents) => ids.contains(id)}).map({case(id, title, contents) => (id, title)}).collect()
val allTopicProbs = model.topicDistributions(lda_countVector_mllib)

  

>     sampleRdd: org.apache.spark.rdd.RDD[(Long, org.apache.spark.mllib.linalg.Vector)] = ParallelCollectionRDD[2329] at makeRDD at command-4443336225072182:1
>     topicDistributions: org.apache.spark.rdd.RDD[(Long, org.apache.spark.mllib.linalg.Vector)] = MapPartitionsRDD[2330] at map at LDAModel.scala:373
>     MLTopics: org.apache.spark.rdd.RDD[(Long, Int)] = MapPartitionsRDD[2331] at map at command-4443336225072182:3
>     ids: Array[Long] = Array(2779114, 226060, 1613204, 4970370, 700470, 636341, 4987605, 766457, 4965484, 82392, 7982574, 7973374, 367933, 1204225, 1225253, 503655, 821159, 8299631, 4684122, 1695311)
>     selection: Array[(Long, String)] = Array((82392,Senmedeltiden), (226060,Río Verde, San Luis Potosí), (367933,Kontrollsumma), (503655,Adam Kårsnäs), (636341,Live at McCabe's Guitar Shop), (700470,R2-0), (766457,Pakistanska muslimska förbundet - L), (821159,Fisher Athletic FC), (1204225,Financial Institutions Reform, Recovery and Enforcement Act of 1989), (1225253,Gardar), (1613204,Bessan), (1695311,Borneol), (2779114,Sara Branham Matthews), (4684122,Cecil Blachford), (4965484,Riddarsynerätt), (4970370,Kenyaskratthärfågel), (4987605,Rihannas diskografi), (7973374,Ahtisaariplanen), (7982574,James S. Brown), (8299631,Mount Roper))
>     allTopicProbs: org.apache.spark.rdd.RDD[(Long, org.apache.spark.mllib.linalg.Vector)] = MapPartitionsRDD[2335] at map at LDAModel.scala:373

  

For each of our selected articles, we can now look for other articles
that very strongly belong in the same topic.

In [ ]:
for ( (article_id, topic_id) <- MLTopics.collect() ) {
  val title = selection.filter({case(id, title) => id == article_id})(0)._2
  val topicProbs = allTopicProbs.map({case(id, vs) => (id, vs(topic_id))})
  val top5articles = topicProbs.takeOrdered(5)(Ordering[Double].reverse.on(_._2))
  val top5ids = top5articles.map(_._1)
  val top5titles = filtered.filter({case(id, title, contents) => top5ids.contains(id) }).map({case(id, title, contents) => (title)}).collect
  println(s"Top 5 articles on the same topic as ${title}:")
  
  for (t <- top5titles) println(s" * ${t}")
  println("")
}

  

>     Top 5 articles on the same topic as Sara Branham Matthews:
>      * Geological Society of London
>      * Lista över Bank of England-chefer
>      * Howard (efternamn)
>      * Kansler för hertigdömet Lancaster
>      * Lordpresident
>
>     Top 5 articles on the same topic as Río Verde, San Luis Potosí:
>      * Nederländernas Grand Prix 1980
>      * Nederländernas Grand Prix 1978
>      * Nederländernas Grand Prix 1977
>      * Nederländernas Grand Prix 1976
>      * Storbritanniens Grand Prix 1975
>
>     Top 5 articles on the same topic as Bessan:
>      * Saint-Martin-du-Mont, Ain
>      * Saint-Martin-de-Ré
>      * Naves, Corrèze
>      * Saint-Martin-Sainte-Catherine
>      * Saint-Pierre-des-Corps
>
>     Top 5 articles on the same topic as Kenyaskratthärfågel:
>      * Långnäbbad piplärka
>      * Svarthuvad juveltrast
>      * Kloss sångare
>      * Amurbuskskvätta
>      * Guldfulvetta
>
>     Top 5 articles on the same topic as R2-0:
>      * Haga, Göteborg
>      * Raus plantering
>      * Malmö stads spårvägar
>      * Mälarstrandens sommarbebyggelse
>      * Kvarteret Vintertullen
>
>     Top 5 articles on the same topic as Live at McCabe's Guitar Shop:
>      * Grammy Award
>      * Lista över sånger inspelade av Elvis Presley
>      * Paramore
>      * L.A. Guns
>      * Skillet
>
>     Top 5 articles on the same topic as Rihannas diskografi:
>      * Grammy Award
>      * Lista över sånger inspelade av Elvis Presley
>      * Paramore
>      * L.A. Guns
>      * Skillet
>
>     Top 5 articles on the same topic as Pakistanska muslimska förbundet - L:
>      * Europeiska folkpartiets grupp (kristdemokrater)
>      * Gruppen De gröna/Europeiska fria alliansen
>      * Gruppen Renew Europe
>      * Gruppen Progressiva förbundet av socialdemokrater i Europaparlamentet
>      * Gruppen Europeiska konservativa och reformister
>
>     Top 5 articles on the same topic as Riddarsynerätt:
>      * Svenska marinen
>      * Livregementets husarer
>      * Trängregementet
>      * Marinbasen
>      * Skånska dragonregementet (pansar)
>
>     Top 5 articles on the same topic as Senmedeltiden:
>      * Polsk-sovjetiska kriget 1919–1921
>      * Vitrysslands historia
>      * Cypernkrisen 1974
>      * Israel–Libanon-konflikten
>      * Kungariket Bulgarien
>
>     Top 5 articles on the same topic as James S. Brown:
>      * Geological Society of London
>      * Lista över Bank of England-chefer
>      * Howard (efternamn)
>      * Kansler för hertigdömet Lancaster
>      * Lordpresident
>
>     Top 5 articles on the same topic as Ahtisaariplanen:
>      * Polsk-sovjetiska kriget 1919–1921
>      * Vitrysslands historia
>      * Cypernkrisen 1974
>      * Israel–Libanon-konflikten
>      * Kungariket Bulgarien
>
>     Top 5 articles on the same topic as Kontrollsumma:
>      * Psykoterapi
>      * Analog elektronik
>      * Linuxdistribution
>      * Datautvinning
>      * Digitalteknik
>
>     Top 5 articles on the same topic as Financial Institutions Reform, Recovery and Enforcement Act of 1989:
>      * Human Rights Watch
>      * Logica
>      * Verisign
>      * RISE Research Institutes of Sweden
>      * Caesars Entertainment, Inc. (2020–)
>
>     Top 5 articles on the same topic as Gardar:
>      * Haga, Göteborg
>      * Raus plantering
>      * Malmö stads spårvägar
>      * Mälarstrandens sommarbebyggelse
>      * Kvarteret Vintertullen
>
>     Top 5 articles on the same topic as Adam Kårsnäs:
>      * Persson
>      * Svensson
>      * Johansson
>      * Jonsson
>      * Eriksson
>
>     Top 5 articles on the same topic as Fisher Athletic FC:
>      * Magnus Johansson (ishockeyspelare)
>      * Per-Åge Skrøder
>      * AIK Fotbolls säsong 1998
>      * Jonas Junland
>      * Kim Staal
>
>     Top 5 articles on the same topic as Mount Roper:
>      * Iraqi Airways
>      * Saudia
>      * Martinair
>      * OR Tambo internationella flygplats
>      * Pacific Air Lines Flight 773
>
>     Top 5 articles on the same topic as Cecil Blachford:
>      * Magnus Johansson (ishockeyspelare)
>      * Per-Åge Skrøder
>      * AIK Fotbolls säsong 1998
>      * Jonas Junland
>      * Kim Staal
>
>     Top 5 articles on the same topic as Borneol:
>      * Kloakdjur
>      * Rovdjur
>      * Pungdjur
>      * Trollsländor
>      * Kirurgfiskar